In [ ]:
import copy
import glob
import json
import os
import random
import re
import subprocess
import sys
import time
from pathlib import Path
from typing import Dict, List
import itertools
import scipy as sp
from collections import defaultdict

import h5py
import imageio.v3 as iio
import matplotlib
import matplotlib.colors as mcolors 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import cycler
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Circle, Polygon
from numba import njit
from scipy.interpolate import CubicSpline
from scipy.ndimage import uniform_filter1d
import scipy as sp

# plt.style.use("ggplot")
# plt.rcParams["figure.figsize"] = (12, 10)
spec_home = "/home/himanshu/spec/my_spec"
matplotlib.matplotlib_fname()


# Various functions to read across levs
### Also functions to make reports

### domain color 

In [ ]:
def filter_by_regex(regex,col_list,exclude=False):
  filtered_set = set()
  if type(regex) is list:
    for reg in regex:
      for i in col_list:
        if re.search(reg,i):
          filtered_set.add(i)
  else:
    for i in col_list:
      if re.search(regex,i):
        filtered_set.add(i)

  filtered_list = list(filtered_set)
  if exclude:
    col_list_copy = list(col_list.copy())
    for i in filtered_list:
      if i in col_list_copy:
        col_list_copy.remove(i)
    filtered_list = col_list_copy

  # Restore the original order
  filtered_original_ordered_list = []
  for i in list(col_list):
    if i in filtered_list:
      filtered_original_ordered_list.append(i)
  return filtered_original_ordered_list

def limit_by_col_val(min_val,max_val,col_name,df):
  filter = (df[col_name]>=min_val) &(df[col_name] <=max_val)
  return df[filter]

def get_domain_name(col_name,just_domain_names):
  if just_domain_names:
    return col_name
  def AMR_domains_to_decimal(subdoamin_name):
    # SphereC28.0.1
    a = subdoamin_name.split(".")
    # a = [SphereC28,0,1]
    decimal_rep = a[0]+"."
    # decimal_rep = SphereC28.
    for i in a[1:]:
      decimal_rep = decimal_rep + i
    # decimal_rep = SphereC28.01
    return decimal_rep

  if "on" in col_name:
    return AMR_domains_to_decimal(col_name.split(" ")[-1])
  if "in" in col_name:
    return AMR_domains_to_decimal(col_name.split(" ")[-1])
  elif "_" in col_name:
    return col_name.split("_")[0]
  elif "MinimumGridSpacing" in col_name:
    return col_name.split("[")[-1][:-1]
  else:
    raise Exception(f"{col_name} type not implemented in return_sorted_domain_names")

def return_sorted_domain_names(domain_names, repeated_symmetric=False, num_Excision=2, just_domain_names=False):

  # def filtered_domain_names(domain_names, filter):
  #   return [i for i in domain_names if get_domain_name(i).startswith(filter)]

  def filtered_domain_names(domain_names, filter):
    return [i for i in domain_names if re.match(filter, get_domain_name(i, just_domain_names))]

  def sort_spheres(sphere_list,reverse=False):
    if len(sphere_list) == 0:
      return []
    if "SphereA" in sphere_list[0] and "Filled" not in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('SphereA')), reverse=reverse)
    elif "SphereB" in sphere_list[0] and "Filled" not in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('SphereB')), reverse=reverse)
    elif "SphereC" in sphere_list[0] and "Filled" not in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('SphereC')), reverse=reverse)
    elif "SphereD" in sphere_list[0] and "Filled" not in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('SphereD')), reverse=reverse)
    elif "SphereE" in sphere_list[0] and "Filled" not in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('SphereE')), reverse=reverse)
    elif "FilledSphereA" in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('FilledSphereA')), reverse=reverse)
    elif "FilledSphereB" in sphere_list[0]:
      return sorted(sphere_list, key=lambda x: float(get_domain_name(x, just_domain_names).lstrip('FilledSphereB')), reverse=reverse)

  FilledCylinderCA = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}CA')
  CylinderCA = filtered_domain_names(domain_names, r'Cylinder.{0,2}CA')
  FilledCylinderEA = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}EA')
  CylinderEA = filtered_domain_names(domain_names, r'Cylinder.{0,2}EA')
  SphereA = sort_spheres(filtered_domain_names(domain_names, 'SphereA'), reverse=True)
  FilledSphereA = sort_spheres(filtered_domain_names(domain_names, 'FilledSphereA'), reverse=True)
  CylinderSMA = filtered_domain_names(domain_names, r'CylinderS.{0,2}MA')
  FilledCylinderMA = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}MA')

  FilledCylinderMB = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}MB')
  CylinderSMB = filtered_domain_names(domain_names, r'CylinderS.{0,2}MB')
  FilledSphereB = sort_spheres(filtered_domain_names(domain_names, 'FilledSphereB'), reverse=True)
  SphereB = sort_spheres(filtered_domain_names(domain_names, 'SphereB'), reverse=True)
  CylinderEB = filtered_domain_names(domain_names, r'Cylinder.{0,2}EB')
  FilledCylinderEB = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}EB')
  CylinderCB = filtered_domain_names(domain_names, r'Cylinder.{0,2}CB')
  FilledCylinderCB = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}CB')

  SphereC = sort_spheres(filtered_domain_names(domain_names, 'SphereC'), reverse=False)
  SphereD = sort_spheres(filtered_domain_names(domain_names, 'SphereD'), reverse=False)
  SphereE = sort_spheres(filtered_domain_names(domain_names, 'SphereE'), reverse=False)

  import math
  if FilledSphereB == [] and repeated_symmetric:
    FilledSphereB = ['Excision']*math.ceil(num_Excision/2)
  if FilledSphereA == [] and repeated_symmetric:
    FilledSphereA = ['Excision']*math.ceil(num_Excision/2)

  combined_columns = [FilledCylinderCA, CylinderCA, FilledCylinderEA, CylinderEA, SphereA, FilledSphereA, CylinderSMA, FilledCylinderMA, FilledCylinderMB, CylinderSMB, SphereB, FilledSphereB, CylinderEB, FilledCylinderEB, CylinderCB, FilledCylinderCB, SphereC, SphereD, SphereE]
  if repeated_symmetric:
    combined_columns = [SphereE[::-1], SphereD[::-1], SphereC[::-1],FilledCylinderCA[::-1], CylinderCA[::-1], FilledCylinderEA[::-1], CylinderEA[::-1], SphereA, FilledSphereA,FilledSphereA[::-1], SphereA[::-1], CylinderSMA[::-1], FilledCylinderMA[::-1], FilledCylinderMB, CylinderSMB, SphereB,FilledSphereB, FilledSphereB[::-1], SphereB[::-1], CylinderEB, FilledCylinderEB, CylinderCB, FilledCylinderCB, SphereC, SphereD, SphereE]
  combined_columns = [item for sublist in combined_columns for item in sublist]

  # Just append the domains not following any patterns in the front. Mostly domains surrounding sphereA for high spin and mass ratios
  combined_columns_set = set(combined_columns)
  domain_names_set = set()
  for i in domain_names:
    domain_names_set.add(i)
  subdomains_not_sorted = list(domain_names_set - combined_columns_set)
  return subdomains_not_sorted+combined_columns

class BBH_domain_sym_ploy:
  def __init__(self, center_xA, rA,RA,rC,RC,nA,nC,color_dict:dict=None):
    self.center_xA = center_xA
    self.color_dict = color_dict
    self.rA = rA # Largest SphereA radius
    self.RA = RA # Radius of FilledCylinderE
    self.rC = rC # Smallest SphereC radius
    self.RC = RC # Radius of the largest SphereC

    self.nA = nA # Number of SphereA
    self.nC = nC # Number of SphereC

    self.alpha_for_FilledCylinderE_from_Center_bh = np.radians(50)
    self.outer_angle_for_CylinderSM_from_Center_bh = np.arccos(self.center_xA/self.RA)
    self.inner_angle_for_CylinderSM_from_Center_bh = self.outer_angle_for_CylinderSM_from_Center_bh/3

    self.patches = []

    self.add_shpereCs()

    self.add_CylinderC(which_bh='A')
    self.add_FilledCylinderE(which_bh='A')
    self.add_CylinderE(which_bh='A')
    self.add_CylinderSM(which_bh='A')
    self.add_FilledCylinderM(which_bh='A')
    self.add_FilledCylinderC(which_bh='A')

    self.add_CylinderC(which_bh='B')
    self.add_FilledCylinderE(which_bh='B')
    self.add_CylinderE(which_bh='B')
    self.add_CylinderSM(which_bh='B')
    self.add_FilledCylinderM(which_bh='B')
    self.add_FilledCylinderC(which_bh='B')

    self.add_inner_shperes(which_bh='A')
    self.add_inner_shperes(which_bh='B')

    # print the unmatched domains
    print(self.color_dict)

  def get_matching_color(self, domain_name:str):
    if self.color_dict is None:
      return np.random.rand(3,)
    for key in self.color_dict.keys():
      if domain_name in key:
        # Remove the domain name from the key, this will allow us to see which domains were not matched
        return self.color_dict.pop(key)
    # No match found
    return 'pink'

  def add_inner_shperes(self,which_bh):
    center = self.center_xA
    if which_bh == 'B':
      center = -self.center_xA
  
    spheres_outer_radii = np.linspace(self.rA, 0, self.nA+2)
    i=nA-1
    for r in spheres_outer_radii[:-2]:
      domain_name = f'Sphere{which_bh}{i}'
      i = i-1
      color = self.get_matching_color(domain_name)
      self.patches.append(Circle((center, 0), r, facecolor=color, edgecolor='black'))

    domain_name = f'Sphere{which_bh}{i}'
    i = i-1
    color = self.get_matching_color(domain_name)
    self.patches.append(Circle((center, 0), spheres_outer_radii[-2], facecolor='black', edgecolor='black'))

  def add_shpereCs(self):
    spheres_outer_radii = np.linspace(self.RC, self.rC, self.nC+1)[:-1]
    i=nC-1
    for r in spheres_outer_radii:
      domain_name = f'SphereC{i}'
      i = i-1
      color = self.get_matching_color(domain_name)
      self.patches.append(Circle((0, 0), r, facecolor=color, edgecolor='black'))
    
  def add_FilledCylinderE(self,which_bh):
    alpha = self.alpha_for_FilledCylinderE_from_Center_bh
    
    x_inner = self.center_xA+self.rA*np.cos(alpha)
    y_inner = self.rA*np.sin(alpha)
    x_outer = self.center_xA+self.RA*np.cos(alpha)
    y_outer = self.RA*np.sin(alpha)

    if which_bh == 'B':
      x_inner = -x_inner
      x_outer = -x_outer
    vertices=[
      (x_inner,y_inner),
      (x_outer,y_outer),
      (x_outer,-y_outer),
      (x_inner,-y_inner),
    ]
    color = self.get_matching_color(f'FilledCylinderE{which_bh}')
    self.patches.append(Polygon(vertices, closed=True, facecolor=color, edgecolor='black'))

  def add_CylinderE(self,which_bh):
    alpha = self.alpha_for_FilledCylinderE_from_Center_bh
    beta = self.outer_angle_for_CylinderSM_from_Center_bh

    x_inner_away_from_center = self.center_xA+self.rA*np.cos(alpha)
    y_inner_away_from_center = self.rA*np.sin(alpha)
    x_outer_away_from_center = self.center_xA+self.RA*np.cos(alpha)
    y_outer_away_from_center = self.RA*np.sin(alpha)

    x_inner_closer_to_center = self.center_xA-self.rA*np.cos(beta)
    y_inner_closer_to_center = self.rA*np.sin(beta)
    x_outer_closer_to_center = 0
    y_outer_closer_to_center = self.RA*np.sin(beta)

    if which_bh == 'B':
      x_inner_away_from_center = -x_inner_away_from_center
      x_outer_away_from_center = -x_outer_away_from_center
      x_inner_closer_to_center = -x_inner_closer_to_center
      x_outer_closer_to_center = -x_outer_closer_to_center

    vertices=[
      (x_inner_away_from_center,y_inner_away_from_center),
      (x_outer_away_from_center,y_outer_away_from_center),
      (x_outer_closer_to_center,y_outer_closer_to_center),
      (x_inner_closer_to_center,y_inner_closer_to_center),
      (x_inner_closer_to_center,-y_inner_closer_to_center),
      (x_outer_closer_to_center,-y_outer_closer_to_center),
      (x_outer_away_from_center,-y_outer_away_from_center),
      (x_inner_away_from_center,-y_inner_away_from_center),
    ]
    color = self.get_matching_color(f'CylinderE{which_bh}')
    self.patches.append(Polygon(vertices, closed=True, facecolor=color, edgecolor='black'))

  def add_CylinderC(self,which_bh):
    alpha = self.alpha_for_FilledCylinderE_from_Center_bh
    beta = self.outer_angle_for_CylinderSM_from_Center_bh

    x_inner_away_from_center = self.center_xA+self.rA*np.cos(alpha)
    y_inner_away_from_center = self.rA*np.sin(alpha)
    x_outer_away_from_center = self.rC*np.cos(np.radians(30))
    y_outer_away_from_center = self.rC*np.sin(np.radians(30))

    x_inner_closer_to_center = 0
    y_inner_closer_to_center = self.RA*np.sin(beta)
    x_outer_closer_to_center = 0
    y_outer_closer_to_center = self.rC

    if which_bh == 'B':
      x_inner_away_from_center = -x_inner_away_from_center
      x_outer_away_from_center = -x_outer_away_from_center
      x_inner_closer_to_center = -x_inner_closer_to_center
      x_outer_closer_to_center = -x_outer_closer_to_center

    vertices=[
      (x_inner_closer_to_center,y_inner_closer_to_center),
      (x_outer_closer_to_center,y_outer_closer_to_center),
      (x_outer_away_from_center,y_outer_away_from_center),
      (x_inner_away_from_center,y_inner_away_from_center),
      (x_inner_away_from_center,-y_inner_away_from_center),
      (x_outer_away_from_center,-y_outer_away_from_center),
      (x_outer_closer_to_center,-y_outer_closer_to_center),
      (x_inner_closer_to_center,-y_inner_closer_to_center),
    ]
    color = self.get_matching_color(f'CylinderC{which_bh}')
    self.patches.append(Polygon(vertices, closed=True, facecolor=color, edgecolor='black'))

  def add_CylinderSM(self,which_bh):
    beta = self.outer_angle_for_CylinderSM_from_Center_bh
    gamma = self.inner_angle_for_CylinderSM_from_Center_bh

    x_inner_away_from_center = self.center_xA-self.rA*np.cos(beta)
    y_inner_away_from_center = self.rA*np.sin(beta)
    x_outer_away_from_center = 0
    y_outer_away_from_center = self.RA*np.sin(beta)

    x_inner_closer_to_center = self.center_xA-self.rA*np.cos(gamma)
    y_inner_closer_to_center = self.rA*np.sin(gamma)
    x_outer_closer_to_center = 0
    y_outer_closer_to_center = self.RA*np.sin(gamma)

    if which_bh == 'B':
      x_inner_away_from_center = -x_inner_away_from_center
      x_outer_away_from_center = -x_outer_away_from_center
      x_inner_closer_to_center = -x_inner_closer_to_center
      x_outer_closer_to_center = -x_outer_closer_to_center

    vertices=[
      (x_inner_away_from_center,y_inner_away_from_center),
      (x_outer_away_from_center,y_outer_away_from_center),
      (x_outer_closer_to_center,y_outer_closer_to_center),
      (x_inner_closer_to_center,y_inner_closer_to_center),
      (x_inner_closer_to_center,-y_inner_closer_to_center),
      (x_outer_closer_to_center,-y_outer_closer_to_center),
      (x_outer_away_from_center,-y_outer_away_from_center),
      (x_inner_away_from_center,-y_inner_away_from_center),
    ]
    color = self.get_matching_color(f'CylinderSM{which_bh}')
    self.patches.append(Polygon(vertices, closed=True, facecolor=color, edgecolor='black'))
    
  def add_FilledCylinderM(self,which_bh):
    gamma = self.inner_angle_for_CylinderSM_from_Center_bh

    x_inner = self.center_xA-self.rA*np.cos(gamma)
    y_inner = self.rA*np.sin(gamma)
    x_outer = 0
    y_outer = self.RA*np.sin(gamma)

    if which_bh == 'B':
      x_inner = -x_inner
      x_outer = -x_outer
    vertices=[
      (x_inner,y_inner),
      (x_outer,y_outer),
      (x_outer,-y_outer),
      (x_inner,-y_inner),
    ]
    color = self.get_matching_color(f'FilledCylinderM{which_bh}')
    self.patches.append(Polygon(vertices, closed=True, facecolor=color, edgecolor='black'))

  def add_FilledCylinderC(self,which_bh):
    alpha = self.alpha_for_FilledCylinderE_from_Center_bh

    x_inner = self.center_xA+self.RA*np.cos(alpha)
    y_inner = self.RA*np.sin(alpha)
    x_outer = self.rC*np.cos(np.radians(30))
    y_outer = self.rC*np.sin(np.radians(30))

    if which_bh == 'B':
      x_inner = -x_inner
      x_outer = -x_outer
    vertices=[
      (x_inner,y_inner),
      (x_outer,y_outer),
      (x_outer,-y_outer),
      (x_inner,-y_inner),
    ]
    color = self.get_matching_color(f'FilledCylinderC{which_bh}')
    self.patches.append(Polygon(vertices, closed=True, facecolor=color, edgecolor='black'))

def scalar_to_color(scalar_dict,min_max_tuple=None,color_map="viridis"):
  arr_keys,arr_vals = [], []
  for key,val in scalar_dict.items():
    if np.isnan(val):
      continue
    else:
      arr_keys.append(key)
      arr_vals.append(val)

  scalar_array = np.array(arr_vals, dtype=np.float64) 
  scalar_array = np.log10(scalar_array)
  min_val = np.min(scalar_array)
  max_val = np.max(scalar_array)
  print(min_val,max_val)
  if min_max_tuple is not None:
    min_val, max_val = min_max_tuple
  scalar_normalized = (scalar_array - min_val) / (max_val - min_val)

  colormap = plt.get_cmap(color_map)
  colors = {}
  for key,value in zip(arr_keys,scalar_normalized):
    colors[key] = colormap(value)

  # Get colorbar
  norm = Normalize(vmin=min_val, vmax=max_val)

  sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
  sm.set_array([])

  return colors,sm

# nA=4
# rA=nA*1.5
# center_xA=rA + 2
# RA=rA+5
# rC=RA*2
# nC=30
# RC=rC+nC

# fig, ax = plt.subplots(figsize=(12, 10))

# domain_color_local = domain_color.copy()
# patches_class = BBH_domain_sym_ploy(center_xA=center_xA, rA=rA, RA=RA, rC=rC, RC=RC, nA=nA, nC=nC, color_dict=domain_color_local) 
# for patch in patches_class.patches:
#   ax.add_patch(patch)

# ax.set_xlim(-RC, RC)
# ax.set_ylim(-RC, RC)
# ax.set_aspect('equal')

### Functions to read h5 files

### Functions to read horizon files

In [ ]:
def make_Bh_pandas(h5_dir):
    # Empty dataframe
    df = pd.DataFrame()
    
    # List of all the vars in the h5 file
    var_list = []
    h5_dir.visit(var_list.append)
    
    
    for var in var_list:
        # This means there is no time column
        # print(f"{var} : {h5_dir[var].shape}")
        if df.shape == (0,0):
            # data[:,0] is time and then we have the data
            data = h5_dir[var]
            
            # vars[:-4] to remove the .dat at the end
            col_names = make_col_names(var[:-4],data.shape[1]-1)
            col_names.append('t')
            # Reverse the list so that we get ["t","var_name"]
            col_names.reverse()            
            append_to_df(data[:],col_names,df)
            
        else:
            data = h5_dir[var]
            col_names = make_col_names(var[:-4],data.shape[1]-1)         
            append_to_df(data[:,1:],col_names,df)
            
    return df

def append_to_df(data,col_names,df):
    for i,col_name in enumerate(col_names):
        df[col_name] = data[:,i]
        
def make_col_names(val_name:str,val_size:int):
    col_names = []
    if val_size == 1:
        col_names.append(val_name)
    else:
        for i in range(val_size):
            col_names.append(val_name+f"_{i}")
    return col_names


def horizon_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if key == 'VersionHist.ver':
                # Newer runs have this
                continue
            df_dict[key[:-4]] = make_Bh_pandas(hf[key])

    return df_dict

def read_horizon_across_Levs(path_list:List[Path]):
    df_listAB = []
    df_listC = []
    final_dict = {}
    for path in path_list:
        df_lev = horizon_to_pandas(path)
        # Either [AhA,AhB] or [AhA,AhB,AhC]
        if len(df_lev.keys()) > 1:
            df_listAB.append(df_lev)
        # Either [AhC] or [AhA,AhB,AhC]
        if (len(df_lev.keys()) == 1) or (len(df_lev.keys()) ==3):
            df_listC.append(df_lev)
    if len(df_listAB)==1:
        # There was only one lev
        final_dict = df_listAB[0]
    else:
        final_dict["AhA"] = pd.concat([df["AhA"] for df in df_listAB])
        final_dict["AhB"] = pd.concat([df["AhB"] for df in df_listAB])
        if len(df_listC) > 0:
            final_dict["AhC"] = pd.concat([df["AhC"] for df in df_listC])
    
    return final_dict

def load_horizon_data_from_levs(base_path:Path, runs_path:Dict[str,Path]):
  data_dict = {}
  for run_name in runs_path.keys():
    path_list = list(base_path.glob(runs_path[run_name]))
    print(path_list)
    data_dict[run_name] = read_horizon_across_Levs(path_list)
  return data_dict

def flatten_dict(horizon_data_dict:Dict[str,pd.DataFrame]) -> Dict[str,pd.DataFrame] :
  flattened_data = {}
  for run_name in horizon_data_dict.keys():
      for horizons in horizon_data_dict[run_name]:
          flattened_data[run_name+"_"+horizons] = horizon_data_dict[run_name][horizons]
          # print(run_name+"_"+horizons)
  return flattened_data

In [ ]:
def read_profiler(file_name):
  with h5py.File(file_name,'r') as f:
    steps = set()
    procs = set()
    names = []
    f.visit(names.append)
    for name in names:
      step = name.split('.')[0][4:]
      steps.add(step)
      if 'Proc' in name:
        procs.add(name.split('/')[-1][4:-4])

    dict_list = []
    for step in steps:
      for proc in procs:
        data = f[f'Step{step}.dir/Proc{proc}.txt'][0].decode()

        lines = data.split("\n")
        time = float((lines[0].split("=")[-1])[:-1])

        curr_dict = {
            "t(M)": time,
            "step": step,
            "proc": proc
        }
        # Find where the columns end
        a = lines[4]
        event_end = a.find("Event")+5
        cum_end = a.find("cum(%)")+6
        exc_end = a.find("exc(%)")+6
        inc_end = a.find("inc(%)")+6

        for line in lines[6:-2]:
          Event = line[:event_end].strip()
          cum = float(line[event_end:cum_end].strip())
          exc = float(line[cum_end:exc_end].strip())
          inc = float(line[exc_end:inc_end].strip())
          N = int(line[inc_end:].strip())
          # print(a)
          # a = line.split("  ")
          # Event,cum,exc,inc,N = [i.strip() for i in a if i!= '']
          curr_dict[f'{Event}_cum'] = cum
          curr_dict[f'{Event}_exc'] = exc
          curr_dict[f'{Event}_inc'] = inc
          curr_dict[f'{Event}_N'] = N

        dict_list.append(curr_dict)
  return pd.DataFrame(dict_list)

def read_profiler_multiindex(folder_path:Path):
  dir_paths,dat_paths = list_all_dir_and_dat_files(folder_path)
  steps = set()
  # Get step names
  for dir in dir_paths:
    step = dir.name.split('.')[0][4:]
    steps.add(step)

  procs = set()
  # Get the proc names
  for txt in dir_paths[0].iterdir():
    if ".txt" in txt.name and "Summary" not in txt.name:
      procs.add(txt.name[4:-4])

  dict_list = []
  col_names = set()
  row_names = []
  for step in steps:
    for proc in procs:
      txt_file_path = folder_path/f'Step{step}.dir/Proc{proc}.txt'

      with txt_file_path.open("r") as f:
        lines = f.readlines()

      time = float((lines[0].split("=")[-1])[:-2])

      curr_dict = {
          "time": time,
          "step": step,
          "proc": proc
      }

      # Find where the columns end
      a = lines[4]
      event_end = a.find("Event")+5
      cum_end = a.find("cum(%)")+6
      exc_end = a.find("exc(%)")+6
      inc_end = a.find("inc(%)")+6

      row_names.append((str(proc),str(time)))

      for line in lines[6:-2]:
        Event = line[:event_end].strip()
        cum = float(line[event_end:cum_end].strip())
        exc = float(line[cum_end:exc_end].strip())
        inc = float(line[exc_end:inc_end].strip())
        N = int(line[inc_end:].strip())
        # print(a)
        # a = line.split("  ")
        # Event,cum,exc,inc,N = [i.strip() for i in a if i!= '']
        col_names.add(Event)
        curr_dict[("cum",Event)] = cum
        curr_dict[("exc",Event)] = exc
        curr_dict[("inc",Event)] = inc
        curr_dict[("N",Event)] = N

      dict_list.append(curr_dict)

  # Multi index rows
  index = pd.MultiIndex.from_tuples(row_names, names=["proc","t(M)"])
  df = pd.DataFrame(dict_list,index=index)
  
  # Multi index cols
  multi_index_columns = [(k if isinstance(k, tuple) else (k, '')) for k in df.columns]
  df.columns = pd.MultiIndex.from_tuples(multi_index_columns)
  df.columns.names = ['metric', 'process']

  # data.xs('24', level="proc")['N']
  # data.xs('0.511442', level="t(M)")['cum']
  # data.xs(('0','0.511442'),level=('proc','t(M)'))
  # data.xs('cum',level='metric',axis=1) = data['cum']
  # data.xs('MPI::MPreduceAdd(MV<double>)',level='process',axis=1)
  # data[data['time']<50]
  # data[data['time']<50]['cum'].xs('0',level='proc')['MPI::MPreduceAdd(MV<double>)']
  return df.sort_index()

### Functions to read dat and hist files

In [ ]:
def get_top_name_from_number(top_number: int, subdomain_name: str) -> str:
    if re.match(r"Sphere", subdomain_name):
        return ["Bf0I1", "Bf1S2", "Bf1S2"][top_number]
    elif re.match(r"Cylinder", subdomain_name):
        return ["Bf0I1", "Bf1S1", "Bf2I1"][top_number]
    elif re.match(r"FilledCylinder", subdomain_name):
        return ["Bf0I1", "Bf1B2Radial", "Bf1B2"][top_number]
    else:
        raise Exception(f"{subdomain_name=} not recognized!")


def read_dat_file(file_name):
    cols_names = []
    # Read column names
    with open(file_name, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "#" not in line:
                # From now onwards it will be all data
                break
            elif "=" in line:
                if ("[" not in line) and ("]" not in line):
                    continue
                cols_names.append(line.split("=")[-1][1:-1].strip())
            else:
                continue

    return pd.read_csv(file_name, sep="\s+", comment="#", names=cols_names)


def hist_files_to_dataframe(file_path):
    # Function to parse a single line and return a dictionary of values
    def parse_line(line):
        data = {}
        # Find all variable=value pairs
        pairs = re.findall(r"([^;=\s]+)=\s*([^;]+)", line)
        for var, val in pairs:
            # Hist-GrDomain.txt should be parsed a little differently
            if "ResizeTheseSubdomains" in var:
                items = val.split("),")
                items[-1] = items[-1][:-1]
                for item in items:
                    name, _, vals = item.split("(")
                    r, l, m = vals[:-1].split(",")
                    data[f"{name}_R"] = int(r)
                    data[f"{name}_L"] = int(l)
                    data[f"{name}_M"] = int(m)
            else:
                data[var] = float(val) if re.match(r"^[\d.e+-]+$", val) else val
        return data

    with open(file_path, "r") as file:
        # Parse the lines
        data = []
        for line in file.readlines():
            data.append(parse_line(line.strip()))

        # Create a DataFrame
        df = pd.DataFrame(data)

    return df


# Files like AhACoefs.dat have unequal number of columns
def read_dat_file_uneq_cols(file_name):
    cols_names = []
    with open(file_name, "r") as f:
        lines = f.readlines()
        max_col_num = 0
        for line in lines:
            max_col_num = max(max_col_num, len(line.split()))
    maxL = np.sqrt(max_col_num-4)-1
    for l in range(int(maxL)+1):
        for m in range(-l, l+1):
            cols_names.append(f"{l},{m}")
    cols_names = ['t(M)','Center-x','Center-y','Center-z'] + cols_names

    return pd.read_csv(file_name, comment='#', sep="\s+", names=cols_names)


def read_power_diagnostics_non_power_spectrum(
    file_path, dat_file_name, psi_or_kappa, top_num
):
    top_num = int(top_num)
    with h5py.File(file_path, "r") as f:
        data_dict = {}
        if psi_or_kappa == "psi":
            data_index = 1
        else:
            data_index = 2
        for sd_name in f.keys():
            top_name = get_top_name_from_number(top_num, sd_name)
            data_dict[
                f"{psi_or_kappa}_{dat_file_name.split('.')[0]}_{top_name} on {sd_name[:-4]}"
            ] = f[sd_name][f"{top_name}_{dat_file_name}"][:, data_index]

        # get time var
        any_subdomain = next(iter(f.keys()))
        top_name = get_top_name_from_number(top_num, any_subdomain)
        data_dict["t(M)"] = f[any_subdomain][f"{top_name}_{dat_file_name}"][:, 0]

        df = pd.DataFrame(data_dict)

    return df

def GetWTDataExtracRadii(folder_path: Path):
    extraction_radii_list = []
    for WT_data in folder_path.glob("BondiCceR*.h5"):
        extraction_radii_list.append(WT_data.name.split("BondiCceR")[1].split(".")[0])
    return sorted(extraction_radii_list)


def GetFiniteRadiiDataVars(folder_path: Path):
    finite_radii_files = []
    for file in folder_path.glob("*_CodeUnits.h5"):
        finite_radii_files.append(file.name.split("_")[0])
    return sorted(finite_radii_files)


def GetFiniteRadiusExtractionList(file_path: Path):
    with h5py.File(file_path, "r") as f:
        extraction_radii_list = []
        for radius_dir in f.keys():
            if "Version" in radius_dir:
                continue
            extraction_radii_list.append(radius_dir[1:5])
    return sorted(extraction_radii_list)


def FindMinMaxL(keys):
    "Finds the minimum and maximum L from the dat files in the finite radius H5 files."
    minL = 10000
    maxL = -10000
    for key in keys:
        if "Y_l" not in key:
            continue
        l = int(key.split("_")[1][1:])
        minL = min(minL, l)
        maxL = max(maxL, l)
    return minL, maxL


def read_finite_radius_quantaties(file_path, radius):
    with h5py.File(file_path, "r") as f:
        if f"R{radius}.dir" not in f.keys():
            # -1 at last drops the version history
            radii_list = [i.split(".")[0][1:] for i in f.keys()][:-1] 
            raise Exception(f"R{radius}.dir not found in {file_path}\n{radii_list=}")
        rad_data = f[f"R{radius}.dir"]
        minL, maxL = FindMinMaxL(rad_data.keys())

        data_is_double_double = False
        data = {"t(M)": rad_data["Y_l2_m0.dat"][:, 0]}
        if data['t(M)'].dtype == np.dtype([('hi', '<f8'), ('lo', '<f8')]):
            data_is_double_double = True
            data['t(M)'] = data['t(M)']['hi']
        for l in range(minL, maxL + 1):
            for m in range(-l, l + 1):
                if data_is_double_double:
                    data[f"{l},{m}"] = (
                        rad_data[f"Y_l{l}_m{m}.dat"][:, 1]['hi']
                        + 1j * rad_data[f"Y_l{l}_m{m}.dat"][:, 2]['hi']
                    )
                else:
                    data[f"{l},{m}"] = (
                        rad_data[f"Y_l{l}_m{m}.dat"][:, 1]
                        + 1j * rad_data[f"Y_l{l}_m{m}.dat"][:, 2]
                    )

        return pd.DataFrame(data)


def read_WT_data(file_path: Path, var: str):
    with h5py.File(file_path, "r") as f:
        # all_m = ['DrJ.dat', 'H.dat', 'J.dat', 'Q.dat', 'U.dat', ]
        # some_m = ['Beta.dat', 'DuR.dat',  'R.dat', 'W.dat']

        col_names = list(f[var].attrs["Legend"])
        col_names[0] = "t(M)"
        name_to_index_dict = {name: i for i, name in enumerate(col_names)}
        
        data_is_double_double = False
        data = {"t(M)": f[var][:, name_to_index_dict["t(M)"]]}
        if data['t(M)'].dtype == np.dtype([('hi', '<f8'), ('lo', '<f8')]):
            data_is_double_double = True
            data['t(M)'] = data['t(M)']['hi']

        maxL = int(col_names[-1].split(",")[0][3:])
        for L in range(maxL + 1):
            for m in range(-L, L + 1):
                real_key = f"Re({L},{m})"
                img_key = f"Im({L},{m})"
                if data_is_double_double:
                    # Note tested because there is no Bondi data from float128 just yets
                    if real_key in name_to_index_dict:
                        data[f"{L},{m}"] = np.array(
                            f[var][:, name_to_index_dict[real_key]]['hi'], dtype=np.complex128
                        )
                    if img_key in name_to_index_dict:
                        data[f"{L},{m}"] += 1j * np.array(
                            f[var][:, name_to_index_dict[img_key]]['hi'], dtype=np.complex128
                        )
                else:
                    if real_key in name_to_index_dict:
                        data[f"{L},{m}"] = np.array(
                            f[var][:, name_to_index_dict[real_key]], dtype=np.complex128
                        )
                    if img_key in name_to_index_dict:
                        data[f"{L},{m}"] += 1j * np.array(
                            f[var][:, name_to_index_dict[img_key]], dtype=np.complex128
                        )

        # df1 = pd.DataFrame(f[var][:], columns=col_names)
        return pd.DataFrame(data)

def read_point_interpolation_file(file_path,  loadALL):
    def get_col_name(file_path):
        # if the file is Int_kappa_ttt.dat which has
        # Points = (0,0,0), (0,0,2), (0,0,4), (0,0,6), (0,0,8)
        # returns a list of string ["kappa_ttt_(0,0,0)", "kappa_ttt_(0,0,2)", ...]
        with open(file_path, 'r') as file:
            file_name = file_path.split("/")[-1][4:-4]  # Get the file name without extension
            lines = file.readlines()
            for line in lines:
                if line.startswith('# Points'):
                    parts = line[11:].split(', ')
                    points = [f"{file_name}_{part.strip()}" for part in parts]
                    return points
    if loadALL:
        parent_folder,var = file_path.split("/Int_")
        parent_folder = Path(parent_folder)
        var = var.split("_")[0]
        all_dat_files = parent_folder.glob(f"Int_{var}_*.dat")
        dfs = []
        for dat_file in all_dat_files:
            print(dat_file)
            column_names = get_col_name(str(dat_file))
            column_names = ["t(M)"] + column_names
            dfs.append(pd.read_csv(dat_file, sep='\s+', comment='#', names=column_names))

        result = dfs[0]
        for df in dfs[1:]:
            result = pd.merge(result, df, on='t(M)', how='outer')
        return result
    else:
        column_names = get_col_name(file_path)
        column_names = ["t(M)"] + column_names
        data = pd.read_csv(file_path, sep='\s+', comment='#', names=column_names)
        return data

def read_OrbitDiagnostics_file(file_name):
    with h5py.File(file_name, "r") as f:
        names = []
        f.visit(names.append)

        first_data_set = names[0]
        data = f[first_data_set]
        # This df now had t and some orbital quantities
        df = pd.DataFrame(data[:], columns=list(data.attrs['Legend']))

        # Now read and append rest of the datasets
        for dataset_name in names[1:]:
            data = f[dataset_name]
            cols_ = list(data.attrs['Legend'])[1:] # Skip time column
            for col in cols_:
                df[col] = data[:, list(data.attrs['Legend']).index(col)]

    return df

def read_dat_file_across_AA(file_pattern):
    # ApparentHorizons/Horizons.h5@AhA
    if "Horizons.h5@" in file_pattern:
        file_pattern, h5_key = file_pattern.split("@")
    if "PowerDiagnostics" in file_pattern:
        file_pattern, dat_file_name, psi_or_kappa, top_num = file_pattern.split("@")
    if "BondiCceR" in file_pattern:
        file_pattern, WT_var = file_pattern.split("@")
        if "Index:" in file_pattern:
            raise Exception("Not Implemented yet!")
    if "_FiniteRadii_CodeUnits" in file_pattern:
        file_pattern, radius = file_pattern.split("@")
        if "Index:" in file_pattern:
            raise Exception("Not Implemented yet!")
    if "@ALL" in file_pattern:
        file_pattern, _ = file_pattern.split("@")
        loadALL = True
    else:
        loadALL = False

    path_pattern = file_pattern
    path_collection = []

    print(path_pattern)
    for folder_name in glob.iglob(path_pattern, recursive=True):
        if os.path.isdir(folder_name) or os.path.isfile(folder_name):
            path_collection.append(folder_name)

    # If we gave wrong radii for the BondiData then return the correct radii list
    if "BondiCceR" in file_pattern and len(path_collection) == 0:
        general_pattern_for_Bondi = file_pattern.split("BondiCceR")[0] + "BondiCceR*.h5"
        all_Bondi_files = glob.glob(general_pattern_for_Bondi)
        radii_list = []
        for file in all_Bondi_files:
            radii_list.append(file.split("BondiCceR")[1].split(".h5")[0])
        raise Exception(
            f"No files found for pattern {file_pattern}!\nAvailable extraction radii are: {sorted(set(radii_list))}"
        )
    path_collection.sort()

    read_data_collection = []
    for path in path_collection:
        print(path)
        # AhACoefs.dat has uneq cols
        if "Coefs.dat" in path:
            read_data_collection.append(read_dat_file_uneq_cols(path))
        elif "Hist-" in path:
            read_data_collection.append(hist_files_to_dataframe(path))
        elif "Profiler" in path:
            read_data_collection.append(read_profiler(path))
        elif "Horizons.h5" in path:
            returned_data = read_horizonh5(path, h5_key)
            if returned_data is not None:
                read_data_collection.append(returned_data)
        elif "PowerDiagnostics" in path:
            returned_data = read_power_diagnostics_non_power_spectrum(
                path, dat_file_name, psi_or_kappa, top_num
            )
            if returned_data is not None:
                read_data_collection.append(returned_data)
        elif "BondiCceR" in path:
            returned_data = read_WT_data(path, WT_var)
            if returned_data is not None:
                read_data_collection.append(returned_data)
        elif "_FiniteRadii_CodeUnits" in path:
            returned_data = read_finite_radius_quantaties(path, radius)
            if returned_data is not None:
                read_data_collection.append(returned_data)
        elif "Int_" in path:
            read_data_collection.append(read_point_interpolation_file(path, loadALL))
        elif "OrbitDiagnostics" in path:
            read_data_collection.append(read_OrbitDiagnostics_file(path))
        else:
            read_data_collection.append(read_dat_file(path))

    data = pd.concat(read_data_collection)
    rename_dict = {
        "t": "t(M)",
        "time": "t(M)",
        "Time": "t(M)",
        "time after step": "t(M)",
    }
    data.rename(columns=rename_dict, inplace=True)

    # f128 has math_utils::sqrt change it back to sqrt
    data.columns = [col.replace("math_utils::","") for col in data.columns]

    # print(data.columns)
    return data


def read_horizonh5(horizonh5_path, h5_key):
    with h5py.File(horizonh5_path, "r") as hf:
        # h5_key = ['AhA','AhB','AhC']
        # Horizons.h5 has keys 'AhA.dir'
        key = h5_key + ".dir"
        # 'AhC' will not be all the horizons.h5
        if key in hf.keys():
            return make_Bh_pandas(hf[key])
        else:
            return None


def read_AH_files(Ev_path):
    fileA = Ev_path + "Run/ApparentHorizons/AhA.dat"
    fileB = Ev_path + "Run/ApparentHorizons/AhB.dat"

    dataA = read_dat_file_across_AA(fileA)
    dataB = read_dat_file_across_AA(fileB)

    return dataA, dataB


# Combines all the pvd files into a single file and save it in the base folder
def combine_pvd_files(base_folder: Path, file_pattern: str, output_path=None):
    pvd_start = """<?xml version="1.0"?>\n<VTKFile type="Collection" version="0.1" byte_order="LittleEndian">\n  <Collection>\n"""
    pvd_end = "  </Collection>\n</VTKFile>"

    vis_folder_name = file_pattern.split("/")[-1][:-4]
    Lev = file_pattern[0:4]

    if output_path is None:
        output_path = f"{base_folder}/{vis_folder_name}_{Lev}.pvd"

    pvd_files = list(base_folder.glob(file_pattern))
    pvd_folders = list(base_folder.glob(file_pattern[:-4]))

    with open(output_path, "w") as write_file:
        write_file.writelines(pvd_start)
        for files in pvd_files:
            print(files)
            with files.open("r") as f:
                for line in f.readlines():
                    line = line.replace(vis_folder_name, str(files)[:-4])
                    if "DataSet" in line:
                        write_file.writelines(line)
        write_file.writelines(pvd_end)

    print(output_path)


def moving_average(array, avg_len):
    return np.convolve(array, np.ones(avg_len)) / avg_len


def moving_average_valid(array, avg_len):
    return np.convolve(array, np.ones(avg_len), "valid") / avg_len

def moving_median_valid(array, avg_len):
    from scipy import ndimage
    # Apply median filter to full array, then trim to 'valid' size
    filtered = ndimage.median_filter(array, size=avg_len)
    # Trim edges to match 'valid' convolution behavior
    trim = (avg_len - 1) // 2
    if avg_len % 2 == 0:  # even window size
        return filtered[trim:-trim-1]
    else:  # odd window size
        return filtered[trim:-trim] if trim > 0 else filtered

def moving_percentile_valid(array, avg_len, percentile=50):
    from scipy import ndimage
    filtered = ndimage.percentile_filter(array, percentile=percentile, size=avg_len)
    trim = (avg_len - 1) // 2
    if avg_len % 2 == 0:
        return filtered[trim:-trim-1]
    else:
        return filtered[trim:-trim] if trim > 0 else filtered

def moving_trimmed_mean_valid(array, avg_len, trim_percent=0.2):
    from scipy import stats
    import pandas as pd
    # Use pandas rolling with trimmed mean
    result = pd.Series(array).rolling(avg_len).apply(
        lambda x: stats.trim_mean(x, trim_percent)
    ).dropna().values
    return result

def moving_robust_mean_valid(array, avg_len, mad_threshold=3):
    """Moving average that excludes outliers based on MAD"""
    import pandas as pd
    from scipy import stats

    def robust_mean(window):
        median = np.median(window)
        mad = stats.median_abs_deviation(window)
        mask = np.abs(window - median) <= mad_threshold * mad
        return np.mean(window[mask]) if np.any(mask) else median

    result = pd.Series(array).rolling(avg_len).apply(robust_mean).dropna().values
    return result


# def moving_average_valid(array, avg_len):
#     # return moving_median_valid(array, avg_len)
#     return moving_percentile_valid(array, avg_len, percentile=5)
#     # return moving_trimmed_mean_valid(array, avg_len, trim_percent=0.005)
#     # return moving_robust_mean_valid(array, avg_len, mad_threshold=3)

def path_to_folder_name(folder_name):
    return folder_name.replace("/", "_")


# Give a dict of {"run_name" = runs_path} and data_file_path to get {"run_name" = dat_file_data}
def load_data_from_levs(runs_path, data_file_path):
    data_dict = {}
    column_list = ""
    for run_name in runs_path.keys():
        data_dict[run_name] = read_dat_file_across_AA(
            runs_path[run_name] + data_file_path
        )
        column_list = data_dict[run_name].columns
    return column_list, data_dict


def add_diff_columns(runs_data_dict, x_axis, y_axis, diff_base):
    if diff_base not in runs_data_dict.keys():
        raise Exception(f"{diff_base} not in {runs_data_dict.keys()}")

    unique_x_data, unique_indices = np.unique(
        runs_data_dict[diff_base][x_axis], return_index=True
    )
    # sorted_indices = np.sort(unique_indices)
    unique_y_data = runs_data_dict[diff_base][y_axis].iloc[unique_indices]
    interpolated_data = sp.interpolate.CubicSpline(
        unique_x_data, unique_y_data, extrapolate=False
    )
    # interpolated_data = sp.interpolate.PchipInterpolator(unique_x_data, unique_y_data, extrapolate=False)

    for key in runs_data_dict:
        if key == diff_base:
            continue
        df = runs_data_dict[key]
        df["diff_abs_" + y_axis] = np.abs(df[y_axis] - interpolated_data(df[x_axis]))
        df["diff_" + y_axis] = df[y_axis] - interpolated_data(df[x_axis])


def plot_graph_for_runs_wrapper(
    runs_data_dict,
    x_axis,
    y_axis_list,
    minT,
    maxT,
    legend_dict=None,
    save_path=None,
    moving_avg_len=0,
    plot_fun=lambda x, y, label: plt.plot(x, y, label=label),
    sort_by=None,
    diff_base=None,
    title=None,
    append_to_title="",
    plot_abs_diff=False,
    constant_shift_val_time=None,
    modification_function=None,
    take_abs=False,
):
    # Do this better using columns of a pandas dataframe
    for y_axis in y_axis_list[:-1]:
        legend_dict = {}
        for key in runs_data_dict:
            legend_dict[key] = key + "_" + str(y_axis)
        plot_graph_for_runs(
            runs_data_dict,
            x_axis,
            y_axis,
            minT,
            maxT,
            legend_dict=legend_dict,
            save_path=None,
            moving_avg_len=moving_avg_len,
            plot_fun=plot_fun,
            sort_by=sort_by,
            diff_base=diff_base,
            title=title,
            append_to_title=append_to_title,
            plot_abs_diff=plot_abs_diff,
            constant_shift_val_time=constant_shift_val_time,
            modification_function=modification_function,
            take_abs=take_abs,
        )

    # Save when plotting the last y_axis.
    y_axis = y_axis_list[-1]
    legend_dict = {}
    for key in runs_data_dict:
        legend_dict[key] = key + "_" + str(y_axis)
    plot_graph_for_runs(
        runs_data_dict,
        x_axis,
        y_axis,
        minT,
        maxT,
        legend_dict=legend_dict,
        save_path=save_path,
        moving_avg_len=moving_avg_len,
        plot_fun=plot_fun,
        sort_by=sort_by,
        diff_base=diff_base,
        title=title,
        append_to_title=append_to_title,
        plot_abs_diff=plot_abs_diff,
        constant_shift_val_time=constant_shift_val_time,
        modification_function=modification_function,
        take_abs=take_abs,
    )

    plt.ylabel("")
    plt.title("" + append_to_title)
    if moving_avg_len > 0:
        plt.title(
            f"{title} (moving avg len = {moving_avg_len})" + append_to_title,
        )

    if save_path is not None:
        fig_x_label = ""
        fig_y_label = ""

        for y_axis in y_axis_list:
            fig_x_label = fig_x_label + x_axis.replace("/", "_").replace(".", "_")
            fig_y_label = fig_y_label + y_axis.replace("/", "_").replace(".", "_")
        save_file_name = (
            f"{fig_y_label}_vs_{fig_x_label}_minT={minT}_maxT={maxT}".replace(".", "_")
        )
        if moving_avg_len > 0:
            save_file_name = save_file_name + f"_moving_avg_len={moving_avg_len}"
        if diff_base is not None:
            save_file_name = save_file_name + f"_diff_base={diff_base}"

        if len(save_file_name) >= 251:  # <save_file_name>.png >=255
            save_file_name = save_file_name[:245] + str(random.randint(10000, 99999))
            print(f"The filename was too long!! New filename is {save_file_name}")

        plt.savefig(save_path + save_file_name)


def plot_graph_for_runs(
    runs_data_dict_original,
    x_axis,
    y_axis,
    minT,
    maxT,
    legend_dict=None,
    save_path=None,
    moving_avg_len=0,
    plot_fun=lambda x, y, label: plt.plot(x, y, label=label),
    sort_by=None,
    diff_base=None,
    title=None,
    append_to_title="",
    plot_abs_diff=False,
    constant_shift_val_time=None,
    modification_function=None,
    take_abs=False,
):
    runs_data_dict = runs_data_dict_original
    if modification_function is not None:
        runs_data_dict = copy.deepcopy(runs_data_dict_original)
        for key in runs_data_dict:
            new_x, new_y, new_y_axis = modification_function(
                runs_data_dict[key][x_axis],
                runs_data_dict[key][y_axis],
                runs_data_dict[key],
                y_axis,
            )
            runs_data_dict[key][new_y_axis] = new_y
            runs_data_dict[key][x_axis] = new_x
        y_axis = new_y_axis

    sort_run_data_dict(runs_data_dict, sort_by=sort_by)
    current_runs_data_dict_keys = list(runs_data_dict.keys())

    if diff_base is not None:
        add_diff_columns(runs_data_dict, x_axis, y_axis, diff_base)
        current_runs_data_dict_keys = []
        for key in runs_data_dict:
            if key == diff_base:
                continue
            else:
                current_runs_data_dict_keys.append(key)
        if plot_abs_diff:
            y_axis = "diff_abs_" + y_axis
        else:
            y_axis = "diff_" + y_axis

    # Find the indices corresponding to maxT and minT
    minT_indx_list = {}
    maxT_indx_list = {}

    if legend_dict is None:
        legend_dict = {}
        for run_name in current_runs_data_dict_keys:
            legend_dict[run_name] = None
    else:
        for run_name in current_runs_data_dict_keys:
            if run_name not in legend_dict:
                raise ValueError(f"{run_name} not in {legend_dict=}")

    for run_name in current_runs_data_dict_keys:
        minT_indx_list[run_name] = len(
            runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < minT]
        )
        maxT_indx_list[run_name] = len(
            runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < maxT]
        )

    if moving_avg_len == 0:
        for run_name in current_runs_data_dict_keys:
            x_data = runs_data_dict[run_name][x_axis][
                minT_indx_list[run_name] : maxT_indx_list[run_name]
            ]
            y_data = runs_data_dict[run_name][y_axis][
                minT_indx_list[run_name] : maxT_indx_list[run_name]
            ]

            if constant_shift_val_time is not None:
                shift_label_val = np.abs(x_data.iloc[-1] - x_data.iloc[0]) / 4
                unique_x_data, unique_indices = np.unique(x_data, return_index=True)
                # sorted_indices = np.sort(unique_indices)
                unique_y_data = y_data.iloc[unique_indices]
                try:
                    interpolated_data = CubicSpline(
                        unique_x_data, unique_y_data, extrapolate=False
                    )
                except Exception as e:
                    print(run_name, unique_y_data)
                y_data = y_data - interpolated_data(constant_shift_val_time)
                if plot_abs_diff:
                    y_data = np.abs(y_data)

            #   print(f"{len(x_data)=},{len(y_data)=},{len(np.argsort(x_data))=},{type(x_data)=}")

            #   sorted_indices = x_data.argsort()
            #   x_data = x_data.iloc[sorted_indices]
            #   y_data = y_data.iloc[sorted_indices]
            legend = legend_dict[run_name]
            if legend is None:
                legend = run_name
            if take_abs:
                y_data = np.abs(y_data)

            if np.all(np.isnan(y_data)):
                print(
                    f"Warning: {run_name} has no data {y_axis=} for {x_axis}={minT} to {maxT}. Skipping."
                )
                continue
            plot_fun(x_data, y_data, legend)

            if constant_shift_val_time is not None:
                plt.axhline(y=y_data.iloc[-1], linestyle=":")
                plt.text(
                    x=np.random.rand() * shift_label_val + x_data.iloc[0],
                    y=y_data.iloc[-1],
                    s=f"{y_data.iloc[-1]:.2e}",
                    verticalalignment="bottom",
                )

        plt.xlabel(x_axis)
        plt.ylabel(y_axis)
        if constant_shift_val_time is not None:
            plt.axvline(x=constant_shift_val_time, linestyle=":", color="red")
        if title is None:
            title = '"' + y_axis + '" vs "' + x_axis + '"'
            if constant_shift_val_time is not None:
                title = title + f" constant_shift_val_time={constant_shift_val_time}"
            if diff_base is not None:
                title = title + f" diff_base={diff_base}"
            if plot_abs_diff:
                title = title + " (abs_diff)"
            if take_abs:
                title = title + " (abs)"
        plt.title(title + append_to_title)
        plt.legend()

    else:
        for run_name in current_runs_data_dict_keys:
            x_data = np.array(
                runs_data_dict[run_name][x_axis][
                    minT_indx_list[run_name] + moving_avg_len - 1 : maxT_indx_list[
                        run_name
                    ]
                ]
            )
            y_data = np.array(
                moving_average_valid(
                    runs_data_dict[run_name][y_axis][
                        minT_indx_list[run_name] : maxT_indx_list[run_name]
                    ],
                    moving_avg_len,
                )
            )

            if constant_shift_val_time is not None:
                shift_label_val = np.abs(x_data.iloc[-1] - x_data.iloc[0]) / 4
                unique_x_data, unique_indices = np.unique(x_data, return_index=True)
                # sorted_indices = np.sort(unique_indices)
                unique_y_data = y_data.iloc[unique_indices]

                interpolated_data = CubicSpline(
                    unique_x_data, unique_y_data, extrapolate=False
                )
                y_data = y_data - interpolated_data(constant_shift_val_time)
                if plot_abs_diff:
                    y_data = np.abs(y_data)

            #   sorted_indices = np.argsort(x_data)
            #   x_data = x_data[sorted_indices]
            #   y_data = y_data[sorted_indices]
            legend = legend_dict[run_name]
            if legend is None:
                legend = run_name
            if take_abs:
                y_data = np.abs(y_data)

            if np.all(np.isnan(y_data)):
                print(
                    f"Warning: {run_name} has no data {y_axis=} for {x_axis}={minT} to {maxT}. Skipping."
                )
                continue
            plot_fun(x_data, y_data, legend)

            if constant_shift_val_time is not None:
                plt.axhline(y=y_data.iloc[-1], linestyle=":")
                plt.text(
                    x=np.random.rand() * shift_label_val + x_data.iloc[0],
                    y=y_data.iloc[-1],
                    s=f"{y_data.iloc[-1]:.1f}",
                    verticalalignment="bottom",
                )

        plt.xlabel(x_axis)
        plt.ylabel(y_axis)
        if constant_shift_val_time is not None:
            plt.axvline(x=constant_shift_val_time, linestyle=":", color="red")
        if title is None:
            title = (
                '"'
                + y_axis
                + '" vs "'
                + x_axis
                + '"  '
                + f"avg_window_len={moving_avg_len}"
            )
            if constant_shift_val_time is not None:
                title = title + f" constant_shift_val_time={constant_shift_val_time}"
            if diff_base is not None:
                title = title + f" diff_base={diff_base}"
            if plot_abs_diff:
                title = title + " (abs_diff)"
            if take_abs:
                title = title + " (abs)"
        plt.title(title + append_to_title)
        plt.legend()

    if save_path is not None:
        fig_x_label = x_axis.replace("/", "_").replace(".", "_")
        fig_y_label = y_axis.replace("/", "_").replace(".", "_")
        save_file_name = (
            f"{fig_y_label}_vs_{fig_x_label}_minT={minT}_maxT={maxT}".replace(".", "_")
        )
        if moving_avg_len > 0:
            save_file_name = save_file_name + f"_moving_avg_len={moving_avg_len}"
        if diff_base is not None:
            save_file_name = save_file_name + f"_diff_base={diff_base}"

        for run_name in current_runs_data_dict_keys:
            save_file_name = (
                save_file_name + "__" + run_name.replace("/", "_").replace(".", "_")
            )

        if len(save_file_name) >= 251:  # <save_file_name>.png >=255
            save_file_name = save_file_name[:245] + str(random.randint(10000, 99999))
            print(f"The filename was too long!! New filename is {save_file_name}")

        plt.savefig(save_path + save_file_name)


def find_file(pattern):
    return glob.glob(pattern, recursive=True)[0]


def plots_for_a_folder(things_to_plot, plot_folder_path, data_folder_path):
    for plot_info in things_to_plot:
        file_name = plot_info["file_name"]
        y_arr = plot_info["columns"][1:]
        x_arr = [plot_info["columns"][0]] * len(y_arr)

        data = read_dat_file_across_AA(data_folder_path + "/**/" + file_name)
        plot_and_save(data, x_arr, y_arr, plot_folder_path, file_name)


def is_the_current_run_going_on(run_folder):
    if len(find_file(run_folder + "/**/" + "TerminationReason.txt")) > 0:
        return False
    else:
        return True


def plot_min_grid_spacing(runs_data_dict):
    """
    runs_data_dict should have dataframes with MinimumGridSpacing.dat data.
    The function will compute the min grid spacing along all domains and plot it.
    """
    keys = runs_data_dict.keys()
    if len(keys) == 0:
        print("There are no dataframes in the dict")

    for key in keys:
        t_step = runs_data_dict[key]["t"]
        min_val = runs_data_dict[key].drop(columns=["t"]).min(axis="columns")
        plt.plot(t_step, min_val, label=key)

    plt.legend()
    plt.xlabel("t")
    plt.ylabel("Min Grid Spacing")
    plt.title("Min grid spacing in all domains")
    plt.show()


def plot_GrAdjustSubChunksToDampingTimes(runs_data_dict):
    keys = runs_data_dict.keys()
    if len(keys) > 1:
        print(
            "To plot the Tdamp for various quantities only put one dataframe in the runs_data_dict"
        )

    data: pd.DataFrame = runs_data_dict[list(keys)[0]]
    tdamp_keys = []
    for key in data.keys():
        if "Tdamp" in key:
            tdamp_keys.append(key)

    # Get a colormap
    cmap = plt.get_cmap("tab10")
    colors = cmap(np.linspace(0, 1, len(tdamp_keys)))

    t_vals = data["time"]
    for i, color, key in zip(range(len(tdamp_keys)), colors, tdamp_keys):
        if i % 2 == 0:
            plt.plot(t_vals, data[key], label=key, color=color)
        else:
            plt.plot(t_vals, data[key], label=key, color=color, linestyle="--")

    min_tdamp = data[tdamp_keys].min(axis="columns")
    plt.plot(
        t_vals,
        min_tdamp,
        label="min_tdamp",
        linewidth=3,
        linestyle="dotted",
        color="red",
    )

    plt.legend()
    plt.xlabel("time")
    plt.title(list(keys)[0])
    plt.show()


def add_max_and_min_val(runs_data_dict):
    # If we load a file with 5 columns with first being time, then find max and min values for all the other columns, at all times and add it to the dataframe.
    # Useful when you want to find like Linf across all domains at all times
    for run_name in runs_data_dict.keys():
        data_frame = runs_data_dict[run_name]
        t = data_frame.iloc[:, 0]
        max_val = np.zeros_like(t)
        min_val = np.zeros_like(t)
        for i in range(len(t)):
            max_val[i] = data_frame.iloc[i, 1:].max()
            min_val[i] = data_frame.iloc[i, 1:].max()

        # Add the values to the dataframe
        data_frame["max_val"] = max_val
        data_frame["min_val"] = min_val


def sort_run_data_dict(runs_data_dict: dict, sort_by=None):
    for run_name in runs_data_dict.keys():
        run_df = runs_data_dict[run_name]
        if sort_by is None:
            if "t(M)" in run_df.keys():
                sort_by = "t(M)"
            else:
                sort_by = run_df.keys()[0]
        runs_data_dict[run_name] = run_df.sort_values(by=sort_by)


### Functions to modify the data, for passing to the plotting function

In [ ]:

def noise_function(x, y, df, y_axis, scipy_or_np="scipy", window=25):
    if scipy_or_np == "scipy":
        running_mean = uniform_filter1d(y, size=window, mode="nearest")
        noise_estimate = y - running_mean
    elif scipy_or_np == "np":
        running_mean = np.convolve(y, np.ones(window), mode="valid") / window
        noise_estimate = np.full(y.shape[0], np.nan, dtype=y.dtype)
        noise_estimate[window // 2 - 1 : -window // 2] = (
            y[window // 2 - 1 : -window // 2] - running_mean
        )
    else:
        raise Exception(f"Invalid scipy_or_np value: {scipy_or_np}")

    return x, noise_estimate, f"noise_{window}_{y_axis}"


def derivative_function(x, y, df, y_axis, order=1):
    if order == 1:
        grad = np.gradient(y, x)
    elif order == 2:
        grad = np.gradient(np.gradient(y, x), x)
    else:
        raise Exception(f"Invalid order value: {order}")

    return x, grad, f"deriv:{order}_{y_axis}"


def compute_center(x, y, df, y_axis):
    center = np.zeros_like(y)
    for i in [
        "CoordCenterInertial_0",
        "CoordCenterInertial_1",
        "CoordCenterInertial_2",
    ]:
        assert i in df.columns, f"Column {i} not found in DataFrame"
        center += df[i] ** 2

    return x, np.sqrt(center), "center"

def compute_dt_center(x, y, df, y_axis):
    center = np.zeros_like(y)
    for i in [
        "CoordCenterInertial_0",
        "CoordCenterInertial_1",
        "CoordCenterInertial_2",
    ]:
        assert i in df.columns, f"Column {i} not found in DataFrame"
        center += df[i] ** 2

    return x,  np.gradient(np.sqrt(center),x), "dt_center"

def min_max_r_ratio(x, y, df, y_axis):
    ratio = np.zeros_like(y)
    for i in [
        "max(r)",
        "min(r)",
    ]:
        assert i in df.columns, f"Column {i} not found in DataFrame"
    ratio = df['min(r)'] / df['max(r)']
    return x, ratio, "ratio"


def index_constraints_norm(x, y, df, y_axis, index_num=3 ,norm="Linf"):
    index_cols = [i for i in df.columns if f"{index_num}Con" in i]

    if norm ==  "Linf": 
        index_norm = np.max(np.abs(df[index_cols]), axis=1)
    elif norm == "L2":
        index_norm = np.linalg.norm(df[index_cols], axis=1)
    elif norm == "RMS":
        index_norm = np.linalg.norm(df[index_cols], axis=1)/ np.sqrt(len(index_cols))
    else:
        raise Exception(f"Invalid norm value: {norm}")

    label = f"{norm}({index_num}Con)"
    return x, index_norm, label

def get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r".*", print_sd_names=False):
    # only consider col names that have _ in them
    filtered_cols = [col for col in df.columns if "_" in col]
    if len(filtered_cols) == 0:
        raise Exception("No subdomains with _ columns found in the dataframe")
    sd_names = set([col.split("_")[0] for col in filtered_cols if re.match(regex_for_sd, col)])
    if print_sd_names:
        print(f"All subdomains in the dataframe: {sd_names}")
    sd_names = sd_names - set(["t(M)","TOfLastChange","StartTime","Version"])
    if len(sd_names) == 0:
        raise Exception(f"No subdomains found matching {regex_for_sd}")
    num_points = 0
    if print_sd_names:
        print(f"Subdomains matching {regex_for_sd}:")
    for sd in sd_names:
        points_in_current_sd = 0
        R_pts = df[f"{sd}_R"].fillna(0)
        L_pts = df[f"{sd}_L"].fillna(0)
        M_pts = df[f"{sd}_M"].fillna(0)
        points_in_current_sd = R_pts * L_pts * M_pts
        if "Sphere" in sd:
            points_in_current_sd = points_in_current_sd // 2
        if print_sd_names:
            print(f"{sd}: {points_in_current_sd} points")
        num_points += points_in_current_sd

    return x, num_points, f"NumPoints in {regex_for_sd}"


def compute_power_in_modes(x,y,df,y_axis,L_list=None):
    def get_l(key):
        return int(key.split(",")[0])

    if L_list is None:
        cols_to_use = []
        for col in df.columns:
            if col == "t(M)":
                continue
            if "diff_" in col:
                raise Exception("You have diff base enabled, this is not what you want!! Use the function compute_diff_power_in_modes")
            cols_to_use.append(col)
        power = np.linalg.norm(df[cols_to_use], axis=1)
        return x, power, "power"
    else:
        cols_to_use = []
        for col in df.columns:
            if col == "t(M)":
                continue
            if "diff_" in col:
                raise Exception("You have diff base enabled, this is not what you want!! Use the function compute_diff_power_in_modes")
            l_value = get_l(col)
            if l_value in L_list:
                cols_to_use.append(col)
        power = np.linalg.norm(df[cols_to_use], ord=2, axis=1)
        return x, power, f"power{L_list}"
    

def data_frame_diff_with_interpolation(df1,df2):
    # Will interpolate the df2 onto the time of df1 and then compute the difference
    # No interpolation is done if the time values are the same
    # Assumes that the time column is sorted and is named "t(M)"
    t1 = df1["t(M)"]
    # mint1 = t1.min()
    # maxt1 = t1.max()
    t2 = df2["t(M)"]
    if not np.array_equal(t1, t2):
        # interpolating_t_vals = t2[(t2 >= mint1) & (t2 <= maxt1)]
        diff = df1.copy()
        for col in df2.columns:
            # df1 cols will have diff_abs cols added by the main function. ignore those
            if col == "t(M)":
                continue
            diff[col] = df1[col] - sp.interpolate.CubicSpline(t2, df2[col], extrapolate=False)(t1)
        return diff
    else:
        diff = df1 - df2
        diff["t(M)"] = df1["t(M)"]
        return diff

def compute_diff_power_in_modes(x,y,df,y_axis,diff_base_df,L_list=None):
    def get_l(key):
        return int(key.split(",")[0])
    
    # def get_pandas

    if L_list is None:
        cols_to_use = []
        for col in df.columns:
            if col == "t(M)":
                continue
            if "diff_" in col:
                continue # These keys will be added, just ignore
            cols_to_use.append(col)
        ylabel = "power"
    else:
        cols_to_use = []
        for col in df.columns:
            if col == "t(M)":
                continue
            if "diff_" in col:
                continue # These keys will be added, just ignore
            l_value = get_l(col)
            if l_value in L_list:
                cols_to_use.append(col)
        ylabel = f"power{L_list}"

    power = np.linalg.norm(data_frame_diff_with_interpolation(df, diff_base_df)[cols_to_use], axis=1)
    return x, power, ylabel

def get_drift_from_center(x, y, df, y_axis):
    center = np.zeros_like(y)
    for i in [
        "COM_X",
        "COM_Y",
        "COM_Z",
    ]:
        assert i in df.columns, f"Column {i} not found in DataFrame"
        center += df[i] ** 2

    center = np.sqrt(center)
    return x, center, "COM_mag"

### add_norm_constraints

In [ ]:
def add_norm_constraints(runs_data_dict,index_num=[1,2,3] ,norm=["Linf","L2","RMS"], subdomains_seperately=False, replace_runs_data_dict=False):  
    for key, run_data in runs_data_dict.items():
        new_data = {} # Add data later in one go otherwise index filtering will pickup added stuff
        for which_index in index_num:
            if not subdomains_seperately:
                index_cols = [i for i in run_data.columns if re.search(rf"{which_index}Con.+\)", i)] # Prevent picking up new added cols on reruns
                for which_norm in norm:
                    label = f"{which_norm}({which_index}Con)"
                    if which_norm ==  "Linf": 
                        new_data[label] = np.max(np.abs(run_data[index_cols]), axis=1)
                    elif which_norm == "L2":
                        new_data[label] = np.linalg.norm(run_data[index_cols], axis=1)
                    elif which_norm == "RMS":
                        new_data[label] = np.linalg.norm(run_data[index_cols], axis=1)/ np.sqrt(len(index_cols))
                    else:
                        raise Exception(f"Invalid norm value: {norm}")
            else:
                subdomains = [col.split(' on ')[1] for col in run_data.columns if " on " in col]
                subdomains = list(set(subdomains))
                for sd in subdomains:
                    index_cols = [i for i in run_data.columns if (re.search(rf"{which_index}Con.+\)", i)) and (sd in i)] # Prevent picking up new added cols on reruns
                    for which_norm in norm:
                        label = f"{which_norm}({which_index}Con) on {sd}"
                        if which_norm ==  "Linf": 
                            new_data[label] = np.max(np.abs(run_data[index_cols]), axis=1)
                        elif which_norm == "L2":
                            new_data[label] = np.linalg.norm(run_data[index_cols], axis=1)
                        elif which_norm == "RMS":
                            new_data[label] = np.linalg.norm(run_data[index_cols], axis=1)/ np.sqrt(len(index_cols))
                        else:
                            raise Exception(f"Invalid norm value: {norm}")
                pass

        new_df = pd.DataFrame(new_data, index=run_data.index)
        if replace_runs_data_dict:
            new_df['t(M)'] = run_data['t(M)']
            runs_data_dict[key] = new_df
        else:
            runs_data_dict[key] = pd.concat([run_data, new_df], axis=1)
            runs_data_dict[key] = runs_data_dict[key].loc[:, ~runs_data_dict[key].columns.duplicated(keep='last')]
        new_indices = list(new_data.keys()) # Just of the last lev
    return new_indices, runs_data_dict

### Add number of points per subdomain

In [ ]:
def num_points_per_subdomain(runs_data_dict, replace_runs_data_dict=False):  
    for key, run_data in runs_data_dict.items():
        new_data = {} # Add data later in one go otherwise index filtering will pickup added stuff
        sd_names = set([col.split("_")[0] for col in run_data.columns])
        sd_names = sd_names - set(["t(M)","TOfLastChange","StartTime","Version"])
        for sd in sd_names:
            points_in_current_sd = 0
            R_pts = run_data[f"{sd}_R"]
            L_pts = run_data[f"{sd}_L"]
            M_pts = run_data[f"{sd}_M"]
            points_in_current_sd = R_pts * L_pts * M_pts
            if "Sphere" in sd:
                points_in_current_sd = points_in_current_sd // 2
            new_data[f"NumPoints in {sd}"] = points_in_current_sd

        new_df = pd.DataFrame(new_data, index=run_data.index)
        if replace_runs_data_dict:
            new_df['t(M)'] = run_data['t(M)']
            new_df.drop_duplicates(keep='last', inplace=True)
            runs_data_dict[key] = new_df
        else:
            runs_data_dict[key] = pd.concat([run_data, new_df], axis=1)
            runs_data_dict[key] = runs_data_dict[key].loc[:, ~runs_data_dict[key].columns.duplicated(keep='last')]
        new_indices = list(new_data.keys()) # Just of the last lev
    return new_indices, runs_data_dict

### Add power modes

In [ ]:
def change_to_power_in_L_modes(runs_data_dict, LArr=None, RMS_Power=True,debug=False):
    for key in runs_data_dict:
        

        cols = list(runs_data_dict[key].columns)
        Lmax = int(cols[-1].split(",")[0])

        power_dict = {"t(M)": runs_data_dict[key]["t(M)"]}
        for L in range(0, Lmax + 1):
            num_m_modes_added = 0
            for m in range(-L, L + 1):
                mode_key = f"{L},{m}"
                if mode_key in cols:
                    if debug:
                        print(f"{L},{m}")
                    if str(L) not in power_dict:
                        # We need to square the magnitude of the mode, it is complex
                        power_dict[str(L)] = np.abs(runs_data_dict[key][mode_key]) ** 2
                    else:
                        power_dict[str(L)] += np.abs(runs_data_dict[key][mode_key]) ** 2
                    num_m_modes_added += 1

            if num_m_modes_added > 0:
                if RMS_Power:
                    if debug:
                        print(f"Normalizing power for L={L} by ({num_m_modes_added})")
                    # In theory num_m_modes_added should be 2*L+1, there are some var where negative m modes are not present.
                    power_dict[str(L)] = np.sqrt(power_dict[str(L)]) / num_m_modes_added
                else:
                    # Total power is the sum of squares
                    power_dict[str(L)] = power_dict[str(L)]

        runs_data_dict[key] = pd.DataFrame(power_dict)




### Mismatch related functions

In [ ]:
def interpolate_df_to_target(df1, df2, time_col="t(M)", t_min=None, t_max=None):
    # 1. Determine Source vs Target based on density (length)
    df1_is_source = len(df1) > len(df2)
    source_df = df1 if df1_is_source else df2
    target_df = df2 if df1_is_source else df1

    # 2. Filter Source FIRST based on User Constraints
    #    We handle None values for t_min/t_max here
    if t_min is not None:
        source_df = source_df[source_df[time_col] >= t_min]
    if t_max is not None:
        source_df = source_df[source_df[time_col] <= t_max]

    # Check if source became empty
    if source_df.empty:
        raise ValueError(
            f"Time constraints resulted in an empty source DataFrame: t_min={t_min}, t_max={t_max}"
        )

    source_df = source_df.reset_index(drop=True)
    source_times = source_df[time_col].values

    # 3. Define Valid Range based on ACTUAL filtered source data
    #    This prevents the "Discrete Gap" bug.
    valid_min = source_times.min()
    valid_max = source_times.max()

    # 4. Filter Target to be strictly within the Source's available range
    target_df = target_df[
        (target_df[time_col] >= valid_min) & (target_df[time_col] <= valid_max)
    ].reset_index(drop=True)

    target_times = target_df[time_col].values

    # Get column names (excluding time column)
    data_cols = [col for col in source_df.columns if col != time_col]

    # Initialize result dictionary
    result = {time_col: target_times}
    print(f"Min and Max times after filtering: {target_times.min()}, {target_times.max()}")

    # Use scipy for better performance on large datasets
    for col in data_cols:
        # Create interpolator
        f = CubicSpline(source_times, source_df[col].values, extrapolate=False)

        # Interpolate
        result[col] = f(target_times)
        if np.any(np.isnan(result[col])):
            print(
                f"Warning: NaN values found in interpolated column {col}. This may be due to target times being outside the source time range."
            )
            return result[col], source_times, target_times
    # return df1 and df2 with both having same time steps now
    if df1_is_source:
        return pd.DataFrame(result), target_df
    else:
        return target_df, pd.DataFrame(result)


def waveform_diff_all_modes(df1, df2, time_col="t(M)", t_min=None, t_max=None, warnings=False):
    if not np.array_equal(df1[time_col].values, df2[time_col].values):
        # print(f"Interpolating dataframes to have same time steps. {t_max=}, {t_min=}")
        df1, df2 = interpolate_df_to_target(
            df1, df2, time_col=time_col, t_min=t_min, t_max=t_max
        )
    diff = df1 - df2
    diff[time_col] = df1[time_col]
    t = diff[time_col].values
    diff_dict = {}
    for cols in diff.columns:
        if cols == time_col:
            continue
        h1h2 = sp.integrate.simpson(np.abs(diff[cols]) ** 2, t)

        h1h1 = sp.integrate.simpson(np.abs(df1[cols]) ** 2, t)
        h2h2 = sp.integrate.simpson(np.abs(df2[cols]) ** 2, t)
        if (h1h1 - h2h2) > 1e-12:
            if warnings:
                print(
                    f"Warning {cols}: h1h1 ({h1h1}) and h2h2 ({h2h2}) differ significantly {abs(h1h1 - h2h2)}."
                )
        norm = np.sqrt(h1h1 * h2h2)
        diff_dict[cols] = (h1h2, norm)

    return diff_dict


def get_cumilative_waveform_diff(diff_dict, mode_list=None):
    total_h1h2 = 0.0
    total_norm = 0.0
    for key in diff_dict:
        if mode_list is not None and key not in mode_list:
            continue
        h1h2, norm = diff_dict[key]
        total_h1h2 += h1h2
        total_norm += norm
    total_waveform_diff = 0.5 * total_h1h2 / total_norm
    return total_waveform_diff

def getLM_from_key(key):
    L_str, M_str = key.split(",")
    return int(L_str), int(M_str)

# Plot dat files

In [ ]:
# Old runs
runs_to_plot = {}
# runs_to_plot["boost_ID_test_wrong"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_wrong/Ev/Lev3_A?/Run/"
# runs_to_plot["boost_ID_test_correct"] =  "/groups/sxs/hchaudha/spec_runs/boost_ID_test_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin1"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin1/Ev/Lev3_A?/Run/"
# runs_to_plot["corrected_coord_spin2"] =  "/groups/sxs/hchaudha/spec_runs/corrected_coord_spin2/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_0_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_99_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_0_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15"] =  "/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["L3_tol8_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol8_eq/Lev3_A?/Run/"
# runs_to_plot["L3_tol9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol9/Lev3_A?/Run/"
# runs_to_plot["L3_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10/Lev3_A?/Run/"
# runs_to_plot["L3_tol10_hi"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10_hi/Lev3_A?/Run/"
# runs_to_plot["L3_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol11/Lev3_A?/Run/"
# runs_to_plot["L3_all_100_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_tol10/Lev3_A?/Run/"
# runs_to_plot["L3_all_1000_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_all_10_tol11/Lev3_A?/Run/"
# runs_to_plot["local_100_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_100_tol5_11/Lev3_A?/Run/"
# runs_to_plot["local_10_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_10_tol5_10/Lev3_A?/Run/"
# runs_to_plot["L3_local_10_tol5_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_10_tol5_10/Lev3_A?/Run/"
# runs_to_plot["L3_local_100_tol5_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/local_100_tol5_11/Lev3_A?/Run/"


# runs_to_plot["3_DH_q1_ns_d18_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol8_eq_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol8_eq/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol9_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol9/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol10_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol10_hi_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol10_hi/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_tol11_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/tol11/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_all_100_tol10_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_tol10/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_all_1000_tol11_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/L6_all_10_tol11/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L6"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6/Ev/Lev6_A?/Run/"
# runs_to_plot["L6_1.1"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1/Lev6_A?/Run/"
# runs_to_plot["L6_1.1_dp8_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1_dp8_tol_10/Lev6_A?/Run/"
# runs_to_plot["L6_1.1_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_1.1_tol_10/Lev6_A?/Run/"
# runs_to_plot["L6_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_higher_acc/L6_tol_10/Lev6_A?/Run/"
# runs_to_plot["all_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10/Lev3_A?/Run/"
# runs_to_plot["all_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100/Lev3_A?/Run/"
# runs_to_plot["near_bhs_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10/Lev3_A?/Run/"
# runs_to_plot["near_bhs_100"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100/Lev3_A?/Run/"
# runs_to_plot["same_obs"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/same_obs/Lev3_A?/Run/"
# runs_to_plot["all_10_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10_obs/Lev3_A?/Run/"
# runs_to_plot["all_100_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_obs/Lev3_A?/Run/"
# runs_to_plot["all_10_obs_tol_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_10_obs_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_obs_tol_10"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_obs_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_1.1_b0"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_1.1_b0/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_1.1_b0_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_1.1_b0_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1_dp8_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1_dp8_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_1.1_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_1.1_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_2"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_2/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_2"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_2/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_3"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_3/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8_tol10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dp8_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dp8_tol11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.02"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.02/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.03"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.03/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.04"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.04/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.025"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.025/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.021"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.021/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.022"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.022/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.023"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.023/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.024"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.024/Lev3_A?/Run/"
runs_to_plot["all_100_t2690_obs_grid_dt_0.0225"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt_0.0225/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_dt005"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_dt0.005/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_1.128e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_1.128e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_1.692e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_1.692e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_tol_3.383e-11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_tol_3.383e-11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690/Lev3_A?/Run/"
# runs_to_plot["near_bhs_10_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_10_obs/Lev3_A?/Run/"
# runs_to_plot["near_bhs_100_obs"] =  "/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100_obs/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_ns_d18_L6_AA"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L6_AA/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SSphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SSphKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["4_SphKS_q1_15_SKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/4_SphKS_q1_15_SKS_ID/Ev/Lev3_A?/Run/"
# runs_to_plot["5_gd_SphKS_gauge_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_gd_SphKS_gauge_ID/Ev/Lev2_A[A-S]/Run/"
# runs_to_plot["5_ngd_SphKS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_SphKS_ID/Ev/Lev2_A?/Run/"
# runs_to_plot["5_ngd_KS_ID"] =  "/groups/sxs/hchaudha/spec_runs/5_ngd_KS_ID/Ev/Lev2_A?/Run/"

# runs_to_plot["all_100_t2690_eteq_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_12"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_12/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eteq_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_eq/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_12"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_12/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_eth_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eth_tol_eq/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_11/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_8/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_9/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_etl_tol_eq"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_eq/Lev3_A?/Run/"

# runs_to_plot["all_100_t2690_obs_grid_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_linf/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_10_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_10_linf/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_11_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_11_linf/Lev3_A?/Run/"
# runs_to_plot["all_100_t2690_obs_grid_tol_9_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_9_linf/Lev3_A?/Run/"

# runs_to_plot["t6115_tol11"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol11/Lev3_A?/Run/"
# runs_to_plot["t6115_tol10"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol10/Lev3_A?/Run/"
# runs_to_plot["t6115_tol9"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol9/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8/Lev3_A?/Run/"
# runs_to_plot["t6115_tol7"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol7/Lev3_A?/Run/"
# runs_to_plot["t6115_tol11_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol11_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol10_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol10_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol9_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol9_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol7_AMR"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol7_AMR/Lev3_A?/Run/"
# runs_to_plot["t6115_tol8_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_linf/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.02"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.02/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.03"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.03/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.041"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.041/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.042"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.042/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.043"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.043/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.044"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.044/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.045"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.045/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.046"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.046/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.047"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.047/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.048"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.048/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.049"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.049/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.050"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.050/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.052"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.052/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.054"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.054/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_dt0.056"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_dt0.056/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_2.368e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_2.368e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.015e-07"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.015e-07/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_6.767e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_6.767e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_5.075e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_5.075e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_3.383e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_3.383e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_2.256e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_2.256e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.128e-08"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.128e-08/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_6.767e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_6.767e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_4.833e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_4.833e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_3.383e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_3.383e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.692e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.692e-09/Lev3_A?/Run/"
# runs_to_plot["t6115_linf_tol_1.128e-09"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_linf_tol_1.128e-09/Lev3_A?/Run/"


# runs_to_plot["all_100_t2710_0.021_0.021"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.21/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.022"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.22/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.0225"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.225/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.023"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.23/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_0.024"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_0.24/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol8"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol8/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol9/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol10/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol10.5"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol10.5/Lev3_AE/Run/"
# runs_to_plot["all_100_t2710_0.021_max_tol11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/all_100_t2710_0.021_max_tol11/Lev3_AE/Run/"

# runs_to_plot["eq_t4000_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol10/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol5_10/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol5_11/Lev3_A?/Run/"
# runs_to_plot["eq_t4000_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/eq_t4000_tol9/Lev3_A?/Run/"
# runs_to_plot["t4000_tol10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol10/Lev3_A?/Run/"
# runs_to_plot["t4000_tol5_10"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol5_10/Lev3_A?/Run/"
# runs_to_plot["t4000_tol5_11"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol5_11/Lev3_A?/Run/"
# runs_to_plot["t4000_tol8"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol8/Lev3_A?/Run/"
# runs_to_plot["t4000_tol9"] = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_all_10/t4000_tol9/Lev3_A?/Run/"

# runs_to_plot["Lev3_AA_tol10_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol10_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol11_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol11_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol12_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol12_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_10_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_10_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_11_all_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_11_all_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol11"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol11/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol12"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol12/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_10"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_10/Lev3_A?_/Run/"
# runs_to_plot["Lev3_AA_tol5_11"]  = "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_ringdown_tol/rd_all_10_tol11/Lev3_AA_tol5_11/Lev3_A?_/Run/"



In [ ]:
runs_to_plot = {}

# runs_to_plot["ode_change_Run1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev1/Run/"
# runs_to_plot["ode_change_Run2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev2/Run/"
# runs_to_plot["ode_change_Run3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev3/Run/"
# runs_to_plot["ode_change_Run4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev4/Run/"
# runs_to_plot["ode_change_Run5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev5/Run/"
# runs_to_plot["ode_change_Run6"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev6/Run/"
# runs_to_plot["ode_change_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev7/Run/"
# runs_to_plot["ode_change_Run8"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev8/Run/"

# runs_to_plot["1000M_Run1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev1/Run/"
# runs_to_plot["1000M_Run2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev2/Run/"
# runs_to_plot["1000M_Run3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev3/Run/"
# runs_to_plot["1000M_Run4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev4/Run/"
# runs_to_plot["1000M_Run5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev5/Run/"
# runs_to_plot["1000M_Run6"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev6/Run/"
# runs_to_plot["1000M_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev7/Run/"
# runs_to_plot["1000M_Run7_a"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev7_a/Run/"

# runs_to_plot["Lin_Run1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev1/Run/"
# runs_to_plot["Lin_Run2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev2/Run/"
# runs_to_plot["Lin_Run3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev3/Run/"
# runs_to_plot["Lin_Run4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev4/Run/"
# runs_to_plot["Lin_Run5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev5/Run/"
# runs_to_plot["Lin_Run6"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev6/Run/"
# runs_to_plot["Lin_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev7/Run/"

# runs_to_plot["400M_base1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev1/Run/"
# runs_to_plot["400M_base2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev2/Run/"
# runs_to_plot["400M_base3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev3/Run/"
# runs_to_plot["400M_base4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev4/Run/"
# runs_to_plot["400M_base5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev5/Run/"
# runs_to_plot["Lin_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev7/Run/"

# runs_to_plot["400M_phys_bc1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev1/Run/"
# runs_to_plot["400M_phys_bc2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev2/Run/"
# runs_to_plot["400M_phys_bc3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev3/Run/"
# runs_to_plot["400M_phys_bc4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev4/Run/"
# runs_to_plot["400M_phys_bc5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev5/Run/"

# runs_to_plot["400M_phys_bc5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev5/Run/"

# runs_to_plot["13_Lev4_250"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_250/Run/"
# runs_to_plot["13_Lev4_300"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_300/Run/"
# runs_to_plot["13_Lev4_350"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_350/Run/"
# runs_to_plot["13_Lev4_400"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_400/Run/"
# runs_to_plot["13_Lev4_450"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_450/Run/"
# runs_to_plot["13_Lev4_500"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_500/Run/"
# runs_to_plot["13_Lev4_550"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_550/Run/"
# runs_to_plot["13_Lev4_600"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_600/Run/"
# runs_to_plot["13_Lev4_650"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_650/Run/"
# runs_to_plot["13_Lev4_700"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_700/Run/"
# runs_to_plot["13_Lev4_750"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_750/Run/"

# runs_to_plot["14_Lev4_250"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_250/Run/"
# runs_to_plot["14_Lev4_300"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_300/Run/"
# runs_to_plot["14_Lev4_350"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_350/Run/"
# runs_to_plot["14_Lev4_400"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_400/Run/"
# runs_to_plot["14_Lev4_450"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_450/Run/"
# runs_to_plot["14_Lev4_500"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_500/Run/"
# runs_to_plot["14_Lev4_550"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_550/Run/"
# runs_to_plot["14_Lev4_600"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_600/Run/"
# runs_to_plot["14_Lev4_650"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_650/Run/"
# runs_to_plot["14_Lev4_700"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_700/Run/"
# runs_to_plot["14_Lev4_750"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_750/Run/"

# runs_to_plot["15_AMR_Lev0_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev0_255/Run/"
# runs_to_plot["15_AMR_Lev0_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev0_355/Run/"
# runs_to_plot["15_AMR_Lev0_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev0_455/Run/"
# runs_to_plot["15_AMR_Lev1_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev1_255/Run/"
# runs_to_plot["15_AMR_Lev1_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev1_355/Run/"
# runs_to_plot["15_AMR_Lev1_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev1_455/Run/"
# runs_to_plot["15_AMR_Lev2_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev2_255/Run/"
# runs_to_plot["15_AMR_Lev2_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev2_355/Run/"
# runs_to_plot["15_AMR_Lev2_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev2_455/Run/"
# runs_to_plot["15_AMR_Lev3_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev3_255/Run/"
# runs_to_plot["15_AMR_Lev3_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev3_355/Run/"
# runs_to_plot["15_AMR_Lev3_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev3_455/Run/"
# runs_to_plot["15_AMR_Lev4_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev4_255/Run/"
# runs_to_plot["15_AMR_Lev4_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev4_355/Run/"
# runs_to_plot["15_AMR_Lev4_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev4_455/Run/"
# runs_to_plot["15_AMR_Lev5_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev5_255/Run/"
# runs_to_plot["15_AMR_Lev5_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev5_355/Run/"
# runs_to_plot["15_AMR_Lev5_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev5_455/Run/"

# runs_to_plot["15_Lev5_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255/Run/"
# runs_to_plot["15_Lev5_255_010"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_010/Run/"
# runs_to_plot["15_Lev5_255_0100"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_0100/Run/"
# runs_to_plot["15_Lev5_255_01000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_01000/Run/"
# runs_to_plot["15_Lev5_255_010000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_010000/Run/"
# runs_to_plot["15_Lev5_255_05"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_05/Run/"
# runs_to_plot["15_Lev5_255_050"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_050/Run/"
# runs_to_plot["15_Lev5_255_0500"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_0500/Run/"
# runs_to_plot["15_Lev5_255_05000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_05000/Run/"
# runs_to_plot["15_Lev5_255_050000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_050000/Run/"

# runs_to_plot["400M_gamma1_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/5_400M_gamma1/Lev2/Run/"
# runs_to_plot["400M_gamma1_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/5_400M_gamma1/Lev3/Run/"
# runs_to_plot["400M_gamma1_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/5_400M_gamma1/Lev4/Run/"

# runs_to_plot["400M_gamma1_001_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/6_400M_gamma1_001/Lev2/Run/"
# runs_to_plot["400M_gamma1_001_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/6_400M_gamma1_001/Lev3/Run/"
# runs_to_plot["400M_gamma1_001_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/6_400M_gamma1_001/Lev4/Run/"

# runs_to_plot["400M_gamma1_01_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/7_400M_gamma1_01/Lev2/Run/"
# runs_to_plot["400M_gamma1_01_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/7_400M_gamma1_01/Lev3/Run/"
# runs_to_plot["400M_gamma1_01_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/7_400M_gamma1_01/Lev4/Run/"

# runs_to_plot["400M_BDres_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/8_400M_BDres/Lev2/Run/"
# runs_to_plot["400M_BDres_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/8_400M_BDres/Lev3/Run/"
# runs_to_plot["400M_BDres_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/8_400M_BDres/Lev4/Run/"

# runs_to_plot["9_400M_BDres_05_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/9_400M_BDres_05/Lev2/Run/"
# runs_to_plot["9_400M_BDres_05_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/9_400M_BDres_05/Lev3/Run/"
# runs_to_plot["9_400M_BDres_05_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/9_400M_BDres_05/Lev4/Run/"

# runs_to_plot["10_freezing"] = "/groups/sxs/hchaudha/spec_runs/single_bh/10_freezing/Lev4/Run/"
# runs_to_plot["11_physical_bc"] = "/groups/sxs/hchaudha/spec_runs/single_bh/11_physical_bc/Lev4/Run/"

# runs_to_plot["11_physical_bc"] = "/groups/sxs/hchaudha/spec_runs/single_bh/11_physical_bc/Lev4/Run/"

runs_to_plot["Lev3_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev3_265/Run/"
runs_to_plot["Lev4_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev4_265/Run/"
runs_to_plot["Lev5_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265/Run/"
# runs_to_plot["Lev6_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev6_265/Run/"
# runs_to_plot["Lev5_265_0.01_0.01_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.01_0.01_0.01/Run/"
# runs_to_plot["Lev5_265_0.05_0.05_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.05_0.05_0.01/Run/"
# runs_to_plot["Lev5_265_0.1_0.1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.1_0.1_0.001/Run/"
# runs_to_plot["Lev5_265_0.1_0.1_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.1_0.1_0.01/Run/"
# runs_to_plot["Lev5_265_1_1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.001/Run/"
# runs_to_plot["Lev5_265_1_1_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.01/Run/"
# runs_to_plot["Lev5_265_3_3_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_3_3_0.001/Run/"
# runs_to_plot["Lev5_265_3_3_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_3_3_0.01/Run/"
# runs_to_plot["Lev5_265_5_5_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_5_5_0.001/Run/"
# runs_to_plot["Lev5_265_5_5_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_5_5_0.01/Run/"
# runs_to_plot["Lev5_265_10_10_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.001/Run/"
# runs_to_plot["Lev5_265_10_10_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.01/Run/"
# runs_to_plot["Lev5_265_25_25_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_25_25_0.01/Run/"
# runs_to_plot["Lev5_265_50_50_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_50_50_0.01/Run/"
# runs_to_plot["Lev5_265_100_100_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_100_100_0.01/Run/"

# runs_to_plot["Lev5_265_1_1_1e-05"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_1e-05/Run/"
# runs_to_plot["Lev5_265_1_1_0.0001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.0001/Run/"
runs_to_plot["Lev5_265_1_1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.001/Run/"
# runs_to_plot["Lev5_265_1_1_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.01/Run/"
# runs_to_plot["Lev5_265_1_1_0.1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.1/Run/"
# runs_to_plot["Lev5_265_1_1_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_1/Run/"
# runs_to_plot["Lev5_265_10_10_1e-05"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_1e-05/Run/"
# runs_to_plot["Lev5_265_10_10_0.0001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.0001/Run/"
runs_to_plot["Lev5_265_10_10_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.001/Run/"
# runs_to_plot["Lev5_265_10_10_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.01/Run/"
# runs_to_plot["Lev5_265_10_10_0.1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.1/Run/"
# runs_to_plot["Lev5_265_10_10_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_1/Run/"

# data_file_path = "FailedTStepperDiag.dat"
# data_file_path = "GhCe_Norms.dat"
# data_file_path = "GhCe.dat"
# data_file_path = "NormalizedGhCe.dat"
# data_file_path = "GhCe_Linf.dat"
# data_file_path = "GhCe.dat"
# data_file_path = "NormalizedGhCe_Linf.dat"
# data_file_path = "1Con.dat"
# data_file_path = "2Con.dat"
data_file_path = "3Con.dat"
# data_file_path = "kappaErr_Linf.dat"
# data_file_path = "psiErr_Linf.dat"
# data_file_path = "kappaErr.dat"
# data_file_path = "psiErr.dat"
# data_file_path = "psi.dat"
# data_file_path = "kappa.dat"
# data_file_path = "Shift.dat"
# data_file_path = "TStepperDiag.dat"

column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)

In [ ]:
runs_to_plot = {}

# runs_to_plot["73_gd_master_new_code"] =  "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/73_gd_master_new_code/Ev/Lev1_A?/Run/"
# runs_to_plot["119_gd_SUKS_3_20"] =  "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["120W_gd_SUKS1_3_20"] =  "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/120W_gd_SUKS1_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["AccTest_q1ns_Lev9"] =  "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/AccTest_q1ns_Lev9/Ev/Lev9_A?/Run/"
# runs_to_plot["77_gd_Kerr_q3"] =  "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/77_gd_Kerr_q3/Ev_Kerr/Lev1_A?/Run/"
# runs_to_plot["77_gd_Kerr_q3"] =  "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/120W_gd_SUKS1_3_20/Ev/Lev3_A?/Run/"


# data_file_path = "ConstraintNorms/GhCe.dat"
# data_file_path = "ConstraintNorms/GhCeExt.dat"
# data_file_path = "ConstraintNorms/GhCeExt_L2.dat"
# data_file_path = "ConstraintNorms/GhCeExt_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_L2.dat"
data_file_path = "ConstraintNorms/GhCe_Linf.dat"
# data_file_path = "ConstraintNorms/Linf.dat"
# data_file_path = "ConstraintNorms/Constraints_Linf.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path = "ConstraintNorms/GhCe_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_VolL2.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Norms.dat"
# data_file_path = "CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0.dat"
# data_file_path = "MinimumGridSpacing.dat"
# data_file_path = "GrAdjustMaxTstepToDampingTimes.dat"
# data_file_path = "GrAdjustSubChunksToDampingTimes.dat"
# data_file_path = "DiagAhSpeedA.dat"
# data_file_path = "ApparentHorizons/AhA.dat"
# data_file_path = "ApparentHorizons/AhB.dat" 
# data_file_path = "ApparentHorizons/MinCharSpeedAhA.dat"
# data_file_path = "ApparentHorizons/RescaledRadAhA.dat"
# data_file_path = "ApparentHorizons/AhACoefs.dat"
# data_file_path = "ApparentHorizons/AhBCoefs.dat"
# data_file_path = "ApparentHorizons/Trajectory_AhB.dat"
# data_file_path = "ApparentHorizons/HorizonSepMeasures.dat"

# data_file_path = "ApparentHorizons/Horizons.h5@AhA"
# data_file_path = "TStepperDiag.dat"
# data_file_path = "TimeInfo.dat"
# data_file_path = "Hist-FuncSkewAngle.txt"
# data_file_path = "Hist-FuncCutX.txt"
# data_file_path = "Hist-FuncExpansionFactor.txt"
# data_file_path = "Hist-FuncLambdaFactorA0.txt"
# data_file_path = "Hist-FuncLambdaFactorA.txt"
# data_file_path = "Hist-FuncLambdaFactorB0.txt"
# data_file_path = "Hist-FuncLambdaFactorB.txt"
# data_file_path = "Hist-FuncQuatRotMatrix.txt"
# data_file_path = "Hist-FuncSkewAngle.txt"
# data_file_path = "Hist-FuncSmoothCoordSep.txt"
# data_file_path = "Hist-FuncSmoothMinDeltaRNoLam00AhA.txt"
# data_file_path = "Hist-FuncSmoothMinDeltaRNoLam00AhB.txt"
# data_file_path = "Hist-FuncSmoothRAhA.txt"
# data_file_path = "Hist-FuncSmoothRAhB.txt"
# data_file_path = "Hist-FuncTrans.txt"
# data_file_path = "Hist-GrDomain.txt"
# data_file_path = "Profiler.h5"
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)

### Interpolated to points runs

In [ ]:
runs_to_plot = {}

# runs_to_plot["44_Set1L6s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs/Ev/Lev1_A?/Run/"
# runs_to_plot["44_Set1L6s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs/Ev/Lev2_A?/Run/"
# runs_to_plot["44_Set1L6s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs/Ev/Lev3_A?/Run/"
# runs_to_plot["44_Set1L6s4"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs/Ev/Lev4_A?/Run/"
# runs_to_plot["44_Set1L6s5"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs/Ev/Lev5_A?/Run/"
# runs_to_plot["44_Set1L6s6"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs/Ev/Lev6_A?/Run/"

# runs_to_plot["44_Set1L6s1_NAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRReset/Ev/Lev1_A?/Run/"
# runs_to_plot["44_Set1L6s2_NAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRReset/Ev/Lev2_A?/Run/"
# runs_to_plot["44_Set1L6s3_NAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRReset/Ev/Lev3_A?/Run/"
# runs_to_plot["44_Set1L6s4_NAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRReset/Ev/Lev4_A?/Run/"
# runs_to_plot["44_Set1L6s5_NAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRReset/Ev/Lev5_A?/Run/"
# runs_to_plot["44_Set1L6s6_NAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRReset/Ev/Lev6_A?/Run/"

# runs_to_plot["44_Set1L6s1_NAMRD"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRResetDense/Ev/Lev1_A?/Run/"
# runs_to_plot["44_Set1L6s2_NAMRD"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRResetDense/Ev/Lev2_A?/Run/"
# runs_to_plot["44_Set1L6s3_NAMRD"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRResetDense/Ev/Lev3_A?/Run/"
# runs_to_plot["44_Set1L6s4_NAMRD"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRResetDense/Ev/Lev4_A?/Run/"
# runs_to_plot["44_Set1L6s5_NAMRD"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRResetDense/Ev/Lev5_A?/Run/"
# runs_to_plot["44_Set1L6s6_NAMRD"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsNoAMRResetDense/Ev/Lev6_A?/Run/"

# runs_to_plot["44_Set1L6s1_D"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsL6ToLow/Ev/Lev1_A?/Run/"
# runs_to_plot["44_Set1L6s2_D"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsL6ToLow/Ev/Lev2_A?/Run/"
# runs_to_plot["44_Set1L6s3_D"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsL6ToLow/Ev/Lev3_A?/Run/"
# runs_to_plot["44_Set1L6s4_D"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsL6ToLow/Ev/Lev4_A?/Run/"
# runs_to_plot["44_Set1L6s5_D"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsL6ToLow/Ev/Lev5_A?/Run/"
# runs_to_plot["44_Set1L6s6_D"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObsL6ToLow/Ev/Lev6_A?/Run/"

# runs_to_plot["44_LineObs_10M_EachStep_Lev1_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs_10M_EachStep/Ev/Lev1_AE/Run/"
# runs_to_plot["44_LineObs_10M_EachStep_Lev1_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs_10M_EachStep/Ev/Lev2_AE/Run/"
# runs_to_plot["44_LineObs_10M_EachStep_Lev1_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs_10M_EachStep/Ev/Lev3_AE/Run/"
# runs_to_plot["44_LineObs_10M_EachStep_Lev1_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs_10M_EachStep/Ev/Lev4_AF/Run/"
# runs_to_plot["44_LineObs_10M_EachStep_Lev1_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs_10M_EachStep/Ev/Lev5_AF/Run/"
# runs_to_plot["44_LineObs_10M_EachStep_Lev1_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/44_set1_fvars_obs/LineObs_10M_EachStep/Ev/Lev6_AG/Run/"

ringdown_only = True
ringdown_only = False
if ringdown_only:
    for key in list(runs_to_plot.keys()):
        runs_to_plot[key] = runs_to_plot[key].replace("/Ev/","/Ev/Lev?_Ringdown/")


# data_file_path = "ConstraintNorms/Int_kappa_ttt.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ttx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_tty.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ttz.dat"
data_file_path = "ConstraintNorms/Int_kappa_txx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_txy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_txz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_tyy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_tyz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_tzz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xtt.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xtx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xty.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xtz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xxx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xxy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xxz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xyy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xyz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_xzz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ytt.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ytx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yty.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ytz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yxx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yxy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yxz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yyy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yyz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_yzz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ztt.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ztx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zty.dat"
# data_file_path = "ConstraintNorms/Int_kappa_ztz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zxx.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zxy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zxz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zyy.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zyz.dat"
# data_file_path = "ConstraintNorms/Int_kappa_zzz.dat"
# data_file_path = "ConstraintNorms/Int_psi_tt.dat"
# data_file_path = "ConstraintNorms/Int_psi_tx.dat"
# data_file_path = "ConstraintNorms/Int_psi_ty.dat"
# data_file_path = "ConstraintNorms/Int_psi_tz.dat"
# data_file_path = "ConstraintNorms/Int_psi_xx.dat"
# data_file_path = "ConstraintNorms/Int_psi_xy.dat"
# data_file_path = "ConstraintNorms/Int_psi_xz.dat"
# data_file_path = "ConstraintNorms/Int_psi_yy.dat"
# data_file_path = "ConstraintNorms/Int_psi_yz.dat"
# data_file_path = "ConstraintNorms/Int_psi_zz.dat"

# data_file_path = "ConstraintNorms/Int_kappa_zzz.dat@ALL"
# data_file_path = "ConstraintNorms/Int_psi_zz.dat@ALL"

runs_data_dict = {}
column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)

# load both psi and kappa

print(column_names)
print(runs_data_dict.keys())

### last data load

In [ ]:
runs_to_plot = {}

# runs_to_plot["high_accuracy_L3_tol8_wrong"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev3_tol8/Run/"
# runs_to_plot["high_accuracy_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_Ringdown/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_Ringdown/Lev5_A?/Run/"

# runs_to_plot["high_accuracy_L0"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev0_A?/Run/"
# runs_to_plot["high_accuracy_L1"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev1_A?/Run/"
# runs_to_plot["high_accuracy_L2"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev2_A?/Run/"
# runs_to_plot["high_accuracy_L3"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L45"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L55"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev55_A?/Run/"
# runs_to_plot["high_accuracy_L6"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev6_A?/Run/"

# runs_to_plot["high_accuracy_L45n"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/new_L45_L55/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L55n"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/new_L45_L55/Ev/Lev55_A?/Run/"

# runs_to_plot["high_accuracy_L4n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L45n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L5n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L55n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev55_A?/Run/"

# runs_to_plot["high_accuracy_L5_three_tier"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L5_three_tier_constra"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L5_three_tier_constra200"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra_200/Lev5_A?/Run/"
# runs_to_plot["L3_step_bound_gauss_error"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_A?/Run/"
# runs_to_plot["L3_step_bound_gauss_error_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_Ringdown/Lev3_A?/Run/"

# runs_to_plot["Lev5_big_gaussian_ah_tol10"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol10/Lev5_A?/Run/"
# runs_to_plot["Lev5_big_gaussian_ah_tol100"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol100/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_cd_01_uamr"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_lapse"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_full/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_lapse_uamr"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/Lev5_A?/Run/"

# runs_to_plot["high_accuracy_L0_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev0_A?/Run/"
# runs_to_plot["high_accuracy_L1_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev1_A?/Run/"
# runs_to_plot["high_accuracy_L2_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev2_A?/Run/"
# runs_to_plot["high_accuracy_L3_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L45_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L55_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev55_A?/Run/"
# runs_to_plot["high_accuracy_L6_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev6_A?/Run/"

# runs_to_plot["ode_impro_Lev0"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev0_A?/Run/'
# runs_to_plot["ode_impro_Lev1"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev1_A?/Run/'
# runs_to_plot["ode_impro_Lev2"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev2_A?/Run/'
# runs_to_plot["ode_impro_Lev3"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev3_A?/Run/'
# runs_to_plot["ode_impro_Lev4"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev4_A?/Run/'
# runs_to_plot["ode_impro_Lev5"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev5_A?/Run/'
# runs_to_plot["main_Lev0"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev0_A?/Run/'
# runs_to_plot["main_Lev2"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev2_A?/Run/'
# runs_to_plot["main_Lev1"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev1_A?/Run/'

# runs_to_plot["ode_impro_Lev0_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev0_Ringdown/Lev0_A?/Run/'
# runs_to_plot["ode_impro_Lev2_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev2_Ringdown/Lev2_A?/Run/'
# runs_to_plot["ode_impro_Lev1_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev1_Ringdown/Lev1_A?/Run/'
# runs_to_plot["main_Lev0_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev0_Ringdown/Lev0_A?/Run/'
# runs_to_plot["main_Lev2_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev2_Ringdown/Lev2_A?/Run/'
# runs_to_plot["main_Lev1_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev1_Ringdown/Lev1_A?/Run/'

# runs_to_plot["6_set1_L3s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set1_L3s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set1_L3s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set1_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set1_L3s3_fil6"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_6/Ev/Lev3_A[B-]/Run/"
# runs_to_plot["6_set1_L3s3_fil8"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_8/Ev/Lev3_A[B-]/Run/"
# runs_to_plot["6_set1_L3s3_fil10"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_10/Ev/Lev3_A[B-]/Run/"

# runs_to_plot["6_set1_L3_template_all"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_template_all/Ev/Lev3_A[B-]/Run/"
# runs_to_plot["6_set1_L3_template_1_29"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_template_1_29/Ev/Lev3_A[B-]/Run/"

# runs_to_plot["6_set1_L3s3_5517_CCopy"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_CCopy/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set2_L3s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L3/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set2_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["24_allcd_gaussEx_10_800"] =  "/groups/sxs/hchaudha/spec_runs/24_allcd_gaussEx_10_800/Ev/Lev3_A?/Run/"
# runs_to_plot["24_allcd_gaussEx_5_800"] =  "/groups/sxs/hchaudha/spec_runs/24_allcd_gaussEx_5_800/Ev/Lev3_A?/Run/"
# runs_to_plot["25_allcd_gaussEx_2_60"] =  "/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_2_60/Ev/Lev3_A?/Run/"
# runs_to_plot["25_allcd_gaussEx_2_800"] =  "/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_2_800/Ev/Lev3_A?/Run/"
# runs_to_plot["25_allcd_gaussEx_5_60"] =  "/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_5_60/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set3_L3s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set3_L3s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set3_L3s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set3_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set1_L6s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set1_L6s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set1_L6s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set1_L6s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set1_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set1_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["set1_L6s4_cd10"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd10/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd200"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd200/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd500"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd500/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100_AMRL6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100_AMRL6/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100_AMRL7"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100_AMRL7/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100_AMRL8"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100_AMRL8/Ev/Lev4_A?/Run/"

# runs_to_plot["6_set2_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set2_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set2_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set3_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set3_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set3_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set1_L6s3_CAMR"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_CAMR/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s3_min_L"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_min_L/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s3_min_LR"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_min_LR/Ev/Lev3_A?/Run/"

# runs_to_plot["7_constAMR_set1_L6_base_0"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev0_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_1"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev1_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_2"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev2_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_3"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev3_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_4"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev4_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_5"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev5_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_6"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev6_A?/Run/"

# runs_to_plot["8_constAMR_set1_L6_base_0"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev0_A?/Run/"
# runs_to_plot["8_constAMR_set1_L6_base_1"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev1_A?/Run/"
# runs_to_plot["8_constAMR_set1_L6_base_2"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev2_A?/Run/"
# runs_to_plot["8_constAMR_set1_L6_base_3"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev3_A?/Run/"

# runs_to_plot["9_set1_L3s3_01"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_01/Ev/Lev3_A?/Run/"
# runs_to_plot["9_set1_L3s3_001"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_001/Ev/Lev3_A?/Run/"
# runs_to_plot["9_set1_L3s3_10"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_10/Ev/Lev3_A?/Run/"
# runs_to_plot["9_set1_L3s3_100"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_100/Ev/Lev3_A?/Run/"

# runs_to_plot["10_4000M_CAMR_set1_L6_base0"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev0_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base1"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev1_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base2"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev2_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base3"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev3_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base4"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev4_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base5"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev5_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base6"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev6_A?/Run/"

# runs_to_plot["11_4000M_CAMR_set1_L6_base0"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev0_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base1"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev1_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base2"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev2_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base3"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev3_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base4"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev4_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base5"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev5_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base6"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev6_A?/Run/"

# runs_to_plot["12_set1_L3_1500"] =  "/groups/sxs/hchaudha/spec_runs/12_set1_L3_1500/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2000"] =  "/groups/sxs/hchaudha/spec_runs/12_set1_L3_2000/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2500"] =  "/groups/sxs/hchaudha/spec_runs/12_set1_L3_2500/Ev/Lev3_A?/Run/"

# runs_to_plot["13_set1_L3_3000"] =  "/groups/sxs/hchaudha/spec_runs/13_set1_L3_3000/Ev/Lev3_A?/Run/"
# runs_to_plot["13_set1_L4_1500"] =  "/groups/sxs/hchaudha/spec_runs/13_set1_L4_1500/Ev/Lev4_A?/Run/"
# runs_to_plot["13_set1_L4_3000"] =  "/groups/sxs/hchaudha/spec_runs/13_set1_L4_3000/Ev/Lev4_A?/Run/"

# runs_to_plot["14_set1_L4_1500_cd5"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd5/Ev/Lev4_A?/Run/"
# runs_to_plot["14_set1_L4_1500_cd10"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd10/Ev/Lev4_A?/Run/"
# runs_to_plot["14_set1_L4_1500_cd25"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd25/Ev/Lev4_A?/Run/"
# runs_to_plot["14_set1_L4_1500_cd50"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd50/Ev/Lev4_A?/Run/"

# runs_to_plot["15_set1_L4_1500_JY"] =  "/groups/sxs/hchaudha/spec_runs/15_set1_L4_1500_JY/Ev/Lev4_A?/Run/"

# runs_to_plot["16_set1_L3"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["16_set1_L3_HP32"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP32/Ev/Lev3_A?/Run/"
# runs_to_plot["16_set1_L3_HP28"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP28/Ev/Lev3_A?/Run/"
# runs_to_plot["16_set1_L3_HP32_AF"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP32_AF/Ev/Lev3_A?/Run/"
# runs_to_plot["17_BDI_32_SAE_NONE"] = "/groups/sxs/hchaudha/spec_runs/17_BDI_32_SAE_NONE/Ev/Lev4_A?/Run/"
# runs_to_plot["17_BDI_32_SAE_32"] = "/groups/sxs/hchaudha/spec_runs/17_BDI_32_SAE_32/Ev/Lev4_A?/Run/"
# runs_to_plot["17_BDI_32_SAE_32_AF"] = "/groups/sxs/hchaudha/spec_runs/17_BDI_32_SAE_32_AF/Ev/Lev4_A?/Run/"

# runs_to_plot["17_set_main_q3_15_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_q3_15_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set_main_99_15_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_99_15_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["17_set_main_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set3_q3_18_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["17_main_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_set3_9_18_L3_correct/Ev/Lev3_A?/Run/"

# runs_to_plot["17_set_main_99_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_99_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_99_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_99_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_99_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set3_99_18_L3/Ev/Lev3_??/Run/"

# runs_to_plot["17_main_9_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_9_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_9_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set3_9_18_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["18_set1_L3_junk_resolved"] = "/groups/sxs/hchaudha/spec_runs/18_set1_L3_junk_resolved/Ev/Lev3_??/Run/"
# runs_to_plot["20_set1_L3_fine_cylinders"] = "/groups/sxs/hchaudha/spec_runs/20_set1_L3_fine_cylinders/Ev/Lev3_??/Run/"
# runs_to_plot["21_set1_L3_fine_cylinders_minExtent"] = "/groups/sxs/hchaudha/spec_runs/21_set1_L3_fine_cylinders_minExtent/Ev/Lev3_??/Run/"

# runs_to_plot["22_set1_L1_long"] = "/groups/sxs/hchaudha/spec_runs/22_set1_L1_long/Ev/Lev1_??/Run/"
# runs_to_plot["L1_AC_L3"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L3/Ev/Lev3_??/Run/"
# runs_to_plot["L1_AC_L2"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L2/Ev/Lev2_??/Run/"
# runs_to_plot["L1_AC_L1"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L1/Ev/Lev1_??/Run/"

# runs_to_plot["L3_AC_L3_cd_const_high"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_high/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_cd_const_low"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_low/Ev/Lev3_??/Run/"

# runs_to_plot["22_L3_AC_L3_no_res_C"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_no_res_C/Ev/Lev3_??/Run/"
# runs_to_plot["22_L3_AC_L3_res_10_C"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_res_10_C/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L1"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L1/Ev/Lev1_??/Run/"
# runs_to_plot["L3_AC_L2"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L2/Ev/Lev2_??/Run/"
# runs_to_plot["22_set1_L3_long"] = "/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L4"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L4/Ev/Lev4_??/Run/"
# runs_to_plot["L3_AC_L5"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L5/Ev/Lev5_??/Run/"
# runs_to_plot["L3_AC_L6"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L6/Ev/Lev6_??/Run/"
# runs_to_plot["L3_AC_L7"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L7/Ev/Lev7_??/Run/"
# runs_to_plot["L3_AC_L8"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L8/Ev/Lev8_??/Run/"

# runs_to_plot["43_f128_L3_AC_L6"] = "/resnick/groups/sxs/hchaudha/spec_runs/43_128_L68_test/L3_AC_L6/Ev/Lev6_??/Run/"
# runs_to_plot["43_f128_L3_AC_L7"] = "/resnick/groups/sxs/hchaudha/spec_runs/43_128_L68_test/L3_AC_L7/Ev/Lev7_??/Run/"
# runs_to_plot["43_f128_L3_AC_L8"] = "/resnick/groups/sxs/hchaudha/spec_runs/43_128_L68_test/L3_AC_L8/Ev/Lev8_??/Run/"

# runs_to_plot["L3_AC_L3_3_01"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_3_01/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_3_02"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_3_02/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_5_04"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_5_04/Ev/Lev3_??/Run/"

# runs_to_plot["26_set1_L6_long"] = "/groups/sxs/hchaudha/spec_runs/26_set1_L6_long/Ev/Lev6_??/Run/"
# runs_to_plot["26_main_L6_long"] = "/groups/sxs/hchaudha/spec_runs/26_main_L6_long/Ev/Lev6_??/Run/"

# runs_to_plot["28_set1_cd_junk_5"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_5/Ev/Lev3_??/Run/"
# runs_to_plot["28_set1_cd_junk_1"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_1/Ev/Lev3_??/Run/"
# runs_to_plot["28_set1_cd_junk_01"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_01/Ev/Lev3_??/Run/"
# runs_to_plot["28_set1_cd_junk_001"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_001/Ev/Lev3_??/Run/"

# runs_to_plot["29_set1_L3_ID_diff_12"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_12/Ev/Lev3_??/Run/"
# runs_to_plot["29_set1_L3_ID_diff_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_8/Ev/Lev3_??/Run/"
# runs_to_plot["29_set1_L3_ID_diff_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_4/Ev/Lev3_??/Run/"

# runs_to_plot["29_set1_L3_ID_diff_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_0/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_1/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_2/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_3/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_4_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_4_2/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_4/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_5/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_6/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_7"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_7/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_8/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_12"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_12/Ev/Lev3_A?/Run/"

# runs_to_plot["L1_AC_L1_AK"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L1/Ev/Lev1_AK/Run/"
# runs_to_plot["46K_L1_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L1_3000_cd10/Ev/Lev1_AK/Run/"
# runs_to_plot["46K_L2_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L2_3000_cd10/Ev/Lev2_AK/Run/"
# runs_to_plot["46K_L3_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L3_3000_cd10/Ev/Lev3_AK/Run/"
# runs_to_plot["46K_L4_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L4_3000_cd10/Ev/Lev4_AK/Run/"
# runs_to_plot["46K_L5_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L5_3000_cd10/Ev/Lev5_AK/Run/"
# runs_to_plot["46K_L6_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_cd10/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_cd100"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_cd100/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_2500"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_2500/Ev/Lev6_AK/Run/"

# runs_to_plot["46K_L6_3000_c_AB_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_2/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_4/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_6/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_10/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_12"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_12/Ev/Lev6_AK/Run/"

# runs_to_plot["46K_L6_3000_c_AB_4_L"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_4_L/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_2_L"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_2_L/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_1_L"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_1_L/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_1_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_1_L2/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_0_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_0_L2/Ev/Lev6_AK/Run/"

# runs_to_plot["set1_L3_Rn1"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn1/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn2/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin18"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin18/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin20_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin20_Rn2/Ev/Lev3_A?/Run/"

# runs_to_plot["RM_1_Lev3"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev3/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_0_Lev6"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_0_test/Ev_456/Lev6_A?/Run/"

# runs_to_plot["30_RM_set1_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_RM_set1_L1/Ev/Lev1_A?/Run/"
# runs_to_plot["30_RM_set1_L3"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_RM_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["RM_L3s3_k0"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s3_k0/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_L3s3_k0_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s3_k0_cd10/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_L3s3_k0_cd100"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s3_k0_cd100/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_L3s4_k0"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s4_k0/Ev/Lev4_A?/Run/"
# runs_to_plot["RM_L3s5_k0"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s5_k0/Ev/Lev5_A?/Run/"
# runs_to_plot["RM_0_Lev6"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_0_test/Ev_456/Lev6_A?/Run/"
# runs_to_plot["RM_1_Lev3"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev3/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_1_Lev6"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev6/Ev/Lev6_A?/Run/"

# runs_to_plot["31_RM_set1_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_RM_set1_L1/Ev/Lev1_A?/Run/"
# runs_to_plot["31_RM_set1_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_RM_set1_L2/Ev/Lev2_A?/Run/"

# runs_to_plot["31_segs_L1s0"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s0/Ev/Lev0_AC/Run/"
# runs_to_plot["31_segs_L1s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s1/Ev/Lev1_AC/Run/"
# runs_to_plot["31_segs_L1s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s2/Ev/Lev2_AC/Run/"
# runs_to_plot["31_segs_L1s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3/Ev/Lev3_AC/Run/"

# runs_to_plot["32_RM_set1_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/32_RM_set1_L1/Ev/Lev1_A?/Run/"
# runs_to_plot["32_RM_set1_L3"] = "/resnick/groups/sxs/hchaudha/spec_runs/32_RM_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["L1s3_cdg1_10"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_10/Ev/Lev3_AC/Run/"
# runs_to_plot["L1s3_cdg1_100"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_100/Ev/Lev3_AC/Run/"
# runs_to_plot["L1s3_cdg1_250"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_250/Ev/Lev3_AC/Run/"

# runs_to_plot["set1_L3_Rn1"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn1/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn2/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin18"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin18/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin20_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin20_Rn2/Ev/Lev3_A?/Run/"

# runs_to_plot["main_L6_AM_ode_MQOS"] = "/resnick/groups/sxs/hchaudha/spec_runs/26_segs_res/main_L6_AM_ode_MQOS/Ev/Lev6_A?/Run/"
# runs_to_plot["set1_L6_AK_ode_MQOS"] = "/resnick/groups/sxs/hchaudha/spec_runs/26_segs_res/set1_L6_AK_ode_MQOS/Ev/Lev6_A?/Run/"

# runs_to_plot["34_master_L16_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev1_A?/Run/"
# runs_to_plot["34_master_L16_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev2_A?/Run/"
# runs_to_plot["34_master_L16_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev3_A?/Run/"
# runs_to_plot["34_master_L16_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev4_A?/Run/"
# runs_to_plot["34_master_L16_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev5_A?/Run/"
# runs_to_plot["34_master_L16_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev6_A?/Run/"

# runs_to_plot["L3_AC_L3_minL17"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_minL17/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_minL19"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_minL19/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_minL21"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_minL21/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_01_cd_asymp"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_01_cd_asymp/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_single_exp"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_single_exp/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_single_Exp_large_sigma"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_single_Exp_large_sigma/Ev/Lev3_??/Run/"
# # runs_to_plot["L3_AC_L3_cd_const_low"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_low/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_cd_const_high"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_high/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_sigma2"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_sigma2/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_sigma05"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_sigma05/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_sigma1_const"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_sigma1_const/Ev/Lev3_??/Run/"

# runs_to_plot["L3_AC_L3_AB_L8"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB_L8/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_AB_R7"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB_R7/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_AB0_L16"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB0_L16/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_AB0_L15"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB0_L15/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_ps_10"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_ps_10/Ev/Lev3_AC/Run/"
# # runs_to_plot["L3_AC_L3_ps_01"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_ps_01/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_BCSC_8"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_BCSC_8/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_BCSC_12"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_BCSC_12/Ev/Lev3_AC/Run/"

# runs_to_plot["119_gd_SUKS_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["120W_gd_SUKS1_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/120W_gd_SUKS1_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["120_gd_SUKS1_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/120_gd_SUKS1_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["119_gd_SUKS_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["67_master_mr3"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/67_master_mr3/Ev/Lev1_A?/Run/"
# runs_to_plot["119_gd_SUKS_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"

# runs_to_plot["35_L3_base"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_base/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L3_base_nas"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_base_nas/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L2_base_nas"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L2_base_nas/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L3_Ctest10"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_Ctest10/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L3_Ctest15"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_Ctest15/Ev/Lev6_A?/Run/"
# runs_to_plot["35_L3_Ctest15Uni"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_Ctest15Uni/Ev/Lev6_A?/Run/"

# runs_to_plot["36_set1_L1_d20"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_set1_L1_d20/Ev/Lev1_A?/Run/"
# runs_to_plot["37_set1_L3_no_ang"] = "/resnick/groups/sxs/hchaudha/spec_runs/37_6_segs_grid_change/6_set1_L3_no_ang/Ev/Lev3_A?/Run/"

# runs_to_plot["34_master_L16_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev1_A?/Run/"
# runs_to_plot["34_master_L16_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev2_A?/Run/"
# runs_to_plot["34_master_L16_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev3_A?/Run/"
# runs_to_plot["34_master_L16_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev4_A?/Run/"
# runs_to_plot["34_master_L16_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev5_A?/Run/"
# runs_to_plot["34_master_L16_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev6_A?/Run/"

# runs_to_plot["36_segs_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev1_A?/Run/"
# runs_to_plot["36_segs_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev2_A?/Run/"
# runs_to_plot["36_segs_L3"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev3_A?/Run/"
# runs_to_plot["36_segs_L4"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev4_A?/Run/"
# runs_to_plot["36_segs_L5"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev5_A?/Run/"
# runs_to_plot["36_segs_L6"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set1_L6s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set1_L6s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set1_L6s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set1_L6s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set1_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set1_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set1_L6s7"] =  "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs/Ev/Lev7_A?/Run/"
# runs_to_plot["6_set1_L6s8"] =  "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs/Ev/Lev8_A?/Run/"
# runs_to_plot["6_set1_L6s9"] =  "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs/Ev/Lev9_A?/Run/"
# runs_to_plot["6_set1_L6s10"] =  "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs/Ev/Lev10_A?/Run/"
# runs_to_plot["6_set1_L6s11"] =  "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs/Ev/Lev11_A?/Run/"

# runs_to_plot["6_set1_L6s6_TA"] = "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs_trigger_AMR/Ev/Lev6_AF/Run/"
# runs_to_plot["6_set1_L6s7_TA"] = "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs_trigger_AMR/Ev/Lev7_AF/Run/"
# runs_to_plot["6_set1_L6s8_TA"] = "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs_trigger_AMR/Ev/Lev8_AF/Run/"
# runs_to_plot["6_set1_L6s9_TA"] = "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs_trigger_AMR/Ev/Lev9_AF/Run/"
# runs_to_plot["6_set1_L6s10_TA"] = "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs_trigger_AMR/Ev/Lev10_AF/Run/"
# runs_to_plot["6_set1_L6s11_TA"] = "/resnick/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6_high_levs_trigger_AMR/Ev/Lev11_AF/Run/"

# runs_to_plot["L35higher_correct"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35higher_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["L35lower_correct"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35lower_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["L45higher_correct"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L45higher_correct/Ev/Lev4_A?/Run/"
# runs_to_plot["L35higher"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35higher/Ev/Lev3_A?/Run/"
# runs_to_plot["L35lower"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35lower/Ev/Lev3_A?/Run/"
# runs_to_plot["L45higher"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L45higher/Ev/Lev4_A?/Run/"

# runs_to_plot["L3AH001"] = "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH001/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH010"] = "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH010/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH1"] = "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH1/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH10"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH10/Ev/Lev3_AD/Run/"
# runs_to_plot["6_set1_L6s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH1000"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH1000/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH5000"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH5000/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH20000"] = "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH20000/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH100000"] = "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH100000/Ev/Lev3_AD/Run/"
# runs_to_plot["L3AH500000"] = "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH500000/Ev/Lev3_AD/Run/"

# runs_to_plot["L5_main_obs"] =  "/resnick/groups/sxs/hchaudha/spec_runs/39_obs_segs/L3L3_main_obs/Ev/Lev5_A?/Run/"
# runs_to_plot["L1_set1_metric_obs"] =  "/resnick/groups/sxs/hchaudha/spec_runs/39_obs_segs/L1_set1_metric_obs/Ev/Lev1_AB/Run/"

# runs_to_plot["q10_0_0_0_0_0_0_d14"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc3/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_9_0_0_0_d165"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_0_d165/Ecc2/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_9_0_0_9_d165"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_9_d165/Ecc3/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_n9_0_0_0_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_n9_0_0_0_d185/Ecc3/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_65_65_0_n65_n65_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_65_65_0_n65_n65_d185/Ecc3/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_8_0_0_0_8_d17"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_8_d17/Ecc2/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_8_0_0_0_n8_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_n8_d185/Ecc2/Ev/Lev3_A?/Run/"
# runs_to_plot["q3_0_0_85_0_0_85_d15"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_85_0_0_85_d15/Ecc3/Ev/Lev3_A?/Run/"
# runs_to_plot["q3_0_0_n85_0_0_n85_d18"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_n85_0_0_n85_d18/Ecc4/Ev/Lev3_A?/Run/"
# runs_to_plot["q5_0_0_0_0_0_0_d15"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q5_0_0_0_0_0_0_d15/Ecc3/Ev/Lev3_A?/Run/"

# runs_to_plot["q1_0_0_9_0_0_0_d165"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_0_d165_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_0_9_0_0_9_d165"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_9_d165_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_65_65_0_n65_n65_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_65_65_0_n65_n65_d185_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_8_0_0_0_8_d17"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_8_d17_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_8_0_0_0_n8_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_n8_d185_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q5_0_0_0_0_0_0_d15"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q5_0_0_0_0_0_0_d15_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q10_0_0_0_0_0_0_d14"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_0_n9_0_0_0_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_n9_0_0_0_d185_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q3_0_0_85_0_0_85_d15"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_85_0_0_85_d15_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q3_0_0_n85_0_0_n85_d18"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_n85_0_0_n85_d18_segs/Ev/Lev3_??/Run/"
# Ecc_num = 0
# runs_to_plot["q10_0_0_0_0_0_0_d14"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_9_0_0_0_d165"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_0_d165/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_9_0_0_9_d165"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_9_d165/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_n9_0_0_0_d185"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_n9_0_0_0_d185/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_65_65_0_n65_n65_d185"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_65_65_0_n65_n65_d185/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_8_0_0_0_8_d17"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_8_d17/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_8_0_0_0_n8_d185"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_n8_d185/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q3_0_0_85_0_0_85_d15"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_85_0_0_85_d15/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q3_0_0_n85_0_0_n85_d18"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_n85_0_0_n85_d18/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q5_0_0_0_0_0_0_d15"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q5_0_0_0_0_0_0_d15/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"

# runs_to_plot["1_DH"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SupImpKerr"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/2_SupImpKerr/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["17_main_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_09"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_DH_q1_09/Ev/Lev3_A?/Run/"
# runs_to_plot["3_SupImpKerr_09"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_SupImpKerr_09/Ev/Lev3_A?/Run/"

# runs_to_plot["17_set_main_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["3_DH_q3"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_DH_q3/Ev/Lev3_A?/Run/"
# runs_to_plot["3_SupImpKerr_q3"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_SupImpKerr_q3/Ev/Lev3_A?/Run/"

# runs_to_plot["DH_8_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_8_8/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_32_32"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_32_32/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_64_64"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_64_64/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_128_128"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_128_128/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_256_256"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_256_256/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_512_512"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_512_512/Ev/Lev3_AC/Run/"

# runs_to_plot["41_dlib_test"] = "/resnick/groups/sxs/hchaudha/spec_runs/41_float128/dlib_test/Ev/Lev3_A?/Run/"
# runs_to_plot["40_q1_ns_d18_faster"] = "/resnick/groups/sxs/hchaudha/spec_runs/40_float128/q1_ns_d18_faster/Ev/Lev3_A?/Run/"
# runs_to_plot["40_q1_ns_d18"] = "/resnick/groups/sxs/hchaudha/spec_runs/40_float128/q1_ns_d18/Ev/Lev3_A?/Run/"

# runs_to_plot["42_f128_check"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/L3_new/Ev/Lev3_A?/Run/"

# runs_to_plot["36_segs_L6"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev6_A[G-Z]/Run/"
# runs_to_plot["42_f128_L6"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/L6/Ev/Lev6_A?/Run/"
# runs_to_plot["42_L6_nAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/36_L6_nAMR/Ev/Lev6_A?/Run/"
# runs_to_plot["42_L6_cd100_nAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/36_L6_cd100_nAMR/Ev/Lev6_A?/Run/"

# runs_to_plot["17_set1_99_18_L3_cd100"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/17_set1_99_18_L3_cd100/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set1_L6s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev2_A[D-Z]/Run/"
# runs_to_plot["42_set1L6s2_nAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s2_nAMR/Ev/Lev2_A?/Run/"
# runs_to_plot["42_set1L6s2_nAMR_moreL"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s2_nAMR_moreL/Ev/Lev2_A?/Run/"
# runs_to_plot["42_set1L6s4_nAMR"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s4_nAMR/Ev/Lev4_A?/Run/"

# runs_to_plot["42_f128_Set1L6s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s1/Ev/Lev1_A?/Run/"
# runs_to_plot["42_f128_Set1L6s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s2/Ev/Lev2_A?/Run/"
# runs_to_plot["42_f128_Set1L6s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s3/Ev/Lev3_A?/Run/"
# runs_to_plot["42_f128_Set1L6s4"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s4/Ev/Lev4_A?/Run/"
# runs_to_plot["42_f128_Set1L6s5"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s5/Ev/Lev5_A?/Run/"
# runs_to_plot["42_f128_Set1L6s6"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s6/Ev/Lev6_A?/Run/"

# runs_to_plot["45_f128_Set1L6s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s1/Ev/Lev1_A?/Run/"
# runs_to_plot["45_f128_Set1L6s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s2/Ev/Lev2_A?/Run/"
# runs_to_plot["45_f128_Set1L6s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s3/Ev/Lev3_A?/Run/"
# runs_to_plot["45_f128_Set1L6s4"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s4/Ev/Lev4_A?/Run/"
# runs_to_plot["45_f128_Set1L6s5"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s5/Ev/Lev5_A?/Run/"
# runs_to_plot["45_f128_Set1L6s6"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s6/Ev/Lev6_A?/Run/"

# runs_to_plot["45.1_set1_more_cd1_moreL_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev1_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev2_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev3_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev4_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev5_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev6_A?/Run/"

# runs_to_plot["45.2_RM_more_cd1_moreL_2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev2_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_3"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev3_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_4"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev4_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_5"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev5_A?/Run/"

# runs_to_plot["45.2_RM_more_cd1_moreL_3_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev3_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_4_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev4_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_5_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev5_A?/Run/"

# runs_to_plot["q4_0_0_0_0_0_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q4_0_0_0_0_0_0/Ecc0/Ev/Lev4_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0_deltaL_large"] = "/resnick/groups/sxs/hchaudha/spec_runs/47.1_LISA_cat_vars/q4_0_0_0_0_0_0_deltaL_large/Ev/Lev4_A?/Run/"

# runs_to_plot["q1_0_0_0_0_0_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_0_0_0_0/Ecc2/Ev/Lev4_A?/Run/"
# runs_to_plot["q1_0_0_0_0_0_0_no_NegExp"] = "/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_0_0_0_0_no_NegExp/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["12_set1_L3_1500"] = "/resnick/groups/sxs/hchaudha/spec_runs/12_set1_L3_1500/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2000"] = "/resnick/groups/sxs/hchaudha/spec_runs/12_set1_L3_2000/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2500"] = "/resnick/groups/sxs/hchaudha/spec_runs/12_set1_L3_2500/Ev/Lev3_A?/Run/"
# runs_to_plot["13_set1_L3_3000"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L3_3000/Ev/Lev3_A?/Run/"

# runs_to_plot["13_set1_L4_1500"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L4_1500/Ev/Lev4_A?/Run/"
# runs_to_plot["13_set1_L4_3000"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L4_3000/Ev/Lev4_A?/Run/"

# runs_to_plot["48_set1_L3_fromID"] = "/resnick/groups/sxs/hchaudha/spec_runs/48_less_sds/set1_L3_fromID/Ev/Lev3_A?/Run/"
# runs_to_plot["48_set1_L3_fromID_1AB"] = "/resnick/groups/sxs/hchaudha/spec_runs/48_less_sds/set1_L3_fromID_1AB/Ev/Lev3_A?/Run/"

# runs_to_plot["442"] = "/groups/sxs/dsun2/BBH/442/Ev/Lev3_A?/Run/"
# runs_to_plot["443"] = "/groups/sxs/dsun2/BBH/443/Ev/Lev3_A?/Run/"

# runs_to_plot["q1_0_0_8_0_0_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q1_0_0_8_0_0_8/Ev/Lev6_A?/Run/"
# runs_to_plot["q1_0_0_8_0_0_8_MaxDL"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q1_0_0_8_0_0_8_MaxDL/Ev/Lev6_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q4_0_0_0_0_0_0/Ev/Lev6_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0_MaxDL"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q4_0_0_0_0_0_0_MaxDL/Ev/Lev6_A?/Run/"

# runs_to_plot["q10_0_0_0_0_0_0_d14_Ecc0"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc0/Ev/Lev3_A?/Run/"
# runs_to_plot["q10_0_0_0_0_0_0_d14_Ecc1"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc1/Ev/Lev3_A?/Run/"
# runs_to_plot["q10_0_0_0_0_0_0_d14_Ecc2"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc2/Ev/Lev3_A?/Run/"
# runs_to_plot["q10_0_0_0_0_0_0_d14_Ecc3"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc3/Ev/Lev3_A?/Run/"

# runs_to_plot["52_q105_0_0_1_0_0_1_Ecc0"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_1_0_0_1/Ecc0/Ev/Lev3_A?/Run/"

# runs_to_plot["52_q105_0_0_n1_0_0_1_Ecc0"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_n1_0_0_1/Ecc0/Ev/Lev3_A?/Run/"
# runs_to_plot["52_q105_0_0_n1_0_0_1_Ecc1"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_n1_0_0_1/Ecc1/Ev/Lev3_A?/Run/"

# runs_to_plot["52_q105_0_0_4_0_0_4_Ecc0"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_4_0_0_4/Ecc0/Ev/Lev3_A?/Run/"
# runs_to_plot["52_q105_0_0_4_0_0_4_Ecc1"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_4_0_0_4/Ecc1/Ev/Lev3_A?/Run/"
# runs_to_plot["52_q105_0_0_4_0_0_4_Ecc2"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_4_0_0_4/Ecc2/Ev/Lev3_A?/Run/"

# runs_to_plot["52_q105_0_0_n4_0_0_4_Ecc0"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_n4_0_0_4/Ecc0/Ev/Lev3_A?/Run/"
# runs_to_plot["52_q105_0_0_n4_0_0_4_Ecc1"] = "/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_n4_0_0_4/Ecc1/Ev/Lev3_A?/Run/"

# runs_to_plot["q1_ns_3000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_3000M_lsds/Ecc0/Ev/Lev4_A?/Run/"
# runs_to_plot["q1_ns_3000M"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_3000M/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q1_ns_2000M"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_2000M/Ecc0/Ev/Lev4_A?/Run/"
# runs_to_plot["q1_ns_2000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_2000M_lsds/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q1_ns_10000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_10000M_lsds/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q4_0_0_0_0_0_0_L6"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q4_0_0_0_0_0_0/Ev/Lev6_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0_MaxDL"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q4_0_0_0_0_0_0_MaxDL/Ev/Lev6_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0_L3"] = "/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q4_0_0_0_0_0_0_no_NegExp/Ecc1/Ev/Lev3_A?/Run/"
# runs_to_plot["q4_ns_3000M_lsds_var2"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q4_ns_3000M_lsds_var2/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q1_0_0_n8_0_0_8_no_NegExp"] = "/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_n8_0_0_8_no_NegExp/Ecc0/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_n8_8_3000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_n8_8_3000M_lsds/Ecc1/Ev/Lev4_A?/Run/"
# runs_to_plot["q1_n8_8_3000M_lsds_var1"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_n8_8_3000M_lsds_var1/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q105_01_01_4a7a1adccf"] = "/resnick/groups/sxs/hchaudha/spec_runs/54_SphereC_glitch/q105_01_01_4a7a1adccf/Ev/Lev2_A?/Run/"
# runs_to_plot["q105_01_01_4c3e70a2ba"] = "/resnick/groups/sxs/hchaudha/spec_runs/54_SphereC_glitch/q105_01_01_4c3e70a2ba/Ev/Lev2_A?/Run/"
# runs_to_plot["q105_01_01_4e146823fe"] = "/resnick/groups/sxs/hchaudha/spec_runs/54_SphereC_glitch/q105_01_01_4e146823fe/Ev/Lev2_A?/Run/"
# runs_to_plot["q105_01_01_8e2c456ce5"] = "/resnick/groups/sxs/hchaudha/spec_runs/54_SphereC_glitch/q105_01_01_8e2c456ce5/Ev/Lev2_A?/Run/"
# runs_to_plot["q105_01_01_fad21f0be4"] = "/resnick/groups/sxs/hchaudha/spec_runs/54_SphereC_glitch/q105_01_01_fad21f0be4/Ev/Lev2_A?/Run/"

# runs_to_plot["f128_q1_n8_8_3000M_lsds_var1"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_n8_8_3000M_lsds_var1/Ev/Lev4_A?/Run/"
# runs_to_plot["f128_q1_ns_2000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_ns_2000M_lsds/Ev/Lev4_A?/Run/"
# runs_to_plot["f128_q1_ns_3000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_ns_3000M_lsds/Ev/Lev4_A?/Run/"
# runs_to_plot["f128_q1_ns_10000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_ns_10000M_lsds/Ev/Lev4_A?/Run/"

# runs_to_plot["HighAccuracy102501_vars"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars/Lev5_A?/Run/"

# runs_to_plot["01_L5_maxL10"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars2/01_L5_maxL10/Ev/Lev5_A?/Run/"
# runs_to_plot["01_L5_maxL13"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars2/01_L5_maxL13/Ev/Lev5_A?/Run/"
# runs_to_plot["01_L5_maxL15"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars2/01_L5_maxL15/Ev/Lev5_A?/Run/"
# runs_to_plot["01_L5_regrid"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars2/01_L5_regrid/Ev/Lev5_A?/Run/"
# runs_to_plot["01_L5_no_cd"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars2/01_L5_no_cd/Ev/Lev5_A?/Run/"

# runs_to_plot["Ev3"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/Ev/Lev3_A?/Run/"
# runs_to_plot["Ev4"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/Ev/Lev4_A?/Run/"

# runs_to_plot["Ecc5DSur_B064_3"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/Ecc5DSur_B064/Ecc3/Ev/Lev3_??/Run/"
# runs_to_plot["Ecc5DSur_B064_4"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/Ecc5DSur_B064/Ecc3/Ev/Lev4_??/Run/"

# #  for lev in range(2,7):
for lev in [6]:
#     runs_to_plot[f"01_old_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501/Ecc0/Ev/Lev{lev}_A?/Run/"
#     runs_to_plot[f"01_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/from_stampede/HighAccuracy102501/Ecc0/Ev/Lev{lev}_A?/Run/"
#     # runs_to_plot[f"02_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102502/Ecc0/Ev/Lev{lev}_A?/Run/"
    runs_to_plot[f"09_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509_stampede/Ecc0/Ev/Lev{lev}_A?/Run/"

# runs_to_plot['09_L2_var'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509_vars/2_BFI/Lev2_A?/Run/"

# for lev in [2,4]:
#     runs_to_plot[f"SpECCCM_L{lev}"] = f"/resnick/groups/sxs/sma/SpECCCM/IMR/Ev_new/Lev{lev}_A?/Run/"

# runs_to_plot["L2_lowcd"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509_vars/L2_lowcd/Ev/Lev2_??/Run/"
# runs_to_plot["L2_lowcd_regrid"] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509_vars/L2_lowcd_regrid/Ev/Lev2_??/Run/"


# runs_to_plot['run09_Lev5_cd_0_amr_0'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev5_cd_0_amr_0/Ecc0/Ev/Lev5_A?/Run/"
# runs_to_plot['run09_Lev5_cd_0_amr_1'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev5_cd_0_amr_1/Ecc0/Ev/Lev5_A?/Run/"
# runs_to_plot['run09_Lev5_cd_1_amr_1'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev5_cd_1_amr_1/Ecc0/Ev/Lev5_A?/Run/"
runs_to_plot['run09_Lev6_cd_0_amr_0'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev6_cd_0_amr_0/Ecc0/Ev/Lev6_A?/Run/"
runs_to_plot['run09_Lev6_cd_0_amr_1'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev6_cd_0_amr_1/Ecc0/Ev/Lev6_A?/Run/"
runs_to_plot['run09_Lev6_cd_1_amr_1'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev6_cd_1_amr_1/Ecc0/Ev/Lev6_A?/Run/"
runs_to_plot['run09_Lev6_main'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/AMR_cd_variations/run09_Lev6_main/Ecc0/Ev/Lev6_A?/Run/"

# runs_to_plot['01_L5_mainish'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars3/01_L5_mainish/Ev/Lev5_A?/Run/"
# runs_to_plot['01_L5_platue_AMR_Lev3_no_cd'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars3/01_L5_platue_AMR_Lev3_no_cd/Ev/Lev5_A?/Run/"
# runs_to_plot['01_L5_platue_AMR_no_cd'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501_vars3/01_L5_platue_AMR_no_cd/Ev/Lev5_A?/Run/"

# runs_to_plot['09_L2_mainish'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509_vars/09_L2_mainish/Ev/Lev2_A?/Run/"
# runs_to_plot['09_L2_platue_AMR_no_cd'] = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509_vars/09_L2_platue_AMR_no_cd/Ev/Lev2_A?/Run/"

ringdown_only = True
ringdown_only = False
if ringdown_only:
    for key in list(runs_to_plot.keys()):
        runs_to_plot[key] = runs_to_plot[key].replace("/Ev/","/Ev/**/")

# data_file_path = "ConstraintNorms/GhCe.dat"
# data_file_path = "ConstraintNorms/GhCeExt.dat"
# data_file_path = "ConstraintNorms/GhCeExt_L2.dat"
# data_file_path = "ConstraintNorms/GhCeExt_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_L2.dat"
data_file_path = "ConstraintNorms/GhCe_Linf.dat"
# data_file_path = "ConstraintNorms/Linf.dat"
# data_file_path = "ConstraintNorms/Constraints_Linf.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path = "ConstraintNorms/GhCe_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_VolL2.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"
data_file_path = "ConstraintNorms/NormalizedGhCe_Norms.dat"
# data_file_path = "CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0.dat"
# data_file_path = "MinimumGridSpacing.dat"
# data_file_path = "GrAdjustMaxTstepToDampingTimes.dat"
# data_file_path = "GrAdjustSubChunksToDampingTimes.dat"
# data_file_path = "DiagAhSpeedA.dat"
# data_file_path = "ApparentHorizons/AhA.dat"
# data_file_path = "ApparentHorizons/AhB.dat" 
# data_file_path = "ApparentHorizons/MinCharSpeedAhA.dat"
# data_file_path = "ApparentHorizons/RescaledRadAhA.dat"
# data_file_path = "ApparentHorizons/AhACoefs.dat"
# data_file_path = "ApparentHorizons/AhBCoefs.dat"
# data_file_path = "ApparentHorizons/Trajectory_AhB.dat"
# data_file_path = "ApparentHorizons/Trajectory_AhA.dat"
# data_file_path = "ApparentHorizons/HorizonSepMeasures.dat"
# data_file_path = "ConstraintNorms/Metric_norms.dat"
# data_file_path = "ConstraintNorms/Metric_L2.dat"

# data_file_path = "PowerDiagnostics.h5@ConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@HighestThirdConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfFilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfNonFilteredNonZeroModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfPiledUpModes.dat"
# data_file_path = "PowerDiagnostics.h5@PowerInFilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@PowerInHighestUnfilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@PredictedTruncationErrorForOneLessMode.dat"
# data_file_path = "PowerDiagnostics.h5@RawConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@SpectrumIsDegenerate.dat"
# data_file_path = "PowerDiagnostics.h5@TruncationError.dat"

# psi_or_kappa = "psi"
psi_or_kappa = "kappa"
psi_or_kappa = "both"
top_number = 0

# data_file_path = "ApparentHorizons/Horizons.h5@AhA"
# data_file_path = "ApparentHorizons/Horizons.h5@AhB"
# data_file_path = "ComputeCOM"
# data_file_path = "TStepperDiag.dat"
# data_file_path = "TimeInfo.dat"
# data_file_path = "Hist-FuncSkewAngle.txt"
# data_file_path = "Hist-FuncCutX.txt"
# data_file_path = "Hist-FuncExpansionFactor.txt"
# data_file_path = "Hist-FuncLambdaFactorA0.txt"
# data_file_path = "Hist-FuncLambdaFactorA.txt"
# data_file_path = "Hist-FuncLambdaFactorB0.txt"
# data_file_path = "Hist-FuncLambdaFactorB.txt"
# data_file_path = "Hist-FuncQuatRotMatrix.txt"
# data_file_path = "Hist-FuncSkewAngle.txt"
# data_file_path = "Hist-FuncSmoothCoordSep.txt"
# data_file_path = "Hist-FuncSmoothMinDeltaRNoLam00AhA.txt"
# data_file_path = "Hist-FuncSmoothMinDeltaRNoLam00AhB.txt"
# data_file_path = "Hist-FuncSmoothRAhA.txt"
# data_file_path = "Hist-FuncSmoothRAhB.txt"
# data_file_path = "Hist-FuncTrans.txt"
# data_file_path = "Hist-GrDomain.txt"
# data_file_path = "Profiler.h5"
# data_file_path = "OrbitDiagnostics.h5"


runs_data_dict = {}
if psi_or_kappa == "both" and "PowerDiagnostics" in data_file_path:
    column_names_kappa, runs_data_dict_kappa = load_data_from_levs(runs_to_plot,f"{data_file_path}@kappa@{top_number}")
    column_names_psi, runs_data_dict_psi = load_data_from_levs(runs_to_plot,f"{data_file_path}@psi@{top_number}")

    for key in runs_data_dict_psi:
        runs_data_dict[key] =pd.merge(runs_data_dict_kappa[key], runs_data_dict_psi[key], on='t(M)', how='outer')
    column_names = runs_data_dict[key].columns.tolist()

elif data_file_path == "ComputeCOM":
    bhA_cols, bhA_data = load_data_from_levs(runs_to_plot,"ApparentHorizons/Horizons.h5@AhA")
    bhB_cols, bhB_data = load_data_from_levs(runs_to_plot,"ApparentHorizons/Horizons.h5@AhB")
    for key in bhA_data:
        bhA_df = bhA_data[key]
        bhB_df = bhB_data[key]
        mA = bhA_df["ChristodoulouMass"]
        mB = bhB_df["ChristodoulouMass"]
        com_x = (mA*bhA_df['CoordCenterInertial_0'] + mB*bhB_df['CoordCenterInertial_0']) / (mA + mB)
        com_y = (mA*bhA_df['CoordCenterInertial_1'] + mB*bhB_df['CoordCenterInertial_1']) / (mA + mB)
        com_z = (mA*bhA_df['CoordCenterInertial_2'] + mB*bhB_df['CoordCenterInertial_2']) / (mA + mB)
        com_df = pd.DataFrame({'t(M)': bhA_df['t(M)'], 'COM_X': com_x, 'COM_Y': com_y, 'COM_Z': com_z})
        runs_data_dict[key] = com_df
        column_names = com_df.columns.tolist()
else:
    if "PowerDiagnostics" in data_file_path:
        data_file_path = f"{data_file_path}@{psi_or_kappa}@{top_number}"
    column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)

# load both psi and kappa

print(column_names)
print(runs_data_dict.keys())

In [ ]:
vars = set()
domains = set()
for run in runs_data_dict:
    for col in runs_data_dict[run]:
        vars.add(col.split(" on ")[0])
        domains.add(col.split(" on ")[1] if " on " in col else "All Domains") 
# list(sorted(vars)),list(sorted(domains))

#### Modify dict

In [ ]:
if "Constraints_Linf" in data_file_path:
    new_indices,runs_data_dict = add_norm_constraints(runs_data_dict,index_num=[1,2,3] ,norm=["Linf"], subdomains_seperately=True, replace_runs_data_dict=True)
    print(runs_data_dict.keys())
    print(new_indices)  

if "GrDomain" in data_file_path:
    new_indices,runs_data_dict = num_points_per_subdomain(runs_data_dict, replace_runs_data_dict=False)
    print(runs_data_dict.keys())
    print(new_indices)

if "ComputeCOM" in data_file_path:
    for key in runs_data_dict:
        df = runs_data_dict[key]
        df['COM_mag'] = np.sqrt(df['COM_X']**2 + df['COM_Y']**2 + df['COM_Z']**2)
        runs_data_dict[key] = df
    print(runs_data_dict.keys())
    print(new_indices)

### dat files plot

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
constant_shift_val_time = None
plot_abs_diff = False
modification_function = None
append_to_title = ""
y_axis_list = None
x_axis = 't(M)'
take_abs = False

take_abs = True

# diff_base = 'q105_01_01_4a7a1adccf'
# diff_base = '36_segs_L6'
# diff_base = '6_set1_L6s6'
# diff_base = '34_master_L16_6'
# diff_base = 'q1_0_0_8_0_0_8_MaxDL'
# diff_base = 'L3AH500000'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

# constant_shift_val_time = 1200
# constant_shift_val_time = 11790
# constant_shift_val_time = 1150
# constant_shift_val_time = 2000
# constant_shift_val_time = 5900
# constant_shift_val_time = 6000
# constant_shift_val_time = 7000
# if constant_shift_val_time is not None:
#     take_abs = True

# sd = 'SphereD0'
# sd = 'SphereD5'
# sd = 'SphereE0'
# sd = 'SphereE10'
# sd = 'SphereB0'
# sd = 'SphereB4'

sd = 'SphereA0'
sd = 'SphereB0'
# sd = 'SphereB5'
sd = 'SphereC0'
# sd = 'SphereC4'
sd = 'SphereC9'
# sd = 'SphereC15'
sd = 'SphereC17'
sd = 'SphereC22'
# sd = 'SphereC28'
# sd = 'SphereC45'
# sd = 'CylinderEB0.0.0'
# sd = 'CylinderEB1.0.0'
# sd = 'CylinderCB0.0.0'
# sd = 'CylinderCB1.0.0'
# sd = 'FilledCylinderEB0'
# sd = 'FilledCylinderEB1'
# sd = 'FilledCylinderCB0'
# sd = 'FilledCylinderCB1'
# sd = 'FilledCylinderMB0'
# sd = 'FilledCylinderMB1'
# sd = 'CylinderSMB0.0'
# sd = 'CylinderSMB1.0'
# sd = 'CylinderA1EA0.0'

y_axis = f'Linf(GhCe) on {sd}'

# y_axis_list = [f'Linf(GhCe) on SphereC15',f'Linf(GhCe) on SphereC16',f'Linf(GhCe) on SphereC17' ]
# y_axis = f'Linf(1Con) on {sd}'
# y_axis = f'Linf(2Con) on {sd}'
# y_axis = f'Linf(3Con) on {sd}'
# y_axis_list = [f'Linf(1Con) on {sd}',f'Linf(2Con) on {sd}',f'Linf(3Con) on {sd}']
# y_axis = f'Linf(sqrt(1Con^2)) on {sd}'
# y_axis = f'Linf(sqrt(2Con^2)) on {sd}'
# y_axis = f'Linf(sqrt(3Con^2)) on {sd}'
# y_axis_list = [f'Linf(sqrt(1Con^2)) on {sd}',f'Linf(sqrt(2Con^2)) on {sd}',f'Linf(sqrt(3Con^2)) on {sd}']
# y_axis_list = [f'Linf(GhCe) on SphereC{d}' for d in range(0,32,3)]
# y_axis_list = [i for i in column_names if f'Linf(GhCe)' in i and 'SphereC' in i]
# y_axis_list = [f'L2(1Con) on {sd}',f'L2(2Con) on {sd}',f'L2(3Con) on {sd}'] + y_axis_list
# y_axis = f'Linf(sqrt(kappa^2)) on {sd}'
# y_axis = f'L2(kappatxx) on {sd}'
# y_axis = f'L2(sqrt(psi^2)) on {sd}'
# y_axis = f'L2(sqrt(kappa^2)) on {sd}'
# y_axis_list = [f'Linf({i}Con) on {sd}' for i in [1,2,3]]
# y_axis = f'Linf(GhCe) on {sd}'
# y_axis = f'Linf(NormalizedGhCe) on {sd}'
# y_axis = f'MinimumGridSpacing[{sd}]'
# y_axis = f'NumPoints in {sd}'
# y_axis = f'{psi_or_kappa}_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}'
# y_axis = f'psi_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}'
# y_axis = f'kappa_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}'
# if psi_or_kappa == "both":
#     y_axis_list = [f'kappa_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}',f'psi_TruncationError_{get_top_name_from_number(top_number,sd)} on {sd}']
# y_axis = 'Linf(sqrt(3Con^2)) on SphereD0'
# y_axis = 'Linf(sqrt(3Con^2)) on SphereE5'

fund_var = 'kappa'
fund_var = 'psi'
# var = 'PowerInHighestUnfilteredModes'
var = 'TruncationError'
# y_axis = f'{fund_var}_{var}_{get_top_name_from_number(top_number,sd)} on {sd}'
# y_axis_list = [f'psi_{var}_{get_top_name_from_number(top_number,sd)} on {sd}',f'kappa_{var}_{get_top_name_from_number(top_number,sd)} on {sd}']

# y_axis = f'{sd}_R'
# y_axis = f'{sd}_L'
# y_axis_list = [f'{sd}_R',f'{sd}_L']
# y_axis_list = [f'SphereC5_L',f'SphereC4_L']
# y_axis = f'{sd}_M'

# y_axis_list = [f'Linf(GhCe) on SphereC{n}' for n in range(26)]

# y_axis = 'L2(GhCe)'
y_axis = 'Linf(NormalizedGhCe)'

# y_axis = 'phi'

# y_axis = 'Linf(3Conzzz) on CylinderSMB0.0'

# y_axis = 'MPI::MPwait_cum'
# x_axis = 't'
# y_axis = 'T [hours]'
# y_axis = 'dt/dT'

# y_axis = 'FilledCylinderMB0_L'
# y_axis = 'MinimumGridSpacing[SphereA0]'
# y_axis = 'MinimumGridSpacing[CylinderSMB0.0]'
# y_axis = 'CoordSepHorizons'

# y_axis = 'ArealMass'
# y_axis = 'ChristodoulouMass'
# y_axis = 'CoordCenterInertial_0'
# y_axis = 'CoordCenterInertial_2'
# y_axis = 'CoordSpinChiMagInertial'
# y_axis = 'DimensionfulInertialCoordSpinMag'
# y_axis = 'DimensionfulInertialSpinMag'
# y_axis = 'chiMagInertial'
# y_axis = 'min(r)'
# y_axis = 'max(r)'
# y_axis = 'NumIterations'
# y_axis = 'L_surface'
# y_axis = 'L_mesh'
# y_axis = 'L_max'
# y_axis = 'Residual'
# y_axis = 'Shape_TruncationError'
# y_axis = 'InertialCenter_x'

# point = "(0,0,0)"
# y_axis = f"{data_file_path.split('/')[-1][4:-4]}_{point}"

# y_axis = 'courant factor'
# y_axis = 'error/1e-08'
# y_axis = 'NfailedSteps'
# y_axis = 'NumRhsEvaluations in this segment'
# y_axis = 'dt'
# y_axis = 'dt desired by control'
# y_axis_list = ['dt','dt desired by control']
# y_axis = 'FilledCylinderMA1_L'
# y_axis = 'NumIterations'
# y_axis = 'ProperSepHorizons'
# y_axis = 'dt'

# x_axis = 'COM_X'
# y_axis = 'COM_X'
# y_axis = 'COM_Y'
# y_axis = 'COM_mag'

minT = -1000
# minT = 0
minT = 85
# minT = 750
# minT = constant_shift_val_time-50
# minT = 2100
# minT = 4100
# minT = 6001
# minT = 6100
# minT = 8000
# minT = 8700
# minT = 11000

maxT = 400000
# maxT = minT+5
# maxT = 700
# maxT = minT + 250
# maxT = 2777
# maxT = 4000
# maxT = 6300
# maxT = 9500
# maxT = 11000
# moving_avg_len = 10
# moving_avg_len = 50
# moving_avg_len = 1000
# maxT = 47100
# maxT = 46600


# y_axis_list = [f"SphereC{i}_R" for i in range(30)]
# y_axis_list = ["SphereC0_L","SphereC1_L","SphereC2_L","SphereC4_L","SphereC8_L","SphereC16_L","SphereC29_L"]
# y_axis_list = ["SphereC4_L","SphereC16_L","SphereC29_L"]
# y_axis_list = [f'Linf(GhCe) on SphereA{i}' for i in [0]]
# y_axis_list = ["SphereC0_R",'CylinderSMA0.0_R','FilledCylinderMA0_R','SphereA0_R']
# y_axis_list = ['SphereA0_L','SphereA1_L','SphereA2_L','SphereA3_L','SphereA4_L']
# y_axis_list = ['Linf(GhCe) on CylinderSMA0.0','Linf(GhCe) on FilledCylinderMA0','Linf(GhCe) on SphereA0']
# y_axis_list = [
#   'Linf(NormalizedGhCe) on SphereA0',
#   'Linf(NormalizedGhCe) on SphereA1',
#   'Linf(NormalizedGhCe) on SphereA2',
#   'Linf(NormalizedGhCe) on SphereA3',
#   'Linf(NormalizedGhCe) on SphereA4',
#   'Linf(NormalizedGhCe) on CylinderSMA0.0',
#   'Linf(NormalizedGhCe) on FilledCylinderMA0',
#   'Linf(NormalizedGhCe) on SphereC0',
#   'Linf(NormalizedGhCe) on SphereC1',
#   'Linf(NormalizedGhCe) on SphereC2',
#   'Linf(NormalizedGhCe) on SphereC4',
#   'Linf(NormalizedGhCe) on SphereC8',
#   'Linf(NormalizedGhCe) on SphereC12',
#   'Linf(NormalizedGhCe) on SphereC16',
#   'Linf(NormalizedGhCe) on SphereC20',
#   'Linf(NormalizedGhCe) on SphereC24',
#   'Linf(NormalizedGhCe) on SphereC28',
#   ]
# y_axis_list = [f'Linf(1Con{v}) on SphereC0' for v in ['t','x','y','z']]
# y_axis_list = [f'Linf(sqrt(kappaErr^2)) on SphereC{i}' for i in range(0,12)]
# y_axis_list = [f'Linf(NormalizedGhCe) on SphereC{i}' for i in range(5,45,10)]
# y_axis_list = [f'Linf(NormalizedGhCe) on SphereA{i}' for i in range(0,5)]
# y_axis_list = [f'Linf(GhCe) on SphereA{i}' for i in range(0,5)]
# y_axis_list = [f'Linf(GhCe) on SphereC{i}' for i in range(0,45,2)]
# y_axis_list = [f'Linf(GhCe) on SphereC{i}' for i in range(0,45,3)]
# y_axis_list = ['MinimumGridSpacing[CylinderSMA0.0]','MinimumGridSpacing[FilledCylinderMA0]','MinimumGridSpacing[SphereA0]']
# y_axis_list = [i for i in column_names if ('SphereA' in i)]
# y_axis_inc_list = [f"SphereC{i}$" for i in range(0,45,5)]
# y_axis_list = []
# for col in column_names:
#   for i in y_axis_inc_list :
#     if re.search(i,col):
#       y_axis_list.append(col)
# print(y_axis_list)
# y_axis_list = column_names

# if "GhCe" in y_axis:
plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)

# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x",alpha=0.4)
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label)
# save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
# save_path = "/groups/sxs/hchaudha/rough/plots/"
# save_path = "/home/hchaudha/notes/spec_accuracy/figures/"
# save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/"
# save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/L15_no_tol/"
legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

# legend_dict = {
#     'high_accuracy_L1_main':"Old Level 1",
#     'high_accuracy_L2_main':"Old Level 2",
#     'high_accuracy_L3_main':"Old Level 3",
#     'high_accuracy_L4_main':"Old Level 4",
#     'high_accuracy_L5_main':"Old Level 5",
#     '6_set1_L6s1':'New Level 1',
#     '6_set1_L6s2':'New Level 2',
#     '6_set1_L6s3':'New Level 3',
#     '6_set1_L6s4':'New Level 4',
#     '6_set1_L6s5':'New Level 5',
#     'high_accuracy_L1':"New Level 1",
#     'high_accuracy_L2':"New Level 2",
#     'high_accuracy_L3':"New Level 3",
#     'high_accuracy_L4':"New Level 4",
#     'high_accuracy_L5':"New Level 5",
#  }


# take_abs,modification_function = True,noise_function
# take_abs,modification_function = False,noise_function
# take_abs,modification_function = True,lambda x, y, df, y_axis: noise_function(x, y, df, y_axis, scipy_or_np='np', window=10)
# take_abs,modification_function = False,derivative_function
# take_abs,modification_function = True,compute_center
# take_abs,modification_function = False,compute_center
# take_abs,modification_function = False,compute_dt_center
# take_abs,modification_function = True,min_max_r_ratio
# take_abs,modification_function = False, lambda x, y, df, y_axis: get_drift_from_center(x, y, df, y_axis)
# take_abs,modification_function = True, lambda x, y, df, y_axis: index_constraints_norm(x, y, df, y_axis, index_num=3,norm="Linf")
# take_abs,modification_function = True, lambda x, y, df, y_axis: get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r".*", print_sd_names=False)
# take_abs,modification_function = True, lambda x, y, df, y_axis: get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r'^(?!.*SphereC).*', print_sd_names=False)
# take_abs,modification_function = True, lambda x, y, df, y_axis: get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r'SphereC.*', print_sd_names=False)

# if 'Horizons.h5@' in data_file_path:
#   append_to_title += " HorizonBH="+data_file_path.split('@')[-1]
if 'AhA' in data_file_path:
    append_to_title += " AhA"
if 'AhB' in data_file_path:
    append_to_title += " AhB"


# with plt.style.context('default'):
with plt.style.context('ggplot'):
# sd = 'SphereC6'
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:20]
    # colors = plt.cm.tab10.colors
    line_styles = ['-','--',':',  '-.']
    combined_cycler = cycler(linestyle=line_styles)*cycler(color=colors[:7])
    # combined_cycler = cycler(color=colors)*cycler(linestyle=line_styles[:3])
    plt.rcParams['axes.prop_cycle'] = combined_cycler
#   plt.rcParams["figure.figsize"] = (15,10)
    # plt.rcParams["figure.figsize"] = (10,8)
    plt.rcParams["figure.figsize"] = (8,6)
    # plt.rcParams["figure.figsize"] = (6,5)
    plt.rcParams["figure.autolayout"] = True
    # plt.ylim(1e-10,5e-6)
    if y_axis_list is None:
        plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)
    else:
        plot_graph_for_runs_wrapper(runs_data_dict, x_axis, y_axis_list, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)

#   plt.title("")
#   plt.ylabel("Constraint Violations near black holes")
#   plt.tight_layout()
    # plt.legend(loc='upper left')
    # plt.ylim(1e-14, 1e-7)
    # plt.ylim(1e-20, 1e-13)
    # save_name = f"L16_set1_ArealMass_Shift_{constant_shift_val_time}.pdf"
    # save_name = f"Deriv_ArealMass_L16_main.pdf"

    # save_name = Path(f"/home/hchaudha/code_for_paper_plots/spec_accuracy/{save_name}")
    # if save_name.exists():
    #     raise Exception("Change name")
    # plt.xscale('log')
    # plt.legend("")
    # plt.yscale('symlog',linthresh=1e-12)
    # plt.yscale('symlog',linthresh=1e-6)
    # plt.yscale('symlog',linthresh=1e-10)
    plt.yscale('log')
    plt.tight_layout()
    # plt.savefig(save_name,dpi=300)
    plt.show()

### Interpolated vars vs time

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d


def interpolate_df_at_time(
    df, t, time_column="t(M)", kind="cubic", fill_value="None"
):
    """
    Interpolate all column values in a dataframe at a specific time point using scipy.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with time column and value columns
    t : float
        Time point at which to interpolate
    time_column : str, default='t(M)'
        Name of the time column
    kind : str or int, default='linear'
        Specifies the kind of interpolation as a string or integer.
        Options include:
        - 'linear': linear interpolation (default)
        - 'nearest': nearest neighbor interpolation
        - 'nearest-up': nearest neighbor interpolation (round up)
        - 'zero': zero-order hold (piecewise constant)
        - 'slinear': first-order spline (same as 'linear')
        - 'quadratic': second-order spline
        - 'cubic': third-order spline
        - 'previous': previous value (step function)
        - 'next': next value (step function)
        - integer: order of spline interpolation (1-5)
    fill_value : str or float, default='extrapolate'
        How to handle values outside the data range:
        - 'extrapolate': extrapolate beyond the data range
        - float or tuple: use this value for extrapolation
        - None: raise error if outside bounds (scipy default)

    Returns:
    --------
    pandas.Series
        Series containing interpolated values for all columns (except time column)

    Raises:
    -------
    ValueError
        If time column not found or if t is outside bounds (when fill_value=None)
    """
    # Check if time column exists
    if time_column not in df.columns:
        raise ValueError(f"Time column '{time_column}' not found in dataframe")

    # Sort dataframe by time if not already sorted
    df_sorted = df.sort_values(time_column).reset_index(drop=True)

    # Get time values
    times = df_sorted[time_column].values

    # Interpolate all columns except the time column
    interpolated_values = {}
    for col in df_sorted.columns:
        if col != time_column:
            values = df_sorted[col].values

            # Create interpolation function
            f = interp1d(
                times,
                values,
                kind=kind,
                fill_value=fill_value,
                bounds_error=(fill_value is None),
            )

            # Interpolate at time t
            interpolated_values[col] = float(f(t))

    return pd.Series(interpolated_values)


In [ ]:
points_list_Z = ["(0,0,0)","(0,0,2)","(0,0,4)","(0,0,6)","(0,0,8)","(0,0,10)","(0,0,15)","(0,0,20)","(0,0,30)","(0,0,50)","(0,0,100)","(0,0,200)","(0,0,300)","(0,0,400)","(0,0,500)","(0,0,600)","(0,0,700)","(0,0,800)","(0,0,900)",]
points_list_X = ["(30,0,0)","(50,0,0)","(100,0,0)","(200,0,0)","(300,0,0)","(400,0,0)","(500,0,0)","(600,0,0)","(700,0,0)","(800,0,0)","(900,0,0)",]

points_list_X = ["(1,0,0)","(2,0,0)","(3,0,0)","(4,0,0)","(5,0,0)","(6,0,0)","(7,0,0)","(8,0,0)","(9,0,0)","(10,0,0)","(11,0,0)","(12,0,0)","(13,0,0)","(14,0,0)","(15,0,0)","(16,0,0)","(17,0,0)","(18,0,0)","(19,0,0)","(20,0,0)","(21,0,0)","(22,0,0)","(23,0,0)","(24,0,0)","(25,0,0)","(26,0,0)","(27,0,0)","(28,0,0)","(29,0,0)","(30,0,0)","(31,0,0)","(32,0,0)","(33,0,0)","(34,0,0)","(35,0,0)","(36,0,0)","(37,0,0)","(38,0,0)","(39,0,0)","(40,0,0)","(41,0,0)","(42,0,0)","(43,0,0)","(44,0,0)","(45,0,0)","(46,0,0)","(47,0,0)","(48,0,0)","(49,0,0)","(50,0,0)","(51,0,0)","(52,0,0)","(53,0,0)","(54,0,0)","(55,0,0)","(56,0,0)","(57,0,0)","(58,0,0)","(59,0,0)","(60,0,0)","(61,0,0)","(62,0,0)","(63,0,0)","(64,0,0)","(65,0,0)","(66,0,0)","(67,0,0)","(68,0,0)","(69,0,0)","(70,0,0)","(71,0,0)","(72,0,0)","(73,0,0)","(74,0,0)","(75,0,0)","(76,0,0)","(77,0,0)","(78,0,0)","(79,0,0)","(80,0,0)","(81,0,0)","(82,0,0)","(83,0,0)","(84,0,0)","(85,0,0)","(86,0,0)","(87,0,0)","(88,0,0)","(89,0,0)","(90,0,0)","(91,0,0)","(92,0,0)","(93,0,0)","(94,0,0)","(95,0,0)","(96,0,0)","(97,0,0)","(98,0,0)","(99,0,0)","(100,0,0)","(105,0,0)","(110,0,0)","(115,0,0)","(120,0,0)","(125,0,0)","(130,0,0)","(135,0,0)","(140,0,0)","(145,0,0)","(150,0,0)","(155,0,0)","(160,0,0)","(165,0,0)","(170,0,0)","(175,0,0)","(180,0,0)","(185,0,0)","(190,0,0)","(195,0,0)","(200,0,0)","(205,0,0)","(210,0,0)","(215,0,0)","(220,0,0)","(225,0,0)","(230,0,0)","(235,0,0)","(240,0,0)","(245,0,0)","(250,0,0)","(255,0,0)","(260,0,0)","(265,0,0)","(270,0,0)","(275,0,0)","(280,0,0)","(285,0,0)","(290,0,0)","(295,0,0)","(300,0,0)","(305,0,0)","(310,0,0)","(315,0,0)","(320,0,0)","(325,0,0)","(330,0,0)","(335,0,0)","(340,0,0)","(345,0,0)","(350,0,0)","(355,0,0)","(360,0,0)","(365,0,0)","(370,0,0)","(375,0,0)","(380,0,0)","(385,0,0)","(390,0,0)","(395,0,0)","(400,0,0)","(405,0,0)","(410,0,0)","(415,0,0)","(420,0,0)","(425,0,0)","(430,0,0)","(435,0,0)","(440,0,0)","(445,0,0)","(450,0,0)","(455,0,0)","(460,0,0)","(465,0,0)","(470,0,0)","(475,0,0)","(480,0,0)","(485,0,0)","(490,0,0)","(495,0,0)","(500,0,0)","(505,0,0)","(510,0,0)","(515,0,0)","(520,0,0)","(525,0,0)","(530,0,0)","(535,0,0)","(540,0,0)","(545,0,0)","(550,0,0)","(555,0,0)","(560,0,0)","(565,0,0)","(570,0,0)","(575,0,0)","(580,0,0)","(585,0,0)","(590,0,0)","(595,0,0)","(600,0,0)","(605,0,0)","(610,0,0)","(615,0,0)","(620,0,0)","(625,0,0)","(630,0,0)","(635,0,0)","(640,0,0)","(645,0,0)","(650,0,0)","(655,0,0)","(660,0,0)","(665,0,0)","(670,0,0)","(675,0,0)","(680,0,0)","(685,0,0)","(690,0,0)","(695,0,0)","(700,0,0)","(705,0,0)","(710,0,0)","(715,0,0)","(720,0,0)","(725,0,0)","(730,0,0)","(735,0,0)","(740,0,0)","(745,0,0)","(750,0,0)","(755,0,0)","(760,0,0)","(765,0,0)","(770,0,0)","(775,0,0)","(780,0,0)","(785,0,0)","(790,0,0)","(795,0,0)","(800,0,0)","(805,0,0)","(810,0,0)","(815,0,0)","(820,0,0)","(825,0,0)","(830,0,0)","(835,0,0)","(840,0,0)","(845,0,0)","(850,0,0)","(855,0,0)","(860,0,0)","(865,0,0)","(870,0,0)","(875,0,0)","(880,0,0)","(885,0,0)","(890,0,0)","(895,0,0)","(900,0,0)","(905,0,0)","(910,0,0)","(915,0,0)","(920,0,0)","(925,0,0)","(930,0,0)","(935,0,0)","(940,0,0)","(945,0,0)","(950,0,0)"]
points_list_Y = ["(0,1,0)","(0,2,0)","(0,3,0)","(0,4,0)","(0,5,0)","(0,6,0)","(0,7,0)","(0,8,0)","(0,9,0)","(0,10,0)","(0,11,0)","(0,12,0)","(0,13,0)","(0,14,0)","(0,15,0)","(0,16,0)","(0,17,0)","(0,18,0)","(0,19,0)","(0,20,0)","(0,21,0)","(0,22,0)","(0,23,0)","(0,24,0)","(0,25,0)","(0,26,0)","(0,27,0)","(0,28,0)","(0,29,0)","(0,30,0)","(0,31,0)","(0,32,0)","(0,33,0)","(0,34,0)","(0,35,0)","(0,36,0)","(0,37,0)","(0,38,0)","(0,39,0)","(0,40,0)","(0,41,0)","(0,42,0)","(0,43,0)","(0,44,0)","(0,45,0)","(0,46,0)","(0,47,0)","(0,48,0)","(0,49,0)","(0,50,0)","(0,51,0)","(0,52,0)","(0,53,0)","(0,54,0)","(0,55,0)","(0,56,0)","(0,57,0)","(0,58,0)","(0,59,0)","(0,60,0)","(0,61,0)","(0,62,0)","(0,63,0)","(0,64,0)","(0,65,0)","(0,66,0)","(0,67,0)","(0,68,0)","(0,69,0)","(0,70,0)","(0,71,0)","(0,72,0)","(0,73,0)","(0,74,0)","(0,75,0)","(0,76,0)","(0,77,0)","(0,78,0)","(0,79,0)","(0,80,0)","(0,81,0)","(0,82,0)","(0,83,0)","(0,84,0)","(0,85,0)","(0,86,0)","(0,87,0)","(0,88,0)","(0,89,0)","(0,90,0)","(0,91,0)","(0,92,0)","(0,93,0)","(0,94,0)","(0,95,0)","(0,96,0)","(0,97,0)","(0,98,0)","(0,99,0)","(0,100,0)","(0,105,0)","(0,110,0)","(0,115,0)","(0,120,0)","(0,125,0)","(0,130,0)","(0,135,0)","(0,140,0)","(0,145,0)","(0,150,0)","(0,155,0)","(0,160,0)","(0,165,0)","(0,170,0)","(0,175,0)","(0,180,0)","(0,185,0)","(0,190,0)","(0,195,0)","(0,200,0)","(0,205,0)","(0,210,0)","(0,215,0)","(0,220,0)","(0,225,0)","(0,230,0)","(0,235,0)","(0,240,0)","(0,245,0)","(0,250,0)","(0,255,0)","(0,260,0)","(0,265,0)","(0,270,0)","(0,275,0)","(0,280,0)","(0,285,0)","(0,290,0)","(0,295,0)","(0,300,0)","(0,305,0)","(0,310,0)","(0,315,0)","(0,320,0)","(0,325,0)","(0,330,0)","(0,335,0)","(0,340,0)","(0,345,0)","(0,350,0)","(0,355,0)","(0,360,0)","(0,365,0)","(0,370,0)","(0,375,0)","(0,380,0)","(0,385,0)","(0,390,0)","(0,395,0)","(0,400,0)","(0,405,0)","(0,410,0)","(0,415,0)","(0,420,0)","(0,425,0)","(0,430,0)","(0,435,0)","(0,440,0)","(0,445,0)","(0,450,0)","(0,455,0)","(0,460,0)","(0,465,0)","(0,470,0)","(0,475,0)","(0,480,0)","(0,485,0)","(0,490,0)","(0,495,0)","(0,500,0)","(0,505,0)","(0,510,0)","(0,515,0)","(0,520,0)","(0,525,0)","(0,530,0)","(0,535,0)","(0,540,0)","(0,545,0)","(0,550,0)","(0,555,0)","(0,560,0)","(0,565,0)","(0,570,0)","(0,575,0)","(0,580,0)","(0,585,0)","(0,590,0)","(0,595,0)","(0,600,0)","(0,605,0)","(0,610,0)","(0,615,0)","(0,620,0)","(0,625,0)","(0,630,0)","(0,635,0)","(0,640,0)","(0,645,0)","(0,650,0)","(0,655,0)","(0,660,0)","(0,665,0)","(0,670,0)","(0,675,0)","(0,680,0)","(0,685,0)","(0,690,0)","(0,695,0)","(0,700,0)","(0,705,0)","(0,710,0)","(0,715,0)","(0,720,0)","(0,725,0)","(0,730,0)","(0,735,0)","(0,740,0)","(0,745,0)","(0,750,0)","(0,755,0)","(0,760,0)","(0,765,0)","(0,770,0)","(0,775,0)","(0,780,0)","(0,785,0)","(0,790,0)","(0,795,0)","(0,800,0)","(0,805,0)","(0,810,0)","(0,815,0)","(0,820,0)","(0,825,0)","(0,830,0)","(0,835,0)","(0,840,0)","(0,845,0)","(0,850,0)","(0,855,0)","(0,860,0)","(0,865,0)","(0,870,0)","(0,875,0)","(0,880,0)","(0,885,0)","(0,890,0)","(0,895,0)","(0,900,0)","(0,905,0)","(0,910,0)","(0,915,0)","(0,920,0)","(0,925,0)","(0,930,0)","(0,935,0)","(0,940,0)","(0,945,0)","(0,950,0)"]
points_list_Z = ["(0,0,1)","(0,0,2)","(0,0,3)","(0,0,4)","(0,0,5)","(0,0,6)","(0,0,7)","(0,0,8)","(0,0,9)","(0,0,10)","(0,0,11)","(0,0,12)","(0,0,13)","(0,0,14)","(0,0,15)","(0,0,16)","(0,0,17)","(0,0,18)","(0,0,19)","(0,0,20)","(0,0,21)","(0,0,22)","(0,0,23)","(0,0,24)","(0,0,25)","(0,0,26)","(0,0,27)","(0,0,28)","(0,0,29)","(0,0,30)","(0,0,31)","(0,0,32)","(0,0,33)","(0,0,34)","(0,0,35)","(0,0,36)","(0,0,37)","(0,0,38)","(0,0,39)","(0,0,40)","(0,0,41)","(0,0,42)","(0,0,43)","(0,0,44)","(0,0,45)","(0,0,46)","(0,0,47)","(0,0,48)","(0,0,49)","(0,0,50)","(0,0,51)","(0,0,52)","(0,0,53)","(0,0,54)","(0,0,55)","(0,0,56)","(0,0,57)","(0,0,58)","(0,0,59)","(0,0,60)","(0,0,61)","(0,0,62)","(0,0,63)","(0,0,64)","(0,0,65)","(0,0,66)","(0,0,67)","(0,0,68)","(0,0,69)","(0,0,70)","(0,0,71)","(0,0,72)","(0,0,73)","(0,0,74)","(0,0,75)","(0,0,76)","(0,0,77)","(0,0,78)","(0,0,79)","(0,0,80)","(0,0,81)","(0,0,82)","(0,0,83)","(0,0,84)","(0,0,85)","(0,0,86)","(0,0,87)","(0,0,88)","(0,0,89)","(0,0,90)","(0,0,91)","(0,0,92)","(0,0,93)","(0,0,94)","(0,0,95)","(0,0,96)","(0,0,97)","(0,0,98)","(0,0,99)","(0,0,100)","(0,0,105)","(0,0,110)","(0,0,115)","(0,0,120)","(0,0,125)","(0,0,130)","(0,0,135)","(0,0,140)","(0,0,145)","(0,0,150)","(0,0,155)","(0,0,160)","(0,0,165)","(0,0,170)","(0,0,175)","(0,0,180)","(0,0,185)","(0,0,190)","(0,0,195)","(0,0,200)","(0,0,205)","(0,0,210)","(0,0,215)","(0,0,220)","(0,0,225)","(0,0,230)","(0,0,235)","(0,0,240)","(0,0,245)","(0,0,250)","(0,0,255)","(0,0,260)","(0,0,265)","(0,0,270)","(0,0,275)","(0,0,280)","(0,0,285)","(0,0,290)","(0,0,295)","(0,0,300)","(0,0,305)","(0,0,310)","(0,0,315)","(0,0,320)","(0,0,325)","(0,0,330)","(0,0,335)","(0,0,340)","(0,0,345)","(0,0,350)","(0,0,355)","(0,0,360)","(0,0,365)","(0,0,370)","(0,0,375)","(0,0,380)","(0,0,385)","(0,0,390)","(0,0,395)","(0,0,400)","(0,0,405)","(0,0,410)","(0,0,415)","(0,0,420)","(0,0,425)","(0,0,430)","(0,0,435)","(0,0,440)","(0,0,445)","(0,0,450)","(0,0,455)","(0,0,460)","(0,0,465)","(0,0,470)","(0,0,475)","(0,0,480)","(0,0,485)","(0,0,490)","(0,0,495)","(0,0,500)","(0,0,505)","(0,0,510)","(0,0,515)","(0,0,520)","(0,0,525)","(0,0,530)","(0,0,535)","(0,0,540)","(0,0,545)","(0,0,550)","(0,0,555)","(0,0,560)","(0,0,565)","(0,0,570)","(0,0,575)","(0,0,580)","(0,0,585)","(0,0,590)","(0,0,595)","(0,0,600)","(0,0,605)","(0,0,610)","(0,0,615)","(0,0,620)","(0,0,625)","(0,0,630)","(0,0,635)","(0,0,640)","(0,0,645)","(0,0,650)","(0,0,655)","(0,0,660)","(0,0,665)","(0,0,670)","(0,0,675)","(0,0,680)","(0,0,685)","(0,0,690)","(0,0,695)","(0,0,700)","(0,0,705)","(0,0,710)","(0,0,715)","(0,0,720)","(0,0,725)","(0,0,730)","(0,0,735)","(0,0,740)","(0,0,745)","(0,0,750)","(0,0,755)","(0,0,760)","(0,0,765)","(0,0,770)","(0,0,775)","(0,0,780)","(0,0,785)","(0,0,790)","(0,0,795)","(0,0,800)","(0,0,805)","(0,0,810)","(0,0,815)","(0,0,820)","(0,0,825)","(0,0,830)","(0,0,835)","(0,0,840)","(0,0,845)","(0,0,850)","(0,0,855)","(0,0,860)","(0,0,865)","(0,0,870)","(0,0,875)","(0,0,880)","(0,0,885)","(0,0,890)","(0,0,895)","(0,0,900)","(0,0,905)","(0,0,910)","(0,0,915)","(0,0,920)","(0,0,925)","(0,0,930)","(0,0,935)","(0,0,940)","(0,0,945)","(0,0,950)"]
points_list_rand = ["(-712, -314, -508)","(-716, -205, 7)","(-243, 856, 328)","(625, 356, 435)","(2, 211, -767)","(-638, -287, 388)","(-619, 592, -163)","(225, 793, 99)","(339, 628, -306)","(-487, -261, 467)","(375, -660, 406)","(593, -635, 362)","(-542, -591, 353)","(-89, 613, -527)","(-565, 290, -326)","(-307, 24, 870)","(-142, -228, 766)","(-784, 109, -305)","(-312, -670, -63)","(-852, 182, -348)","(-493, -16, -640)","(-60, -94, -817)","(-666, -612, 214)","(-78, -289, 843)","(209, 647, 459)","(302, 619, 200)","(19, 438, -710)","(-865, 24, 345)","(-134, 767, 135)","(-623, -475, 529)","(182, -440, 523)","(-398, -458, 470)","(203, -52, -862)","(736, 66, -275)","(-621, -313, -88)","(-89, -154, 814)","(-350, -739, -441)","(-119, -82, -784)","(453, -7, 756)","(-193, 69, 692)","(-622, -139, 600)","(-193, -29, -822)","(490, -206, -552)","(-835, -84, -164)","(-125, 742, 445)","(-33, 106, -869)","(239, -688, 173)","(-408, -130, 731)","(-460, -550, 522)","(237, 799, -370),);"]

points_list_X = ["(0,0,0)","(1,0,0)","(2,0,0)","(3,0,0)","(4,0,0)","(5,0,0)","(6,0,0)","(7,0,0)","(8,0,0)","(9,0,0)","(10,0,0)","(11,0,0)","(12,0,0)","(13,0,0)","(14,0,0)","(15,0,0)","(16,0,0)","(17,0,0)","(18,0,0)","(19,0,0)","(20,0,0)","(21,0,0)","(22,0,0)","(23,0,0)","(24,0,0)","(25,0,0)","(26,0,0)","(27,0,0)","(28,0,0)","(29,0,0)","(30,0,0)","(31,0,0)","(32,0,0)","(33,0,0)","(34,0,0)","(35,0,0)","(36,0,0)","(37,0,0)","(38,0,0)","(39,0,0)","(40,0,0)","(41,0,0)","(42,0,0)","(43,0,0)","(44,0,0)","(45,0,0)","(46,0,0)","(47,0,0)","(48,0,0)","(49,0,0)","(50,0,0)","(51,0,0)","(52,0,0)","(53,0,0)","(54,0,0)","(55,0,0)","(56,0,0)","(57,0,0)","(58,0,0)","(59,0,0)","(60,0,0)","(61,0,0)","(62,0,0)","(63,0,0)","(64,0,0)","(65,0,0)","(66,0,0)","(67,0,0)","(68,0,0)","(69,0,0)","(70,0,0)","(71,0,0)","(72,0,0)","(73,0,0)","(74,0,0)","(75,0,0)","(76,0,0)","(77,0,0)","(78,0,0)","(79,0,0)","(80,0,0)","(81,0,0)","(82,0,0)","(83,0,0)","(84,0,0)","(85,0,0)","(86,0,0)","(87,0,0)","(88,0,0)","(89,0,0)","(90,0,0)","(91,0,0)","(92,0,0)","(93,0,0)","(94,0,0)","(95,0,0)","(96,0,0)","(97,0,0)","(98,0,0)","(99,0,0)","(100,0,0)","(101,0,0)","(102,0,0)","(103,0,0)","(104,0,0)","(105,0,0)","(106,0,0)","(107,0,0)","(108,0,0)","(109,0,0)","(110,0,0)","(111,0,0)","(112,0,0)","(113,0,0)","(114,0,0)","(115,0,0)","(116,0,0)","(117,0,0)","(118,0,0)","(119,0,0)","(120,0,0)","(121,0,0)","(122,0,0)","(123,0,0)","(124,0,0)","(125,0,0)","(126,0,0)","(127,0,0)","(128,0,0)","(129,0,0)","(130,0,0)","(131,0,0)","(132,0,0)","(133,0,0)","(134,0,0)","(135,0,0)","(136,0,0)","(137,0,0)","(138,0,0)","(139,0,0)","(140,0,0)","(141,0,0)","(142,0,0)","(143,0,0)","(144,0,0)","(145,0,0)","(146,0,0)","(147,0,0)","(148,0,0)","(149,0,0)","(150,0,0)","(151,0,0)","(152,0,0)","(153,0,0)","(154,0,0)","(155,0,0)","(156,0,0)","(157,0,0)","(158,0,0)","(159,0,0)","(160,0,0)","(161,0,0)","(162,0,0)","(163,0,0)","(164,0,0)","(165,0,0)","(166,0,0)","(167,0,0)","(168,0,0)","(169,0,0)","(170,0,0)","(171,0,0)","(172,0,0)","(173,0,0)","(174,0,0)","(175,0,0)","(176,0,0)","(177,0,0)","(178,0,0)","(179,0,0)","(180,0,0)","(181,0,0)","(182,0,0)","(183,0,0)","(184,0,0)","(185,0,0)","(186,0,0)","(187,0,0)","(188,0,0)","(189,0,0)","(190,0,0)","(191,0,0)","(192,0,0)","(193,0,0)","(194,0,0)","(195,0,0)","(196,0,0)","(197,0,0)","(198,0,0)","(199,0,0)","(200,0,0)","(201,0,0)","(202,0,0)","(203,0,0)","(204,0,0)","(205,0,0)","(206,0,0)","(207,0,0)","(208,0,0)","(209,0,0)","(210,0,0)","(211,0,0)","(212,0,0)","(213,0,0)","(214,0,0)","(215,0,0)","(216,0,0)","(217,0,0)","(218,0,0)","(219,0,0)","(220,0,0)","(221,0,0)","(222,0,0)","(223,0,0)","(224,0,0)","(225,0,0)","(226,0,0)","(227,0,0)","(228,0,0)","(229,0,0)","(230,0,0)","(231,0,0)","(232,0,0)","(233,0,0)","(234,0,0)","(235,0,0)","(236,0,0)","(237,0,0)","(238,0,0)","(239,0,0)","(240,0,0)","(241,0,0)","(242,0,0)","(243,0,0)","(244,0,0)","(245,0,0)","(246,0,0)","(247,0,0)","(248,0,0)","(249,0,0)","(250,0,0)","(251,0,0)","(252,0,0)","(253,0,0)","(254,0,0)","(255,0,0)","(256,0,0)","(257,0,0)","(258,0,0)","(259,0,0)","(260,0,0)","(261,0,0)","(262,0,0)","(263,0,0)","(264,0,0)","(265,0,0)","(266,0,0)","(267,0,0)","(268,0,0)","(269,0,0)","(270,0,0)","(271,0,0)","(272,0,0)","(273,0,0)","(274,0,0)","(275,0,0)","(276,0,0)","(277,0,0)","(278,0,0)","(279,0,0)","(280,0,0)","(281,0,0)","(282,0,0)","(283,0,0)","(284,0,0)","(285,0,0)","(286,0,0)","(287,0,0)","(288,0,0)","(289,0,0)","(290,0,0)","(291,0,0)","(292,0,0)","(293,0,0)","(294,0,0)","(295,0,0)","(296,0,0)","(297,0,0)","(298,0,0)","(299,0,0)","(300,0,0)","(301,0,0)","(302,0,0)","(303,0,0)","(304,0,0)","(305,0,0)","(306,0,0)","(307,0,0)","(308,0,0)","(309,0,0)","(310,0,0)","(311,0,0)","(312,0,0)","(313,0,0)","(314,0,0)","(315,0,0)","(316,0,0)","(317,0,0)","(318,0,0)","(319,0,0)","(320,0,0)","(321,0,0)","(322,0,0)","(323,0,0)","(324,0,0)","(325,0,0)","(326,0,0)","(327,0,0)","(328,0,0)","(329,0,0)","(330,0,0)","(331,0,0)","(332,0,0)","(333,0,0)","(334,0,0)","(335,0,0)","(336,0,0)","(337,0,0)","(338,0,0)","(339,0,0)","(340,0,0)","(341,0,0)","(342,0,0)","(343,0,0)","(344,0,0)","(345,0,0)","(346,0,0)","(347,0,0)","(348,0,0)","(349,0,0)","(350,0,0)","(351,0,0)","(352,0,0)","(353,0,0)","(354,0,0)","(355,0,0)","(356,0,0)","(357,0,0)","(358,0,0)","(359,0,0)","(360,0,0)","(361,0,0)","(362,0,0)","(363,0,0)","(364,0,0)","(365,0,0)","(366,0,0)","(367,0,0)","(368,0,0)","(369,0,0)","(370,0,0)","(371,0,0)","(372,0,0)","(373,0,0)","(374,0,0)","(375,0,0)","(376,0,0)","(377,0,0)","(378,0,0)","(379,0,0)","(380,0,0)","(381,0,0)","(382,0,0)","(383,0,0)","(384,0,0)","(385,0,0)","(386,0,0)","(387,0,0)","(388,0,0)","(389,0,0)","(390,0,0)","(391,0,0)","(392,0,0)","(393,0,0)","(394,0,0)","(395,0,0)","(396,0,0)","(397,0,0)","(398,0,0)","(399,0,0)","(400,0,0)","(401,0,0)","(402,0,0)","(403,0,0)","(404,0,0)","(405,0,0)","(406,0,0)","(407,0,0)","(408,0,0)","(409,0,0)","(410,0,0)","(411,0,0)","(412,0,0)","(413,0,0)","(414,0,0)","(415,0,0)","(416,0,0)","(417,0,0)","(418,0,0)","(419,0,0)","(420,0,0)","(421,0,0)","(422,0,0)","(423,0,0)","(424,0,0)","(425,0,0)","(426,0,0)","(427,0,0)","(428,0,0)","(429,0,0)","(430,0,0)","(431,0,0)","(432,0,0)","(433,0,0)","(434,0,0)","(435,0,0)","(436,0,0)","(437,0,0)","(438,0,0)","(439,0,0)","(440,0,0)","(441,0,0)","(442,0,0)","(443,0,0)","(444,0,0)","(445,0,0)","(446,0,0)","(447,0,0)","(448,0,0)","(449,0,0)","(450,0,0)","(451,0,0)","(452,0,0)","(453,0,0)","(454,0,0)","(455,0,0)","(456,0,0)","(457,0,0)","(458,0,0)","(459,0,0)","(460,0,0)","(461,0,0)","(462,0,0)","(463,0,0)","(464,0,0)","(465,0,0)","(466,0,0)","(467,0,0)","(468,0,0)","(469,0,0)","(470,0,0)","(471,0,0)","(472,0,0)","(473,0,0)","(474,0,0)","(475,0,0)","(476,0,0)","(477,0,0)","(478,0,0)","(479,0,0)","(480,0,0)","(481,0,0)","(482,0,0)","(483,0,0)","(484,0,0)","(485,0,0)","(486,0,0)","(487,0,0)","(488,0,0)","(489,0,0)","(490,0,0)","(491,0,0)","(492,0,0)","(493,0,0)","(494,0,0)","(495,0,0)","(496,0,0)","(497,0,0)","(498,0,0)","(499,0,0)","(500,0,0)","(501,0,0)","(502,0,0)","(503,0,0)","(504,0,0)","(505,0,0)","(506,0,0)","(507,0,0)","(508,0,0)","(509,0,0)","(510,0,0)","(511,0,0)","(512,0,0)","(513,0,0)","(514,0,0)","(515,0,0)","(516,0,0)","(517,0,0)","(518,0,0)","(519,0,0)","(520,0,0)","(521,0,0)","(522,0,0)","(523,0,0)","(524,0,0)","(525,0,0)","(526,0,0)","(527,0,0)","(528,0,0)","(529,0,0)","(530,0,0)","(531,0,0)","(532,0,0)","(533,0,0)","(534,0,0)","(535,0,0)","(536,0,0)","(537,0,0)","(538,0,0)","(539,0,0)","(540,0,0)","(541,0,0)","(542,0,0)","(543,0,0)","(544,0,0)","(545,0,0)","(546,0,0)","(547,0,0)","(548,0,0)","(549,0,0)","(550,0,0)","(551,0,0)","(552,0,0)","(553,0,0)","(554,0,0)","(555,0,0)","(556,0,0)","(557,0,0)","(558,0,0)","(559,0,0)","(560,0,0)","(561,0,0)","(562,0,0)","(563,0,0)","(564,0,0)","(565,0,0)","(566,0,0)","(567,0,0)","(568,0,0)","(569,0,0)","(570,0,0)","(571,0,0)","(572,0,0)","(573,0,0)","(574,0,0)","(575,0,0)","(576,0,0)","(577,0,0)","(578,0,0)","(579,0,0)","(580,0,0)","(581,0,0)","(582,0,0)","(583,0,0)","(584,0,0)","(585,0,0)","(586,0,0)","(587,0,0)","(588,0,0)","(589,0,0)","(590,0,0)","(591,0,0)","(592,0,0)","(593,0,0)","(594,0,0)","(595,0,0)","(596,0,0)","(597,0,0)","(598,0,0)","(599,0,0)","(600,0,0)","(601,0,0)","(602,0,0)","(603,0,0)","(604,0,0)","(605,0,0)","(606,0,0)","(607,0,0)","(608,0,0)","(609,0,0)","(610,0,0)","(611,0,0)","(612,0,0)","(613,0,0)","(614,0,0)","(615,0,0)","(616,0,0)","(617,0,0)","(618,0,0)","(619,0,0)","(620,0,0)","(621,0,0)","(622,0,0)","(623,0,0)","(624,0,0)","(625,0,0)","(626,0,0)","(627,0,0)","(628,0,0)","(629,0,0)","(630,0,0)","(631,0,0)","(632,0,0)","(633,0,0)","(634,0,0)","(635,0,0)","(636,0,0)","(637,0,0)","(638,0,0)","(639,0,0)","(640,0,0)","(641,0,0)","(642,0,0)","(643,0,0)","(644,0,0)","(645,0,0)","(646,0,0)","(647,0,0)","(648,0,0)","(649,0,0)","(650,0,0)","(651,0,0)","(652,0,0)","(653,0,0)","(654,0,0)","(655,0,0)","(656,0,0)","(657,0,0)","(658,0,0)","(659,0,0)","(660,0,0)","(661,0,0)","(662,0,0)","(663,0,0)","(664,0,0)","(665,0,0)","(666,0,0)","(667,0,0)","(668,0,0)","(669,0,0)","(670,0,0)","(671,0,0)","(672,0,0)","(673,0,0)","(674,0,0)","(675,0,0)","(676,0,0)","(677,0,0)","(678,0,0)","(679,0,0)","(680,0,0)","(681,0,0)","(682,0,0)","(683,0,0)","(684,0,0)","(685,0,0)","(686,0,0)","(687,0,0)","(688,0,0)","(689,0,0)","(690,0,0)","(691,0,0)","(692,0,0)","(693,0,0)","(694,0,0)","(695,0,0)","(696,0,0)","(697,0,0)","(698,0,0)","(699,0,0)","(700,0,0)","(701,0,0)","(702,0,0)","(703,0,0)","(704,0,0)","(705,0,0)","(706,0,0)","(707,0,0)","(708,0,0)","(709,0,0)","(710,0,0)","(711,0,0)","(712,0,0)","(713,0,0)","(714,0,0)","(715,0,0)","(716,0,0)","(717,0,0)","(718,0,0)","(719,0,0)","(720,0,0)","(721,0,0)","(722,0,0)","(723,0,0)","(724,0,0)","(725,0,0)","(726,0,0)","(727,0,0)","(728,0,0)","(729,0,0)","(730,0,0)","(731,0,0)","(732,0,0)","(733,0,0)","(734,0,0)","(735,0,0)","(736,0,0)","(737,0,0)","(738,0,0)","(739,0,0)","(740,0,0)","(741,0,0)","(742,0,0)","(743,0,0)","(744,0,0)","(745,0,0)","(746,0,0)","(747,0,0)","(748,0,0)","(749,0,0)","(750,0,0)","(751,0,0)","(752,0,0)","(753,0,0)","(754,0,0)","(755,0,0)","(756,0,0)","(757,0,0)","(758,0,0)","(759,0,0)","(760,0,0)","(761,0,0)","(762,0,0)","(763,0,0)","(764,0,0)","(765,0,0)","(766,0,0)","(767,0,0)","(768,0,0)","(769,0,0)","(770,0,0)","(771,0,0)","(772,0,0)","(773,0,0)","(774,0,0)","(775,0,0)","(776,0,0)","(777,0,0)","(778,0,0)","(779,0,0)","(780,0,0)","(781,0,0)","(782,0,0)","(783,0,0)","(784,0,0)","(785,0,0)","(786,0,0)","(787,0,0)","(788,0,0)","(789,0,0)","(790,0,0)","(791,0,0)","(792,0,0)","(793,0,0)","(794,0,0)","(795,0,0)","(796,0,0)","(797,0,0)","(798,0,0)","(799,0,0)","(800,0,0)","(801,0,0)","(802,0,0)","(803,0,0)","(804,0,0)","(805,0,0)","(806,0,0)","(807,0,0)","(808,0,0)","(809,0,0)","(810,0,0)","(811,0,0)","(812,0,0)","(813,0,0)","(814,0,0)","(815,0,0)","(816,0,0)","(817,0,0)","(818,0,0)","(819,0,0)","(820,0,0)","(821,0,0)","(822,0,0)","(823,0,0)","(824,0,0)","(825,0,0)","(826,0,0)","(827,0,0)","(828,0,0)","(829,0,0)","(830,0,0)","(831,0,0)","(832,0,0)","(833,0,0)","(834,0,0)","(835,0,0)","(836,0,0)","(837,0,0)","(838,0,0)","(839,0,0)","(840,0,0)","(841,0,0)","(842,0,0)","(843,0,0)","(844,0,0)","(845,0,0)","(846,0,0)","(847,0,0)","(848,0,0)","(849,0,0)","(850,0,0)","(851,0,0)","(852,0,0)","(853,0,0)","(854,0,0)","(855,0,0)","(856,0,0)","(857,0,0)","(858,0,0)","(859,0,0)","(860,0,0)","(861,0,0)","(862,0,0)","(863,0,0)","(864,0,0)","(865,0,0)","(866,0,0)","(867,0,0)","(868,0,0)","(869,0,0)","(870,0,0)","(871,0,0)","(872,0,0)","(873,0,0)","(874,0,0)","(875,0,0)","(876,0,0)","(877,0,0)","(878,0,0)","(879,0,0)","(880,0,0)","(881,0,0)","(882,0,0)","(883,0,0)","(884,0,0)","(885,0,0)","(886,0,0)","(887,0,0)","(888,0,0)","(889,0,0)","(890,0,0)","(891,0,0)","(892,0,0)","(893,0,0)","(894,0,0)","(895,0,0)","(896,0,0)","(897,0,0)","(898,0,0)","(899,0,0)","(900,0,0)","(901,0,0)","(902,0,0)","(903,0,0)","(904,0,0)","(905,0,0)","(906,0,0)","(907,0,0)","(908,0,0)","(909,0,0)","(910,0,0)","(911,0,0)","(912,0,0)","(913,0,0)","(914,0,0)","(915,0,0)","(916,0,0)","(917,0,0)","(918,0,0)","(919,0,0)","(920,0,0)","(921,0,0)","(922,0,0)","(923,0,0)","(924,0,0)","(925,0,0)","(926,0,0)","(927,0,0)","(928,0,0)","(929,0,0)","(930,0,0)","(931,0,0)","(932,0,0)","(933,0,0)","(934,0,0)","(935,0,0)","(936,0,0)","(937,0,0)","(938,0,0)","(939,0,0)","(940,0,0)","(941,0,0)","(942,0,0)","(943,0,0)","(944,0,0)","(945,0,0)","(946,0,0)","(947,0,0)","(948,0,0)","(949,0,0)","(950,0,0)","(951,0,0)"]
points_list_Y = ["(0,0,0)","(0,1,0)","(0,2,0)","(0,3,0)","(0,4,0)","(0,5,0)","(0,6,0)","(0,7,0)","(0,8,0)","(0,9,0)","(0,10,0)","(0,11,0)","(0,12,0)","(0,13,0)","(0,14,0)","(0,15,0)","(0,16,0)","(0,17,0)","(0,18,0)","(0,19,0)","(0,20,0)","(0,21,0)","(0,22,0)","(0,23,0)","(0,24,0)","(0,25,0)","(0,26,0)","(0,27,0)","(0,28,0)","(0,29,0)","(0,30,0)","(0,31,0)","(0,32,0)","(0,33,0)","(0,34,0)","(0,35,0)","(0,36,0)","(0,37,0)","(0,38,0)","(0,39,0)","(0,40,0)","(0,41,0)","(0,42,0)","(0,43,0)","(0,44,0)","(0,45,0)","(0,46,0)","(0,47,0)","(0,48,0)","(0,49,0)","(0,50,0)","(0,51,0)","(0,52,0)","(0,53,0)","(0,54,0)","(0,55,0)","(0,56,0)","(0,57,0)","(0,58,0)","(0,59,0)","(0,60,0)","(0,61,0)","(0,62,0)","(0,63,0)","(0,64,0)","(0,65,0)","(0,66,0)","(0,67,0)","(0,68,0)","(0,69,0)","(0,70,0)","(0,71,0)","(0,72,0)","(0,73,0)","(0,74,0)","(0,75,0)","(0,76,0)","(0,77,0)","(0,78,0)","(0,79,0)","(0,80,0)","(0,81,0)","(0,82,0)","(0,83,0)","(0,84,0)","(0,85,0)","(0,86,0)","(0,87,0)","(0,88,0)","(0,89,0)","(0,90,0)","(0,91,0)","(0,92,0)","(0,93,0)","(0,94,0)","(0,95,0)","(0,96,0)","(0,97,0)","(0,98,0)","(0,99,0)","(0,100,0)","(0,101,0)","(0,102,0)","(0,103,0)","(0,104,0)","(0,105,0)","(0,106,0)","(0,107,0)","(0,108,0)","(0,109,0)","(0,110,0)","(0,111,0)","(0,112,0)","(0,113,0)","(0,114,0)","(0,115,0)","(0,116,0)","(0,117,0)","(0,118,0)","(0,119,0)","(0,120,0)","(0,121,0)","(0,122,0)","(0,123,0)","(0,124,0)","(0,125,0)","(0,126,0)","(0,127,0)","(0,128,0)","(0,129,0)","(0,130,0)","(0,131,0)","(0,132,0)","(0,133,0)","(0,134,0)","(0,135,0)","(0,136,0)","(0,137,0)","(0,138,0)","(0,139,0)","(0,140,0)","(0,141,0)","(0,142,0)","(0,143,0)","(0,144,0)","(0,145,0)","(0,146,0)","(0,147,0)","(0,148,0)","(0,149,0)","(0,150,0)","(0,151,0)","(0,152,0)","(0,153,0)","(0,154,0)","(0,155,0)","(0,156,0)","(0,157,0)","(0,158,0)","(0,159,0)","(0,160,0)","(0,161,0)","(0,162,0)","(0,163,0)","(0,164,0)","(0,165,0)","(0,166,0)","(0,167,0)","(0,168,0)","(0,169,0)","(0,170,0)","(0,171,0)","(0,172,0)","(0,173,0)","(0,174,0)","(0,175,0)","(0,176,0)","(0,177,0)","(0,178,0)","(0,179,0)","(0,180,0)","(0,181,0)","(0,182,0)","(0,183,0)","(0,184,0)","(0,185,0)","(0,186,0)","(0,187,0)","(0,188,0)","(0,189,0)","(0,190,0)","(0,191,0)","(0,192,0)","(0,193,0)","(0,194,0)","(0,195,0)","(0,196,0)","(0,197,0)","(0,198,0)","(0,199,0)","(0,200,0)","(0,201,0)","(0,202,0)","(0,203,0)","(0,204,0)","(0,205,0)","(0,206,0)","(0,207,0)","(0,208,0)","(0,209,0)","(0,210,0)","(0,211,0)","(0,212,0)","(0,213,0)","(0,214,0)","(0,215,0)","(0,216,0)","(0,217,0)","(0,218,0)","(0,219,0)","(0,220,0)","(0,221,0)","(0,222,0)","(0,223,0)","(0,224,0)","(0,225,0)","(0,226,0)","(0,227,0)","(0,228,0)","(0,229,0)","(0,230,0)","(0,231,0)","(0,232,0)","(0,233,0)","(0,234,0)","(0,235,0)","(0,236,0)","(0,237,0)","(0,238,0)","(0,239,0)","(0,240,0)","(0,241,0)","(0,242,0)","(0,243,0)","(0,244,0)","(0,245,0)","(0,246,0)","(0,247,0)","(0,248,0)","(0,249,0)","(0,250,0)","(0,251,0)","(0,252,0)","(0,253,0)","(0,254,0)","(0,255,0)","(0,256,0)","(0,257,0)","(0,258,0)","(0,259,0)","(0,260,0)","(0,261,0)","(0,262,0)","(0,263,0)","(0,264,0)","(0,265,0)","(0,266,0)","(0,267,0)","(0,268,0)","(0,269,0)","(0,270,0)","(0,271,0)","(0,272,0)","(0,273,0)","(0,274,0)","(0,275,0)","(0,276,0)","(0,277,0)","(0,278,0)","(0,279,0)","(0,280,0)","(0,281,0)","(0,282,0)","(0,283,0)","(0,284,0)","(0,285,0)","(0,286,0)","(0,287,0)","(0,288,0)","(0,289,0)","(0,290,0)","(0,291,0)","(0,292,0)","(0,293,0)","(0,294,0)","(0,295,0)","(0,296,0)","(0,297,0)","(0,298,0)","(0,299,0)","(0,300,0)","(0,301,0)","(0,302,0)","(0,303,0)","(0,304,0)","(0,305,0)","(0,306,0)","(0,307,0)","(0,308,0)","(0,309,0)","(0,310,0)","(0,311,0)","(0,312,0)","(0,313,0)","(0,314,0)","(0,315,0)","(0,316,0)","(0,317,0)","(0,318,0)","(0,319,0)","(0,320,0)","(0,321,0)","(0,322,0)","(0,323,0)","(0,324,0)","(0,325,0)","(0,326,0)","(0,327,0)","(0,328,0)","(0,329,0)","(0,330,0)","(0,331,0)","(0,332,0)","(0,333,0)","(0,334,0)","(0,335,0)","(0,336,0)","(0,337,0)","(0,338,0)","(0,339,0)","(0,340,0)","(0,341,0)","(0,342,0)","(0,343,0)","(0,344,0)","(0,345,0)","(0,346,0)","(0,347,0)","(0,348,0)","(0,349,0)","(0,350,0)","(0,351,0)","(0,352,0)","(0,353,0)","(0,354,0)","(0,355,0)","(0,356,0)","(0,357,0)","(0,358,0)","(0,359,0)","(0,360,0)","(0,361,0)","(0,362,0)","(0,363,0)","(0,364,0)","(0,365,0)","(0,366,0)","(0,367,0)","(0,368,0)","(0,369,0)","(0,370,0)","(0,371,0)","(0,372,0)","(0,373,0)","(0,374,0)","(0,375,0)","(0,376,0)","(0,377,0)","(0,378,0)","(0,379,0)","(0,380,0)","(0,381,0)","(0,382,0)","(0,383,0)","(0,384,0)","(0,385,0)","(0,386,0)","(0,387,0)","(0,388,0)","(0,389,0)","(0,390,0)","(0,391,0)","(0,392,0)","(0,393,0)","(0,394,0)","(0,395,0)","(0,396,0)","(0,397,0)","(0,398,0)","(0,399,0)","(0,400,0)","(0,401,0)","(0,402,0)","(0,403,0)","(0,404,0)","(0,405,0)","(0,406,0)","(0,407,0)","(0,408,0)","(0,409,0)","(0,410,0)","(0,411,0)","(0,412,0)","(0,413,0)","(0,414,0)","(0,415,0)","(0,416,0)","(0,417,0)","(0,418,0)","(0,419,0)","(0,420,0)","(0,421,0)","(0,422,0)","(0,423,0)","(0,424,0)","(0,425,0)","(0,426,0)","(0,427,0)","(0,428,0)","(0,429,0)","(0,430,0)","(0,431,0)","(0,432,0)","(0,433,0)","(0,434,0)","(0,435,0)","(0,436,0)","(0,437,0)","(0,438,0)","(0,439,0)","(0,440,0)","(0,441,0)","(0,442,0)","(0,443,0)","(0,444,0)","(0,445,0)","(0,446,0)","(0,447,0)","(0,448,0)","(0,449,0)","(0,450,0)","(0,451,0)","(0,452,0)","(0,453,0)","(0,454,0)","(0,455,0)","(0,456,0)","(0,457,0)","(0,458,0)","(0,459,0)","(0,460,0)","(0,461,0)","(0,462,0)","(0,463,0)","(0,464,0)","(0,465,0)","(0,466,0)","(0,467,0)","(0,468,0)","(0,469,0)","(0,470,0)","(0,471,0)","(0,472,0)","(0,473,0)","(0,474,0)","(0,475,0)","(0,476,0)","(0,477,0)","(0,478,0)","(0,479,0)","(0,480,0)","(0,481,0)","(0,482,0)","(0,483,0)","(0,484,0)","(0,485,0)","(0,486,0)","(0,487,0)","(0,488,0)","(0,489,0)","(0,490,0)","(0,491,0)","(0,492,0)","(0,493,0)","(0,494,0)","(0,495,0)","(0,496,0)","(0,497,0)","(0,498,0)","(0,499,0)","(0,500,0)","(0,501,0)","(0,502,0)","(0,503,0)","(0,504,0)","(0,505,0)","(0,506,0)","(0,507,0)","(0,508,0)","(0,509,0)","(0,510,0)","(0,511,0)","(0,512,0)","(0,513,0)","(0,514,0)","(0,515,0)","(0,516,0)","(0,517,0)","(0,518,0)","(0,519,0)","(0,520,0)","(0,521,0)","(0,522,0)","(0,523,0)","(0,524,0)","(0,525,0)","(0,526,0)","(0,527,0)","(0,528,0)","(0,529,0)","(0,530,0)","(0,531,0)","(0,532,0)","(0,533,0)","(0,534,0)","(0,535,0)","(0,536,0)","(0,537,0)","(0,538,0)","(0,539,0)","(0,540,0)","(0,541,0)","(0,542,0)","(0,543,0)","(0,544,0)","(0,545,0)","(0,546,0)","(0,547,0)","(0,548,0)","(0,549,0)","(0,550,0)","(0,551,0)","(0,552,0)","(0,553,0)","(0,554,0)","(0,555,0)","(0,556,0)","(0,557,0)","(0,558,0)","(0,559,0)","(0,560,0)","(0,561,0)","(0,562,0)","(0,563,0)","(0,564,0)","(0,565,0)","(0,566,0)","(0,567,0)","(0,568,0)","(0,569,0)","(0,570,0)","(0,571,0)","(0,572,0)","(0,573,0)","(0,574,0)","(0,575,0)","(0,576,0)","(0,577,0)","(0,578,0)","(0,579,0)","(0,580,0)","(0,581,0)","(0,582,0)","(0,583,0)","(0,584,0)","(0,585,0)","(0,586,0)","(0,587,0)","(0,588,0)","(0,589,0)","(0,590,0)","(0,591,0)","(0,592,0)","(0,593,0)","(0,594,0)","(0,595,0)","(0,596,0)","(0,597,0)","(0,598,0)","(0,599,0)","(0,600,0)","(0,601,0)","(0,602,0)","(0,603,0)","(0,604,0)","(0,605,0)","(0,606,0)","(0,607,0)","(0,608,0)","(0,609,0)","(0,610,0)","(0,611,0)","(0,612,0)","(0,613,0)","(0,614,0)","(0,615,0)","(0,616,0)","(0,617,0)","(0,618,0)","(0,619,0)","(0,620,0)","(0,621,0)","(0,622,0)","(0,623,0)","(0,624,0)","(0,625,0)","(0,626,0)","(0,627,0)","(0,628,0)","(0,629,0)","(0,630,0)","(0,631,0)","(0,632,0)","(0,633,0)","(0,634,0)","(0,635,0)","(0,636,0)","(0,637,0)","(0,638,0)","(0,639,0)","(0,640,0)","(0,641,0)","(0,642,0)","(0,643,0)","(0,644,0)","(0,645,0)","(0,646,0)","(0,647,0)","(0,648,0)","(0,649,0)","(0,650,0)","(0,651,0)","(0,652,0)","(0,653,0)","(0,654,0)","(0,655,0)","(0,656,0)","(0,657,0)","(0,658,0)","(0,659,0)","(0,660,0)","(0,661,0)","(0,662,0)","(0,663,0)","(0,664,0)","(0,665,0)","(0,666,0)","(0,667,0)","(0,668,0)","(0,669,0)","(0,670,0)","(0,671,0)","(0,672,0)","(0,673,0)","(0,674,0)","(0,675,0)","(0,676,0)","(0,677,0)","(0,678,0)","(0,679,0)","(0,680,0)","(0,681,0)","(0,682,0)","(0,683,0)","(0,684,0)","(0,685,0)","(0,686,0)","(0,687,0)","(0,688,0)","(0,689,0)","(0,690,0)","(0,691,0)","(0,692,0)","(0,693,0)","(0,694,0)","(0,695,0)","(0,696,0)","(0,697,0)","(0,698,0)","(0,699,0)","(0,700,0)","(0,701,0)","(0,702,0)","(0,703,0)","(0,704,0)","(0,705,0)","(0,706,0)","(0,707,0)","(0,708,0)","(0,709,0)","(0,710,0)","(0,711,0)","(0,712,0)","(0,713,0)","(0,714,0)","(0,715,0)","(0,716,0)","(0,717,0)","(0,718,0)","(0,719,0)","(0,720,0)","(0,721,0)","(0,722,0)","(0,723,0)","(0,724,0)","(0,725,0)","(0,726,0)","(0,727,0)","(0,728,0)","(0,729,0)","(0,730,0)","(0,731,0)","(0,732,0)","(0,733,0)","(0,734,0)","(0,735,0)","(0,736,0)","(0,737,0)","(0,738,0)","(0,739,0)","(0,740,0)","(0,741,0)","(0,742,0)","(0,743,0)","(0,744,0)","(0,745,0)","(0,746,0)","(0,747,0)","(0,748,0)","(0,749,0)","(0,750,0)","(0,751,0)","(0,752,0)","(0,753,0)","(0,754,0)","(0,755,0)","(0,756,0)","(0,757,0)","(0,758,0)","(0,759,0)","(0,760,0)","(0,761,0)","(0,762,0)","(0,763,0)","(0,764,0)","(0,765,0)","(0,766,0)","(0,767,0)","(0,768,0)","(0,769,0)","(0,770,0)","(0,771,0)","(0,772,0)","(0,773,0)","(0,774,0)","(0,775,0)","(0,776,0)","(0,777,0)","(0,778,0)","(0,779,0)","(0,780,0)","(0,781,0)","(0,782,0)","(0,783,0)","(0,784,0)","(0,785,0)","(0,786,0)","(0,787,0)","(0,788,0)","(0,789,0)","(0,790,0)","(0,791,0)","(0,792,0)","(0,793,0)","(0,794,0)","(0,795,0)","(0,796,0)","(0,797,0)","(0,798,0)","(0,799,0)","(0,800,0)","(0,801,0)","(0,802,0)","(0,803,0)","(0,804,0)","(0,805,0)","(0,806,0)","(0,807,0)","(0,808,0)","(0,809,0)","(0,810,0)","(0,811,0)","(0,812,0)","(0,813,0)","(0,814,0)","(0,815,0)","(0,816,0)","(0,817,0)","(0,818,0)","(0,819,0)","(0,820,0)","(0,821,0)","(0,822,0)","(0,823,0)","(0,824,0)","(0,825,0)","(0,826,0)","(0,827,0)","(0,828,0)","(0,829,0)","(0,830,0)","(0,831,0)","(0,832,0)","(0,833,0)","(0,834,0)","(0,835,0)","(0,836,0)","(0,837,0)","(0,838,0)","(0,839,0)","(0,840,0)","(0,841,0)","(0,842,0)","(0,843,0)","(0,844,0)","(0,845,0)","(0,846,0)","(0,847,0)","(0,848,0)","(0,849,0)","(0,850,0)","(0,851,0)","(0,852,0)","(0,853,0)","(0,854,0)","(0,855,0)","(0,856,0)","(0,857,0)","(0,858,0)","(0,859,0)","(0,860,0)","(0,861,0)","(0,862,0)","(0,863,0)","(0,864,0)","(0,865,0)","(0,866,0)","(0,867,0)","(0,868,0)","(0,869,0)","(0,870,0)","(0,871,0)","(0,872,0)","(0,873,0)","(0,874,0)","(0,875,0)","(0,876,0)","(0,877,0)","(0,878,0)","(0,879,0)","(0,880,0)","(0,881,0)","(0,882,0)","(0,883,0)","(0,884,0)","(0,885,0)","(0,886,0)","(0,887,0)","(0,888,0)","(0,889,0)","(0,890,0)","(0,891,0)","(0,892,0)","(0,893,0)","(0,894,0)","(0,895,0)","(0,896,0)","(0,897,0)","(0,898,0)","(0,899,0)","(0,900,0)","(0,901,0)","(0,902,0)","(0,903,0)","(0,904,0)","(0,905,0)","(0,906,0)","(0,907,0)","(0,908,0)","(0,909,0)","(0,910,0)","(0,911,0)","(0,912,0)","(0,913,0)","(0,914,0)","(0,915,0)","(0,916,0)","(0,917,0)","(0,918,0)","(0,919,0)","(0,920,0)","(0,921,0)","(0,922,0)","(0,923,0)","(0,924,0)","(0,925,0)","(0,926,0)","(0,927,0)","(0,928,0)","(0,929,0)","(0,930,0)","(0,931,0)","(0,932,0)","(0,933,0)","(0,934,0)","(0,935,0)","(0,936,0)","(0,937,0)","(0,938,0)","(0,939,0)","(0,940,0)","(0,941,0)","(0,942,0)","(0,943,0)","(0,944,0)","(0,945,0)","(0,946,0)","(0,947,0)","(0,948,0)","(0,949,0)","(0,950,0)","(0,951,0)"]
points_list_Z = ["(0,0,0)","(0,0,1)","(0,0,2)","(0,0,3)","(0,0,4)","(0,0,5)","(0,0,6)","(0,0,7)","(0,0,8)","(0,0,9)","(0,0,10)","(0,0,11)","(0,0,12)","(0,0,13)","(0,0,14)","(0,0,15)","(0,0,16)","(0,0,17)","(0,0,18)","(0,0,19)","(0,0,20)","(0,0,21)","(0,0,22)","(0,0,23)","(0,0,24)","(0,0,25)","(0,0,26)","(0,0,27)","(0,0,28)","(0,0,29)","(0,0,30)","(0,0,31)","(0,0,32)","(0,0,33)","(0,0,34)","(0,0,35)","(0,0,36)","(0,0,37)","(0,0,38)","(0,0,39)","(0,0,40)","(0,0,41)","(0,0,42)","(0,0,43)","(0,0,44)","(0,0,45)","(0,0,46)","(0,0,47)","(0,0,48)","(0,0,49)","(0,0,50)","(0,0,51)","(0,0,52)","(0,0,53)","(0,0,54)","(0,0,55)","(0,0,56)","(0,0,57)","(0,0,58)","(0,0,59)","(0,0,60)","(0,0,61)","(0,0,62)","(0,0,63)","(0,0,64)","(0,0,65)","(0,0,66)","(0,0,67)","(0,0,68)","(0,0,69)","(0,0,70)","(0,0,71)","(0,0,72)","(0,0,73)","(0,0,74)","(0,0,75)","(0,0,76)","(0,0,77)","(0,0,78)","(0,0,79)","(0,0,80)","(0,0,81)","(0,0,82)","(0,0,83)","(0,0,84)","(0,0,85)","(0,0,86)","(0,0,87)","(0,0,88)","(0,0,89)","(0,0,90)","(0,0,91)","(0,0,92)","(0,0,93)","(0,0,94)","(0,0,95)","(0,0,96)","(0,0,97)","(0,0,98)","(0,0,99)","(0,0,100)","(0,0,101)","(0,0,102)","(0,0,103)","(0,0,104)","(0,0,105)","(0,0,106)","(0,0,107)","(0,0,108)","(0,0,109)","(0,0,110)","(0,0,111)","(0,0,112)","(0,0,113)","(0,0,114)","(0,0,115)","(0,0,116)","(0,0,117)","(0,0,118)","(0,0,119)","(0,0,120)","(0,0,121)","(0,0,122)","(0,0,123)","(0,0,124)","(0,0,125)","(0,0,126)","(0,0,127)","(0,0,128)","(0,0,129)","(0,0,130)","(0,0,131)","(0,0,132)","(0,0,133)","(0,0,134)","(0,0,135)","(0,0,136)","(0,0,137)","(0,0,138)","(0,0,139)","(0,0,140)","(0,0,141)","(0,0,142)","(0,0,143)","(0,0,144)","(0,0,145)","(0,0,146)","(0,0,147)","(0,0,148)","(0,0,149)","(0,0,150)","(0,0,151)","(0,0,152)","(0,0,153)","(0,0,154)","(0,0,155)","(0,0,156)","(0,0,157)","(0,0,158)","(0,0,159)","(0,0,160)","(0,0,161)","(0,0,162)","(0,0,163)","(0,0,164)","(0,0,165)","(0,0,166)","(0,0,167)","(0,0,168)","(0,0,169)","(0,0,170)","(0,0,171)","(0,0,172)","(0,0,173)","(0,0,174)","(0,0,175)","(0,0,176)","(0,0,177)","(0,0,178)","(0,0,179)","(0,0,180)","(0,0,181)","(0,0,182)","(0,0,183)","(0,0,184)","(0,0,185)","(0,0,186)","(0,0,187)","(0,0,188)","(0,0,189)","(0,0,190)","(0,0,191)","(0,0,192)","(0,0,193)","(0,0,194)","(0,0,195)","(0,0,196)","(0,0,197)","(0,0,198)","(0,0,199)","(0,0,200)","(0,0,201)","(0,0,202)","(0,0,203)","(0,0,204)","(0,0,205)","(0,0,206)","(0,0,207)","(0,0,208)","(0,0,209)","(0,0,210)","(0,0,211)","(0,0,212)","(0,0,213)","(0,0,214)","(0,0,215)","(0,0,216)","(0,0,217)","(0,0,218)","(0,0,219)","(0,0,220)","(0,0,221)","(0,0,222)","(0,0,223)","(0,0,224)","(0,0,225)","(0,0,226)","(0,0,227)","(0,0,228)","(0,0,229)","(0,0,230)","(0,0,231)","(0,0,232)","(0,0,233)","(0,0,234)","(0,0,235)","(0,0,236)","(0,0,237)","(0,0,238)","(0,0,239)","(0,0,240)","(0,0,241)","(0,0,242)","(0,0,243)","(0,0,244)","(0,0,245)","(0,0,246)","(0,0,247)","(0,0,248)","(0,0,249)","(0,0,250)","(0,0,251)","(0,0,252)","(0,0,253)","(0,0,254)","(0,0,255)","(0,0,256)","(0,0,257)","(0,0,258)","(0,0,259)","(0,0,260)","(0,0,261)","(0,0,262)","(0,0,263)","(0,0,264)","(0,0,265)","(0,0,266)","(0,0,267)","(0,0,268)","(0,0,269)","(0,0,270)","(0,0,271)","(0,0,272)","(0,0,273)","(0,0,274)","(0,0,275)","(0,0,276)","(0,0,277)","(0,0,278)","(0,0,279)","(0,0,280)","(0,0,281)","(0,0,282)","(0,0,283)","(0,0,284)","(0,0,285)","(0,0,286)","(0,0,287)","(0,0,288)","(0,0,289)","(0,0,290)","(0,0,291)","(0,0,292)","(0,0,293)","(0,0,294)","(0,0,295)","(0,0,296)","(0,0,297)","(0,0,298)","(0,0,299)","(0,0,300)","(0,0,301)","(0,0,302)","(0,0,303)","(0,0,304)","(0,0,305)","(0,0,306)","(0,0,307)","(0,0,308)","(0,0,309)","(0,0,310)","(0,0,311)","(0,0,312)","(0,0,313)","(0,0,314)","(0,0,315)","(0,0,316)","(0,0,317)","(0,0,318)","(0,0,319)","(0,0,320)","(0,0,321)","(0,0,322)","(0,0,323)","(0,0,324)","(0,0,325)","(0,0,326)","(0,0,327)","(0,0,328)","(0,0,329)","(0,0,330)","(0,0,331)","(0,0,332)","(0,0,333)","(0,0,334)","(0,0,335)","(0,0,336)","(0,0,337)","(0,0,338)","(0,0,339)","(0,0,340)","(0,0,341)","(0,0,342)","(0,0,343)","(0,0,344)","(0,0,345)","(0,0,346)","(0,0,347)","(0,0,348)","(0,0,349)","(0,0,350)","(0,0,351)","(0,0,352)","(0,0,353)","(0,0,354)","(0,0,355)","(0,0,356)","(0,0,357)","(0,0,358)","(0,0,359)","(0,0,360)","(0,0,361)","(0,0,362)","(0,0,363)","(0,0,364)","(0,0,365)","(0,0,366)","(0,0,367)","(0,0,368)","(0,0,369)","(0,0,370)","(0,0,371)","(0,0,372)","(0,0,373)","(0,0,374)","(0,0,375)","(0,0,376)","(0,0,377)","(0,0,378)","(0,0,379)","(0,0,380)","(0,0,381)","(0,0,382)","(0,0,383)","(0,0,384)","(0,0,385)","(0,0,386)","(0,0,387)","(0,0,388)","(0,0,389)","(0,0,390)","(0,0,391)","(0,0,392)","(0,0,393)","(0,0,394)","(0,0,395)","(0,0,396)","(0,0,397)","(0,0,398)","(0,0,399)","(0,0,400)","(0,0,401)","(0,0,402)","(0,0,403)","(0,0,404)","(0,0,405)","(0,0,406)","(0,0,407)","(0,0,408)","(0,0,409)","(0,0,410)","(0,0,411)","(0,0,412)","(0,0,413)","(0,0,414)","(0,0,415)","(0,0,416)","(0,0,417)","(0,0,418)","(0,0,419)","(0,0,420)","(0,0,421)","(0,0,422)","(0,0,423)","(0,0,424)","(0,0,425)","(0,0,426)","(0,0,427)","(0,0,428)","(0,0,429)","(0,0,430)","(0,0,431)","(0,0,432)","(0,0,433)","(0,0,434)","(0,0,435)","(0,0,436)","(0,0,437)","(0,0,438)","(0,0,439)","(0,0,440)","(0,0,441)","(0,0,442)","(0,0,443)","(0,0,444)","(0,0,445)","(0,0,446)","(0,0,447)","(0,0,448)","(0,0,449)","(0,0,450)","(0,0,451)","(0,0,452)","(0,0,453)","(0,0,454)","(0,0,455)","(0,0,456)","(0,0,457)","(0,0,458)","(0,0,459)","(0,0,460)","(0,0,461)","(0,0,462)","(0,0,463)","(0,0,464)","(0,0,465)","(0,0,466)","(0,0,467)","(0,0,468)","(0,0,469)","(0,0,470)","(0,0,471)","(0,0,472)","(0,0,473)","(0,0,474)","(0,0,475)","(0,0,476)","(0,0,477)","(0,0,478)","(0,0,479)","(0,0,480)","(0,0,481)","(0,0,482)","(0,0,483)","(0,0,484)","(0,0,485)","(0,0,486)","(0,0,487)","(0,0,488)","(0,0,489)","(0,0,490)","(0,0,491)","(0,0,492)","(0,0,493)","(0,0,494)","(0,0,495)","(0,0,496)","(0,0,497)","(0,0,498)","(0,0,499)","(0,0,500)","(0,0,501)","(0,0,502)","(0,0,503)","(0,0,504)","(0,0,505)","(0,0,506)","(0,0,507)","(0,0,508)","(0,0,509)","(0,0,510)","(0,0,511)","(0,0,512)","(0,0,513)","(0,0,514)","(0,0,515)","(0,0,516)","(0,0,517)","(0,0,518)","(0,0,519)","(0,0,520)","(0,0,521)","(0,0,522)","(0,0,523)","(0,0,524)","(0,0,525)","(0,0,526)","(0,0,527)","(0,0,528)","(0,0,529)","(0,0,530)","(0,0,531)","(0,0,532)","(0,0,533)","(0,0,534)","(0,0,535)","(0,0,536)","(0,0,537)","(0,0,538)","(0,0,539)","(0,0,540)","(0,0,541)","(0,0,542)","(0,0,543)","(0,0,544)","(0,0,545)","(0,0,546)","(0,0,547)","(0,0,548)","(0,0,549)","(0,0,550)","(0,0,551)","(0,0,552)","(0,0,553)","(0,0,554)","(0,0,555)","(0,0,556)","(0,0,557)","(0,0,558)","(0,0,559)","(0,0,560)","(0,0,561)","(0,0,562)","(0,0,563)","(0,0,564)","(0,0,565)","(0,0,566)","(0,0,567)","(0,0,568)","(0,0,569)","(0,0,570)","(0,0,571)","(0,0,572)","(0,0,573)","(0,0,574)","(0,0,575)","(0,0,576)","(0,0,577)","(0,0,578)","(0,0,579)","(0,0,580)","(0,0,581)","(0,0,582)","(0,0,583)","(0,0,584)","(0,0,585)","(0,0,586)","(0,0,587)","(0,0,588)","(0,0,589)","(0,0,590)","(0,0,591)","(0,0,592)","(0,0,593)","(0,0,594)","(0,0,595)","(0,0,596)","(0,0,597)","(0,0,598)","(0,0,599)","(0,0,600)","(0,0,601)","(0,0,602)","(0,0,603)","(0,0,604)","(0,0,605)","(0,0,606)","(0,0,607)","(0,0,608)","(0,0,609)","(0,0,610)","(0,0,611)","(0,0,612)","(0,0,613)","(0,0,614)","(0,0,615)","(0,0,616)","(0,0,617)","(0,0,618)","(0,0,619)","(0,0,620)","(0,0,621)","(0,0,622)","(0,0,623)","(0,0,624)","(0,0,625)","(0,0,626)","(0,0,627)","(0,0,628)","(0,0,629)","(0,0,630)","(0,0,631)","(0,0,632)","(0,0,633)","(0,0,634)","(0,0,635)","(0,0,636)","(0,0,637)","(0,0,638)","(0,0,639)","(0,0,640)","(0,0,641)","(0,0,642)","(0,0,643)","(0,0,644)","(0,0,645)","(0,0,646)","(0,0,647)","(0,0,648)","(0,0,649)","(0,0,650)","(0,0,651)","(0,0,652)","(0,0,653)","(0,0,654)","(0,0,655)","(0,0,656)","(0,0,657)","(0,0,658)","(0,0,659)","(0,0,660)","(0,0,661)","(0,0,662)","(0,0,663)","(0,0,664)","(0,0,665)","(0,0,666)","(0,0,667)","(0,0,668)","(0,0,669)","(0,0,670)","(0,0,671)","(0,0,672)","(0,0,673)","(0,0,674)","(0,0,675)","(0,0,676)","(0,0,677)","(0,0,678)","(0,0,679)","(0,0,680)","(0,0,681)","(0,0,682)","(0,0,683)","(0,0,684)","(0,0,685)","(0,0,686)","(0,0,687)","(0,0,688)","(0,0,689)","(0,0,690)","(0,0,691)","(0,0,692)","(0,0,693)","(0,0,694)","(0,0,695)","(0,0,696)","(0,0,697)","(0,0,698)","(0,0,699)","(0,0,700)","(0,0,701)","(0,0,702)","(0,0,703)","(0,0,704)","(0,0,705)","(0,0,706)","(0,0,707)","(0,0,708)","(0,0,709)","(0,0,710)","(0,0,711)","(0,0,712)","(0,0,713)","(0,0,714)","(0,0,715)","(0,0,716)","(0,0,717)","(0,0,718)","(0,0,719)","(0,0,720)","(0,0,721)","(0,0,722)","(0,0,723)","(0,0,724)","(0,0,725)","(0,0,726)","(0,0,727)","(0,0,728)","(0,0,729)","(0,0,730)","(0,0,731)","(0,0,732)","(0,0,733)","(0,0,734)","(0,0,735)","(0,0,736)","(0,0,737)","(0,0,738)","(0,0,739)","(0,0,740)","(0,0,741)","(0,0,742)","(0,0,743)","(0,0,744)","(0,0,745)","(0,0,746)","(0,0,747)","(0,0,748)","(0,0,749)","(0,0,750)","(0,0,751)","(0,0,752)","(0,0,753)","(0,0,754)","(0,0,755)","(0,0,756)","(0,0,757)","(0,0,758)","(0,0,759)","(0,0,760)","(0,0,761)","(0,0,762)","(0,0,763)","(0,0,764)","(0,0,765)","(0,0,766)","(0,0,767)","(0,0,768)","(0,0,769)","(0,0,770)","(0,0,771)","(0,0,772)","(0,0,773)","(0,0,774)","(0,0,775)","(0,0,776)","(0,0,777)","(0,0,778)","(0,0,779)","(0,0,780)","(0,0,781)","(0,0,782)","(0,0,783)","(0,0,784)","(0,0,785)","(0,0,786)","(0,0,787)","(0,0,788)","(0,0,789)","(0,0,790)","(0,0,791)","(0,0,792)","(0,0,793)","(0,0,794)","(0,0,795)","(0,0,796)","(0,0,797)","(0,0,798)","(0,0,799)","(0,0,800)","(0,0,801)","(0,0,802)","(0,0,803)","(0,0,804)","(0,0,805)","(0,0,806)","(0,0,807)","(0,0,808)","(0,0,809)","(0,0,810)","(0,0,811)","(0,0,812)","(0,0,813)","(0,0,814)","(0,0,815)","(0,0,816)","(0,0,817)","(0,0,818)","(0,0,819)","(0,0,820)","(0,0,821)","(0,0,822)","(0,0,823)","(0,0,824)","(0,0,825)","(0,0,826)","(0,0,827)","(0,0,828)","(0,0,829)","(0,0,830)","(0,0,831)","(0,0,832)","(0,0,833)","(0,0,834)","(0,0,835)","(0,0,836)","(0,0,837)","(0,0,838)","(0,0,839)","(0,0,840)","(0,0,841)","(0,0,842)","(0,0,843)","(0,0,844)","(0,0,845)","(0,0,846)","(0,0,847)","(0,0,848)","(0,0,849)","(0,0,850)","(0,0,851)","(0,0,852)","(0,0,853)","(0,0,854)","(0,0,855)","(0,0,856)","(0,0,857)","(0,0,858)","(0,0,859)","(0,0,860)","(0,0,861)","(0,0,862)","(0,0,863)","(0,0,864)","(0,0,865)","(0,0,866)","(0,0,867)","(0,0,868)","(0,0,869)","(0,0,870)","(0,0,871)","(0,0,872)","(0,0,873)","(0,0,874)","(0,0,875)","(0,0,876)","(0,0,877)","(0,0,878)","(0,0,879)","(0,0,880)","(0,0,881)","(0,0,882)","(0,0,883)","(0,0,884)","(0,0,885)","(0,0,886)","(0,0,887)","(0,0,888)","(0,0,889)","(0,0,890)","(0,0,891)","(0,0,892)","(0,0,893)","(0,0,894)","(0,0,895)","(0,0,896)","(0,0,897)","(0,0,898)","(0,0,899)","(0,0,900)","(0,0,901)","(0,0,902)","(0,0,903)","(0,0,904)","(0,0,905)","(0,0,906)","(0,0,907)","(0,0,908)","(0,0,909)","(0,0,910)","(0,0,911)","(0,0,912)","(0,0,913)","(0,0,914)","(0,0,915)","(0,0,916)","(0,0,917)","(0,0,918)","(0,0,919)","(0,0,920)","(0,0,921)","(0,0,922)","(0,0,923)","(0,0,924)","(0,0,925)","(0,0,926)","(0,0,927)","(0,0,928)","(0,0,929)","(0,0,930)","(0,0,931)","(0,0,932)","(0,0,933)","(0,0,934)","(0,0,935)","(0,0,936)","(0,0,937)","(0,0,938)","(0,0,939)","(0,0,940)","(0,0,941)","(0,0,942)","(0,0,943)","(0,0,944)","(0,0,945)","(0,0,946)","(0,0,947)","(0,0,948)","(0,0,949)","(0,0,950)","(0,0,951)"]

print(runs_data_dict.keys())

In [ ]:
# base_key = '44_Set1L6s6'
# base_key = '44_Set1L6s6_NAMR'
# base_key = '44_Set1L6s6_D'
# base_key = '44_Set1L6s6_NAMRD'
base_key = '44_LineObs_10M_EachStep_Lev1_6'
target_time = 4001
target_time = 4050

# var_comp = "kappa_ttt"
# var_comp = "kappa_ttx"
# var_comp = "kappa_tty"
# var_comp = "kappa_ttz"
var_comp = "kappa_txx"
# var_comp = "kappa_txy"
# var_comp = "kappa_txz"
# var_comp = "kappa_tyy"
# var_comp = "kappa_tyz"
# var_comp = "kappa_tzz"
# var_comp = "kappa_xtt"
# var_comp = "kappa_xtx"
# var_comp = "kappa_xty"
# var_comp = "kappa_xtz"
# var_comp = "kappa_xxx"
# var_comp = "kappa_xxy"
# var_comp = "kappa_xxz"
# var_comp = "kappa_xyy"
# var_comp = "kappa_xyz"
# var_comp = "kappa_xzz"
# var_comp = "kappa_ytt"
# var_comp = "kappa_ytx"
# var_comp = "kappa_yty"
# var_comp = "kappa_ytz"
# var_comp = "kappa_yxx"
# var_comp = "kappa_yxy"
# var_comp = "kappa_yxz"
# var_comp = "kappa_yyy"
# var_comp = "kappa_yyz"
# var_comp = "kappa_yzz"
# var_comp = "kappa_ztt"
# var_comp = "kappa_ztx"
# var_comp = "kappa_zty"
# var_comp = "kappa_ztz"
# var_comp = "kappa_zxx"
# var_comp = "kappa_zxy"
# var_comp = "kappa_zxz"
# var_comp = "kappa_zyy"
# var_comp = "kappa_zyz"
# var_comp = "kappa_zzz"
# var_comp = "psi_tt"
# var_comp = "psi_tx"
# var_comp = "psi_ty"
# var_comp = "psi_tz"
# var_comp = "psi_xx"
# var_comp = "psi_xy"
# var_comp = "psi_xz"
# var_comp = "psi_yy"
# var_comp = "psi_yz"
# var_comp = "psi_zz"

if "@ALL" not in data_file_path:
    var_comp = data_file_path.split("Int_")[-1][:-4]

points_list = points_list_X
# points_list = points_list_Y
# points_list = points_list_Z

# points_list = points_list[:40]

base_data = runs_data_dict[base_key]

diff_series_dict = {}

# First find the closest time in the base key
base_data = runs_data_dict[base_key]
closest_idx = (base_data['t(M)'] - target_time).abs().idxmin()
closest_time = base_data.iloc[closest_idx,0]
print(closest_idx, closest_time)
# save the base value for normalization
diff_series_dict[base_key] = base_data.iloc[closest_idx,1:]

for key in runs_data_dict:
    if key == base_key:
        continue
    # Interpolate other keys to the same time
    data = runs_data_dict[key]
    interpolated_series = interpolate_df_at_time(data, closest_time)
    diff_series_dict[key] = interpolated_series - base_data.iloc[closest_idx,1:]


plot_vals = {}
for key in diff_series_dict:
    data = diff_series_dict[key]
    plot_vals[key] = []
    for point in points_list:
        col_name = f'{var_comp}_{point}'
        plot_vals[key].append(data[col_name])
    # change to numpy arrays for easier processing later
    plot_vals[key] = np.array(plot_vals[key])

# remove the base key from plotting
field_val = plot_vals.pop(base_key)


In [ ]:
normalize = False
# normalize = True
abs = False
# abs = True

plot_field_vals = field_val
with plt.style.context('ggplot'):
    plt.figure(figsize=(7, 6))

    for plot_key in plot_vals:
        y_vals = plot_vals[plot_key]
        if normalize:
            y_vals = y_vals / field_val
        if abs:
            y_vals = np.abs(y_vals)

        plt.plot(range(len(points_list)), y_vals, label=plot_key)
        # plt.plot(points_list, y_vals, label=plot_key)

    if abs:
        plot_field_vals = np.abs(plot_field_vals)
    # plt.plot(range(len(points_list)), plot_field_vals, label=var_comp, color='black', linestyle='--')

    xtick_count = min(10, len(points_list))
    if xtick_count > 0:
        xtick_indices = np.linspace(0, len(points_list) - 1, xtick_count, dtype=int)
        xtick_labels = [points_list[idx] for idx in xtick_indices]
        plt.xticks(xtick_indices, xtick_labels, rotation=90)
    else:
        plt.xticks(rotation=90)
    plt.legend()
    plt.title(f"base: {base_key}, {closest_time:.1f}M, {np.abs(field_val).max():.1e},{np.abs(field_val[field_val != 0]).min():.1e}")
    if normalize:
        plt.ylabel(f"{var_comp} normalized diff")
    else:
        plt.ylabel(f"{var_comp} diff")

plt.yscale('log')
plt.yscale('symlog',linthresh=1e-10)
plt.tight_layout()
plt.show()

### Peak Analysis Function for All Runs

Let's create a comprehensive function to analyze peaks in InertialCenter_x data for all runs in the dictionary.

In [ ]:
def analyze_inertial_center_peaks(runs_dict, plot_results=False, save_plots=False, save_dir=None, 
                                  specific_runs=None, figsize=(14, 8), dpi=100):
    """
    Analyze InertialCenter_x data for all runs in the dictionary to find peaks.
    
    Parameters:
    -----------
    runs_dict : dict
        Dictionary containing run data with InertialCenter_x and t(M) columns
    plot_results : bool, default=False
        Whether to plot the waveforms and peaks for each run
    save_plots : bool, default=False
        Whether to save the plots to files
    save_dir : str, optional
        Directory to save plots (required if save_plots=True)
    specific_runs : list, optional
        List of specific run keys to analyze. If None, analyzes all runs.
    figsize : tuple, default=(14, 8)
        Figure size for plots
    dpi : int, default=100
        DPI for saved plots
        
    Returns:
    --------
    dict : Dictionary with run names as keys and analysis results as values
           Each result contains: num_peaks, peak_times, peak_values, peak_stats
    """
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import find_peaks
    import os
    
    results = {}
    
    # Determine which runs to analyze
    runs_to_analyze = specific_runs if specific_runs is not None else list(runs_dict.keys())
    
    print(f"Analyzing {len(runs_to_analyze)} runs for InertialCenter_x peaks...")
    print("="*60)
    
    for i, run_key in enumerate(runs_to_analyze):
        try:
            # Get the data for this run
            run_data = runs_dict[run_key]
            
            # Check if required columns exist
            if 'InertialCenter_x' not in run_data.columns:
                print(f"Warning: Run '{run_key}' missing 'InertialCenter_x' column. Skipping.")
                continue
            if 't(M)' not in run_data.columns:
                print(f"Warning: Run '{run_key}' missing 't(M)' column. Skipping.")
                continue
                
            # Get time and InertialCenter_x data
            time_data = run_data['t(M)']
            inertial_x_data = run_data['InertialCenter_x']
            
            # Sort by time to ensure proper chronological order
            df_combined = pd.DataFrame({'time': time_data, 'InertialCenter_x': inertial_x_data})
            df_sorted = df_combined.sort_values('time').reset_index(drop=True)
            
            time_sorted = df_sorted['time']
            data_sorted = df_sorted['InertialCenter_x']
            
            # Find peaks in the sorted data
            peaks_indices, _ = find_peaks(data_sorted)
            num_peaks = len(peaks_indices)
            
            # Get peak times and values
            peak_times = time_sorted.iloc[peaks_indices].values
            peak_values = data_sorted.iloc[peaks_indices].values
            
            # Calculate peak statistics
            peak_stats = {
                'max_peak_value': peak_values.max() if len(peak_values) > 0 else np.nan,
                'min_peak_value': peak_values.min() if len(peak_values) > 0 else np.nan,
                'mean_peak_value': peak_values.mean() if len(peak_values) > 0 else np.nan,
                'std_peak_value': peak_values.std() if len(peak_values) > 0 else np.nan,
                'first_peak_time': peak_times[0] if len(peak_times) > 0 else np.nan,
                'last_peak_time': peak_times[-1] if len(peak_times) > 0 else np.nan,
                'total_time_span': time_sorted.max() - time_sorted.min(),
                'data_points': len(data_sorted)
            }
            
            # Store results
            results[run_key] = {
                'num_peaks': num_peaks,
                'peak_times': peak_times,
                'peak_values': peak_values,
                'peak_stats': peak_stats,
                'time_data': time_sorted,
                'inertial_x_data': data_sorted,
                'peak_indices': peaks_indices
            }
            
            # Print summary for this run
            print(f"Run: {run_key}")
            print(f"  Number of peaks: {num_peaks}")
            print(f"  Data points: {len(data_sorted)}")
            print(f"  Time span: {time_sorted.min():.1f} to {time_sorted.max():.1f} M")
            if num_peaks > 0:
                print(f"  Peak value range: {peak_values.min():.3f} to {peak_values.max():.3f}")
            print()
            
            # Plot if requested
            if plot_results:
                plt.figure(figsize=figsize, dpi=dpi)
                
                # Plot the waveform
                plt.plot(time_sorted, data_sorted, 'b-', linewidth=0.8, 
                        label=f'InertialCenter_x', alpha=0.7)
                
                # Plot the peaks
                if num_peaks > 0:
                    plt.plot(peak_times, peak_values, 'ro', markersize=4, 
                            label=f'Peaks ({num_peaks})')
                
                plt.xlabel('Time t(M)')
                plt.ylabel('InertialCenter_x')
                plt.title(f'Run: {run_key} - InertialCenter_x with {num_peaks} peaks')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                
                # Save plot if requested
                if save_plots:
                    if save_dir is None:
                        raise ValueError("save_dir must be specified when save_plots=True")
                    
                    os.makedirs(save_dir, exist_ok=True)
                    filename = f"peaks_analysis_{run_key}.png"
                    filepath = os.path.join(save_dir, filename)
                    plt.savefig(filepath, dpi=dpi, bbox_inches='tight')
                    print(f"  Plot saved: {filepath}")
                
                plt.show()
                
        except Exception as e:
            print(f"Error analyzing run '{run_key}': {str(e)}")
            continue
    
    print("="*60)
    print(f"Analysis complete! Processed {len(results)} runs successfully.")
    
    return results

In [ ]:
def create_peaks_summary_report(analysis_results, save_to_file=False, filename=None):
    """
    Create a summary report of the peak analysis results.
    
    Parameters:
    -----------
    analysis_results : dict
        Results from analyze_inertial_center_peaks function
    save_to_file : bool, default=False
        Whether to save the report to a file
    filename : str, optional
        Filename for the report (required if save_to_file=True)
        
    Returns:
    --------
    pandas.DataFrame : Summary table of all runs
    """
    import pandas as pd
    
    if not analysis_results:
        print("No analysis results to summarize.")
        return pd.DataFrame()
    
    # Create summary data
    summary_data = []
    
    for run_key, result in analysis_results.items():
        stats = result['peak_stats']
        
        summary_data.append({
            'Run': run_key,
            'Num_Peaks': result['num_peaks'],
            'Data_Points': stats['data_points'],
            'Time_Span_M': stats['total_time_span'],
            'Max_Peak_Value': stats['max_peak_value'],
            'Min_Peak_Value': stats['min_peak_value'],
            'Mean_Peak_Value': stats['mean_peak_value'],
            'Std_Peak_Value': stats['std_peak_value'],
            'First_Peak_Time': stats['first_peak_time'],
            'Last_Peak_Time': stats['last_peak_time']
        })
    
    # Create DataFrame
    summary_df = pd.DataFrame(summary_data)
    
    # Sort by number of peaks (descending)
    summary_df = summary_df.sort_values('Num_Peaks', ascending=False).reset_index(drop=True)
    
    # Display summary statistics
    print("PEAK ANALYSIS SUMMARY REPORT")
    print("="*80)
    print(f"Total runs analyzed: {len(summary_df)}")
    print(f"Runs with peaks: {(summary_df['Num_Peaks'] > 0).sum()}")
    print(f"Runs without peaks: {(summary_df['Num_Peaks'] == 0).sum()}")
    print()
    
    print("Peak count statistics:")
    print(f"  Maximum peaks in any run: {summary_df['Num_Peaks'].max()}")
    print(f"  Minimum peaks in any run: {summary_df['Num_Peaks'].min()}")
    print(f"  Average peaks per run: {summary_df['Num_Peaks'].mean():.1f}")
    print(f"  Median peaks per run: {summary_df['Num_Peaks'].median():.1f}")
    print()
    
    # Display the table
    print("Detailed results (sorted by number of peaks):")
    print("-"*80)
    
    # Format for better display
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 20)
    
    print(summary_df.to_string(index=False, float_format='%.3f'))
    
    # Save to file if requested
    if save_to_file:
        if filename is None:
            raise ValueError("filename must be specified when save_to_file=True")
        
        summary_df.to_csv(filename, index=False)
        print(f"\nSummary report saved to: {filename}")
    
    return summary_df

In [ ]:
# Test the function with all available runs (no plots first)
print("Testing peak analysis function on all runs...")
results = analyze_inertial_center_peaks(runs_data_dict, plot_results=False)

print("\nCreating summary report...")
summary_df = create_peaks_summary_report(results)

In [ ]:
# Now demonstrate with plots for one specific run
print("Demonstrating with plots for one run...")
single_run_results = analyze_inertial_center_peaks(
    runs_data_dict, 
    plot_results=True, 
    specific_runs=['q1_0_8_0_0_0_8_d17']
)

In [ ]:
# key = ['26_set1_L6_long', '6_set1_L6s4', '6_set1_L6s6']
key = '26_set1_L6_long'
key = '6_set1_L6s4'
cols = sorted(runs_data_dict[key].columns)
cols = [i for i in cols if 'SphereC' in i]
data = runs_data_dict[key][['t(M)']+return_sorted_domain_names(cols)]
t_index = -1000
y = data.iloc[t_index,1:]
plt.plot(list(y))
plt.xlabel('SphereC Domain Index')
plt.ylabel('Truncation Error')
plt.title(f'{key} t={data.iloc[t_index,0]} M')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = "2.0526315789602303e+00   1.5000000000024982e+00   1.2000000000004154e+00   1.1111111111111118e+00   1.0666666666666673e+00   1.0400000000000003e+00   1.0222222222222226e+00   1.0117647058823531e+00   1.0075471698113208e+00   1.0054794520547947e+00   1.0043010752688173e+00   1.0035398230088499e+00   1.0030075187969927e+00   1.0026143790849673e+00   1.0023121387283238e+00   1.0020725388601037e+00   1.0018779342723010e+00   1.0017167381974250e+00   1.0015810276679842e+00   1.0014652014652015e+00   1.0013651877133112e+00   1.0012779552715658e+00   1.0012012012012013e+00   1.0011331444759211e+00   1.0010723860589814e+00   1.0010178117048347e+00   1.7642573252257077e+00   1.3386504614734347e+00   1.1498568893199947e+00   1.0863415326571306e+00   1.0517778970367335e+00   1.0299023392442415e+00   1.0162171820482417e+00   1.0094377225327236e+00   1.0064365220804508e+00   1.0048572774232334e+00   1.0039033918543223e+00   1.0032637559683677e+00   1.0028046513303512e+00   1.0024590469717682e+00   1.0021893648165854e+00   1.0019730590840459e+00   1.0017956943297366e+00   1.0016476270850283e+00   1.0015221242203169e+00   1.0014143998651277e+00   1.0013209240726657e+00   1.0012390438480769e+00   1.0011667266254245e+00   1.0011023888476995e+00   1.0010447784252550e+00   1.0009928956925862e+00".split("  ")
data = [float(i) for i in data]
half_len = len(data) // 2
linf = 10*(np.array(data[:half_len])-1.0)**2
l2 = 10*(np.array(data[half_len:])-1.0)**2
plt.plot(linf, label='Linf')
plt.plot(l2, label='L2')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
data = "8.1649658092777477e-01   9.1287092917529067e-01   9.4868329805051455e-01   9.6824583655185470e-01   9.8058067569092056e-01   9.8907071009368053e-01   9.9416904650228166e-01   9.9624764155097500e-01   9.9727148195431048e-01   9.9785637481240130e-01   9.9823477355746815e-01   9.9849962406120851e-01   9.9869536800287817e-01   9.9884593152549250e-01   9.9896533857447534e-01   9.9906235328654625e-01   9.9914273451881297e-01   9.9921042230089230e-01   9.9926820334143396e-01   9.9931810425074330e-01   9.9936163415135015e-01   9.9939993993996701e-01   9.9943390881396121e-01   9.9946423783992533e-01   9.9949148229612805e-01   9.9951608997892261e-01   7.6035258190363275e-01   8.6863388546918263e-01   9.3310113004523676e-01   9.5959430068789664e-01   9.7513604909582574e-01   9.8540443812312650e-01   9.9199884213487344e-01   9.9531598745720817e-01   9.9679761057567140e-01   9.9758031047559925e-01   9.9805405706609340e-01   9.9837213423141202e-01   9.9860063280764233e-01   9.9877274874866995e-01   9.9890711764913698e-01   9.9901493177511413e-01   9.9910336286748158e-01   9.9917720495584661e-01   9.9923980699047965e-01   9.9929355040959578e-01   9.9934019234140414e-01   9.9938105379883657e-01   9.9941714713680063e-01   9.9944926126043943e-01   9.9947802007335484e-01   9.9950392179119096e-01".split("  ")
data = [float(i) for i in data]
half_len = len(data) // 2
linf = 99*(1.0/np.array(data[:half_len])-1.0)**2
l2 = 99*(1.0/np.array(data[half_len:])-1.0)**2
plt.plot(linf, label='Linf')
plt.plot(l2, label='L2')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
test_data = np.arange(1000)
test_data = np.diff(runs_data_dict['L3AH001']['ArealMass'])
# test_data = np.diff(runs_data_dict['6_set1_L6s3']['ArealMass'])
y_50_100 = sp.ndimage.percentile_filter(test_data, percentile=50, size=100)
y_0_100 = sp.ndimage.percentile_filter(test_data, percentile=0, size=100)
y_100_100 = sp.ndimage.percentile_filter(test_data, percentile=100, size=100)
y_50_10 = sp.ndimage.percentile_filter(test_data, percentile=50, size=10)
y_mean_100 = sp.ndimage.uniform_filter(test_data, size=100)
y_mean_10 = sp.ndimage.uniform_filter(test_data, size=10)
plt.figure(figsize=(10, 8))
plt.plot(test_data, label='Original Data')
plt.plot(y_0_100, label='y_percentile_0_ws_100')
plt.plot(y_100_100, label='y_percentile_100_ws_100')
plt.plot(y_mean_10, label='y_mean_ws_10')
plt.plot(y_50_10, label='y_percentile_50_ws_10')
plt.plot(y_50_100, label='y_percentile_50_ws_100')
plt.plot(y_mean_100, label='y_mean_ws_100')
plt.yscale('symlog', linthresh=1e-9)
plt.legend()
plt.show()

### Single bh

In [ ]:
runs_to_plot = {}

# runs_to_plot["ode_change_Run1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev1/Run/"
# runs_to_plot["ode_change_Run2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev2/Run/"
# runs_to_plot["ode_change_Run3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev3/Run/"
# runs_to_plot["ode_change_Run4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev4/Run/"
# runs_to_plot["ode_change_Run5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev5/Run/"
# runs_to_plot["ode_change_Run6"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev6/Run/"
# runs_to_plot["ode_change_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev7/Run/"
# runs_to_plot["ode_change_Run8"] = "/groups/sxs/hchaudha/spec_runs/single_bh/0/Lev8/Run/"

# runs_to_plot["1000M_Run1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev1/Run/"
# runs_to_plot["1000M_Run2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev2/Run/"
# runs_to_plot["1000M_Run3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev3/Run/"
# runs_to_plot["1000M_Run4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev4/Run/"
# runs_to_plot["1000M_Run5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev5/Run/"
# runs_to_plot["1000M_Run6"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev6/Run/"
# runs_to_plot["1000M_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev7/Run/"
# runs_to_plot["1000M_Run7_a"] = "/groups/sxs/hchaudha/spec_runs/single_bh/2_1000M/Lev7_a/Run/"

# runs_to_plot["Lin_Run1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev1/Run/"
# runs_to_plot["Lin_Run2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev2/Run/"
# runs_to_plot["Lin_Run3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev3/Run/"
# runs_to_plot["Lin_Run4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev4/Run/"
# runs_to_plot["Lin_Run5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev5/Run/"
# runs_to_plot["Lin_Run6"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev6/Run/"
# runs_to_plot["Lin_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev7/Run/"

# runs_to_plot["400M_base1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev1/Run/"
# runs_to_plot["400M_base2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev2/Run/"
# runs_to_plot["400M_base3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev3/Run/"
# runs_to_plot["400M_base4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev4/Run/"
# runs_to_plot["400M_base5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/4_400M_base/Lev5/Run/"
# runs_to_plot["Lin_Run7"] = "/groups/sxs/hchaudha/spec_runs/single_bh/3_Lin/Lev7/Run/"

# runs_to_plot["400M_phys_bc1"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev1/Run/"
# runs_to_plot["400M_phys_bc2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev2/Run/"
# runs_to_plot["400M_phys_bc3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev3/Run/"
# runs_to_plot["400M_phys_bc4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev4/Run/"
# runs_to_plot["400M_phys_bc5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev5/Run/"

# runs_to_plot["400M_phys_bc5"] = "/groups/sxs/hchaudha/spec_runs/single_bh/12_400M_phys_bc/Lev5/Run/"

# runs_to_plot["13_Lev4_250"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_250/Run/"
# runs_to_plot["13_Lev4_300"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_300/Run/"
# runs_to_plot["13_Lev4_350"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_350/Run/"
# runs_to_plot["13_Lev4_400"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_400/Run/"
# runs_to_plot["13_Lev4_450"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_450/Run/"
# runs_to_plot["13_Lev4_500"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_500/Run/"
# runs_to_plot["13_Lev4_550"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_550/Run/"
# runs_to_plot["13_Lev4_600"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_600/Run/"
# runs_to_plot["13_Lev4_650"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_650/Run/"
# runs_to_plot["13_Lev4_700"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_700/Run/"
# runs_to_plot["13_Lev4_750"] = "/groups/sxs/hchaudha/spec_runs/single_bh/13_error_falloff/Lev4_750/Run/"

# runs_to_plot["14_Lev4_250"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_250/Run/"
# runs_to_plot["14_Lev4_300"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_300/Run/"
# runs_to_plot["14_Lev4_350"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_350/Run/"
# runs_to_plot["14_Lev4_400"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_400/Run/"
# runs_to_plot["14_Lev4_450"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_450/Run/"
# runs_to_plot["14_Lev4_500"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_500/Run/"
# runs_to_plot["14_Lev4_550"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_550/Run/"
# runs_to_plot["14_Lev4_600"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_600/Run/"
# runs_to_plot["14_Lev4_650"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_650/Run/"
# runs_to_plot["14_Lev4_700"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_700/Run/"
# runs_to_plot["14_Lev4_750"] = "/groups/sxs/hchaudha/spec_runs/single_bh/14_error_fo_2000M/Lev4_750/Run/"

# runs_to_plot["15_AMR_Lev0_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev0_255/Run/"
# runs_to_plot["15_AMR_Lev0_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev0_355/Run/"
# runs_to_plot["15_AMR_Lev0_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev0_455/Run/"
# runs_to_plot["15_AMR_Lev1_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev1_255/Run/"
# runs_to_plot["15_AMR_Lev1_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev1_355/Run/"
# runs_to_plot["15_AMR_Lev1_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev1_455/Run/"
# runs_to_plot["15_AMR_Lev2_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev2_255/Run/"
# runs_to_plot["15_AMR_Lev2_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev2_355/Run/"
# runs_to_plot["15_AMR_Lev2_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev2_455/Run/"
# runs_to_plot["15_AMR_Lev3_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev3_255/Run/"
# runs_to_plot["15_AMR_Lev3_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev3_355/Run/"
# runs_to_plot["15_AMR_Lev3_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev3_455/Run/"
# runs_to_plot["15_AMR_Lev4_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev4_255/Run/"
# runs_to_plot["15_AMR_Lev4_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev4_355/Run/"
# runs_to_plot["15_AMR_Lev4_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev4_455/Run/"
# runs_to_plot["15_AMR_Lev5_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev5_255/Run/"
# runs_to_plot["15_AMR_Lev5_355"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev5_355/Run/"
# runs_to_plot["15_AMR_Lev5_455"] = "/groups/sxs/hchaudha/spec_runs/single_bh/15_AMR_test/Lev5_455/Run/"

# runs_to_plot["15_Lev5_255"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255/Run/"
# runs_to_plot["15_Lev5_255_010"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_010/Run/"
# runs_to_plot["15_Lev5_255_0100"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_0100/Run/"
# runs_to_plot["15_Lev5_255_01000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_01000/Run/"
# runs_to_plot["15_Lev5_255_010000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_010000/Run/"
# runs_to_plot["15_Lev5_255_05"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_05/Run/"
# runs_to_plot["15_Lev5_255_050"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_050/Run/"
# runs_to_plot["15_Lev5_255_0500"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_0500/Run/"
# runs_to_plot["15_Lev5_255_05000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_05000/Run/"
# runs_to_plot["15_Lev5_255_050000"] = "/groups/sxs/hchaudha/spec_runs/single_bh/16_AMR_ode_tol_test/Lev5_255_050000/Run/"

# runs_to_plot["400M_gamma1_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/5_400M_gamma1/Lev2/Run/"
# runs_to_plot["400M_gamma1_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/5_400M_gamma1/Lev3/Run/"
# runs_to_plot["400M_gamma1_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/5_400M_gamma1/Lev4/Run/"

# runs_to_plot["400M_gamma1_001_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/6_400M_gamma1_001/Lev2/Run/"
# runs_to_plot["400M_gamma1_001_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/6_400M_gamma1_001/Lev3/Run/"
# runs_to_plot["400M_gamma1_001_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/6_400M_gamma1_001/Lev4/Run/"

# runs_to_plot["400M_gamma1_01_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/7_400M_gamma1_01/Lev2/Run/"
# runs_to_plot["400M_gamma1_01_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/7_400M_gamma1_01/Lev3/Run/"
# runs_to_plot["400M_gamma1_01_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/7_400M_gamma1_01/Lev4/Run/"

# runs_to_plot["400M_BDres_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/8_400M_BDres/Lev2/Run/"
# runs_to_plot["400M_BDres_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/8_400M_BDres/Lev3/Run/"
# runs_to_plot["400M_BDres_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/8_400M_BDres/Lev4/Run/"

# runs_to_plot["9_400M_BDres_05_2"] = "/groups/sxs/hchaudha/spec_runs/single_bh/9_400M_BDres_05/Lev2/Run/"
# runs_to_plot["9_400M_BDres_05_3"] = "/groups/sxs/hchaudha/spec_runs/single_bh/9_400M_BDres_05/Lev3/Run/"
# runs_to_plot["9_400M_BDres_05_4"] = "/groups/sxs/hchaudha/spec_runs/single_bh/9_400M_BDres_05/Lev4/Run/"

# runs_to_plot["10_freezing"] = "/groups/sxs/hchaudha/spec_runs/single_bh/10_freezing/Lev4/Run/"
# runs_to_plot["11_physical_bc"] = "/groups/sxs/hchaudha/spec_runs/single_bh/11_physical_bc/Lev4/Run/"

# runs_to_plot["11_physical_bc"] = "/groups/sxs/hchaudha/spec_runs/single_bh/11_physical_bc/Lev4/Run/"

# runs_to_plot["Lev3_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev3_265/Run/"
# runs_to_plot["Lev4_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev4_265/Run/"
# runs_to_plot["Lev5_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265/Run/"
# runs_to_plot["Lev6_265"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev6_265/Run/"
# runs_to_plot["Lev5_265_0.01_0.01_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.01_0.01_0.01/Run/"
# runs_to_plot["Lev5_265_0.05_0.05_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.05_0.05_0.01/Run/"
# runs_to_plot["Lev5_265_0.1_0.1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.1_0.1_0.001/Run/"
# runs_to_plot["Lev5_265_0.1_0.1_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_0.1_0.1_0.01/Run/"
# runs_to_plot["Lev5_265_1_1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.001/Run/"
# runs_to_plot["Lev5_265_1_1_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.01/Run/"
# runs_to_plot["Lev5_265_3_3_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_3_3_0.001/Run/"
# runs_to_plot["Lev5_265_3_3_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_3_3_0.01/Run/"
# runs_to_plot["Lev5_265_5_5_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_5_5_0.001/Run/"
# runs_to_plot["Lev5_265_5_5_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_5_5_0.01/Run/"
# runs_to_plot["Lev5_265_10_10_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.001/Run/"
# runs_to_plot["Lev5_265_10_10_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.01/Run/"
# runs_to_plot["Lev5_265_25_25_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_25_25_0.01/Run/"
# runs_to_plot["Lev5_265_50_50_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_50_50_0.01/Run/"
# runs_to_plot["Lev5_265_100_100_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_100_100_0.01/Run/"

# runs_to_plot["Lev5_265_1_1_1e-05"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_1e-05/Run/"
# runs_to_plot["Lev5_265_1_1_0.0001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.0001/Run/"
# runs_to_plot["Lev5_265_1_1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.001/Run/"
# runs_to_plot["Lev5_265_1_1_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.01/Run/"
# runs_to_plot["Lev5_265_1_1_0.1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_0.1/Run/"
# runs_to_plot["Lev5_265_1_1_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_1_1_1/Run/"
# runs_to_plot["Lev5_265_10_10_1e-05"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_1e-05/Run/"
# runs_to_plot["Lev5_265_10_10_0.0001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.0001/Run/"
# runs_to_plot["Lev5_265_10_10_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.001/Run/"
# runs_to_plot["Lev5_265_10_10_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.01/Run/"
# runs_to_plot["Lev5_265_10_10_0.1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0.1/Run/"
# runs_to_plot["Lev5_265_10_10_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_1/Run/"

# runs_to_plot["Lev5_265_10_10_0_-0.1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_-0.1/Run/"
# runs_to_plot["Lev5_265_10_10_0_-0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_-0.01/Run/"
# runs_to_plot["Lev5_265_10_10_0_-0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_-0.001/Run/"
# runs_to_plot["Lev5_265_10_10_0_-0.0001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_-0.0001/Run/"
# runs_to_plot["Lev5_265_10_10_0_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_-1/Run/"

# runs_to_plot["Lev5_265_10_10_0_0_-3"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_-3/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_-2"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_-2/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_-1/Run/"
# runs_to_plot["Lev5_265_10_10_0_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_0/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_1/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_2/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_3/Run/"
# runs_to_plot["Lev5_265_10_10_0_0_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_265_10_10_0_0_4/Run/"

# runs_to_plot["Lev7_265_1_1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev7_265_1_1_0.001/Run/"
# runs_to_plot["Lev7_265_10_10_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev7_265_10_10_0.001/Run/"

# runs_to_plot["Lev5_2065_10_10_0.01"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_10_10_0.01/Run/"
# runs_to_plot["Lev5_2065_10_10_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_10_10_0.001/Run/"
# runs_to_plot["Lev5_2065_3_3_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_3_3_0.001/Run/"
# runs_to_plot["Lev5_2065_1_1_0.001"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_1_1_0.001/Run/"

# runs_to_plot["Lev5_2065_4_4_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_4_4_0/Run/"
# runs_to_plot["Lev5_2065_12_12_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_12_12_0/Run/"
# runs_to_plot["Lev5_2065_40_40_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_40_40_0/Run/"

# runs_to_plot["Lev5_2065_10_10_0.001Gamma1n1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_10_10_0.001Gamma1n1/Run/"

# runs_to_plot["Lev5_2065_10_10_0Detg"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_10_10_0Detg/Run/"
# runs_to_plot["Lev5_2065_99_99_0Lapse"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_99_99_0Lapse/Run/"
# runs_to_plot["Lev5_2065_10_10_0ExpFalloff"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_10_10_0ExpFalloff/Run/"
# runs_to_plot["Lev5_2065_1_1_0ExpFalloff"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_1_1_0ExpFalloff/Run/"

# runs_to_plot["Lev5_2065_10_10_0_0_-8"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/22_const_AMR/Lev5_2065_10_10_0_0_-8/Run/"

# runs_to_plot["Lev0_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev0_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev1_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev1_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev2_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev2_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev3_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev3_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev4_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev4_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev5_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev5_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev6_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev6_265_10_10_0.001_-1/Run/"
# runs_to_plot["Lev7_265_10_10_0.001_-1"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev7_265_10_10_0.001_-1/Run/"

runs_to_plot["46_bh_bbh_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/46_single_bh_bbh_domain/1_1000M_L16/Lev1_1000/Run/"
runs_to_plot["46_bh_bbh_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/46_single_bh_bbh_domain/1_1000M_L16/Lev2_1000/Run/"
runs_to_plot["46_bh_bbh_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/46_single_bh_bbh_domain/1_1000M_L16/Lev3_1000/Run/"
runs_to_plot["46_bh_bbh_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/46_single_bh_bbh_domain/1_1000M_L16/Lev4_1000/Run/"
runs_to_plot["46_bh_bbh_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/46_single_bh_bbh_domain/1_1000M_L16/Lev5_1000/Run/"
runs_to_plot["46_bh_bbh_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/46_single_bh_bbh_domain/1_1000M_L16/Lev6_1000/Run/"


df_list = []
# df_list.append("ApparentHorizons/Horizons.h5@AhC")
# df_list.append("ApparentHorizons/AhC.dat")

# df_list.append("FailedTStepperDiag.dat")
# df_list.append("GhCe_Norms.dat")
# df_list.append("GhCe.dat")
# df_list.append("NormalizedGhCe.dat")
# df_list.append("GhCe_Linf.dat")
df_list.append("GhCe.dat")
# df_list.append("NormalizedGhCe_Linf.dat")
# df_list.append("1Con.dat")
# df_list.append("2Con.dat")
# df_list.append("3Con.dat")
# df_list.append("kappaErr_Linf.dat")
# df_list.append("psiErr_Linf.dat")
# df_list.append("kappaErr.dat")
# df_list.append("kappa.dat")
# df_list.append("psiErr.dat")
# df_list.append("psi.dat")
# df_list.append("Shift.dat")
# df_list.append("TStepperDiag.dat")

data_file_path = df_list[0]

normalize_error = False
# normalize_error = True

cols_names_all = []
if normalize_error:
    if 'kappa.dat' in df_list and 'kappaErr.dat' in df_list:
        column_names, runs_data_dict = load_data_from_levs(runs_to_plot,'kappaErr.dat')
        column_names_val, runs_data_dict_val = load_data_from_levs(runs_to_plot,'kappa.dat')
        for key in runs_data_dict:
            df_err = runs_data_dict[key].iloc[:, 1:]
            df_val = runs_data_dict_val[key].iloc[:, 1:]

            runs_data_dict[key] = pd.DataFrame(df_err.values / df_val.values, columns=df_err.columns)
            t_col = runs_data_dict_val[key]['t(M)']
            runs_data_dict[key].insert(0, 't(M)', t_col)
        cols_names_all = cols_names_all+list(column_names)

    elif 'psi.dat' in df_list and 'psiErr.dat' in df_list:
        column_names, runs_data_dict = load_data_from_levs(runs_to_plot,'psiErr.dat')
        column_names_val, runs_data_dict_val = load_data_from_levs(runs_to_plot,'psi.dat')
        for key in runs_data_dict:
            df_err = runs_data_dict[key].iloc[:, 1:]
            df_val = runs_data_dict_val[key].iloc[:, 1:]

            runs_data_dict[key] = pd.DataFrame(df_err.values / df_val.values, columns=df_err.columns)
            t_col = runs_data_dict_val[key]['t(M)']
            runs_data_dict[key].insert(0, 't(M)', t_col)
        cols_names_all = cols_names_all+list(column_names)

    else:
        raise ValueError("Normalization is requested but neither 'kappa.dat' nor 'psi.dat' are present in the data files.")
else:
    print(f"Loading {df_list[0]}")
    column_names, runs_data_dict = load_data_from_levs(runs_to_plot,df_list[0])
    cols_names_all = cols_names_all+list(column_names)
    for data_file_path in df_list[1:]:
        print(f"Processing {data_file_path}")
        column_names, runs_data_dict_temp = load_data_from_levs(runs_to_plot,data_file_path)
        for key in runs_data_dict_temp:
            runs_data_dict[key] = pd.merge(runs_data_dict[key], runs_data_dict_temp[key], on='t(M)')
        cols_names_all = cols_names_all+list(column_names)

print(cols_names_all)
print(runs_data_dict.keys())

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
constant_shift_val_time = None
plot_abs_diff = False
modification_function = None
append_to_title = ""
y_axis_list = None
x_axis = 't(M)'
take_abs = False

take_abs = True

# diff_base = '17_set1_9_18_L3_correct'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

# constant_shift_val_time = 7206
# constant_shift_val_time = 1200
# constant_shift_val_time = 0
# constant_shift_val_time = 6400
# if constant_shift_val_time is not None:
#     take_abs = True

sd = 'FilledSphere0'
# sd = 'SphereA0'
# sd = 'SphereB3'
# sd = 'FilledCylinderMB0'
# sd = 'CylinderEB0.0.0'
# sd = 'SphereC5'
# sd = 'SphereD0'
# sd = 'SphereE0'
# sd = 'SphereE5'
# sd = 'SphereE23'


y_axis = f'L2(sqrt(3Con^2)) on {sd}'
y_axis = f'Linf(sqrt(3Con^2)) on {sd}'
# y_axis = f'L2(sqrt(psiErr^2)) on {sd}'
# y_axis = f'Linf(sqrt(psiErr^2)) on {sd}'
# y_axis = f'L2(sqrt(kappaErr^2)) on {sd}'
# y_axis = f'Linf(sqrt(kappaErr^2)) on {sd}'
# y_axis = f'L2(sqrt(kappa^2)) on {sd}'
# y_axis = f'Linf(sqrt(kappa^2)) on {sd}'
# y_axis = f'L2(sqrt(1Con^2)) on {sd}'
# y_axis = f'Linf(sqrt(1Con^2)) on {sd}'
# y_axis = f'L2(sqrt(2Con^2)) on {sd}'
# y_axis = f'Linf(sqrt(2Con^2)) on {sd}'
# y_axis = f'L2(sqrt(3Con^2)) on {sd}'
y_axis = f'Linf(sqrt(3Con^2)) on {sd}'
# y_axis = f'Linf(1Con) on {sd}'
# y_axis = f'Linf(2Con) on {sd}'
# y_axis = f'{psi_or_kappa}_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}'
# y_axis = f'psi_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}'
# y_axis = f'kappa_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}'
# if psi_or_kappa == "both":
#     y_axis_list = [f'kappa_{data_file_path.split("@")[1][:-4]}_{get_top_name_from_number(top_number,sd)} on {sd}',f'psi_TruncationError_{get_top_name_from_number(top_number,sd)} on {sd}']
# y_axis = 'Linf(sqrt(3Con^2)) on SphereD0'
# y_axis = 'Linf(sqrt(3Con^2)) on SphereE5'
y_axis = f'Linf(GhCe) on {sd}'

# y_axis = 'ArealMass'


minT = 0
# minT = 85
# minT = 470
# minT = 1200
# minT = 2200
# minT = 3400
# minT = 11000

maxT = 400000
# maxT = minT+1750
# maxT = 700
# moving_avg_len = 50
# moving_avg_len = 10

# y_axis_list = [f"SphereC{i}_R" for i in range(30)]

# if "GhCe" in y_axis:
plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)


legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

# take_abs,modification_function = True,noise_function
# take_abs,modification_function = True,derivative_function
# take_abs,modification_function = False,derivative_function
# take_abs,modification_function = True,compute_center
# take_abs,modification_function = True,min_max_r_ratio
# take_abs,modification_function = True, lambda x, y, df, y_axis: index_constraints_norm(x, y, df, y_axis, index_num=3,norm="Linf")
# take_abs,modification_function = True, lambda x, y, df, y_axis: get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r".*", print_sd_names=False)
# take_abs,modification_function = True, lambda x, y, df, y_axis: get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r'^(?!.*SphereC).*', print_sd_names=False)
# take_abs,modification_function = True, lambda x, y, df, y_axis: get_num_points_subdomains(x, y, df, y_axis, regex_for_sd=r'SphereC.*', print_sd_names=False)

# if 'Horizons.h5@' in data_file_path:
#   append_to_title += " HorizonBH="+data_file_path.split('@')[-1]
if 'AhA' in data_file_path:
    append_to_title += " AhA"
if 'AhB' in data_file_path:
    append_to_title += " AhB"
if normalize_error:
    append_to_title += " Normalized"


# with plt.style.context('default'):
with plt.style.context('ggplot'):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:20]
    # colors = plt.cm.tab10.colors
    line_styles = ['-','--',':',  '-.']
    combined_cycler = cycler(linestyle=line_styles)*cycler(color=colors[:6])
    # combined_cycler = cycler(color=colors)*cycler(linestyle=line_styles[:2])
    plt.rcParams['axes.prop_cycle'] = combined_cycler
#   plt.rcParams["figure.figsize"] = (15,10)
#   plt.rcParams["figure.figsize"] = (4,4)
    plt.rcParams["figure.figsize"] = (10,8)
    # plt.rcParams["figure.figsize"] = (6,6)
    plt.rcParams["figure.autolayout"] = True
    # plt.ylim(1e-10,5e-6)
    if y_axis_list is None:
        plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)
    else:
        plot_graph_for_runs_wrapper(runs_data_dict, x_axis, y_axis_list, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)

#   plt.title("")
#   plt.ylabel("Constraint Violations near black holes")
#   plt.tight_layout()
    # plt.legend(loc='upper left')
    # plt.ylim(1e-14, 1e-7)
    # plt.ylim(1e-20, 1e-13)
    # save_name = "change_L16_set1_SphereC22_Linf_NormalizedGhCe.png"

    # save_name = Path(f"/home/hchaudha/code_for_paper_plots/spec_accuracy/{save_name}")
    # if save_name.exists():
    #     raise Exception("Change name")
    # plt.savefig(save_name)
    # plt.xscale('log')
    plt.yscale('log')
    plt.yscale('symlog',linthresh=1e-15)
    plt.tight_layout()
    plt.show()

In [ ]:
t_index = 0

for i in range(len(domain_names)):
    if domain_names[i] == 'FilledSphere0':
        domain_names[i] = 'FilledSphereB0'
return_sorted_domain_names(domain_names, just_domain_names=True)

with plt.style.context('ggplot'):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:20]
    # colors = plt.cm.tab10.colors
    line_styles = ['-','--',':',  '-.']
    combined_cycler = cycler(linestyle=line_styles)*cycler(color=colors[:6])
    # combined_cycler = cycler(color=colors)*cycler(linestyle=line_styles[:2])
    plt.rcParams['axes.prop_cycle'] = combined_cycler
#   plt.rcParams["figure.figsize"] = (15,10)
#   plt.rcParams["figure.figsize"] = (4,4)
    plt.rcParams["figure.figsize"] = (10,8)
    # plt.rcParams["figure.figsize"] = (6,6)
    plt.rcParams["figure.autolayout"] = True


    for key in runs_data_dict.keys():
        cols = runs_data_dict[key].columns
        cols = [col for col in cols if "L2" not in col]
        data = runs_data_dict[key][cols]
        y = data.iloc[t_index,1:]
        plt.plot(list(y), label=key)


# plt.xlabel('SphereC Domain Index')
domain_names = [col.split('on')[-1].strip() for col in cols[1:]]
plt.xticks(range(len(domain_names)),domain_names,rotation=90)
# plt.grid(False)
plt.ylabel(cols[-1].split('on')[0].strip())
title = f"t={runs_data_dict[key]['t(M)'].iloc[t_index]}M"
if normalize_error:
    title = "Normalized" + ", " + title
    plt.ylabel("Normalized " + cols[-1].split('on')[0].strip())
plt.title(title)
plt.yscale('log')
plt.legend()
plt.show()

### Intergrate over something

In [ ]:
def interpolate_data(t,y, t_new):
    return CubicSpline(t, y, extrapolate=False)(t_new)

if diff_base is None:
    diff_base = list(runs_data_dict.keys())[-1]

t = runs_data_dict[diff_base]["t(M)"]
filter_t = (t >= 1200) & (t <= 4000)

runs_data_dict[diff_base].keys()

In [ ]:
diff_int = {}

# y_axis = y_axis
y_axis = 'InertialCenter_x'
y_axis = 'ProperSepHorizons'
y_axis = 'ChristodoulouMass'
y_axis = 'phi'
# y_axis = 'CoordSepHorizons'

t_min = 1200
# t_min = 6000
# t_max = 4000
t_max = 11788
# t_max = 1900
# t_min = 6000
# t_max = 6700

sorted_keys = sorted(list(runs_data_dict.keys()))
for key1,key2 in itertools.combinations(sorted_keys,2):
# for key1,key2 in zip(sorted_keys[:-1],[sorted_keys[-1]]*len(sorted_keys[:-1])):
    # if 's6' in key1 or 's6' in key2:
    #     continue
    data1 = runs_data_dict[key1].drop_duplicates(subset=['t(M)'])
    t1 = np.array(data1["t(M)"])
    filter_t1 = (t1 >= t_min) & (t1 <= t_max)
    t1 = t1[filter_t1]
    # print(t1.shape)
    y1 = np.array(data1[y_axis])[filter_t1]

    data2 = runs_data_dict[key2].drop_duplicates(subset=['t(M)'])
    t2 = np.array(data2["t(M)"])
    y2 = np.array(data2[y_axis])

    diff = y1 - interpolate_data(t2,y2,t1)

    diff = np.abs(diff)
    diff_int[key1+"@"+key2] = sp.integrate.simpson(diff, t1)


diff_int

##### Subtract the location of the center

In [ ]:
diff_int = {}


t_min = 1200
t_max = 4000

# t_min = 6000
# t_max = 8800
# t_max = 11788

sorted_keys = sorted(list(runs_data_dict.keys()))
for key1,key2 in itertools.combinations(sorted_keys,2):
# for key1,key2 in zip(sorted_keys[:-1],[sorted_keys[-1]]*len(sorted_keys[:-1])):
    # if "set3" not in key1 or "set3" not in key2:
    #     continue
    diff_int[key1+"@"+key2] = 0
    for y_axis in ['CoordCenterInertial_0','CoordCenterInertial_1','CoordCenterInertial_2']:

        data1 = runs_data_dict[key1].drop_duplicates(subset=['t(M)'])
        t1 = np.array(data1["t(M)"])
        filter_t1 = (t1 >= t_min) & (t1 <= t_max)
        t1 = t1[filter_t1]
        y1 = np.array(data1[y_axis])[filter_t1]

        data2 = runs_data_dict[key2].drop_duplicates(subset=['t(M)'])
        t2 = np.array(data2["t(M)"])
        y2 = np.array(data2[y_axis])

        diff = y1 - interpolate_data(t2,y2,t1)

        diff = np.abs(diff)
        diff_int[key1+"@"+key2] += (sp.integrate.simpson(diff, t1))**2

diff_int = {k: np.sqrt(v) for k, v in diff_int.items()}
y_axis = 'Center Diff' # For plot label in the next cell

diff_int

In [ ]:
sorted_diff_int = dict(sorted(diff_int.items(), key=lambda item: item[1], reverse=True))
plt.scatter(range(len(sorted_diff_int)), list(sorted_diff_int.values()))
plt.yscale('log')
plt.xticks(range(len(sorted_diff_int)), list(sorted_diff_int.keys()), rotation=90)
plt.show()

In [ ]:
set1_center_diff = {
    "6_set1_L6s4@6_set1_L6s5": 0.07440602543930704,
    "6_set1_L6s4@6_set1_L6s6": 0.02289045491236974,
    "6_set1_L6s5@6_set1_L6s6": 0.05318158328457974,
}
set1_mismatch = {
    "6_set1_L6s4_0304@6_set1_L6s5_0304": 3.358293980026113e-11,
    "6_set1_L6s4_0304@6_set1_L6s6_0304": 4.951323655121002e-12,
    "6_set1_L6s5_0304@6_set1_L6s6_0304": 1.6948436280142414e-11,
}

set2_center_diff = {
    "6_set2_L6s4@6_set2_L6s5": 0.03978163867751885,
    "6_set2_L6s4@6_set2_L6s6": 0.015867127835162077,
    "6_set2_L6s5@6_set2_L6s6": 0.055553043738375316,
}
set2_mismatch = {
    "6_set2_L6s4_0304@6_set2_L6s5_0304": 9.722797419527507e-12,
    "6_set2_L6s4_0304@6_set2_L6s6_0304": 2.2674292829701988e-12,
    "6_set2_L6s5_0304@6_set2_L6s6_0304": 1.814391684672462e-11,
}

set3_center_diff = {
    # "6_set3_L6s4@6_set3_L6s5": 0.03631083482463626,
    "6_set3_L6s4@6_set3_L6s6": 0.02551424036316518,
    "6_set3_L6s5@6_set3_L6s6": 0.061725885090107555,
}
set3_mismatch = {
    # "6_set3_L6s4_0304@6_set3_L6s5_0304": 0.5627793513624006,
    "6_set3_L6s4_0304@6_set3_L6s6_0304": 4.4744723763334985e-12,
    "6_set3_L6s5_0304@6_set3_L6s6_0304": 1.8023140369205797e-11,
}


plt.plot(set1_center_diff.values(),set1_mismatch.values(),'o',label='Set 1')
plt.plot(set2_center_diff.values(),set2_mismatch.values(),'o',label='Set 2')
plt.plot(set3_center_diff.values(),set3_mismatch.values(),'o',label='Set 3')
# plt.yscale('log')
# plt.xscale('log')
plt.legend()
plt.show()


#### Growth trend int ceter diff

In [ ]:
def how_does_int_center_grow(t_min,t_max,dt,runs_data_dict,key1,key2,normalize=False):
    diff_int = defaultdict(float) ## 0.0 by default
    t_arr = np.linspace(t_min,t_max,int((t_max-t_min)/dt),dtype=int)
    for t_max_current in t_arr[1:]:
        norm_denom = 0
        for y_axis in ['CoordCenterInertial_0','CoordCenterInertial_1','CoordCenterInertial_2']:
            data1 = runs_data_dict[key1].drop_duplicates(subset=['t(M)'])
            t1 = np.array(data1["t(M)"])
            filter_t1 = (t1 >= t_min) & (t1 <= t_max_current)
            t1 = t1[filter_t1]
            y1 = np.array(data1[y_axis])[filter_t1]

            data2 = runs_data_dict[key2].drop_duplicates(subset=['t(M)'])
            t2 = np.array(data2["t(M)"])
            y2 = np.array(data2[y_axis])

            diff = y1 - interpolate_data(t2,y2,t1)

            diff = np.abs(diff)
            diff_int[f"{t_min}@{t_max_current}"] += (sp.integrate.simpson(diff, t1))**2
            if normalize:
                norm_denom += (sp.integrate.simpson(y1**2, t1)) ## Note that we are squaring to avoid getting zero at points

        if normalize:
            diff_int[f"{t_min}@{t_max_current}"] /= norm_denom

    return t_arr,diff_int


# t_min = 1200
# t_max = 4000
# sorted_keys = sorted(list(runs_data_dict.keys()))
# t_arr, diff_int = how_does_int_center_grow(t_min,t_max,100,runs_data_dict, sorted_keys[-1],sorted_keys[-2])
# t_arr2, diff_int2 = how_does_int_center_grow(t_min,t_max,100,runs_data_dict, sorted_keys[-1],sorted_keys[-3])

# plt.scatter(t_arr[1:],diff_int.values(),label='L4 vs L6')
# plt.scatter(t_arr2[1:],diff_int2.values(),label='L5 vs L6')
# plt.yscale('log')
# # plt.xscale('log')
# plt.legend()
# plt.show()

In [ ]:
t_min = 1200
t_max = 4000

t_min = 6000
t_max = 8800
t_max = 11788

dt = 250
normalize = False
# normalize = True

int_cent_diff_dict = {}
sorted_keys = sorted(list(runs_data_dict.keys()))
# for key1,key2 in itertools.combinations(sorted_keys,2):
for key1,key2 in zip(sorted_keys[:-1],[sorted_keys[-1]]*len(sorted_keys[:-1])):
    # if "set3" not in key1 or "set3" not in key2:
    #     continue
    t_arr, diff_int = how_does_int_center_grow(t_min,t_max,dt,runs_data_dict,key1,key2, normalize=normalize)
    int_cent_diff_dict[f"{key1}@{key2}"] = diff_int
    plt.scatter(t_arr[1:],diff_int.values(),label=f'{key1} vs {key2}')
    print(f"{key1} vs {key2} done!")

plt.legend()
plt.title(f"Int center diff vs time {t_min=}")
plt.ylabel("Int center diff")
plt.xlabel("t_max")
plt.yscale('log')
plt.show()

In [ ]:
int_cent_diff_dict = {
    "36_segs_L1@36_segs_L6": defaultdict(
        float,
        {
            "6000@6263": 2.48199799965419e-06,
            "6000@6526": 9.048365288112467e-05,
            "6000@6789": 0.0008181289342205235,
            "6000@7052": 0.0064152794053880195,
            "6000@7315": 0.039225427005874156,
            "6000@7578": 0.15067783269117516,
            "6000@7841": 0.47709148700699666,
            "6000@8104": 1.2335158405249562,
            "6000@8367": 2.905481794008875,
            "6000@8630": 6.085815375830216,
            "6000@8894": 11.916294196109769,
            "6000@9157": 21.83018217544503,
            "6000@9420": 38.08881126912497,
            "6000@9683": 64.46120532812449,
            "6000@9946": 103.70742387357288,
            "6000@10209": 163.004336697848,
            "6000@10472": 249.7131996771567,
            "6000@10735": 371.96716118574693,
            "6000@10998": 545.3366988343942,
            "6000@11261": 785.0192906477042,
            "6000@11524": 1130.6339593861312,
            "6000@11788": 1758.5189285863653,
        },
    ),
    "36_segs_L2@36_segs_L6": defaultdict(
        float,
        {
            "6000@6263": 3.398627004215709e-07,
            "6000@6526": 0.00026261397064322235,
            "6000@6789": 0.005319845139352715,
            "6000@7052": 0.03441770274234537,
            "6000@7315": 0.15294680216163203,
            "6000@7578": 0.4929546513995494,
            "6000@7841": 1.3381883195813646,
            "6000@8104": 3.1616270086808185,
            "6000@8367": 6.8786861790535685,
            "6000@8630": 13.68957622488756,
            "6000@8894": 25.689835615022403,
            "6000@9157": 45.64987814785052,
            "6000@9420": 77.98098129826272,
            "6000@9683": 129.84893581249733,
            "6000@9946": 207.17489256081117,
            "6000@10209": 324.86101748494093,
            "6000@10472": 500.29346336924266,
            "6000@10735": 755.6933750004882,
            "6000@10998": 1134.5563798260382,
            "6000@11261": 1691.4868825811868,
            "6000@11524": 2553.2051378678193,
            "6000@11788": 4288.203820615171,
        },
    ),
    "36_segs_L3@36_segs_L6": defaultdict(
        float,
        {
            "6000@6263": 1.0340582208554716e-07,
            "6000@6526": 9.020061752735009e-06,
            "6000@6789": 2.3603435332264497e-05,
            "6000@7052": 0.00010239301685563608,
            "6000@7315": 0.0008266244799622537,
            "6000@7578": 0.003759612784674639,
            "6000@7841": 0.0140544659088626,
            "6000@8104": 0.04134046128543894,
            "6000@8367": 0.10875729643092696,
            "6000@8630": 0.2526580542528181,
            "6000@8894": 0.537936262900619,
            "6000@9157": 1.0729066745865368,
            "6000@9420": 2.02367311114017,
            "6000@9683": 3.6949654712085316,
            "6000@9946": 6.406700463210132,
            "6000@10209": 10.859216909161207,
            "6000@10472": 18.023737127857657,
            "6000@10735": 29.310932222516314,
            "6000@10998": 47.480615198586875,
            "6000@11261": 76.56832677835898,
            "6000@11524": 125.95157538753212,
            "6000@11788": 237.34073991005468,
        },
    ),
    "36_segs_L4@36_segs_L6": defaultdict(
        float,
        {
            "6000@6263": 2.5892492545273836e-08,
            "6000@6526": 3.264232654732462e-06,
            "6000@6789": 1.9388298690227884e-05,
            "6000@7052": 6.29385024568405e-05,
            "6000@7315": 0.00013513572108193755,
            "6000@7578": 0.00023245007984759063,
            "6000@7841": 0.0003202848199795378,
            "6000@8104": 0.0003833612876842819,
            "6000@8367": 0.00041253968891799493,
            "6000@8630": 0.0005562492536569368,
            "6000@8894": 0.0009001044794398779,
            "6000@9157": 0.0016693403572314773,
            "6000@9420": 0.0032154242153067452,
            "6000@9683": 0.006293767496177432,
            "6000@9946": 0.01187559507638713,
            "6000@10209": 0.021848523307868427,
            "6000@10472": 0.03943760160805933,
            "6000@10735": 0.07005505922939181,
            "6000@10998": 0.12432861129139176,
            "6000@11261": 0.21965734604276227,
            "6000@11524": 0.39927602463650896,
            "6000@11788": 0.8673370991935989,
        },
    ),
    "36_segs_L5@36_segs_L6": defaultdict(
        float,
        {
            "6000@6263": 6.896419086010006e-09,
            "6000@6526": 9.707847962950453e-07,
            "6000@6789": 6.658587000652166e-06,
            "6000@7052": 2.5002540621853754e-05,
            "6000@7315": 6.857850608034978e-05,
            "6000@7578": 0.00015770607426876147,
            "6000@7841": 0.0003228060291929489,
            "6000@8104": 0.0006173483609973041,
            "6000@8367": 0.0011229345459923522,
            "6000@8630": 0.001944838141834732,
            "6000@8894": 0.003282629639060616,
            "6000@9157": 0.005358445554351675,
            "6000@9420": 0.008579731431532506,
            "6000@9683": 0.013567957533135243,
            "6000@9946": 0.020781868102338952,
            "6000@10209": 0.03152402026690377,
            "6000@10472": 0.04711531541549985,
            "6000@10735": 0.06900750836717422,
            "6000@10998": 0.0997431472435136,
            "6000@11261": 0.14082143296696525,
            "6000@11524": 0.19479828259443316,
            "6000@11788": 0.2738317783981799,
        },
    ),
}


mist_dict = {
    "36_set1_s1_0300@36_set1_s2_0300": {
        "6000@6263": 2.482311800354938e-15,
        "6000@6526": 1.052908698587459e-12,
        "6000@6789": 7.675595961111951e-11,
        "6000@7052": 2.383861047091147e-10,
        "6000@7315": 5.941452471125841e-10,
        "6000@7578": 1.1900933417200517e-09,
        "6000@7841": 2.2142335630636957e-09,
        "6000@8104": 3.775551314071474e-09,
        "6000@8367": 6.117882853021139e-09,
        "6000@8630": 9.903961004381895e-09,
        "6000@8894": 1.4949793994359136e-08,
        "6000@9157": 2.3341770953323338e-08,
        "6000@9420": 3.468386479145395e-08,
        "6000@9683": 5.206676298247742e-08,
        "6000@9946": 7.963496565304491e-08,
        "6000@10209": 1.1805689404683278e-07,
        "6000@10472": 1.8153341111641538e-07,
        "6000@10735": 2.912694313116308e-07,
        "6000@10998": 4.7452611959960277e-07,
        "6000@11261": 8.140172398727015e-07,
        "6000@11524": 1.547283160666174e-06,
        "6000@11788": 3.4689183790020886e-06,
    },
    "36_set1_s2_0300@36_set1_s3_0300": {
        "6000@6263": 4.3988539139068913e-16,
        "6000@6526": 1.400205770475208e-13,
        "6000@6789": 4.788637614765797e-11,
        "6000@7052": 3.8727608212749457e-10,
        "6000@7315": 1.517034883833197e-09,
        "6000@7578": 4.297868817275912e-09,
        "6000@7841": 9.970421060997904e-09,
        "6000@8104": 2.093274924256068e-08,
        "6000@8367": 3.816110167159361e-08,
        "6000@8630": 7.00360440258212e-08,
        "6000@8894": 1.124857251977518e-07,
        "6000@9157": 1.8816467384289958e-07,
        "6000@9420": 2.8844203173849014e-07,
        "6000@9683": 4.403433573634394e-07,
        "6000@9946": 6.759006532540208e-07,
        "6000@10209": 9.83329849453248e-07,
        "6000@10472": 1.4606422750020576e-06,
        "6000@10735": 2.1907324579167077e-06,
        "6000@10998": 3.2414265911555174e-06,
        "6000@11261": 4.886094327047441e-06,
        "6000@11524": 7.80791857013824e-06,
        "6000@11788": 1.4133791487438332e-05,
    },
    "36_set1_s3_0300@36_set1_s4_0300": {
        "6000@6263": 1.8346609540250692e-15,
        "6000@6526": 6.5206363186214396e-15,
        "6000@6789": 1.624936371374129e-13,
        "6000@7052": 1.6694390696657563e-12,
        "6000@7315": 9.114062764420402e-12,
        "6000@7578": 4.988726055047968e-11,
        "6000@7841": 1.45270170748164e-10,
        "6000@8104": 4.0195822096860943e-10,
        "6000@8367": 8.537428737352619e-10,
        "6000@8630": 1.8587434874311771e-09,
        "6000@8894": 3.3722744475547273e-09,
        "6000@9157": 6.387185954025127e-09,
        "6000@9420": 1.0922805318460619e-08,
        "6000@9683": 1.8397764258590564e-08,
        "6000@9946": 3.1259021064647803e-08,
        "6000@10209": 4.968961071590086e-08,
        "6000@10472": 8.082079633180537e-08,
        "6000@10735": 1.3316709036733317e-07,
        "6000@10998": 2.1644064925269322e-07,
        "6000@11261": 3.614079434827398e-07,
        "6000@11524": 6.500416478224421e-07,
        "6000@11788": 1.3599622689358382e-06,
    },
    "36_set1_s4_0300@36_set1_s5_0300": {
        "6000@6263": 1.2250319897926268e-15,
        "6000@6526": 3.3389895640633266e-15,
        "6000@6789": 8.660645453366674e-14,
        "6000@7052": 2.0801503062603028e-13,
        "6000@7315": 3.1831294349370645e-13,
        "6000@7578": 3.050093317279441e-13,
        "6000@7841": 2.8519155322055265e-13,
        "6000@8104": 7.962299741559907e-13,
        "6000@8367": 2.448194863428661e-12,
        "6000@8630": 7.934536707987507e-12,
        "6000@8894": 1.8091071002357643e-11,
        "6000@9157": 4.141030684113445e-11,
        "6000@9420": 8.038931821248631e-11,
        "6000@9683": 1.4862149551240705e-10,
        "6000@9946": 2.719334010222301e-10,
        "6000@10209": 4.5411040739564233e-10,
        "6000@10472": 7.642294954519472e-10,
        "6000@10735": 1.2941878823887077e-09,
        "6000@10998": 2.1527189312999768e-09,
        "6000@11261": 3.6557956116034944e-09,
        "6000@11524": 6.5904221630009726e-09,
        "6000@11788": 1.3597621641530125e-08,
    },
    "36_set1_s5_0300@36_set1_s6_0300": {
        "6000@6263": 1.502310926981562e-15,
        "6000@6526": 3.528004066665358e-15,
        "6000@6789": 1.1839510577719326e-13,
        "6000@7052": 3.9910352916202327e-13,
        "6000@7315": 9.565222817152412e-13,
        "6000@7578": 1.7568608085772528e-12,
        "6000@7841": 3.0425136586542792e-12,
        "6000@8104": 4.945622337245856e-12,
        "6000@8367": 7.635926369266828e-12,
        "6000@8630": 1.2092012146838665e-11,
        "6000@8894": 1.773988425530862e-11,
        "6000@9157": 2.76294177760372e-11,
        "6000@9420": 4.058685080368717e-11,
        "6000@9683": 6.057463804218416e-11,
        "6000@9946": 9.18059177021887e-11,
        "6000@10209": 1.3301602391407266e-10,
        "6000@10472": 1.9777088363292356e-10,
        "6000@10735": 2.967146758712693e-10,
        "6000@10998": 4.3695988528047013e-10,
        "6000@11261": 6.463879255872134e-10,
        "6000@11524": 9.7845020310907e-10,
        "6000@11788": 1.5545618333395878e-09,
    },
}


int_cent_diff_dict_reversed= {}
mist_dict_reversed= {}
for a in int_cent_diff_dict.keys():
    for b in int_cent_diff_dict[a]:
        if b not in int_cent_diff_dict_reversed:
            int_cent_diff_dict_reversed[b] = {}
        int_cent_diff_dict_reversed[b][a] = int_cent_diff_dict[a][b]
for a in mist_dict.keys():
    for b in mist_dict[a]:
        if b not in mist_dict_reversed: 
            mist_dict_reversed[b] = {}
        mist_dict_reversed[b][a] = mist_dict[a][b]

with plt.style.context('ggplot'):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:6]
    line_styles = ['-',':', '--', '-.']
    combined_cycler = cycler(linestyle=line_styles)*cycler(color=colors)
    plt.rcParams['axes.prop_cycle'] = combined_cycler

    # for a, b in zip(int_cent_diff_dict.keys(), mist_dict.keys()):
    #     plt.scatter(int_cent_diff_dict[a].values(), mist_dict[b].values(), label=b)

    for a, b in zip(int_cent_diff_dict_reversed.keys(), mist_dict_reversed.keys()):
        plt.scatter(int_cent_diff_dict_reversed[a].values(), mist_dict_reversed[b].values(), label=b)


plt.yscale("log")
plt.xscale("log")
plt.xlabel("Int center diff")
plt.ylabel("Mismatch")
plt.legend()
plt.show()


In [ ]:
marker_styles = ['o', 's', 'D', '^', 'v', '>', '<', 'p', '*', 'h', 'H', '+', 'x', 'X', 'd']
from itertools import cycle
with plt.style.context('ggplot'):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:6]
    marker_cycler = cycle(marker_styles)
    color_cycler = cycle(colors)

    for a, b in zip(int_cent_diff_dict_reversed.keys(), mist_dict_reversed.keys()):
        marker = next(marker_cycler)
        color = next(color_cycler)
        plt.scatter(
            list(int_cent_diff_dict_reversed[a].values()), 
            list(mist_dict_reversed[b].values()), 
            label=b, 
            marker=marker, 
            color=color
        )

plt.yscale("log")
plt.xscale("log")
plt.legend()
plt.show()

#### Plot

In [ ]:
mistmatch_dict = {
    "34_main_1_257@34_main_2_257": 2.431234098674027e-06,
    "34_main_1_257@34_main_3_257": 5.292452936890286e-06,
    "34_main_1_257@34_main_4_257": 3.3682335436664007e-06,
    "34_main_1_257@34_main_5_257": 3.6149838367932497e-06,
    "34_main_1_257@34_main_6_257": 3.2930461894781516e-06,
    "34_main_2_257@34_main_3_257": 1.2224933554647637e-05,
    "34_main_2_257@34_main_4_257": 8.953595741992174e-06,
    "34_main_2_257@34_main_5_257": 9.415137037535157e-06,
    "34_main_2_257@34_main_6_257": 8.834944222807571e-06,
    "34_main_3_257@34_main_4_257": 2.545680500823587e-07,
    "34_main_3_257@34_main_5_257": 1.8376877710760494e-07,
    "34_main_3_257@34_main_6_257": 2.7530129680480055e-07,
    "34_main_4_257@34_main_5_257": 6.231294758007266e-09,
    "34_main_4_257@34_main_6_257": 6.3965423848261e-10,
    "34_main_5_257@34_main_6_257": 9.521390013079992e-09,
}
mistmatch_dict = {
    "main_s1_0300@main_s2_0300": 9.921421656552399e-06,
    "main_s1_0300@main_s3_0300": 6.492987547737189e-06,
    "main_s1_0300@main_s4_0300": 1.9526066396127114e-05,
    "main_s1_0300@main_s5_0300": 2.1209099661798887e-05,
    "main_s2_0300@main_s3_0300": 3.192025830842511e-05,
    "main_s2_0300@main_s4_0300": 5.691090779027955e-05,
    "main_s2_0300@main_s5_0300": 5.977668638684721e-05,
    "main_s3_0300@main_s4_0300": 3.6067778535634785e-06,
    "main_s3_0300@main_s5_0300": 4.355849761194729e-06,
    "main_s4_0300@main_s5_0300": 3.531497809009148e-08,
}

mistmatch_dict = {'main_s1_0300@main_s2_0300': 3.4689183790020886e-06,
 'main_s1_0300@main_s3_0300': 3.7088356535501958e-06,
 'main_s1_0300@main_s4_0300': 9.457825196789895e-06,
 'main_s1_0300@main_s5_0300': 1.0182194930464147e-05,
 'main_s1_0300@main_s6_0300': 9.93239407062679e-06,
 'main_s2_0300@main_s3_0300': 1.4133791487438332e-05,
 'main_s2_0300@main_s4_0300': 2.4224258009162826e-05,
 'main_s2_0300@main_s5_0300': 2.5382543378415016e-05,
 'main_s2_0300@main_s6_0300': 2.4988840715137517e-05,
 'main_s3_0300@main_s4_0300': 1.3599622689358382e-06,
 'main_s3_0300@main_s5_0300': 1.6454952312849937e-06,
 'main_s3_0300@main_s6_0300': 1.5473313287790117e-06,
 'main_s4_0300@main_s5_0300': 1.3597621641530125e-08,
 'main_s4_0300@main_s6_0300': 6.086974729152572e-09,
 'main_s5_0300@main_s6_0300': 1.5545618333395878e-09}

mistmatch_dict = {'main_s1_0436@main_s2_0436': 2.363270138292967e-06,
 'main_s1_0436@main_s3_0436': 3.0347331645582294e-06,
 'main_s1_0436@main_s4_0436': 7.328878110261153e-06,
 'main_s1_0436@main_s5_0436': 7.865183693134332e-06,
 'main_s1_0436@main_s6_0436': 7.668165537277777e-06,
 'main_s2_0436@main_s3_0436': 1.0611458171918711e-05,
 'main_s2_0436@main_s4_0436': 1.7909599127012312e-05,
 'main_s2_0436@main_s5_0436': 1.874732174367752e-05,
 'main_s2_0436@main_s6_0436': 1.8443284847852854e-05,
 'main_s3_0436@main_s4_0436': 9.5599347111606e-07,
 'main_s3_0436@main_s5_0436': 1.157533462406733e-06,
 'main_s3_0436@main_s6_0436': 1.0835193000021487e-06,
 'main_s4_0436@main_s5_0436': 9.633386340355138e-09,
 'main_s4_0436@main_s6_0436': 4.022801228332391e-09,
 'main_s5_0436@main_s6_0436': 1.2507653412560044e-09}

mistmatch_dict = {'main_s1_0801@main_s2_0801': 8.532556591175622e-07,
 'main_s1_0801@main_s3_0801': 1.734219413130314e-06,
 'main_s1_0801@main_s4_0801': 3.705930539730551e-06,
 'main_s1_0801@main_s5_0801': 3.945492421435203e-06,
 'main_s1_0801@main_s6_0801': 3.8444716737545304e-06,
 'main_s2_0801@main_s3_0801': 4.986906776637877e-06,
 'main_s2_0801@main_s4_0801': 8.090897065230428e-06,
 'main_s2_0801@main_s5_0801': 8.444319917530127e-06,
 'main_s2_0801@main_s6_0801': 8.29630069123743e-06,
 'main_s3_0801@main_s4_0801': 3.76157793472762e-07,
 'main_s3_0801@main_s5_0801': 4.556668794641239e-07,
 'main_s3_0801@main_s6_0801': 4.2196727469732837e-07,
 'main_s4_0801@main_s5_0801': 3.811110730635384e-09,
 'main_s4_0801@main_s6_0801': 1.3327843602143988e-09,
 'main_s5_0801@main_s6_0801': 6.554741145018479e-10}

# mistmatch_dict = {'main_s1_1070@main_s2_1070': 2.3583473072497105e-09,
#  'main_s1_1070@main_s3_1070': 3.9331953216567545e-09,
#  'main_s1_1070@main_s4_1070': 5.991673977883718e-09,
#  'main_s1_1070@main_s5_1070': 6.036376534040764e-09,
#  'main_s1_1070@main_s6_1070': 5.775934150761061e-09,
#  'main_s2_1070@main_s3_1070': 1.2168886742881799e-08,
#  'main_s2_1070@main_s4_1070': 1.5592698106144222e-08,
#  'main_s2_1070@main_s5_1070': 1.564965727902996e-08,
#  'main_s2_1070@main_s6_1070': 1.521760018608874e-08,
#  'main_s3_1070@main_s4_1070': 2.1649412808365774e-10,
#  'main_s3_1070@main_s5_1070': 2.2574224711210414e-10,
#  'main_s3_1070@main_s6_1070': 1.7903375393618902e-10,
#  'main_s4_1070@main_s5_1070': 4.993634137078929e-13,
#  'main_s4_1070@main_s6_1070': 3.199037510641133e-12,
#  'main_s5_1070@main_s6_1070': 3.2207309110832257e-12}


# mistmatch_dict = {
#     "6_set1_L6s1_0257@6_set1_L6s2_0257": 9.691173399793742e-09,
#     "6_set1_L6s1_0257@6_set1_L6s3_0257": 5.582821623776524e-08,
#     "6_set1_L6s1_0257@6_set1_L6s4_0257": 9.634173231920067e-08,
#     "6_set1_L6s1_0257@6_set1_L6s5_0257": 1.0012000387332597e-07,
#     "6_set1_L6s1_0257@6_set1_L6s6_0257": 9.747267234242399e-08,
#     "6_set1_L6s2_0257@6_set1_L6s3_0257": 1.1176474885869486e-07,
#     "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.668258294750136e-07,
#     "6_set1_L6s2_0257@6_set1_L6s5_0257": 1.7177939577073267e-07,
#     "6_set1_L6s2_0257@6_set1_L6s6_0257": 1.6830341576626154e-07,
#     "6_set1_L6s3_0257@6_set1_L6s4_0257": 5.496792371651053e-09,
#     "6_set1_L6s3_0257@6_set1_L6s5_0257": 6.426830715542033e-09,
#     "6_set1_L6s3_0257@6_set1_L6s6_0257": 5.769846655140213e-09,
#     "6_set1_L6s4_0257@6_set1_L6s5_0257": 3.6726266095554097e-11,
#     "6_set1_L6s4_0257@6_set1_L6s6_0257": 4.509717888492286e-12,
#     "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.810591947900653e-11,
# }

# mistmatch_dict = {'29_diff_4_0250@29_diff_5_0250': 2.59621215886119e-09,
#  '29_diff_4_0250@29_diff_6_0250': 2.5710485477997795e-09,
#  '29_diff_4_0250@29_diff_7_0250': 4.115163463835284e-09,
#  '29_diff_4_0250@29_diff_8_0250': 5.545457991498795e-10,
#  '29_diff_4_0250@6_set1_L3s3_250': 1.0813003705393281e-09,
#  '29_diff_4_0250@6_set1_L6s3_250': 7.453437915133449e-08,
#  '29_diff_5_0250@29_diff_6_0250': 1.5245314016334496e-11,
#  '29_diff_5_0250@29_diff_7_0250': 1.3054108309792178e-08,
#  '29_diff_5_0250@29_diff_8_0250': 8.84534723733401e-10,
#  '29_diff_5_0250@6_set1_L3s3_250': 2.1072701615844928e-09,
#  '29_diff_5_0250@6_set1_L6s3_250': 1.0413485234913669e-07,
#  '29_diff_6_0250@29_diff_7_0250': 1.2961201255918778e-08,
#  '29_diff_6_0250@29_diff_8_0250': 8.35114014980279e-10,
#  '29_diff_6_0250@6_set1_L3s3_250': 1.870692735003319e-09,
#  '29_diff_6_0250@6_set1_L6s3_250': 1.037662365633765e-07,
#  '29_diff_7_0250@29_diff_8_0250': 7.2694701627200424e-09,
#  '29_diff_7_0250@6_set1_L3s3_250': 6.956558424734656e-09,
#  '29_diff_7_0250@6_set1_L6s3_250': 4.445371926845566e-08,
#  '29_diff_8_0250@6_set1_L3s3_250': 6.305699412803853e-10,
#  '29_diff_8_0250@6_set1_L6s3_250': 8.632535881920532e-08,
#  '6_set1_L3s3_250@6_set1_L6s3_250': 8.263000101741527e-08}

mistmatch_dict = {'6_set1_L6s3_0257@6_set1_L6s4_0257': 5.496792371651053e-09,
 '6_set1_L6s3_0257@6_set1_L6s5_0257': 6.426830715542033e-09,
 '6_set1_L6s3_0257@6_set1_L6s6_0257': 5.769846655140213e-09,
 '6_set1_L6s3_0257@L35higher_0257': 5.555645905793208e-09,
 '6_set1_L6s3_0257@L35lower_0257': 2.0104704224190356e-11,
 '6_set1_L6s3_0257@L45higher_0257': 6.415592728572626e-09,
 '6_set1_L6s4_0257@6_set1_L6s5_0257': 3.6726266095554097e-11,
 '6_set1_L6s4_0257@6_set1_L6s6_0257': 4.509717888492286e-12,
 '6_set1_L6s4_0257@L35higher_0257': 1.5739794462962766e-13,
 '6_set1_L6s4_0257@L35lower_0257': 6.177778966230982e-09,
 '6_set1_L6s4_0257@L45higher_0257': 3.586256038050209e-11,
 '6_set1_L6s5_0257@6_set1_L6s6_0257': 1.810591947900653e-11,
 '6_set1_L6s5_0257@L35higher_0257': 3.2095820642077646e-11,
 '6_set1_L6s5_0257@L35lower_0257': 7.161731609296002e-09,
 '6_set1_L6s5_0257@L45higher_0257': 4.994032711491492e-15,
 '6_set1_L6s6_0257@L35higher_0257': 3.2130285158738355e-12,
 '6_set1_L6s6_0257@L35lower_0257': 6.468791692509854e-09,
 '6_set1_L6s6_0257@L45higher_0257': 1.751022970889984e-11,
 'L35higher_0257@L35lower_0257': 6.242522418094593e-09,
 'L35higher_0257@L45higher_0257': 3.130371171625678e-11,
 'L35lower_0257@L45higher_0257': 7.153401016739958e-09}

diff_int_dict = diff_int

# After BMSTP fixing
# mistmatch_dict = {
#     "high_accuracy_Lev1_R0257@high_accuracy_Lev5_R0258": 2.603e-05,
#     "high_accuracy_Lev2_R0257@high_accuracy_Lev5_R0258": 4.574e-05,
#     "high_accuracy_Lev3_R0258@high_accuracy_Lev5_R0258": 1.351e-07,
#     "high_accuracy_Lev4_R0258@high_accuracy_Lev5_R0258": 6.447e-10,
# }

x = [val for val in diff_int_dict.values()]
y = [val for val in mistmatch_dict.values()]
# x = np.abs(x)
plt.ylabel("Mismatch")
plt.xlabel(f"Int Diff: {y_axis}")
plt.scatter(x, y, marker="o")
plt.xscale("log")
plt.yscale("log")
plt.show()


In [ ]:
diff_int_dict = {
    "6_set1_L6s1@6_set1_L6s2": 1.2858028858326789,
    "6_set1_L6s1@6_set1_L6s3": 2.952014392741012,
    "6_set1_L6s1@6_set1_L6s4": 3.8928176255488096,
    "6_set1_L6s1@6_set1_L6s5": 3.9671436667026736,
    "6_set1_L6s1@6_set1_L6s6": 3.914069956217513,
    "6_set1_L6s2@6_set1_L6s3": 4.233594523893546,
    "6_set1_L6s2@6_set1_L6s4": 5.176475788978051,
    "6_set1_L6s2@6_set1_L6s5": 5.250682223344887,
    "6_set1_L6s2@6_set1_L6s6": 5.197502238165702,
    "6_set1_L6s3@6_set1_L6s4": 0.9429371466287805,
    "6_set1_L6s3@6_set1_L6s5": 1.0171364351233168,
    "6_set1_L6s3@6_set1_L6s6": 0.9639556750278575,
    "6_set1_L6s4@6_set1_L6s5": 0.07440602543930704,
    "6_set1_L6s4@6_set1_L6s6": 0.02289045491236974,
    "6_set1_L6s5@6_set1_L6s6": 0.05318158328457974,
}

# diff_phi = {
diff_int_dict = {
 '6_set1_L6s1@6_set1_L6s2': 0.18291673620686666,
 '6_set1_L6s1@6_set1_L6s3': 0.42241567036236083,
 '6_set1_L6s1@6_set1_L6s4': 0.5564967966590947,
 '6_set1_L6s1@6_set1_L6s5': 0.5671463644311162,
 '6_set1_L6s1@6_set1_L6s6': 0.5595623478412944,
 '6_set1_L6s2@6_set1_L6s3': 0.6048938796815856,
 '6_set1_L6s2@6_set1_L6s4': 0.7392498744431973,
 '6_set1_L6s2@6_set1_L6s5': 0.7498889427836771,
 '6_set1_L6s2@6_set1_L6s6': 0.7422949585840114,
 '6_set1_L6s3@6_set1_L6s4': 0.1343604960424211,
 '6_set1_L6s3@6_set1_L6s5': 0.14499928674451112,
 '6_set1_L6s3@6_set1_L6s6': 0.13740503738952842,
 '6_set1_L6s4@6_set1_L6s5': 0.010660803068585521,
 '6_set1_L6s4@6_set1_L6s6': 0.0032864203962184035,
 '6_set1_L6s5@6_set1_L6s6': 0.007594258469580951}

mistmatch_dict = {
    "34_main_1_257@34_main_2_257": 2.431234098674027e-06,
    "34_main_1_257@34_main_3_257": 5.292452936890286e-06,
    "34_main_1_257@34_main_4_257": 3.3682335436664007e-06,
    "34_main_1_257@34_main_5_257": 3.6149838367932497e-06,
    "34_main_1_257@34_main_6_257": 3.2930461894781516e-06,
    "34_main_2_257@34_main_3_257": 1.2224933554647637e-05,
    "34_main_2_257@34_main_4_257": 8.953595741992174e-06,
    "34_main_2_257@34_main_5_257": 9.415137037535157e-06,
    "34_main_2_257@34_main_6_257": 8.834944222807571e-06,
    "34_main_3_257@34_main_4_257": 2.545680500823587e-07,
    "34_main_3_257@34_main_5_257": 1.8376877710760494e-07,
    "34_main_3_257@34_main_6_257": 2.7530129680480055e-07,
    "34_main_4_257@34_main_5_257": 6.231294758007266e-09,
    "34_main_4_257@34_main_6_257": 6.3965423848261e-10,
    "34_main_5_257@34_main_6_257": 9.521390013079992e-09,
}

mistmatch_dict_FR_all = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 9.691173399793742e-09,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 5.582821623776524e-08,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 9.634173231920067e-08,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 1.0012000387332597e-07,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 9.747267234242399e-08,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 1.1176474885869486e-07,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.668258294750136e-07,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 1.7177939577073267e-07,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 1.6830341576626154e-07,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 5.496792371651053e-09,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 6.426830715542033e-09,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 5.769846655140213e-09,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 3.6726266095554097e-11,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 4.509717888492286e-12,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.810591947900653e-11,
}
mistmatch_dict_FR_3 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 1.176258588817942e-08,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 6.165531185012641e-08,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 1.0651261262886684e-07,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 1.1124053411780667e-07,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 1.0928744390796267e-07,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 1.2474555910530625e-07,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.8620953058299842e-07,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 1.9221979319575064e-07,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 1.892722059434233e-07,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 6.330097059371655e-09,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 7.957686068520404e-09,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 8.201665623700853e-09,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 3.7222334803438665e-10,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 1.1900946097029719e-09,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 4.1755309956887246e-10,
}
mistmatch_dict_FR_4 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 4.0876736528390056e-08,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 2.4568427856263186e-07,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 4.228173276804809e-07,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 4.395691155964665e-07,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 4.28147262544654e-07,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 4.843832173288682e-07,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 7.237696025641046e-07,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 7.455332191591245e-07,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 7.30544109437928e-07,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 2.4035054658703722e-08,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 2.8255726158445044e-08,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 2.5508232018686196e-08,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 2.3046966213704724e-10,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 2.0107304992161404e-10,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.32760976243525e-10,
}
mistmatch_dict_FR_5 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 5.986315008341556e-06,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 8.746194906405613e-06,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 1.5468248992122915e-05,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 2.6358887100022743e-05,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 3.559502256598818e-05,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 3.4390890083687526e-06,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.1192658283801559e-05,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 2.0220105316254554e-05,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 2.8429052986874537e-05,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 4.496382861258997e-06,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 1.3253619549021454e-05,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 2.236338906935221e-05,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 4.4018005215639296e-06,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 1.2820626652204688e-05,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 3.7251104296524853e-06,
}

mistmatch_dict_FR_6 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 1.931007373824053e-06,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 3.1815504687829584e-06,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 3.5735224668999955e-06,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 5.14564266914458e-06,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 9.036105849235031e-06,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 2.1736240027795306e-06,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 3.5522855501308405e-06,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 4.971980059687422e-06,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 8.599596845437626e-06,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 6.870842012686385e-07,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 2.6549243914712077e-06,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 6.417413739876474e-06,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 1.1463076566244746e-06,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 4.256999589754828e-06,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.3612209394894923e-06,
}
mistmatch_dict_FR_7 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 0.01857643112835293,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 0.01577560284329549,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 0.018978635596919508,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 0.026428740367623316,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 0.04450042148671994,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 0.009998960263060793,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 0.017971297245776344,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 0.02358519522315965,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 0.04067588101847044,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 0.005946561824416723,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 0.015140001799637403,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 0.03348889914522287,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 0.007571306120762246,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 0.023108932718440913,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 0.007328929906606428,
}
mistmatch_dict_FR_8 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 0.0008436574718200077,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 0.0011948021047756044,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 0.0010416524780482038,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 0.0012101079339250042,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 0.0020797418165750814,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 0.0005219328987594512,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 0.0007417865285151334,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 0.0008851361054985489,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 0.0016800558898570797,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 0.00021021564410308817,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 0.0006245669646834853,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 0.0014236982870111218,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 0.00031973852799309984,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 0.0010042733170450983,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 0.000351805521060114,
}

def find_levs_from_key(key):
    parts = key.split("@")
    lev1 = int(parts[0].split("_")[-2][-1])
    lev2 = int(parts[1].split("_")[-2][-1])
    return lev1, lev2
def filtered_and_to_lev_dict(diff_int_dict,mistmatch_dict):
    x_out = []
    y_out = []
    for diff_int,mistmatch in zip(diff_int_dict.keys(),mistmatch_dict.keys()):
        # print(diff_int,mistmatch)
        lev1,lev2 = find_levs_from_key(mistmatch)
        # print(lev1,lev2)
        if lev1+1 == lev2:
            x_out.append(diff_int_dict[diff_int])
            y_out.append(mistmatch_dict[mistmatch])
    # print(x_out,y_out)
    return x_out,y_out



with plt.style.context("ggplot"):
    plt.rcParams["figure.figsize"] = (6, 4.5)
    plt.rcParams["figure.autolayout"] = True

    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_all), marker="o", label="2/all")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_3), marker="o", label="3")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_4), marker="o", label="4")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_5), marker="o", label="5")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_6), marker="o", label="6")

    # plt.scatter(x, mistmatch_dict_FR_all.values(), marker="o", label="all")
    # plt.scatter(x, mistmatch_dict_FR_3.values(), marker="o", label="3")
    # plt.scatter(x, mistmatch_dict_FR_4.values(), marker="o", label="4")
    # plt.scatter(x, mistmatch_dict_FR_5.values(), marker="o", label="5")
    # plt.scatter(x, mistmatch_dict_FR_6.values(), marker="o", label="6")
    # # plt.scatter(x, mistmatch_dict_FR_7.values(), marker="o", label="7")
    # # plt.scatter(x, mistmatch_dict_FR_8.values(), marker="o", label="8")

    plt.legend(loc="lower right")
    plt.xscale("log")
    plt.yscale("log")
    # plt.title("New Simulations")
    plt.ylabel("Finite Radius Mismatch")
    plt.xlabel("Integrated difference between the centers of the bhs")


    # save_name = Path("/groups/sxs/hchaudha/scripts/code_for_paper_plots/mismatch")/"mismatch_vs_center_diff_set1.pdf"
    # plt.savefig(save_name, dpi=300)
    plt.show()


In [ ]:
diff_int_dict = {
    "34_master_L16_1@34_master_L16_2": 18.679696277968215,
    "34_master_L16_1@34_master_L16_3": 29.877487724657676,
    "34_master_L16_1@34_master_L16_4": 23.14608562703885,
    "34_master_L16_1@34_master_L16_5": 24.137425389872167,
    "34_master_L16_1@34_master_L16_6": 22.8503980285938,
    "34_master_L16_2@34_master_L16_3": 46.4274473494519,
    "34_master_L16_2@34_master_L16_4": 39.69417377095836,
    "34_master_L16_2@34_master_L16_5": 40.68676748326586,
    "34_master_L16_2@34_master_L16_6": 39.39940546563903,
    "34_master_L16_3@34_master_L16_4": 6.733334181817364,
    "34_master_L16_3@34_master_L16_5": 5.74108592147311,
    "34_master_L16_3@34_master_L16_6": 7.028478217020872,
    "34_master_L16_4@34_master_L16_5": 1.0032431185713757,
    "34_master_L16_4@34_master_L16_6": 0.30397876082827,
    "34_master_L16_5@34_master_L16_6": 1.2874399058020611,
}

mistmatch_dict_FR_all = {
    "main_s1_0257@main_s2_0257": 2.264474177411709e-06,
    "main_s1_0257@main_s3_0257": 5.281078661057142e-06,
    "main_s1_0257@main_s4_0257": 3.1546480807368913e-06,
    "main_s1_0257@main_s5_0257": 3.429082324735192e-06,
    "main_s1_0257@main_s6_0257": 3.0753154316035434e-06,
    "main_s2_0257@main_s3_0257": 1.359871836183846e-05,
    "main_s2_0257@main_s4_0257": 9.941788897569273e-06,
    "main_s2_0257@main_s5_0257": 1.0461132443957904e-05,
    "main_s2_0257@main_s6_0257": 9.81546182183829e-06,
    "main_s3_0257@main_s4_0257": 2.857217377646894e-07,
    "main_s3_0257@main_s5_0257": 2.0578584783277007e-07,
    "main_s3_0257@main_s6_0257": 3.0793825115578513e-07,
    "main_s4_0257@main_s5_0257": 6.76401737969155e-09,
    "main_s4_0257@main_s6_0257": 5.514855892662288e-10,
    "main_s5_0257@main_s6_0257": 1.0283170582733175e-08,
}
mistmatch_dict_FR_3 = {
    "main_s1_0257@main_s2_0257": 2.5245654198304116e-06,
    "main_s1_0257@main_s3_0257": 7.340342871167533e-06,
    "main_s1_0257@main_s4_0257": 4.880209580253311e-06,
    "main_s1_0257@main_s5_0257": 5.235973491779865e-06,
    "main_s1_0257@main_s6_0257": 4.820559395652378e-06,
    "main_s2_0257@main_s3_0257": 1.5074094983275444e-05,
    "main_s2_0257@main_s4_0257": 1.1091963462423315e-05,
    "main_s2_0257@main_s5_0257": 1.1688045664866132e-05,
    "main_s2_0257@main_s6_0257": 1.0987723514917608e-05,
    "main_s3_0257@main_s4_0257": 3.106162671833652e-07,
    "main_s3_0257@main_s5_0257": 2.2982500522203126e-07,
    "main_s3_0257@main_s6_0257": 3.5755325856278667e-07,
    "main_s4_0257@main_s5_0257": 1.532390901082463e-08,
    "main_s4_0257@main_s6_0257": 2.4991685737779374e-08,
    "main_s5_0257@main_s6_0257": 2.2356948947066446e-08,
}
mistmatch_dict_FR_4 = {
    "main_s1_0257@main_s2_0257": 1.1221932960826675e-05,
    "main_s1_0257@main_s3_0257": 2.1717887377405357e-05,
    "main_s1_0257@main_s4_0257": 1.2583583614718435e-05,
    "main_s1_0257@main_s5_0257": 1.3759456637084248e-05,
    "main_s1_0257@main_s6_0257": 1.2260729694168781e-05,
    "main_s2_0257@main_s3_0257": 6.093325151208205e-05,
    "main_s2_0257@main_s4_0257": 4.445243782427762e-05,
    "main_s2_0257@main_s5_0257": 4.680137424057483e-05,
    "main_s2_0257@main_s6_0257": 4.39115059943602e-05,
    "main_s3_0257@main_s4_0257": 1.2977344015944192e-06,
    "main_s3_0257@main_s5_0257": 9.339375999507775e-07,
    "main_s3_0257@main_s6_0257": 1.3942934211857813e-06,
    "main_s4_0257@main_s5_0257": 3.112474354313072e-08,
    "main_s4_0257@main_s6_0257": 2.719172659051033e-09,
    "main_s5_0257@main_s6_0257": 4.622434338020812e-08,
}
mistmatch_dict_FR_5 = {
    "main_s1_0257@main_s2_0257": 0.00022582777908583022,
    "main_s1_0257@main_s3_0257": 0.0008115846412258022,
    "main_s1_0257@main_s4_0257": 0.0008340321338808046,
    "main_s1_0257@main_s5_0257": 0.0008404289392315733,
    "main_s1_0257@main_s6_0257": 0.0008519943401235525,
    "main_s2_0257@main_s3_0257": 0.0007596224513639449,
    "main_s2_0257@main_s4_0257": 0.0007461260147103551,
    "main_s2_0257@main_s5_0257": 0.0007524124117603355,
    "main_s2_0257@main_s6_0257": 0.0007595214456137661,
    "main_s3_0257@main_s4_0257": 5.72934589683565e-06,
    "main_s3_0257@main_s5_0257": 1.0248124595860536e-05,
    "main_s3_0257@main_s6_0257": 2.1365777465709717e-05,
    "main_s4_0257@main_s5_0257": 4.907560345229431e-06,
    "main_s4_0257@main_s6_0257": 1.4969220823198804e-05,
    "main_s5_0257@main_s6_0257": 7.157997697776172e-06,
}

mistmatch_dict_FR_6 = {
    "main_s1_0257@main_s2_0257": 3.384136349521663e-05,
    "main_s1_0257@main_s3_0257": 8.960797973877853e-05,
    "main_s1_0257@main_s4_0257": 7.759837566533006e-05,
    "main_s1_0257@main_s5_0257": 7.852177499147466e-05,
    "main_s1_0257@main_s6_0257": 7.600291991995957e-05,
    "main_s2_0257@main_s3_0257": 0.00010158668042783623,
    "main_s2_0257@main_s4_0257": 7.881926854204633e-05,
    "main_s2_0257@main_s5_0257": 8.16573551070029e-05,
    "main_s2_0257@main_s6_0257": 7.768390917954209e-05,
    "main_s3_0257@main_s4_0257": 2.702479053631406e-06,
    "main_s3_0257@main_s5_0257": 2.9704999317513145e-06,
    "main_s3_0257@main_s6_0257": 4.976589973474067e-06,
    "main_s4_0257@main_s5_0257": 5.229341009212379e-07,
    "main_s4_0257@main_s6_0257": 1.9642224133652187e-06,
    "main_s5_0257@main_s6_0257": 1.097690610626177e-06,
}
mistmatch_dict_FR_7 = {
    "main_s1_0257@main_s2_0257": 0.14345548821728304,
    "main_s1_0257@main_s3_0257": 0.21381016093105412,
    "main_s1_0257@main_s4_0257": 0.2095762730431205,
    "main_s1_0257@main_s5_0257": 0.22134986036819626,
    "main_s1_0257@main_s6_0257": 0.20943169737185927,
    "main_s2_0257@main_s3_0257": 0.15525556513692765,
    "main_s2_0257@main_s4_0257": 0.15111151578689128,
    "main_s2_0257@main_s5_0257": 0.16436438142159512,
    "main_s2_0257@main_s6_0257": 0.15236660129630247,
    "main_s3_0257@main_s4_0257": 0.09287494916687139,
    "main_s3_0257@main_s5_0257": 0.11029417593262407,
    "main_s3_0257@main_s6_0257": 0.10466599778724521,
    "main_s4_0257@main_s5_0257": 0.025303307198614275,
    "main_s4_0257@main_s6_0257": 0.039107582776304085,
    "main_s5_0257@main_s6_0257": 0.04026686054943817,
}
mistmatch_dict_FR_8 = {
    "main_s1_0257@main_s2_0257": 965.7914395414202,
    "main_s1_0257@main_s3_0257": 1011.8068881949827,
    "main_s1_0257@main_s4_0257": 1029.0009264616317,
    "main_s1_0257@main_s5_0257": 1031.8071909144708,
    "main_s1_0257@main_s6_0257": 1033.3061440921952,
    "main_s2_0257@main_s3_0257": 0.005354273564666649,
    "main_s2_0257@main_s4_0257": 0.0060241328536135876,
    "main_s2_0257@main_s5_0257": 0.007053857704784392,
    "main_s2_0257@main_s6_0257": 0.008801589715994824,
    "main_s3_0257@main_s4_0257": 0.001317864505275456,
    "main_s3_0257@main_s5_0257": 0.0025483446879863616,
    "main_s3_0257@main_s6_0257": 0.004245256185096948,
    "main_s4_0257@main_s5_0257": 0.000938979324364733,
    "main_s4_0257@main_s6_0257": 0.0027442479086632796,
    "main_s5_0257@main_s6_0257": 0.0015617443685368628,
}

def find_levs_from_key(key):
    parts = key.split("@")
    lev1 = int(parts[0].split("_")[-2][-1])
    lev2 = int(parts[1].split("_")[-2][-1])
    return lev1, lev2
def filtered_and_to_lev_dict(diff_int_dict,mistmatch_dict):
    x_out = []
    y_out = []
    for diff_int,mistmatch in zip(diff_int_dict.keys(),mistmatch_dict.keys()):
        # print(diff_int,mistmatch)
        lev1,lev2 = find_levs_from_key(mistmatch)
        # print(lev1,lev2)
        if lev1+1 == lev2:
            x_out.append(diff_int_dict[diff_int])
            y_out.append(mistmatch_dict[mistmatch])
    # print(x_out,y_out)
    return x_out,y_out



with plt.style.context("ggplot"):
    plt.rcParams["figure.figsize"] = (6, 4.5)
    plt.rcParams["figure.autolayout"] = True

    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_all), marker="o", label="2/all")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_3), marker="o", label="3")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_4), marker="o", label="4")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_5), marker="o", label="5")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_6), marker="o", label="6")

    # plt.scatter(x, mistmatch_dict_FR_all.values(), marker="o", label="all")
    # plt.scatter(x, mistmatch_dict_FR_3.values(), marker="o", label="3")
    # plt.scatter(x, mistmatch_dict_FR_4.values(), marker="o", label="4")
    # plt.scatter(x, mistmatch_dict_FR_5.values(), marker="o", label="5")
    # plt.scatter(x, mistmatch_dict_FR_6.values(), marker="o", label="6")
    # # plt.scatter(x, mistmatch_dict_FR_7.values(), marker="o", label="7")
    # # plt.scatter(x, mistmatch_dict_FR_8.values(), marker="o", label="8")

    plt.legend(loc="lower right")
    plt.xscale("log")
    plt.yscale("log")
    # plt.title("Old Simulations")
    plt.ylabel("Finite Radius Mismatch")
    plt.xlabel("Integrated difference between the centers of the bhs")


    save_name = Path("/groups/sxs/hchaudha/scripts/code_for_paper_plots/mismatch")/"mismatch_vs_center_diff_main.pdf"
    plt.savefig(save_name, dpi=300)
    plt.show()

In [ ]:
mistmatch_dict = {
    "34_main_1_257@34_main_2_257": 2.431234098674027e-06,
    "34_main_1_257@34_main_3_257": 5.292452936890286e-06,
    "34_main_1_257@34_main_4_257": 3.3682335436664007e-06,
    "34_main_1_257@34_main_5_257": 3.6149838367932497e-06,
    "34_main_1_257@34_main_6_257": 3.2930461894781516e-06,
    "34_main_2_257@34_main_3_257": 1.2224933554647637e-05,
    "34_main_2_257@34_main_4_257": 8.953595741992174e-06,
    "34_main_2_257@34_main_5_257": 9.415137037535157e-06,
    "34_main_2_257@34_main_6_257": 8.834944222807571e-06,
    "34_main_3_257@34_main_4_257": 2.545680500823587e-07,
    "34_main_3_257@34_main_5_257": 1.8376877710760494e-07,
    "34_main_3_257@34_main_6_257": 2.7530129680480055e-07,
    "34_main_4_257@34_main_5_257": 6.231294758007266e-09,
    "34_main_4_257@34_main_6_257": 6.3965423848261e-10,
    "34_main_5_257@34_main_6_257": 9.521390013079992e-09,
}

center_int_dict = {
    "34_master_L16_1@34_master_L16_2": 18.675446041002207,
    "34_master_L16_1@34_master_L16_3": 29.87826664240784,
    "34_master_L16_1@34_master_L16_4": 23.146773573643824,
    "34_master_L16_1@34_master_L16_5": 24.13814090990057,
    "34_master_L16_1@34_master_L16_6": 22.851117246074708,
    "34_master_L16_2@34_master_L16_3": 46.42734741655651,
    "34_master_L16_2@34_master_L16_4": 39.69408063055001,
    "34_master_L16_2@34_master_L16_5": 40.68668781676757,
    "34_master_L16_2@34_master_L16_6": 39.39932870066914,
    "34_master_L16_3@34_master_L16_4": 6.733337369212714,
    "34_master_L16_3@34_master_L16_5": 5.741082947733354,
    "34_master_L16_3@34_master_L16_6": 7.028472447367607,
    "34_master_L16_4@34_master_L16_5": 1.0032793233759483,
    "34_master_L16_4@34_master_L16_6": 0.3039198554492159,
    "34_master_L16_5@34_master_L16_6": 1.2874378872604013,
}
ProperSepHorizonDiff = {
    "34_master_L16_1@34_master_L16_2": 0.16738782951283918,
    "34_master_L16_1@34_master_L16_3": 0.44092309833436677,
    "34_master_L16_1@34_master_L16_4": 0.3981014909487843,
    "34_master_L16_1@34_master_L16_5": 0.3989720358799253,
    "34_master_L16_1@34_master_L16_6": 0.391467175725209,
    "34_master_L16_2@34_master_L16_3": 0.2735573431924383,
    "34_master_L16_2@34_master_L16_4": 0.2307535062908818,
    "34_master_L16_2@34_master_L16_5": 0.23168279517450918,
    "34_master_L16_2@34_master_L16_6": 0.2241960814372271,
    "34_master_L16_3@34_master_L16_4": 0.04283349663345011,
    "34_master_L16_3@34_master_L16_5": 0.04197076100334307,
    "34_master_L16_3@34_master_L16_6": 0.049477203563720686,
    "34_master_L16_4@34_master_L16_5": 0.003802382912505564,
    "34_master_L16_4@34_master_L16_6": 0.006644060971648713,
    "34_master_L16_5@34_master_L16_6": 0.007506531365181128,
}

mistmatch_dict2 = {
    "6_set1_L3s0_250@6_set1_L3s1_250": 2.914547417946724e-07,
    "6_set1_L3s0_250@6_set1_L3s2_250": 4.045394770033672e-07,
    "6_set1_L3s0_250@6_set1_L3s3_250": 1.0986966028010009e-07,
    "6_set1_L3s0_250@6_set1_L6s0_250": 3.287767858965617e-07,
    "6_set1_L3s0_250@6_set1_L6s1_250": 1.0880213532684918e-07,
    "6_set1_L3s0_250@6_set1_L6s2_250": 1.709873359771204e-07,
    "6_set1_L3s0_250@6_set1_L6s3_250": 2.77975665637271e-08,
    "6_set1_L3s0_250@6_set1_L6s4_250": 2.3052082573183536e-08,
    "6_set1_L3s0_250@6_set1_L6s5_250": 2.315488178003834e-08,
    "6_set1_L3s0_250@6_set1_L6s6_250": 2.3201082653673834e-08,
    "6_set1_L3s1_250@6_set1_L3s2_250": 1.0262396257579386e-08,
    "6_set1_L3s1_250@6_set1_L3s3_250": 4.424095574984919e-08,
    "6_set1_L3s1_250@6_set1_L6s0_250": 8.86135669598806e-07,
    "6_set1_L3s1_250@6_set1_L6s1_250": 7.611119804681456e-08,
    "6_set1_L3s1_250@6_set1_L6s2_250": 3.668031803501378e-08,
    "6_set1_L3s1_250@6_set1_L6s3_250": 2.3354157737640558e-07,
    "6_set1_L3s1_250@6_set1_L6s4_250": 3.0432377304771097e-07,
    "6_set1_L3s1_250@6_set1_L6s5_250": 3.103701722368893e-07,
    "6_set1_L3s1_250@6_set1_L6s6_250": 3.0602682181612255e-07,
    "6_set1_L3s2_250@6_set1_L3s3_250": 9.385210330103113e-08,
    "6_set1_L3s2_250@6_set1_L6s0_250": 1.0572106651663791e-06,
    "6_set1_L3s2_250@6_set1_L6s1_250": 1.3529142799468035e-07,
    "6_set1_L3s2_250@6_set1_L6s2_250": 7.828207739807506e-08,
    "6_set1_L3s2_250@6_set1_L6s3_250": 3.3660575280438017e-07,
    "6_set1_L3s2_250@6_set1_L6s4_250": 4.210876113032951e-07,
    "6_set1_L3s2_250@6_set1_L6s5_250": 4.28195150365541e-07,
    "6_set1_L3s2_250@6_set1_L6s6_250": 4.2317723672218817e-07,
    "6_set1_L3s3_250@6_set1_L6s0_250": 6.006562891150375e-07,
    "6_set1_L3s3_250@6_set1_L6s1_250": 1.8165727326648093e-08,
    "6_set1_L3s3_250@6_set1_L6s2_250": 1.823223343563967e-08,
    "6_set1_L3s3_250@6_set1_L6s3_250": 8.263000101741527e-08,
    "6_set1_L3s3_250@6_set1_L6s4_250": 1.241072594145097e-07,
    "6_set1_L3s3_250@6_set1_L6s5_250": 1.2778879441835902e-07,
    "6_set1_L3s3_250@6_set1_L6s6_250": 1.252071598539424e-07,
    "6_set1_L6s0_250@6_set1_L6s1_250": 5.540326938122578e-07,
    "6_set1_L6s0_250@6_set1_L6s2_250": 6.533489885234821e-07,
    "6_set1_L6s0_250@6_set1_L6s3_250": 3.455301783433466e-07,
    "6_set1_L6s0_250@6_set1_L6s4_250": 3.0152720001801325e-07,
    "6_set1_L6s0_250@6_set1_L6s5_250": 2.9971801335273965e-07,
    "6_set1_L6s0_250@6_set1_L6s6_250": 3.01014881360718e-07,
    "6_set1_L6s1_250@6_set1_L6s2_250": 1.0670351873460054e-08,
    "6_set1_L6s1_250@6_set1_L6s3_250": 5.153201302229405e-08,
    "6_set1_L6s1_250@6_set1_L6s4_250": 8.767840458897763e-08,
    "6_set1_L6s1_250@6_set1_L6s5_250": 9.103826703722838e-08,
    "6_set1_L6s1_250@6_set1_L6s6_250": 8.878230493466232e-08,
    "6_set1_L6s2_250@6_set1_L6s3_250": 1.0052959142934316e-07,
    "6_set1_L6s2_250@6_set1_L6s4_250": 1.5003295988390529e-07,
    "6_set1_L6s2_250@6_set1_L6s5_250": 1.5435739522264284e-07,
    "6_set1_L6s2_250@6_set1_L6s6_250": 1.511916359139237e-07,
    "6_set1_L6s3_250@6_set1_L6s4_250": 4.972096038051328e-09,
    "6_set1_L6s3_250@6_set1_L6s5_250": 5.785412740608588e-09,
    "6_set1_L6s3_250@6_set1_L6s6_250": 5.1936316893097015e-09,
    "6_set1_L6s4_250@6_set1_L6s5_250": 4.9057576515983685e-11,
    "6_set1_L6s4_250@6_set1_L6s6_250": 2.3629009273018322e-11,
    "6_set1_L6s5_250@6_set1_L6s6_250": 2.497406806820712e-11,
}

center_int_dict2 = {
    "6_set1_L3s0@6_set1_L3s1": 7.166034617999204,
    "6_set1_L3s0@6_set1_L3s2": 8.521299516331075,
    "6_set1_L3s0@6_set1_L3s3": 4.416230045970116,
    "6_set1_L3s0@6_set1_L6s0": 4.656179611191159,
    "6_set1_L3s0@6_set1_L6s1": 4.4528849703599125,
    "6_set1_L3s0@6_set1_L6s2": 5.376672729675688,
    "6_set1_L3s0@6_set1_L6s3": 2.566192520402834,
    "6_set1_L3s0@6_set1_L6s4": 2.2959852225742354,
    "6_set1_L3s0@6_set1_L6s5": 2.276547241819229,
    "6_set1_L3s0@6_set1_L6s6": 2.2828387703187776,
    "6_set1_L3s1@6_set1_L3s2": 1.3565390968766091,
    "6_set1_L3s1@6_set1_L3s3": 2.7535144783308643,
    "6_set1_L3s1@6_set1_L6s0": 11.814648237745471,
    "6_set1_L3s1@6_set1_L6s1": 4.4507768508142345,
    "6_set1_L3s1@6_set1_L6s2": 3.167928207349557,
    "6_set1_L3s1@6_set1_L6s3": 7.396380357400776,
    "6_set1_L3s1@6_set1_L6s4": 8.33870040026953,
    "6_set1_L3s1@6_set1_L6s5": 8.4128465877579,
    "6_set1_L3s1@6_set1_L6s6": 8.35969500745689,
    "6_set1_L3s2@6_set1_L3s3": 4.105861262693099,
    "6_set1_L3s2@6_set1_L6s0": 13.169976407192102,
    "6_set1_L3s2@6_set1_L6s1": 5.805146943883765,
    "6_set1_L3s2@6_set1_L6s2": 4.521391827123608,
    "6_set1_L3s2@6_set1_L6s3": 8.751248128998984,
    "6_set1_L3s2@6_set1_L6s4": 9.693668579023154,
    "6_set1_L3s2@6_set1_L6s5": 9.767821516590862,
    "6_set1_L3s2@6_set1_L6s6": 9.714665285934865,
    "6_set1_L3s3@6_set1_L6s0": 9.066248291254368,
    "6_set1_L3s3@6_set1_L6s1": 1.862183472544896,
    "6_set1_L3s3@6_set1_L6s2": 1.885874760774389,
    "6_set1_L3s3@6_set1_L6s3": 4.648903642558594,
    "6_set1_L3s3@6_set1_L6s4": 5.59089858742151,
    "6_set1_L3s3@6_set1_L6s5": 5.665017424961901,
    "6_set1_L3s3@6_set1_L6s6": 5.611880084194546,
    "6_set1_L6s0@6_set1_L6s1": 7.37253731118235,
    "6_set1_L6s0@6_set1_L6s2": 8.656678148295981,
    "6_set1_L6s0@6_set1_L6s3": 4.423854073187556,
    "6_set1_L6s0@6_set1_L6s4": 3.4815130369892002,
    "6_set1_L6s0@6_set1_L6s5": 3.407300823777067,
    "6_set1_L6s0@6_set1_L6s6": 3.4604244049909236,
    "6_set1_L6s1@6_set1_L6s2": 1.2858492147426237,
    "6_set1_L6s1@6_set1_L6s3": 2.9519509424537276,
    "6_set1_L6s1@6_set1_L6s4": 3.8927614950025387,
    "6_set1_L6s1@6_set1_L6s5": 3.9670869894153418,
    "6_set1_L6s1@6_set1_L6s6": 3.914013175892593,
    "6_set1_L6s2@6_set1_L6s3": 4.233581462649017,
    "6_set1_L6s2@6_set1_L6s4": 5.176466118579964,
    "6_set1_L6s2@6_set1_L6s5": 5.25067204884617,
    "6_set1_L6s2@6_set1_L6s6": 5.197491386491477,
    "6_set1_L6s3@6_set1_L6s4": 0.9429400369565729,
    "6_set1_L6s3@6_set1_L6s5": 1.017138745462085,
    "6_set1_L6s3@6_set1_L6s6": 0.9639573495875533,
    "6_set1_L6s4@6_set1_L6s5": 0.0744050297370248,
    "6_set1_L6s4@6_set1_L6s6": 0.02288851474935882,
    "6_set1_L6s5@6_set1_L6s6": 0.05318223609968889,
}

ProperSepHorizonDiff2 = {
    "6_set1_L3s0@6_set1_L3s1": 0.8720927254043036,
    "6_set1_L3s0@6_set1_L3s2": 1.0227228656442406,
    "6_set1_L3s0@6_set1_L3s3": 0.5549950601405849,
    "6_set1_L3s0@6_set1_L6s0": 0.41390572931051417,
    "6_set1_L3s0@6_set1_L6s1": 0.8443554353061012,
    "6_set1_L3s0@6_set1_L6s2": 0.959085025240764,
    "6_set1_L3s0@6_set1_L6s3": 0.6169540446359396,
    "6_set1_L3s0@6_set1_L6s4": 0.5650716919314867,
    "6_set1_L3s0@6_set1_L6s5": 0.5605011355952645,
    "6_set1_L3s0@6_set1_L6s6": 0.5625283994577718,
    "6_set1_L3s1@6_set1_L3s2": 0.15150687010201747,
    "6_set1_L3s1@6_set1_L3s3": 0.31945068132371546,
    "6_set1_L3s1@6_set1_L6s0": 1.1193199388141872,
    "6_set1_L3s1@6_set1_L6s1": 0.40523242623691014,
    "6_set1_L3s1@6_set1_L6s2": 0.38404346159477587,
    "6_set1_L3s1@6_set1_L6s3": 0.6303825040672069,
    "6_set1_L3s1@6_set1_L6s4": 0.7085536093874417,
    "6_set1_L3s1@6_set1_L6s5": 0.7158144794994897,
    "6_set1_L3s1@6_set1_L6s6": 0.711134711972426,
    "6_set1_L3s2@6_set1_L3s3": 0.46855651406525267,
    "6_set1_L3s2@6_set1_L6s0": 1.2650252921339775,
    "6_set1_L3s2@6_set1_L6s1": 0.4590472740272989,
    "6_set1_L3s2@6_set1_L6s2": 0.4064689672230193,
    "6_set1_L3s2@6_set1_L6s3": 0.7298826889050499,
    "6_set1_L3s2@6_set1_L6s4": 0.8292574109469746,
    "6_set1_L3s2@6_set1_L6s5": 0.8385231970100682,
    "6_set1_L3s2@6_set1_L6s6": 0.8328330637545828,
    "6_set1_L3s3@6_set1_L6s0": 0.8112808237052758,
    "6_set1_L3s3@6_set1_L6s1": 0.34861160394644514,
    "6_set1_L3s3@6_set1_L6s2": 0.4361589294088951,
    "6_set1_L3s3@6_set1_L6s3": 0.4139223261169204,
    "6_set1_L3s3@6_set1_L6s4": 0.46720911234246193,
    "6_set1_L3s3@6_set1_L6s5": 0.4722097201296593,
    "6_set1_L3s3@6_set1_L6s6": 0.4686554878747261,
    "6_set1_L6s0@6_set1_L6s1": 0.894403244124587,
    "6_set1_L6s0@6_set1_L6s2": 1.0386210532087374,
    "6_set1_L6s0@6_set1_L6s3": 0.5562725220344316,
    "6_set1_L6s0@6_set1_L6s4": 0.452238785215,
    "6_set1_L6s0@6_set1_L6s5": 0.4432219481219483,
    "6_set1_L6s0@6_set1_L6s6": 0.44863822644756574,
    "6_set1_L6s1@6_set1_L6s2": 0.14550083811809925,
    "6_set1_L6s1@6_set1_L6s3": 0.34012943305364457,
    "6_set1_L6s1@6_set1_L6s4": 0.44694135784876904,
    "6_set1_L6s1@6_set1_L6s5": 0.4564484982351037,
    "6_set1_L6s1@6_set1_L6s6": 0.4505858309741063,
    "6_set1_L6s2@6_set1_L6s3": 0.48322445588626606,
    "6_set1_L6s2@6_set1_L6s4": 0.5911846440240354,
    "6_set1_L6s2@6_set1_L6s5": 0.6006463430357745,
    "6_set1_L6s2@6_set1_L6s6": 0.5947348177036096,
    "6_set1_L6s3@6_set1_L6s4": 0.10796650862762883,
    "6_set1_L6s3@6_set1_L6s5": 0.11742861409142039,
    "6_set1_L6s3@6_set1_L6s6": 0.11151670195388252,
    "6_set1_L6s4@6_set1_L6s5": 0.009527212046179418,
    "6_set1_L6s4@6_set1_L6s6": 0.00386635164514922,
    "6_set1_L6s5@6_set1_L6s6": 0.005911915564210742,
}


def get_lev(key):
    leva, levb = key.split("@")
    # print(f"Key: {key}, leva: {leva}, levb: {levb}")
    lev_a = int(leva[-1])
    lev_b = int(levb[-1])
    return lev_a, lev_b

fig, ax = plt.subplots(figsize=(8, 6))

joined_center_int_dict = {**center_int_dict, **center_int_dict2}
joined_mistmatch_dict = {**mistmatch_dict, **mistmatch_dict2}

# x = [val for val in ProperSepHorizonDiff.values()]
x = [val for val in center_int_dict.values()]
y = [val for val in mistmatch_dict.values()]
keys = [key for key in center_int_dict if ("L3" not in key) and ("s0" not in key)]
# sizes = [get_lev(key)[0] * get_lev(key)[1] * 5 + 10 for key in keys]
ax.scatter(x, y, marker="o", label="Main")

# x = [
#     ProperSepHorizonDiff2[key]
#     for key in ProperSepHorizonDiff2
#     if ("L3" not in key) and ("s0" not in key)
# ]
x = [
    center_int_dict2[key]
    for key in center_int_dict2
    if ("L3" not in key) and ("s0" not in key)
]
y = [
    mistmatch_dict2[key]
    for key in mistmatch_dict2
    if ("L3" not in key) and ("s0" not in key)
]
keys = [key for key in center_int_dict2 if ("L3" not in key) and ("s0" not in key)]
# sizes = [get_lev(key)[0] * get_lev(key)[1] * 5 + 10 for key in keys]
ax.scatter(x, y, marker="x", label="Set1")
# for xi, yi, key in zip(x, y, keys):
#     leva, levb = get_lev(key)
#     label = f"({leva},{levb})"
#     amp = 10
#     rx,ry = 0, (levb+leva)*2 * amp
#     plt.annotate(
#         label,
#         xy=(xi, yi),
#         xycoords="data",
#         xytext=(rx, ry),
#         textcoords="offset points",
#         ha="left",
#         arrowprops=dict(arrowstyle="-", lw=1, color="gray"),
#     )


# x = [
#     ProperSepHorizonDiff2[key]
#     for key in ProperSepHorizonDiff2
#     if ("L3" in key) and ("L6" not in key)
# ]
# x = [
#     center_int_dict2[key]
#     for key in center_int_dict2
#     if ("L3" in key) and ("L6" not in key)
# ]
# y = [
#     mistmatch_dict2[key]
#     for key in mistmatch_dict2
#     if ("L3" in key) and ("L6" not in key)
# ]
# plt.scatter(x, y, marker="*", label="Set1_L3")

ax.set_ylabel("Mismatch")
ax.set_xlabel(f"Int Diff: center")
# ax.set_xlabel(f"Int Diff: {y_axis}")
ax.set_xscale("log")
ax.set_yscale("log")
plt.legend()
plt.show()


### Noise in things

In [ ]:
runs_data_dict.keys()

In [ ]:
# data = runs_data_dict['26_set1_L6_long'].copy()
# data = runs_data_dict['26_main_L6_long'].copy()
data = runs_data_dict['6_set1_L6s6'].copy()
# data = runs_data_dict['6_set1_L3s3'].copy()
# data = runs_data_dict['high_accuracy_L3_main'].copy()

In [ ]:
y_key = 'ArealMass'
scipy_or_np = 'scipy'
window = 6 # Choose appropriate window size
moving_avg_len = None
moving_avg_len = 50
# scipy_or_np = 'np'

for key in runs_data_dict:
    # if 'L3_AC' not in key:
    #     if '22_set1_L3_long' not in key:
    #         continue
    if 'L3_AC' not in key:
        if '22_set1_L3_long' not in key:
            continue
    data = runs_data_dict[key].copy()
    t = np.array(data['t(M)'])
    x = np.array(data[y_key])

    if scipy_or_np == 'scipy':
        running_mean = uniform_filter1d(x, size=window, mode='nearest')
        noise_estimate = x - running_mean
    elif scipy_or_np == 'np':
        running_mean = np.convolve(x,np.ones(window), mode='valid')/window
        t = t[window//2-1:-window//2]
        noise_estimate = x[window//2-1:-window//2] - running_mean
    else:
        raise Exception(f"Invalid scipy_or_np value: {scipy_or_np}")

    if moving_avg_len is None:
        plt.plot(t,np.abs(noise_estimate), label=key)
    else:
        t = t[moving_avg_len//2-1:-moving_avg_len//2]
        y = np.convolve(np.abs(noise_estimate),np.ones(moving_avg_len), mode='valid')/moving_avg_len
        plt.plot(t,y, label=key)

title = f"Noise estimate {y_key}, window={int(window*0.5)}M"
if moving_avg_len is not None:
    title += f", moving_avg_len={moving_avg_len//2}M"
plt.title(title)
plt.xlabel("t(M)")
plt.ylabel(f"Noise estimate: {y_key}")
plt.legend()
plt.yscale('log')
# plt.ylim(1e-13,1e-6)
# plt.tight_layout()
plt.show()

In [ ]:
t = np.array(data['t(M)'])
x = np.array(data['ArealMass'])
# x is your time series
window = 25 # Choose appropriate window size
running_mean = uniform_filter1d(x, size=window, mode='nearest')
noise_estimate = x - running_mean

# running_mean = np.convolve(x,np.ones(window), mode='valid')/window
# t = t[window//2-1:-window//2]
# noise_estimate = x[window//2-1:-window//2] - running_mean


In [ ]:
plt.plot(t,np.abs(noise_estimate))
plt.yscale('log')
plt.ylim(1e-13,1e-6)
plt.show()

In [ ]:
cols = data.columns
cols  = [col for col in cols if 'SphereC13' in col]
# cols  = [col for col in cols if '2Con' in col]
fil_data = data[cols]
# fil_data

In [ ]:
t = data['t(M)']

colors = ['r','b','g']
plt.plot(t,fil_data[[col for col in cols if '1Con' in col]].min(axis=1),color=colors[0])
plt.plot(t,fil_data[[col for col in cols if '1Con' in col]].max(axis=1),color=colors[0])
plt.plot(t,fil_data[[col for col in cols if '1Con' in col]].median(axis=1),label='1Con',color=colors[0])

plt.plot(t,fil_data[[col for col in cols if '2Con' in col]].min(axis=1),color=colors[1])
plt.plot(t,fil_data[[col for col in cols if '2Con' in col]].max(axis=1),color=colors[1])
plt.plot(t,fil_data[[col for col in cols if '2Con' in col]].median(axis=1),label='2Con',color=colors[1])

all_but_zero = list(set([col for col in cols if '3Con' in col])  - set([col for col in cols if '3Cont' in col] ))
plt.plot(t,fil_data[all_but_zero].min(axis=1),color=colors[2])
plt.plot(t,fil_data[all_but_zero].max(axis=1),color=colors[2])
plt.plot(t,fil_data[all_but_zero].median(axis=1),label='3Con',color=colors[2])

plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
t = data['t(M)']
# y = fil_data['Linf(3Conzyy) on SphereC10']
# y = fil_data['Linf(2Conzy) on SphereC10']
y = fil_data.min(axis=1)
# y = fil_data['Linf(1Cont) on SphereC10']
plt.plot(t,fil_data.min(axis=1),label='min')
plt.plot(t,fil_data.max(axis=1),label='max')
plt.plot(t,fil_data.median(axis=1),label='median')
# plt.plot(t,y)
plt.yscale('log')
plt.legend()
plt.show()

### Domain vals vs time

In [ ]:
runs_data_dict.keys()

# key = '46_bh_bbh_6'
# runs_data_dict_ghce={}

##### Linf

In [ ]:
filtered_runs = {}
for key,val in runs_to_plot.items():
  if "q10_0_0_0_0_0_0_d14" not in key:
    # filtered_runs[key] = val
    continue
  filtered_runs[key] = val
  
filtered_runs.keys()

In [ ]:

# data_file_path = "ConstraintNorms/GhCe.dat"
# data_file_path = "ConstraintNorms/GhCeExt.dat"
# data_file_path = "ConstraintNorms/GhCeExt_L2.dat"
# data_file_path = "ConstraintNorms/GhCeExt_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_L2.dat"
data_file_path = "ConstraintNorms/GhCe_Linf.dat"
# data_file_path = "ConstraintNorms/Metric_L2.dat"
# data_file_path = "ConstraintNorms/Metric_norms.dat"
# data_file_path = "ConstraintNorms/GhCe_Linf.dat"
# data_file_path = "ConstraintNorms/Linf.dat"
# data_file_path = "ConstraintNorms/Constraints_Linf.dat"
# data_file_path = "ConstraintNorms/GhCe_VolL2.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"

# data_file_path = "Hist-GrDomain.txt"

# data_file_path = "GhCe.dat"
# data_file_path = "NormalizedGhCe.dat"
# data_file_path = "GhCe_Norms.dat"
# data_file_path = "kappaErr_Linf.dat"
# data_file_path = "psiErr_Linf.dat"

# data_file_path = "PowerDiagnostics.h5@ConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@HighestThirdConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfFilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfNonFilteredNonZeroModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfPiledUpModes.dat"
# data_file_path = "PowerDiagnostics.h5@PowerInFilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@PowerInHighestUnfilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@PredictedTruncationErrorForOneLessMode.dat"
# data_file_path = "PowerDiagnostics.h5@RawConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@SpectrumIsDegenerate.dat"
# data_file_path = "PowerDiagnostics.h5@TruncationError.dat"

psi_or_kappa = "psi"
psi_or_kappa = "kappa"
top_number = 0

if "PowerDiagnostics" in data_file_path:
    data_file_path = f"{data_file_path}@{psi_or_kappa}@{top_number}"
column_names, runs_data_dict_ghce = load_data_from_levs(runs_to_plot,data_file_path)
# column_names, runs_data_dict_ghce = load_data_from_levs(runs_to_plot,data_file_path)
if "Constraints_Linf" in data_file_path:
    new_indices,runs_data_dict_ghce = add_norm_constraints(runs_data_dict_ghce,index_num=[1,2,3] ,norm=["Linf"], subdomains_seperately=True, replace_runs_data_dict=True)
if "GrDomain" in data_file_path:
    new_indices,runs_data_dict_ghce = num_points_per_subdomain(runs_data_dict_ghce,replace_runs_data_dict=True)
print(runs_data_dict_ghce.keys())

In [ ]:
key = list(runs_data_dict_ghce.keys())[-1]
key = list(runs_data_dict_ghce.keys())[3]
# key = 'run09_Lev5_cd_1_amr_1'
# key = 'run09_Lev5_cd_0_amr_1'
# key = 'run09_Lev6_main'

repeated_symmetric = False
# repeated_symmetric = True

minT = 0
# minT = 10
# minT = 7046
minT = 2000
# minT = 4000
# minT = 6000
# minT = 8302
# minT = 9602
# minT = 10000
# minT = 46500

maxT = 400000
# maxT = 4480
# maxT = 590
# maxT = 11000
# maxT = 200
# maxT = 800
# maxT = 2
# runs_data_dict_ghce={}
if runs_data_dict_ghce == {}:
   runs_data_dict_ghce = runs_data_dict
   
data = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key])
data = data.iloc[::1].dropna(axis=1, how='all')

domain_col_list = filter_by_regex(regex=["Sphere","Cylinder"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["Linf"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["L2"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["\(Shift"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["kappa_"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["psi_"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["\(kappaxxx"],col_list=domain_col_list)
domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC[4-9]"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC[2][0-9]"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list,exclude=True)
domain_col_list = filter_by_regex(regex=["GhCe"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["1Con"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["2Con"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["3Con"],col_list=domain_col_list)
visual_data = data[domain_col_list]
# visual_data = -data[domain_col_list]
# visual_data = visual_data - 1.0
visual_data = np.log10(visual_data)
# visual_data = np.log10(visual_data-1.0)
# print(visual_data.columns)
# Plot using imshow

column_names = list(visual_data.columns)
# column_names = [i.split(" ")[-1] for i in visual_data.columns]
column_names = return_sorted_domain_names(column_names, repeated_symmetric=repeated_symmetric)
print(column_names)

vmin_log,vmax_log = None,None
# vmin_log,vmax_log = -12.617493128350407 , -4.146905492009364
# vmin_log,vmax_log = -12.6946440611571 , -4
# vmin_log,vmax_log = -9 , None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

print(len(domain_col_list))

if repeated_symmetric:
    visual_data["Excision"] = [np.nan for i in range(len(data['t(M)']))]

plt.figure(figsize=(18, 10))
imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

if repeated_symmetric:
    # Set x-ticks and labels
    plt.xticks(
        ticks=np.arange(len(column_names)), 
        labels=[i.split(" ")[-1] for i in column_names], 
        rotation=90
    )
else:
    # Set x-ticks and labels
    plt.xticks(
        ticks=np.arange(len(visual_data.columns)), 
        labels=[i.split(" ")[-1] for i in column_names], 
        rotation=90
    )

ytick_step = 1
ytick_step = len(visual_data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(visual_data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{column_names[0].split(" ")[0]}')

# Determine colorbar ticks that align with the fixed vmin and vmax
# tick_vals = np.linspace(vmin_log, vmax_log, num=5)

# Set these ticks on the colorbar
# colorbar.set_ticks(tick_vals)

# Convert ticks back to the original scale for labeling
# colorbar.set_ticklabels([f'{10**val:.2e}' for val in tick_vals])

plt.ylabel('t(M)')
plt.title(f'{key}')
plt.tight_layout() 

plt.grid(False)
plt.show()

##### APS plot version

In [ ]:
# key = '6_set1_L3_EXP_FK_14012_10_6_30'
# key = '16_set1_L3'
key = '22_set1_L3_long'
# key = 'L3_AC_L4'
key = '28_set1_cd_junk_1'
# key = 'high_accuracy_L5'
# key = '22_set1_L3_long'
# key = 'set1_L6_AK_cd_100'
# key = '6_set1_L6s6'

repeated_symmetric = False
# repeated_symmetric = True
num_Excision = 3


minT = 0
# minT = .2
# minT = 480
# minT = 1400
# minT = 3000
# minT = 3500
# minT = 7000

maxT = 40000
maxT = 4000
# maxT = 590
# maxT = 3500
# maxT = 340
# maxT = 800
# maxT = 2

data = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key])
data = data.iloc[::1].dropna(axis=1, how='all')

domain_col_list = filter_by_regex(regex=["Sphere","Cylinder"],col_list=data.columns)
domain_col_list = filter_by_regex(regex=["Linf"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC[4-9]"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC[2][0-9]"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list,exclude=True)
# domain_col_list = filter_by_regex(regex=["1Conx"],col_list=domain_col_list)
visual_data = data[domain_col_list]
visual_data = np.log10(visual_data)
# print(visual_data.columns)
# Plot using imshow

column_names = list(visual_data.columns)
# column_names = [i.split(" ")[-1] for i in visual_data.columns]
column_names = return_sorted_domain_names(column_names, repeated_symmetric=repeated_symmetric, num_Excision=num_Excision)
# print(column_names)

vmin_log,vmax_log = None,None
# vmin_log,vmax_log = -12.417834211445228 , -8.343109330686875
# vmin_log,vmax_log = -10.183427321168152 , -2.540360921276033
vmin_log,vmax_log = -10.2 , -0.10
# vmin_log,vmax_log = -9 , None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

print(len(domain_col_list))

if repeated_symmetric:
    visual_data["Excision"] = [np.nan for i in range(len(data['t(M)']))]

fig, ax = plt.subplots(figsize=(7, 4))

imshow_plot = ax.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',
    interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

# Set x-ticks and labels
# ax.set_xticks(np.arange(len(visual_data.columns)))
# ax.set_xticklabels([i.split(" ")[-1] for i in column_names], rotation=90)

ax.tick_params(left=False, bottom=True, labelleft=False, labelbottom=True)

# ytick_step = 1
# ytick_step = len(visual_data) // 10  # Show about 10 ticks
# ax.set_yticks(np.arange(0, len(visual_data), ytick_step))
# ax.set_yticklabels(data['t(M)'][::ytick_step].astype(int))

# Create colorbar
colorbar = fig.colorbar(imshow_plot, ax=ax, label=f'log(constraint violation)')

# Determine colorbar ticks that align with the fixed vmin and vmax
# tick_vals = np.linspace(vmin_log, vmax_log, num=5)

# Set these ticks on the colorbar
# colorbar.set_ticks(tick_vals)

# Convert ticks back to the original scale for labeling
# colorbar.set_ticklabels([f'{10**val:.2e}' for val in tick_vals])

ax.set_ylabel(r'time $\longrightarrow$')
# ax.set_xlabel(r'subdomain number')
ax.set_xticks(
    ticks=[0,44,65,len(column_names)], 
    labels=['Outer Boundary','bhA','bhB','Outer Boundary'], 
    # rotation=90
)
ax.set_xticks(
    ticks=[0,44,54,65,len(column_names)], 
    labels=['Outer Boundary','bhA','center','bhB','Outer Boundary'], 
    # rotation=90
)
# ax.set_title(f'{key}')
plt.tight_layout()
# annotation1 = ax.annotate('bh A', 
#             xy=(0.2,0.1),             # point to annotate
#             xytext=(0.1,0.5),  # text position
#             xycoords = 'subfigure fraction',
#             arrowprops=dict(facecolor='black', shrink=0.05)
#             )
# annotation2 = ax.annotate('bh B', 
#             xy=(0.36,0.1),             # point to annotate
#             xytext=(0.25,0.5),  # text position
#             xycoords = 'subfigure fraction',
#             arrowprops=dict(facecolor='black', shrink=0.05)
#             )
# annotation3 = ax.annotate('Outer boundary', 
#             xy=(0.82,0.1),             # point to annotate
#             xytext=(0.6,0.5),  # text position
#             xycoords = 'subfigure fraction',
#             arrowprops=dict(facecolor='black', shrink=0.05)
#             )


ax.grid(False)

# save_name = "Extra_L5_set1_sym.png"
# save_name = "Extra_L6_set1_sym_lim.png"
# save_name = "Extra_L5_main_sym.png"
# save_name = Path(f"/groups/sxs/hchaudha/scripts/report/figures/{save_name}")
# if save_name.exists():
#     raise Exception("Change name")
# plt.savefig(save_name,dpi=600,bbox_inches='tight', bbox_extra_artists=[annotation1, annotation2, annotation3])
# plt.savefig(save_name,dpi=600,bbox_inches='tight')


plt.show()

#### take diff between two different runs

In [ ]:
print(runs_data_dict_ghce.keys())

In [ ]:
# Green better for key1
key1 = 'q105_01_01_fad21f0be4'
# key1 = 'q1_0_0_8_0_0_8'
# key1 = '17_set3_99_18_L3'
# key2 = 'q1_0_0_8_0_0_8_MaxDL'
key2 = 'q105_01_01_4a7a1adccf'
# key2 = '3_SupImpKerr_09'
# key2 = '17_main_9_18_L3_correct'


minT = 0
# minT = 480
# minT = 1680
# minT = 3500
# minT = 4200
# minT = 7360
# minT = 46500
# minT = 46600

maxT = 400000
# maxT = 150
# maxT = 590
# maxT = 1100
# maxT = 8500

data1 = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key1])
data1 = data1.iloc[::1].dropna(axis=1, how='all')

data2 = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key2])
data2 = data2.iloc[::1].dropna(axis=1, how='all')

# Set column 't(M)' as the index for both data1 and data2
data1.set_index('t(M)', inplace=True)
data2.set_index('t(M)', inplace=True)

data = data1.copy()
if data1.index.equals(data2.index) is False:
    print("Indices of data1 and data2 do not match, interpolating data2 to match data1's index.")
    interp_data2 = data1.copy()
    for col in data1.columns:
        if col not in data2.columns:
            print(f"Column {col} not found in data2, skipping.")
            continue
        interp_data2[col] = np.interp(data1.index, data2.index, data2[col])
    data2 = interp_data2    

print("Interpolation done!")

for col in data.columns:
    if col not in data2.columns:
        print(f"Column {col} not found in data2, skipping.")
        continue
    # Negative(green) is good
    data[col] = np.log10(data2[col]) - np.log10(data1[col])
    # data[col] = data2[col] - data1[col]
    # data[col] = (data2[col] - data1[col])*2/(data2[col] + data1[col])
    # Modify each value: set to 1 if positive, 0 if negative or zero
    # => red(1) if more error and green(0) is less
    # data[col] = np.where(data[col] > 0, 1, 0)

# Reset the index if you need 't(M)' back as a column
data.reset_index(inplace=True)

domain_col_list = filter_by_regex(regex=["Sphere","Cylinder"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list,exclude=True)
# domain_col_list = filter_by_regex(regex=["Linf(sqrt(1Con^2))"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["Linf(sqrt(2Con^2))"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["Linf(sqrt(3Con^2))"],col_list=domain_col_list)
domain_col_list = filter_by_regex(regex=["Linf(GhCe)"],col_list=domain_col_list)

visual_data = data[domain_col_list]
# visual_data = np.log10(visual_data)
# Plot using imshow

column_names = list(visual_data.columns)
# column_names = [i.split(" ")[-1] for i in visual_data.columns]
column_names = return_sorted_domain_names(column_names)
print(column_names)

vmin_log,vmax_log = None,None
# vmin_log,vmax_log = -1,1

if vmin_log is None:
  # Get min non -inf value
  temp = visual_data.copy()
  temp.replace([np.inf, -np.inf], np.nan, inplace=True)
  vmin_log = temp.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

# vmin_log, vmax_log = vmin_log, -vmin_log # This is to make it so white is 0, green is good and red is bad

# Emphasize the bad parts, i.e. more color resolution for the red part
# vmin_log, vmax_log = -vmax_log, vmax_log # This is to make it so white is 0, green is good and red is bad

# max_val = max(abs(vmin_log),abs(vmax_log))
# vmin_log, vmax_log = -max_val, max_val # This is to make it so white is 0, green is good and red is bad
# print(vmin_log,",",vmax_log)


# Example colormap centered around zero
plt.figure(figsize=(15, 10))
divnorm = mcolors.TwoSlopeNorm(vmin=vmin_log, vcenter=0, vmax=vmax_log)

imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r',
    origin='lower',interpolation='none',
    norm=divnorm
)

# Set x-ticks and labels
plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

ytick_step = len(visual_data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(visual_data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{column_names[0].split(" ")[0]}')

plt.ylabel('t(M)')
plt.title(f'{key2}(Green better) - {key1}')
plt.tight_layout()
plt.grid(False)
plt.show()

In [ ]:
visual_data

In [ ]:
plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r',
    origin='lower',interpolation='none',
    # norm=divnorm
)
plt.show()

In [ ]:
diff = np.log10(data2) - np.log10(data1)
plt.imshow(diff)
imshow_plot = plt.imshow(
    diff, 
    aspect='auto', 
    cmap='RdYlGn_r',
    origin='lower',interpolation='none',
)
plt.show()

In [ ]:

minT = 0
minT = 1200
# minT = 4000

maxT = 40000
maxT = 2300
# data = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key1])
data = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key2])
data = data.iloc[::1].dropna(axis=1, how='all')

domain_col_list = filter_by_regex(regex=["Sphere","Cylinder"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["Cylinder"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=[r"SphereC14"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=[r"3Con"],col_list=domain_col_list)

data = data[domain_col_list+["t(M)"]]

max_info = {}
for col in data.columns:
    # Get the maximum value
    max_val = data[col].max()
    # Get all indices where the maximum value occurs
    idx_max_list = data[data[col] == max_val].index
    times = data.loc[idx_max_list, 't(M)']
    values = data.loc[idx_max_list, col]

    max_info[col] = {
        "max_idx_list": idx_max_list,
        "max_val_times": list(times),
        "max_vals": list(values),
    }
    for time, val in zip(max_info[col]['max_val_times'], max_info[col]['max_vals']):
        if val > 1e-9:
            print(col, time, val)

In [ ]:
minT = 0
# minT = 1200
key = "6_set1_L6s2"
maxT = 40000
maxT = 4000
data = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_ghce[key])
data = data.iloc[::1].dropna(axis=1, how='all')

domain_col_list = filter_by_regex(regex=["Sphere","Cylinder"],col_list=data.columns)

# Calculate maximum error at each time point
data['max_error'] = data[domain_col_list].max(axis=1)

# Calculate L2 norm (Euclidean norm) of the errors at each time point
data['l2_norm'] = np.sqrt((data[domain_col_list] ** 2).sum(axis=1))

plt.plot(data['t(M)'], data["max_error"],label="max_error")
# plt.plot(data['t(M)'], data["l2_norm"],label="l2_norm")
plt.legend()
plt.yscale('log')
plt.title(key)
plt.show()

### For domain resolutions

In [ ]:
data_file_path = "MinimumGridSpacing.dat"
data_file_path = "Hist-GrDomain.txt"

# data_file_path = "Hist-Domain.txt"


column_names, runs_data_dict_domain = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)
print(runs_data_dict_domain.keys())

In [ ]:


# data = runs_data_dict[key].iloc[1:8000:10].dropna(axis=1, how='all')

key = '48_set1_L3_fromID'
# key = '48_set1_L3_fromID_1AB'

minT = 0
minT = 10
minT = 1400
# minT = 3400
# minT = 10000
maxT = 40000
# maxT = 1200
# maxT = 4000
data = limit_by_col_val(minT,maxT,'t(M)',runs_data_dict_domain[key])
data = data.iloc[::].dropna(axis=1, how='all')
domain_col_list = filter_by_regex(regex=["Sphere","Cylinder"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC[4-9]"],col_list=domain_col_list)
# domain_col_list = filter_by_regex(regex=["SphereC[2][0-9]"],col_list=data.columns)
# domain_col_list = filter_by_regex(regex=["SphereC"],col_list=domain_col_list,exclude=True)


visual_data = data[domain_col_list]
if "Grid" in data_file_path:
  visual_data = np.log10(visual_data)
# Plot using imshow

column_names = list(visual_data.columns)
if "Grid" not in data_file_path:
#   column_names = [i for i in column_names if ("_R" in i)]
  column_names = [i for i in column_names if ("_L" in i)]
  # column_names = [i for i in column_names if ("_M" in i)]
column_names = return_sorted_domain_names(column_names)
plt.figure(figsize=(15, 10))
plt.imshow(visual_data[column_names], aspect='auto', cmap='RdYlGn_r', origin='lower',interpolation='none')

# Set x-ticks and labels
plt.xticks(ticks=np.arange(len(column_names)), labels=[get_domain_name(i,just_domain_names=False) for i in column_names], rotation=90)
# plt.xticks(ticks=np.arange(len(column_names)), labels=[i.split("_")[0] for i in column_names], rotation=90)

# ytick_step = len(visual_data)
ytick_step = len(visual_data) // 5  # show about 10 ticks
# ytick_step = len(visual_data)   # show about 10 ticks
if ytick_step < 1:
  ytick_step = 10
plt.yticks(ticks=np.arange(0, len(visual_data), ytick_step), 
           labels=data['t(M)'][::ytick_step].astype(int))
# plt.yticks(ticks=np.arange(len(visual_data)), labels=data['t(M)'].astype(int))

# plt.colorbar(label=f'{column_names[0].split(" ")[0]}')
plt.colorbar()
# plt.xlabel('Features')
plt.ylabel('t(M)')
plt.title(f'{key}')
plt.tight_layout()
plt.grid(False)
plt.show()

#### Plot L,M and R on the same graph

In [ ]:
# Assuming return_sorted_domain_names and get_domain_name are predefined functions
# Here's how you can create three subplots with three different column_name filters:

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(15, 15), constrained_layout=True)
column_name_filters = ["_R", "_L", "_M"]
column_name = ["Extent 0", "Extent 1", "Extent 2"]

for i, ax, filter_suffix in zip(range(len(column_name_filters)),axes, column_name_filters):
    filtered_column_names = [i for i in visual_data.columns if (filter_suffix in i)]
    filtered_column_names = return_sorted_domain_names(filtered_column_names)
    
    im = ax.imshow(visual_data[filtered_column_names], aspect='auto', cmap='RdYlGn_r', origin='lower',interpolation='none',)
    
    if filter_suffix == "_M":
      # Set x-ticks and labels
      ax.set_xticks(np.arange(len(filtered_column_names)))
      ax.set_xticklabels([get_domain_name(i,just_domain_names=False) for i in filtered_column_names], rotation=90)
    
    # Set y-ticks and labels
    ytick_step = len(visual_data) // 5  # show about 10 ticks
    if ytick_step < 1:
        ytick_step = 10
    ax.set_yticks(np.arange(0, len(visual_data), ytick_step))
    ax.set_yticklabels(data['t(M)'][::ytick_step].astype(int))
    
    # Set labels and title
    ax.set_ylabel('t(M)')
    ax.set_title(f'{key} : {column_name[i]}, {filter_suffix[-1]}')
    
    # Add a colorbar to each subplot
    fig.colorbar(im, ax=ax)
    
    ax.grid(False)

plt.show()

### Plot 2d grid

In [ ]:
# data_file_path = "ConstraintNorms/GhCe.dat"
# data_file_path = "ConstraintNorms/GhCeExt.dat"
# data_file_path = "ConstraintNorms/GhCeExt_L2.dat"
# data_file_path = "ConstraintNorms/GhCeExt_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_L2.dat"
# data_file_path = "ConstraintNorms/GhCe_Linf.dat"
# data_file_path = "ConstraintNorms/Linf.dat"
# data_file_path = "ConstraintNorms/Constraints_Linf.dat"
# data_file_path = "ConstraintNorms/GhCe_Norms.dat"
# data_file_path = "ConstraintNorms/GhCe_VolL2.dat"
data_file_path = "ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Norms.dat"
column_names, runs_data_dict_ghce = load_data_from_levs(runs_to_plot,data_file_path)
print(runs_data_dict_ghce.keys())

In [ ]:
# for key in runs_data_dict_ghce.keys():
for key in ["48_set1_L3_fromID"]:
  t_list = np.arange(0,4000,100)
  save_path = Path("/home/hchaudha/notes/spec_accuracy/del/domain_plots")
  save_path = save_path/key
  if not save_path.exists():
    save_path.mkdir()
  print(save_path)

  nA=5
  rA=nA*1.5
  center_xA=rA + 2
  RA=rA+5
  rC=RA*2
  nC=30
  RC=rC+nC

  saved_path_list_for_gif = []
  for t in t_list:
    fig, ax = plt.subplots(figsize=(12, 10))
    pandas_dict = runs_data_dict_ghce[key].iloc[2*t,:].to_dict()
    time = pandas_dict.pop("t(M)")
    # domain_color_local,sm = scalar_to_color(pandas_dict,(-11,-4),color_map='RdYlGn_r')
    domain_color_local,sm = scalar_to_color(pandas_dict,(-8,-4),color_map='RdYlGn_r')
    domain_color_local,sm = scalar_to_color(pandas_dict,color_map='RdYlGn_r')

    patches_class = BBH_domain_sym_ploy(center_xA=center_xA, rA=rA, RA=RA, rC=rC, RC=RC, nA=nA, nC=nC, color_dict=domain_color_local) 
    for patch in patches_class.patches:
      ax.add_patch(patch)

    ax.set_xlim(-RC, RC)
    ax.set_ylim(-RC, RC)
    ax.set_aspect('equal')

    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f"{list(pandas_dict.keys())[0].split(' ')[0]}")
    plt.title(f"{key}  t(M) = {time}")
    plt.tight_layout()
    img_save_path = save_path/f"{t}.png"
    print(img_save_path)
    plt.savefig(img_save_path)
    saved_path_list_for_gif.append(img_save_path)
    plt.close()


  def create_gif(filenames, output_gif='output.gif'):
    images = []
    # Read each image file using iio.imread and append it to the list
    for filename in filenames:
        images.append(iio.imread(filename))
    # Write the images as an animated GIF using iio.imwrite
    iio.imwrite(output_gif, images, duration=500, loop=0)

  create_gif(saved_path_list_for_gif,output_gif=save_path/'output.gif')
  print(f"Gif created: {save_path/'output.gif'}")


In [ ]:
t = np.linspace(0,4000,100)
data1 = runs_data_dict[key].iloc[2*t,:].to_dict()
data1['t(M)'].values()
for key in data1.keys():
  print(key.split(" ")[-1])

In [ ]:
key = "high_accuracy_L5"
t_list = np.arange(0,4000,100)
t_list = [2000]

# save_path = Path("/home/hchaudha/notes/spec_accuracy/del/domain_plots")/key
# if not save_path.exists():
#   save_path.mkdir()
# print(save_path)

save_path = None

nA=4
rA=nA*1.5
center_xA=rA + 2
RA=rA+5
rC=RA*2
nC=30
RC=rC+nC


for t in t_list:
  fig, ax = plt.subplots(figsize=(12, 10))
  pandas_dict = runs_data_dict[key].iloc[2*t,:].to_dict()
  time = pandas_dict.pop("t(M)")
  domain_color_local,sm = scalar_to_color(pandas_dict,(-10,-2),color_map='RdYlGn_r')

  patches_class = BBH_domain_sym_ploy(center_xA=center_xA, rA=rA, RA=RA, rC=rC, RC=RC, nA=nA, nC=nC, color_dict=domain_color_local) 
  for patch in patches_class.patches:
    ax.add_patch(patch)

  ax.set_xlim(-RC, RC)
  ax.set_ylim(-RC, RC)
  ax.set_aspect('equal')

  cbar = plt.colorbar(sm, ax=ax)
  cbar.set_label(f"{list(pandas_dict.keys())[0].split(' ')[0]}")
  plt.title(f"{key}  t(M) = {time}")
  plt.tight_layout()
  if save_path is not None:
    print(save_path/f"{t}.png")
    plt.savefig(save_path/f"{t}.png")
  plt.show()

### Plot and save all y_axis in this data

In [ ]:
# moving_avg_len=25
plt.close()
save_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/figures/lev6-11/")
for file in save_path.glob('*'):  # Use '*' to match all files
    if file.is_file():  # Check if it's a file
        file.unlink()  # Remove the file
save_path = str(save_path)+"/"

# save_path = "/home/hchaudha/notes/spec_accuracy/uniAMR_comparisons/all_set3/"
diff_base = None

plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# y_lower,y_upper=1e-8,1
# y_lower,y_upper=1e-13,1e-4
# y_lower,y_upper=1e-12,1e-3

legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

minT = 0
minT = 3000
maxT = 4000
maxT = 3500
# maxT = 1200

x_axis = 't(M)'

for y_axis in column_names:
  if y_axis == x_axis:
    continue
  try:
    with plt.style.context('default'):
      plt.rcParams["figure.figsize"] = (8,6)
      plt.rcParams["figure.autolayout"] = True
      # plt.ylim(y_lower,y_upper)
      plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base)
      plt.close()
    print(f"{y_axis} done!")
  except Exception as e:
      print(f"Error plotting {y_axis}: {str(e)}")
      continue

In [ ]:
df = runs_data_dict['all_100_t2690_obs']
minT = 2690
minT = 2700
maxT = 2710
maxT = minT+2
df = df[df['time']>minT]
df = df[df['time']<maxT]
from scipy.signal import find_peaks

# plt.plot(df['time'],df['Linf(GhCe) on SphereA0'],marker='x')
# plt.plot(df['time'],df['Linf(GhCe) on SphereA4'],marker='x')
plt.plot(df['time'],df['Linf(GhCe) on SphereA3'],marker='x')
# plt.yscale('log')
plt.show()

In [ ]:
data = runs_data_dict['all_100_t2690_obs_grid_tol_10']
data = data[data['time after step']>2690]
dt_arr = np.array(data.dt)
averaged_dt = np.zeros_like(dt_arr)
averaged_dt[0] =  dt_arr.mean() 
N = 100
for i in range(len(dt_arr)-1):
  averaged_dt[i+1] = averaged_dt[i]*(N-1)/(N)+dt_arr[i+1]*1/N

In [ ]:
plt.plot(data['time after step'],averaged_dt)
plt.plot(data['time after step'],dt_arr)
plt.show()

### Plot max/min value for a run

In [ ]:
# data_file_path="ConstraintNorms/GhCe_L2.dat"
data_file_path="ConstraintNorms/GhCe_Linf.dat"
data_file_path="ConstraintNorms/NormalizedGhCe_Linf.dat"
data_file_path="ConstraintNorms/Constraints_Linf.dat"
data_file_path="MinimumGridSpacing.dat"
# data_file_path="ConstraintNorms/GhCe_VolL2.dat"
# data_file_path = "ConstraintNorms/NormalizedGhCe_Norms.dat"
column_names_linf2, runs_data_dict_linf2 = load_data_from_levs(runs_to_plot,data_file_path)
# print(column_names_linf2)
print(runs_data_dict_linf2.keys())

In [ ]:
# run_name = list(runs_data_dict_linf2.keys())[4]
# run_name = '3555_1.0e-07_060'
run_name = 'L3_Ctest10'
run_name = '6_set1_L3s3'
run_name = '34_master_L16_3'
# run_name = 'L3_Ctest10'
# run_name = 'L3_Ctest15'
# run_name = '6_set2_L3s3'
# run_name = '6_set2_L6s6'
# run_name = '6_set1_L3s0'
# run_name = '6_set1_L3s3'
df = runs_data_dict_linf2[run_name].copy()
df = df.sort_values(by=df.columns[0])
save_path = None
# save_path = "/home/hchaudha/notes/spec_accuracy/del/normalized_norms/all/"

max_or_min = "MAX"
max_or_min = "MIN"

tmin=0
tmin= 120
# tmin= 1200
# tmin=9372
# tmin=2691
tmax=50000
tmax=4000
# tmax=tmin+4
# tmax=1200
# tmax=2800

df = df[df['t(M)']>=tmin]
df = df[df['t(M)']<tmax]
t_name = df.columns[0]
y_axis = df.columns[1].split(" ")[0]
all_cols_but_t = df.columns[1:]
all_cols_but_t = []

def find_sphere_num(string:str):
  match = re.search(r'Sphere([A-C])(\d{1,2})', string)

  if match:
      letter = match.group(1)  # Extract the letter (A, B, or C)
      number = match.group(2)  # Extract the number (one or two digits)
      return letter,int(number)
  else:
      print(f"No Sphere found in the {string}.")
      return None

only_include = None
# only_include = [
#   r"SphereA[0-9]",
#   r"SphereB[0-9]",
#   r"SphereC[0-9]$",
#   r"SphereC1[0-9]",
#   r"SphereC2[0-9]",
#   r"CylinderE[A,B]\d",
#   r"CylinderC[A,B]\d",
#   r"CylinderSM[A,B]\d",
#   r"FilledCylinderE[A,B]\d",
#   r"FilledCylinderC[A,B]\d",
#   r"FilledCylinderM[A,B]\d",
# ]
# only_include = [
#   r"SphereC\d$",
#   r"SphereC\d\d$",
# ]
exclude = None
# exclude = [
#    r"SphereC[6-9]",
# ]
# exclude = [
#    r"1Con",
#    r"2Con",
# #    r"3Con",
# ]
def matches_any_pattern(label, patterns):
    for pattern in patterns:
        if re.search(pattern, label):
            return True
    return False

SphereCMin,SphereCMax = 0,22
# SphereCMin,SphereCMax = 30,50

SphereAMin,SphereAMax = 0,9
SphereBMin,SphereBMax = 0,9
# SphereAMin,SphereAMax = 5,9
# SphereBMin,SphereBMax = 5,9
for i in df.columns[1:]:
  # Things to include, this overrides things to exclude
  if only_include is None:
    pass
  elif matches_any_pattern(i,only_include):
    # We are allowed to include this domain
    pass
  else:
    # We are not supposed to include this domain
    continue
  
  if exclude is None:
    pass
  elif matches_any_pattern(i,exclude):
    continue
  else:
    pass
  # Things to exclude
  if 'SphereC' in i:
    _, sphereC_num = find_sphere_num(i)
    if (sphereC_num < SphereCMin) or (sphereC_num > SphereCMax):
      print(i)
      continue
  if 'SphereA' in i :
    _, sphere_num = find_sphere_num(i)
    if (sphere_num < SphereAMin) or (sphere_num > SphereAMax):
      print(i)
      continue
  if 'SphereB' in i:
    _, sphere_num = find_sphere_num(i)
    if (sphere_num < SphereBMin) or (sphere_num > SphereBMax):
      print(i)
      continue
  if 't(M)' in i:
    print(i)
    continue
  all_cols_but_t.append(i)

if max_or_min == "MAX":
  # Find the maximum value across columns B, C, D, and F for each row
  df['extreme_val'] = df[all_cols_but_t].max(axis=1)

  # Determine which column had the maximum value
  df['extreme_source'] = df[all_cols_but_t].idxmax(axis=1)

if max_or_min == "MIN":
  # Find the maximum value across columns B, C, D, and F for each row
  df['extreme_val'] = df[all_cols_but_t].min(axis=1)

  # Determine which column had the maximum value
  df['extreme_source'] = df[all_cols_but_t].idxmin(axis=1)

# List all columns that have at least one extreme value
columns_with_extreme = df['extreme_source'].unique()

# Generate a colormap for the columns with at least one extreme value
num_colors = len(columns_with_extreme)
colors = plt.get_cmap('tab20', num_colors)  # Using 'tab20' colormap
color_map = {column: colors(i) for i, column in enumerate(columns_with_extreme)}

# Plot max_BCD vs t with different colors for different sources
plt.figure(figsize=(8, 6))
for i,source in enumerate(columns_with_extreme):
    subset = df[df['extreme_source'] == source]
    if i%4 == 0:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker="^")
    if i%4 == 1:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker="v")
    if i%4 == 2:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker=">")
    if i%4 == 3:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker="<")

plt.xlabel(t_name)
plt.ylabel(y_axis)
# plt.yscale('log')
plt.title(f'{max_or_min}:{y_axis} vs {t_name} for {run_name} : A_{SphereAMin}_{SphereAMax}_B_{SphereBMin}_{SphereBMax}_C_{SphereCMin}_{SphereCMax}')
plt.legend()
plt.grid(True)  
plt.tight_layout()
if save_path is None:
   save_path = "/groups/sxs/hchaudha/rough/"
plt.savefig(f"{save_path}{run_name}_{max_or_min}:{y_axis}_vs_{t_name}_{tmin}_{tmax}_A_{SphereAMin}_{SphereAMax}_B_{SphereBMin}_{SphereBMax}_C_{SphereCMin}_{SphereCMax}.png", dpi=500)
plt.show()

## Plots all ys for a single run

In [ ]:
# data_file_path="ConstraintNorms/GhCe_L2.dat"
data_file_path="ConstraintNorms/GhCe_Linf.dat"
# data_file_path="ConstraintNorms/NormalizedGhCe_Linf.dat"
# data_file_path="MinimumGridSpacing.dat"
# data_file_path="ConstraintNorms/GhCe_VolL2.dat"
column_names_linf3, runs_data_dict_linf3 = load_data_from_levs(runs_to_plot,data_file_path)
# print(column_names_linf3)
print(runs_data_dict_linf3.keys())

In [ ]:
# run_name = list(runs_data_dict_linf2.keys())[4]
# run_name = '119_gd_SUKS_3_20'
# run_name = 'high_accuracy_L4'
# run_name = 'high_accuracy_L3'
# run_name = '6_set2_L3s3'
# run_name = '6_set2_L6s6'
# run_name = '6_set3_L6s6'
df = runs_data_dict_linf3[run_name].copy()
df = df.sort_values(by=df.columns[0])
save_path = None
# save_path = "/home/hchaudha/notes/spec_accuracy/del/normalized_norms/all/"

# tmin= 1200
tmin=0
tmin= 3000
# tmin=9300
# tmin=9372
# tmin=2691
tmax=50000
tmax=3500
# tmax=tmin+4
# tmax=7000
# tmax=2800

df = df[df['t(M)']>=tmin]
df = df[df['t(M)']<tmax]
t_name = df.columns[0]
y_axis = df.columns[1].split(" ")[0]
all_cols_but_t = df.columns[1:]
all_cols_but_t = []

def find_sphere_num(string:str):
  match = re.search(r'Sphere([A-C])(\d{1,2})', string)

  if match:
      letter = match.group(1)  # Extract the letter (A, B, or C)
      number = match.group(2)  # Extract the number (one or two digits)
      return letter,int(number)
  else:
      print(f"No Sphere found in the {string}.")
      return None

SphereCMin,SphereCMax = 0,30
# SphereCMin,SphereCMax = 30,50

SphereAMin,SphereAMax = 1,9
SphereBMin,SphereBMax = 1,9
# SphereAMin,SphereAMax = 5,9
# SphereBMin,SphereBMax = 5,9
for i in df.columns[1:]:
  if 'SphereC' in i:
    _, sphereC_num = find_sphere_num(i)
    if (sphereC_num < SphereCMin) or (sphereC_num > SphereCMax):
      print(i)
      continue
  if 'SphereA' in i :
    _, sphere_num = find_sphere_num(i)
    if (sphere_num < SphereAMin) or (sphere_num > SphereAMax):
      print(i)
      continue
  if 'SphereB' in i:
    _, sphere_num = find_sphere_num(i)
    if (sphere_num < SphereBMin) or (sphere_num > SphereBMax):
      print(i)
      continue
  if 't(M)' in i:
    print(i)
    continue
  all_cols_but_t.append(i)

if max_or_min == "MAX":
  # Find the maximum value across columns B, C, D, and F for each row
  df['extreme_val'] = df[all_cols_but_t].max(axis=1)

  # Determine which column had the maximum value
  df['extreme_source'] = df[all_cols_but_t].idxmax(axis=1)

if max_or_min == "MIN":
  # Find the maximum value across columns B, C, D, and F for each row
  df['extreme_val'] = df[all_cols_but_t].min(axis=1)

  # Determine which column had the maximum value
  df['extreme_source'] = df[all_cols_but_t].idxmin(axis=1)

# List all columns that have at least one extreme value
columns_with_extreme = df['extreme_source'].unique()

# Generate a colormap for the columns with at least one extreme value
num_colors = len(columns_with_extreme)
colors = plt.get_cmap('tab20', num_colors)  # Using 'tab20' colormap
color_map = {column: colors(i) for i, column in enumerate(columns_with_extreme)}

# Plot max_BCD vs t with different colors for different sources
plt.figure(figsize=(18, 10))
for i,source in enumerate(columns_with_extreme):
    subset = df[df['extreme_source'] == source]
    if i%4 == 0:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker="^")
    if i%4 == 1:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker="v")
    if i%4 == 2:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker=">")
    if i%4 == 3:
        plt.scatter(subset[t_name], subset['extreme_val'], color=color_map[source], label=source, s=10, marker="<")

plt.xlabel(t_name)
plt.ylabel(y_axis)
# plt.yscale('log')
plt.title(f'{max_or_min}:{y_axis} vs {t_name} for {run_name} : A_{SphereAMin}_{SphereAMax}_B_{SphereBMin}_{SphereBMax}_C_{SphereCMin}_{SphereCMax}')
plt.legend()
plt.grid(True)  
plt.tight_layout()
if save_path is None:
   save_path = "/groups/sxs/hchaudha/rough/"
plt.savefig(f"{save_path}{run_name}_{max_or_min}:{y_axis}_vs_{t_name}_{tmin}_{tmax}_A_{SphereAMin}_{SphereAMax}_B_{SphereBMin}_{SphereBMax}_C_{SphereCMin}_{SphereCMax}.png", dpi=500)
plt.show()

# Plots for WT and finite radius data

### Folder paths

In [ ]:
runs_to_plot = {}

# runs_to_plot["high_accuracy_L3_tol8_wrong"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev3_tol8/Run/"
# runs_to_plot["high_accuracy_L3_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_Ringdown/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_Ringdown/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_Ringdown/Lev5_A?/Run/"

# runs_to_plot["high_accuracy_L0"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev0_A?/Run/"
# runs_to_plot["high_accuracy_L1"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev1_A?/Run/"
# runs_to_plot["high_accuracy_L2"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev2_A?/Run/"
# runs_to_plot["high_accuracy_L3"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L45"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L55"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev55_A?/Run/"
# runs_to_plot["high_accuracy_L6"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev6_A?/Run/"

# runs_to_plot["high_accuracy_L45n"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/new_L45_L55/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L55n"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/new_L45_L55/Ev/Lev55_A?/Run/"

# runs_to_plot["high_accuracy_L4n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L45n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L5n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L55n_no_tol_change"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_seg_runs/main_L4_to_L55/Ev/Lev55_A?/Run/"

# runs_to_plot["high_accuracy_L5_three_tier"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L5_three_tier_constra"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L5_three_tier_constra200"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra_200/Lev5_A?/Run/"
# runs_to_plot["L3_step_bound_gauss_error"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_A?/Run/"
# runs_to_plot["L3_step_bound_gauss_error_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_Ringdown/Lev3_A?/Run/"

# runs_to_plot["Lev5_big_gaussian_ah_tol10"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol10/Lev5_A?/Run/"
# runs_to_plot["Lev5_big_gaussian_ah_tol100"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol100/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_cd_01_uamr"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_lapse"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_full/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_lapse_uamr"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/Lev5_A?/Run/"

# runs_to_plot["high_accuracy_L0_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev0_A?/Run/"
# runs_to_plot["high_accuracy_L1_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev1_A?/Run/"
# runs_to_plot["high_accuracy_L2_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev2_A?/Run/"
# runs_to_plot["high_accuracy_L3_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L5_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L45_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L55_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev55_A?/Run/"
# runs_to_plot["high_accuracy_L6_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev6_A?/Run/"


# runs_to_plot["ode_impro_Lev0"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev0_A?/Run/'
# runs_to_plot["ode_impro_Lev1"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev1_A?/Run/'
# runs_to_plot["ode_impro_Lev2"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev2_A?/Run/'
# runs_to_plot["ode_impro_Lev3"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev3_A?/Run/'
# runs_to_plot["ode_impro_Lev4"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev4_A?/Run/'
# runs_to_plot["ode_impro_Lev5"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev5_A?/Run/'
# runs_to_plot["main_Lev0"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev0_A?/Run/'
# runs_to_plot["main_Lev2"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev2_A?/Run/'
# runs_to_plot["main_Lev1"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev1_A?/Run/'

# runs_to_plot["ode_impro_Lev0_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev0_Ringdown/Lev0_A?/Run/'
# runs_to_plot["ode_impro_Lev2_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev2_Ringdown/Lev2_A?/Run/'
# runs_to_plot["ode_impro_Lev1_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev1_Ringdown/Lev1_A?/Run/'
# runs_to_plot["main_Lev0_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev0_Ringdown/Lev0_A?/Run/'
# runs_to_plot["main_Lev2_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev2_Ringdown/Lev2_A?/Run/'
# runs_to_plot["main_Lev1_rd"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev1_Ringdown/Lev1_A?/Run/'

# runs_to_plot["6_set1_L3s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set1_L3s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set1_L3s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set1_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set1_L3s3_fil6"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_6/Ev/Lev3_A[B-]/Run/"
# runs_to_plot["6_set1_L3s3_fil8"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_8/Ev/Lev3_A[B-]/Run/"
# runs_to_plot["6_set1_L3s3_fil10"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_10/Ev/Lev3_A[B-]/Run/"

# runs_to_plot["6_set1_L3_template_all"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_template_all/Ev/Lev3_A[B-]/Run/"
# runs_to_plot["6_set1_L3_template_1_29"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_template_1_29/Ev/Lev3_A[B-]/Run/"

# runs_to_plot["6_set1_L3s3_5517_CCopy"] =  "/groups/sxs/hchaudha/spec_runs/19_filtered_checkpoint_runs/6_set1_L3_5517_CCopy/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set2_L3s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L3/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set2_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["24_allcd_gaussEx_10_800"] =  "/groups/sxs/hchaudha/spec_runs/24_allcd_gaussEx_10_800/Ev/Lev3_A?/Run/"
# runs_to_plot["24_allcd_gaussEx_5_800"] =  "/groups/sxs/hchaudha/spec_runs/24_allcd_gaussEx_5_800/Ev/Lev3_A?/Run/"
# runs_to_plot["25_allcd_gaussEx_2_60"] =  "/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_2_60/Ev/Lev3_A?/Run/"
# runs_to_plot["25_allcd_gaussEx_2_800"] =  "/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_2_800/Ev/Lev3_A?/Run/"
# runs_to_plot["25_allcd_gaussEx_5_60"] =  "/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_5_60/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set3_L3s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set3_L3s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set3_L3s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set3_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["6_set1_L6s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set1_L6s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set1_L6s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set1_L6s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set1_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set1_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["set1_L6s4_cd10"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd10/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd200"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd200/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd500"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd500/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100_AMRL6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100_AMRL6/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100_AMRL7"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100_AMRL7/Ev/Lev4_A?/Run/"
# runs_to_plot["set1_L6s4_cd100_AMRL8"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/set1_L6s4_cd100_AMRL8/Ev/Lev4_A?/Run/"

# runs_to_plot["6_set2_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set2_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set2_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set3_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set3_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set3_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set1_L6s3_CAMR"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_CAMR/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s3_min_L"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_min_L/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s3_min_LR"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6_vars/L6s3_min_LR/Ev/Lev3_A?/Run/"

# runs_to_plot["7_constAMR_set1_L6_base_0"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev0_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_1"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev1_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_2"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev2_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_3"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev3_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_4"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev4_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_5"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev5_A?/Run/"
# runs_to_plot["7_constAMR_set1_L6_base_6"] =  "/groups/sxs/hchaudha/spec_runs/7_constAMR_set1_L6_base/Ev/Lev6_A?/Run/"

# runs_to_plot["8_constAMR_set1_L6_base_0"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev0_A?/Run/"
# runs_to_plot["8_constAMR_set1_L6_base_1"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev1_A?/Run/"
# runs_to_plot["8_constAMR_set1_L6_base_2"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev2_A?/Run/"
# runs_to_plot["8_constAMR_set1_L6_base_3"] =  "/groups/sxs/hchaudha/spec_runs/8_constAMR_set1_L6_base/Ev/Lev3_A?/Run/"

# runs_to_plot["9_set1_L3s3_01"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_01/Ev/Lev3_A?/Run/"
# runs_to_plot["9_set1_L3s3_001"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_001/Ev/Lev3_A?/Run/"
# runs_to_plot["9_set1_L3s3_10"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_10/Ev/Lev3_A?/Run/"
# runs_to_plot["9_set1_L3s3_100"] =  "/groups/sxs/hchaudha/spec_runs/9_const_damp_var/set1_L3s3_100/Ev/Lev3_A?/Run/"

# runs_to_plot["10_4000M_CAMR_set1_L6_base0"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev0_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base1"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev1_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base2"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev2_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base3"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev3_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base4"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev4_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base5"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev5_A?/Run/"
# runs_to_plot["10_4000M_CAMR_set1_L6_base6"] =  "/groups/sxs/hchaudha/spec_runs/10_4000M_CAMR_set1_L6_base/Ev/Lev6_A?/Run/"

# runs_to_plot["11_4000M_CAMR_set1_L6_base0"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev0_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base1"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev1_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base2"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev2_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base3"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev3_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base4"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev4_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base5"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev5_A?/Run/"
# runs_to_plot["11_4000M_CAMR_set1_L6_base6"] =  "/groups/sxs/hchaudha/spec_runs/11_4000M_CAMR_set1_L6_maxExt/Ev/Lev6_A?/Run/"

# runs_to_plot["12_set1_L3_1500"] =  "/groups/sxs/hchaudha/spec_runs/12_set1_L3_1500/Ev/Lev3_A?/Run/"
# runs_to_plot["13_set1_L4_1500"] =  "/groups/sxs/hchaudha/spec_runs/13_set1_L4_1500/Ev/Lev4_A?/Run/"

# runs_to_plot["12_set1_L3_2000"] =  "/groups/sxs/hchaudha/spec_runs/12_set1_L3_2000/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2500"] =  "/groups/sxs/hchaudha/spec_runs/12_set1_L3_2500/Ev/Lev3_A?/Run/"

# runs_to_plot["13_set1_L3_3000"] =  "/groups/sxs/hchaudha/spec_runs/13_set1_L3_3000/Ev/Lev3_A?/Run/"
# runs_to_plot["13_set1_L4_3000"] =  "/groups/sxs/hchaudha/spec_runs/13_set1_L4_3000/Ev/Lev4_A?/Run/"

# runs_to_plot["14_set1_L4_1500_cd5"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd5/Ev/Lev4_A?/Run/"
# runs_to_plot["14_set1_L4_1500_cd10"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd10/Ev/Lev4_A?/Run/"
# runs_to_plot["14_set1_L4_1500_cd25"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd25/Ev/Lev4_A?/Run/"
# runs_to_plot["14_set1_L4_1500_cd50"] =  "/groups/sxs/hchaudha/spec_runs/14_set1_L4_1500_cd50/Ev/Lev4_A?/Run/"

# runs_to_plot["15_set1_L4_1500_JY"] =  "/groups/sxs/hchaudha/spec_runs/15_set1_L4_1500_JY/Ev/Lev4_A?/Run/"

# runs_to_plot["16_set1_L3"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["16_set1_L3_HP32"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP32/Ev/Lev3_A?/Run/"
# runs_to_plot["16_set1_L3_HP28"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP28/Ev/Lev3_A?/Run/"
# runs_to_plot["16_set1_L3_HP32_AF"] = "/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP32_AF/Ev/Lev3_A?/Run/"
# runs_to_plot["17_BDI_32_SAE_NONE"] = "/groups/sxs/hchaudha/spec_runs/17_BDI_32_SAE_NONE/Ev/Lev4_A?/Run/"
# runs_to_plot["17_BDI_32_SAE_32"] = "/groups/sxs/hchaudha/spec_runs/17_BDI_32_SAE_32/Ev/Lev4_A?/Run/"
# runs_to_plot["17_BDI_32_SAE_32_AF"] = "/groups/sxs/hchaudha/spec_runs/17_BDI_32_SAE_32_AF/Ev/Lev4_A?/Run/"

# runs_to_plot["17_set_main_q3_15_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_q3_15_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set_main_99_15_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_99_15_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["17_set_main_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set3_q3_18_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["17_main_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_set3_9_18_L3_correct/Ev/Lev3_A?/Run/"

# runs_to_plot["17_set_main_99_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_99_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_99_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_99_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_99_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set3_99_18_L3/Ev/Lev3_??/Run/"

# runs_to_plot["17_main_9_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_9_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set3_9_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set3_9_18_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["18_set1_L3_junk_resolved"] = "/groups/sxs/hchaudha/spec_runs/18_set1_L3_junk_resolved/Ev/Lev3_??/Run/"
# runs_to_plot["20_set1_L3_fine_cylinders"] = "/groups/sxs/hchaudha/spec_runs/20_set1_L3_fine_cylinders/Ev/Lev3_??/Run/"
# runs_to_plot["21_set1_L3_fine_cylinders_minExtent"] = "/groups/sxs/hchaudha/spec_runs/21_set1_L3_fine_cylinders_minExtent/Ev/Lev3_??/Run/"

# runs_to_plot["22_set1_L1_long"] = "/groups/sxs/hchaudha/spec_runs/22_set1_L1_long/Ev/Lev1_??/Run/"
# runs_to_plot["L1_AC_L3"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L3/Ev/Lev3_??/Run/"
# runs_to_plot["L1_AC_L2"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L2/Ev/Lev2_??/Run/"
# runs_to_plot["L1_AC_L1"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L1/Ev/Lev1_??/Run/"

# runs_to_plot["L3_AC_L3_cd_const_high"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_high/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_cd_const_low"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_low/Ev/Lev3_??/Run/"

# runs_to_plot["22_L3_AC_L3_no_res_C"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_no_res_C/Ev/Lev3_??/Run/"
# runs_to_plot["22_L3_AC_L3_res_10_C"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_res_10_C/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L1"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L1/Ev/Lev1_??/Run/"
# runs_to_plot["L3_AC_L2"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L2/Ev/Lev2_??/Run/"
# runs_to_plot["22_set1_L3_long"] = "/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L4"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L4/Ev/Lev4_??/Run/"
# runs_to_plot["L3_AC_L5"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L5/Ev/Lev5_??/Run/"
# runs_to_plot["L3_AC_L6"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L6/Ev/Lev6_??/Run/"
# runs_to_plot["L3_AC_L7"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L7/Ev/Lev7_??/Run/"
# runs_to_plot["L3_AC_L8"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L8/Ev/Lev8_??/Run/"

# runs_to_plot["L3_AC_L3_3_01"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_3_01/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_3_02"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_3_02/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_5_04"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_5_04/Ev/Lev3_??/Run/"

# runs_to_plot["26_set1_L6_long"] = "/groups/sxs/hchaudha/spec_runs/26_set1_L6_long/Ev/Lev6_??/Run/"
# runs_to_plot["26_main_L6_long"] = "/groups/sxs/hchaudha/spec_runs/26_main_L6_long/Ev/Lev6_??/Run/"

# runs_to_plot["28_set1_cd_junk_5"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_5/Ev/Lev3_??/Run/"
# runs_to_plot["28_set1_cd_junk_1"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_1/Ev/Lev3_??/Run/"
# runs_to_plot["28_set1_cd_junk_01"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_01/Ev/Lev3_??/Run/"
# runs_to_plot["28_set1_cd_junk_001"] = "/groups/sxs/hchaudha/spec_runs/28_set1_cd_junk_001/Ev/Lev3_??/Run/"

# runs_to_plot["29_set1_L3_ID_diff_12"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_12/Ev/Lev3_??/Run/"
# runs_to_plot["29_set1_L3_ID_diff_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_8/Ev/Lev3_??/Run/"
# runs_to_plot["29_set1_L3_ID_diff_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_4/Ev/Lev3_??/Run/"

# runs_to_plot["29_set1_L3_ID_diff_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_0/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_1/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_2/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_3/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_4_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_4_2/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_4/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_5/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_6/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_7"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_7/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_8/Ev/Lev3_A?/Run/"
# runs_to_plot["29_set1_L3_ID_diff_12"] = "/resnick/groups/sxs/hchaudha/spec_runs/29_set1_L3_ID_diff_12/Ev/Lev3_A?/Run/"

# runs_to_plot["L1_AC_L1_AK"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L1_AC_L1/Ev/Lev1_AK/Run/"
# runs_to_plot["46K_L1_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L1_3000_cd10/Ev/Lev1_AK/Run/"
# runs_to_plot["46K_L2_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L2_3000_cd10/Ev/Lev2_AK/Run/"
# runs_to_plot["46K_L3_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L3_3000_cd10/Ev/Lev3_AK/Run/"
# runs_to_plot["46K_L4_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L4_3000_cd10/Ev/Lev4_AK/Run/"
# runs_to_plot["46K_L5_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L5_3000_cd10/Ev/Lev5_AK/Run/"
# runs_to_plot["46K_L6_3000_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_cd10/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_cd100"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_cd100/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_2500"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_2500/Ev/Lev6_AK/Run/"

# runs_to_plot["46K_L6_3000_c_AB_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_2/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_4/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_6/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_10"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_10/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_12"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_12/Ev/Lev6_AK/Run/"

# runs_to_plot["46K_L6_3000_c_AB_4_L"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_4_L/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_2_L"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_2_L/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_1_L"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_1_L/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_1_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_1_L2/Ev/Lev6_AK/Run/"
# runs_to_plot["46K_L6_3000_c_AB_0_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/22_segs_res/46K_L6_3000_c_AB_0_L2/Ev/Lev6_AK/Run/"

# runs_to_plot["set1_L3_Rn1"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn1/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn2/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin18"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin18/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin20_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin20_Rn2/Ev/Lev3_A?/Run/"

# runs_to_plot["RM_1_Lev3"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev3/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_0_Lev6"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_0_test/Ev_456/Lev6_A?/Run/"

# runs_to_plot["30_RM_set1_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_RM_set1_L1/Ev/Lev1_A?/Run/"
# runs_to_plot["30_RM_set1_L3"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_RM_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["RM_L3s3_k0"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s3_k0/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_L3s3_k0_cd10"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s3_k0_cd10/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_L3s3_k0_cd100"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s3_k0_cd100/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_L3s4_k0"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s4_k0/Ev/Lev4_A?/Run/"
# runs_to_plot["RM_L3s5_k0"] = "/resnick/groups/sxs/hchaudha/spec_runs/30_segs_res/L3s5_k0/Ev/Lev5_A?/Run/"
# runs_to_plot["RM_0_Lev6"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_0_test/Ev_456/Lev6_A?/Run/"
# runs_to_plot["RM_1_Lev3"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev3/Ev/Lev3_A?/Run/"
# runs_to_plot["RM_1_Lev6"] = "/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev6/Ev/Lev6_A?/Run/"

# runs_to_plot["31_RM_set1_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_RM_set1_L1/Ev/Lev1_A?/Run/"
# runs_to_plot["31_RM_set1_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_RM_set1_L2/Ev/Lev2_A?/Run/"

# runs_to_plot["31_segs_L1s0"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s0/Ev/Lev0_AC/Run/"
# runs_to_plot["31_segs_L1s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s1/Ev/Lev1_AC/Run/"
# runs_to_plot["31_segs_L1s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s2/Ev/Lev2_AC/Run/"
# runs_to_plot["31_segs_L1s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3/Ev/Lev3_AC/Run/"

# runs_to_plot["L1s3_cdg1_10"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_10/Ev/Lev3_AC/Run/"
# runs_to_plot["L1s3_cdg1_100"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_100/Ev/Lev3_AC/Run/"
# runs_to_plot["L1s3_cdg1_250"] = "/resnick/groups/sxs/hchaudha/spec_runs/31_segs/L1s3_cdg1_250/Ev/Lev3_AC/Run/"

# runs_to_plot["set1_L3_Rn1"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn1/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Rn2/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin18"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin18/Ev/Lev3_A?/Run/"
# runs_to_plot["set1_L3_Lmin20_Rn2"] = "/resnick/groups/sxs/hchaudha/spec_runs/33_const_inn_dom_runs/set1_L3_Lmin20_Rn2/Ev/Lev3_A?/Run/"

# runs_to_plot["main_L6_AM_ode_MQOS"] = "/resnick/groups/sxs/hchaudha/spec_runs/26_segs_res/main_L6_AM_ode_MQOS/Ev/Lev6_A?/Run/"
# runs_to_plot["set1_L6_AK_ode_MQOS"] = "/resnick/groups/sxs/hchaudha/spec_runs/26_segs_res/set1_L6_AK_ode_MQOS/Ev/Lev6_A?/Run/"

# runs_to_plot["34_master_L16_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev1_A?/Run/"
# runs_to_plot["34_master_L16_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev2_A?/Run/"
# runs_to_plot["34_master_L16_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev3_A?/Run/"
# runs_to_plot["34_master_L16_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev4_A?/Run/"
# runs_to_plot["34_master_L16_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev5_A?/Run/"
# runs_to_plot["34_master_L16_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev6_A?/Run/"

# runs_to_plot["L3_AC_L3_minL17"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_minL17/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_minL19"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_minL19/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_minL21"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_minL21/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_01_cd_asymp"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_01_cd_asymp/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_single_exp"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_single_exp/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_single_Exp_large_sigma"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_single_Exp_large_sigma/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_cd_const_low"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_low/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_cd_const_high"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_cd_const_high/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_sigma2"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_sigma2/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_sigma05"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_sigma05/Ev/Lev3_??/Run/"
# runs_to_plot["L3_AC_L3_sigma1_const"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_sigma1_const/Ev/Lev3_??/Run/"

# runs_to_plot["L3_AC_L3_AB_L8"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB_L8/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_AB_R7"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB_R7/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_AB0_L16"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB0_L16/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_AB0_L15"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_AB0_L15/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_ps_10"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_ps_10/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_ps_01"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_ps_01/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_BCSC_8"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_BCSC_8/Ev/Lev3_AC/Run/"
# runs_to_plot["L3_AC_L3_BCSC_12"] = "/groups/sxs/hchaudha/spec_runs/22_segs/L3_AC_L3_BCSC_12/Ev/Lev3_AC/Run/"

# runs_to_plot["119_gd_SUKS_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["120W_gd_SUKS1_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/120W_gd_SUKS1_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["120_gd_SUKS1_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/120_gd_SUKS1_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["119_gd_SUKS_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"
# runs_to_plot["67_master_mr3"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/67_master_mr3/Ev/Lev1_A?/Run/"
# runs_to_plot["119_gd_SUKS_3_20"] = "/net/panfs/SXS/himanshu/gauge_stuff/gauge_driver_runs/runs/119_gd_SUKS_3_20/Ev/Lev3_A?/Run/"

# runs_to_plot["35_L3_base"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_base/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L3_base_nas"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_base_nas/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L2_base_nas"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L2_base_nas/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L3_Ctest10"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_Ctest10/Ev/Lev3_A?/Run/"
# runs_to_plot["35_L3_Ctest15"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_Ctest15/Ev/Lev6_A?/Run/"
# runs_to_plot["35_L3_Ctest15Uni"] = "/resnick/groups/sxs/hchaudha/spec_runs/35_NAMR_L3_tests/L3_Ctest15Uni/Ev/Lev6_A?/Run/"

# runs_to_plot["36_set1_L1_d20"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_set1_L1_d20/Ev/Lev1_A?/Run/"
# runs_to_plot["37_set1_L3_no_ang"] = "/resnick/groups/sxs/hchaudha/spec_runs/37_6_segs_grid_change/6_set1_L3_no_ang/Ev/Lev3_A?/Run/"

# runs_to_plot["34_master_L16_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev1_A?/Run/"
# runs_to_plot["34_master_L16_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev2_A?/Run/"
# runs_to_plot["34_master_L16_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev3_A?/Run/"
# runs_to_plot["34_master_L16_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev4_A?/Run/"
# runs_to_plot["34_master_L16_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev5_A?/Run/"
# runs_to_plot["34_master_L16_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/34_master_L16/Ev/Lev6_A?/Run/"

# runs_to_plot["36_segs_L1"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev1_A?/Run/"
# runs_to_plot["36_segs_L2"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev2_A?/Run/"
# runs_to_plot["36_segs_L3"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev3_A?/Run/"
# runs_to_plot["36_segs_L4"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev4_A?/Run/"
# runs_to_plot["36_segs_L5"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev5_A?/Run/"
# runs_to_plot["36_segs_L6"] = "/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/Ev/Lev6_A?/Run/"

# runs_to_plot["6_set1_L6s0"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev0_A?/Run/"
# runs_to_plot["6_set1_L6s1"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev1_A?/Run/"
# runs_to_plot["6_set1_L6s2"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev2_A?/Run/"
# runs_to_plot["6_set1_L6s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L6s4"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev4_A?/Run/"
# runs_to_plot["6_set1_L6s5"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev5_A?/Run/"
# runs_to_plot["6_set1_L6s6"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/Ev/Lev6_A?/Run/"

# runs_to_plot["L35higher_correct"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35higher_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["L35lower_correct"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35lower_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["L45higher_correct"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L45higher_correct/Ev/Lev4_A?/Run/"
# runs_to_plot["L35higher"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35higher/Ev/Lev3_A?/Run/"
# runs_to_plot["L35lower"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L35lower/Ev/Lev3_A?/Run/"
# runs_to_plot["L3AH10"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH10/Ev/Lev3_A?/Run/"
# runs_to_plot["L3AH1000"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH1000/Ev/Lev3_A?/Run/"
# runs_to_plot["L3AH5000"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L3AH5000/Ev/Lev3_A?/Run/"
# runs_to_plot["L45higher"] =  "/resnick/groups/sxs/hchaudha/spec_runs/38_6_segs_half/L45higher/Ev/Lev4_A?/Run/"

# runs_to_plot["L5_main_obs"] =  "/resnick/groups/sxs/hchaudha/spec_runs/39_obs_segs/L3L3_main_obs/Ev/Lev5_A?/Run/"
# runs_to_plot["L1_set1_metric_obs"] =  "/resnick/groups/sxs/hchaudha/spec_runs/39_obs_segs/L1_set1_metric_obs/Ev/Lev1_AB/Run/"

# runs_to_plot["q1_0_0_9_0_0_0_d165"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_0_d165_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_0_9_0_0_9_d165"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_9_d165_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_65_65_0_n65_n65_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_65_65_0_n65_n65_d185_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_8_0_0_0_8_d17"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_8_d17_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_8_0_0_0_n8_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_n8_d185_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q5_0_0_0_0_0_0_d15"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q5_0_0_0_0_0_0_d15_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q10_0_0_0_0_0_0_d14"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q1_0_0_n9_0_0_0_d185"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_n9_0_0_0_d185_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q3_0_0_85_0_0_85_d15"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_85_0_0_85_d15_segs/Ev/Lev3_??/Run/"
# runs_to_plot["q3_0_0_n85_0_0_n85_d18"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_n85_0_0_n85_d18_segs/Ev/Lev3_??/Run/"
Ecc_num = 0
# runs_to_plot["q10_0_0_0_0_0_0_d14"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q10_0_0_0_0_0_0_d14/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_9_0_0_0_d165"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_0_d165/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_9_0_0_9_d165"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_9_0_0_9_d165/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_0_n9_0_0_0_d185"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_0_n9_0_0_0_d185/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_65_65_0_n65_n65_d185"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_65_65_0_n65_n65_d185/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_8_0_0_0_8_d17"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_8_d17/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q1_0_8_0_0_0_n8_d185"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q1_0_8_0_0_0_n8_d185/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q3_0_0_85_0_0_85_d15"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_85_0_0_85_d15/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q3_0_0_n85_0_0_n85_d18"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q3_0_0_n85_0_0_n85_d18/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"
# runs_to_plot["q5_0_0_0_0_0_0_d15"] = f"/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_acc/q5_0_0_0_0_0_0_d15/Ecc{Ecc_num}/Ev/Lev3_A?/Run/"

# runs_to_plot["1_DH"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH/Ev/Lev3_A?/Run/"
# runs_to_plot["2_SupImpKerr"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/2_SupImpKerr/Ev/Lev3_A?/Run/"
# runs_to_plot["6_set1_L3s3"] =  "/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/Ev/Lev3_A?/Run/"

# runs_to_plot["17_main_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_main_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_9_18_L3_correct"] = "/groups/sxs/hchaudha/spec_runs/17_set1_9_18_L3_correct/Ev/Lev3_A?/Run/"
# runs_to_plot["3_DH_q1_09"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_DH_q1_09/Ev/Lev3_A?/Run/"
# runs_to_plot["3_SupImpKerr_09"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_SupImpKerr_09/Ev/Lev3_A?/Run/"

# runs_to_plot["17_set_main_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set_main_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["17_set1_q3_18_L3"] = "/groups/sxs/hchaudha/spec_runs/17_set1_q3_18_L3/Ev/Lev3_A?/Run/"
# runs_to_plot["3_DH_q3"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_DH_q3/Ev/Lev3_A?/Run/"
# runs_to_plot["3_SupImpKerr_q3"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/3_SupImpKerr_q3/Ev/Lev3_A?/Run/"

# runs_to_plot["DH_8_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_8_8/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_32_32"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_32_32/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_64_64"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_64_64/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_128_128"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_128_128/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_256_256"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_256_256/Ev/Lev3_AC/Run/"
# runs_to_plot["DH_512_512"] = "/resnick/groups/sxs/hchaudha/spec_runs/spec_runs_gauge_driver/1_DH_segs/DH_512_512/Ev/Lev3_AC/Run/"


# runs_to_plot["42_f128_Set1L6s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s1/Ev/Lev1_A?/Run/"
# runs_to_plot["42_f128_Set1L6s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s2/Ev/Lev2_A?/Run/"
# runs_to_plot["42_f128_Set1L6s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s3/Ev/Lev3_A?/Run/"
# runs_to_plot["42_f128_Set1L6s4"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s4/Ev/Lev4_A?/Run/"
# runs_to_plot["42_f128_Set1L6s5"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s5/Ev/Lev5_A?/Run/"
# runs_to_plot["42_f128_Set1L6s6"] = "/resnick/groups/sxs/hchaudha/spec_runs/42_f128_from_check/set1L6s6/Ev/Lev6_A?/Run/"

# runs_to_plot["45_f128_Set1L6s1"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s1/Ev/Lev1_A?/Run/"
# runs_to_plot["45_f128_Set1L6s2"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s2/Ev/Lev2_A?/Run/"
# runs_to_plot["45_f128_Set1L6s3"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s3/Ev/Lev3_A?/Run/"
# runs_to_plot["45_f128_Set1L6s4"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s4/Ev/Lev4_A?/Run/"
# runs_to_plot["45_f128_Set1L6s5"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s5/Ev/Lev5_A?/Run/"
# runs_to_plot["45_f128_Set1L6s6"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s6/Ev/Lev6_A?/Run/"

# runs_to_plot["45.1_set1_more_cd1_moreL_1"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev1_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_2"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev2_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_3"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev3_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_4"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev4_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev5_A?/Run/"
# runs_to_plot["45.1_set1_more_cd1_moreL_6"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev6_A?/Run/"

# runs_to_plot["45.2_RM_more_cd1_moreL_2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev2_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_3"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev3_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_4"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev4_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_5"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev5_A?/Run/"

# runs_to_plot["45.2_RM_more_cd1_moreL_3_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev3_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_4_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev4_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_5_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev5_A?/Run/"

# runs_to_plot["45.1_set1_more_cd1_moreL_5"] = "/resnick/groups/sxs/hchaudha/spec_runs/45.1_set1_more_cd1_moreL/Ev/Lev5_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_5"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s25/Ev/Lev5_A?/Run/"
# runs_to_plot["45.2_RM_more_cd1_moreL_5_lesscd2"]="/resnick/groups/sxs/hchaudha/spec_runs/45.2_RM_more_cd1_moreL/L3s35_25L_lesscd2/Ev/Lev5_A?/Run/"
# runs_to_plot["45_f128_Set1L6s5"] = "/resnick/groups/sxs/hchaudha/spec_runs/45_128_set1L16_more_cd1_moreL/set1L6s5/Ev/Lev5_A?/Run/"


# runs_to_plot["13_set1_L3_3000"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L3_3000/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2500"] = "/resnick/groups/sxs/hchaudha/spec_runs/12_set1_L3_2500/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_2000"] = "/resnick/groups/sxs/hchaudha/spec_runs/12_set1_L3_2000/Ev/Lev3_A?/Run/"
# runs_to_plot["12_set1_L3_1500"] = "/resnick/groups/sxs/hchaudha/spec_runs/12_set1_L3_1500/Ev/Lev3_A?/Run/"

# runs_to_plot["13_set1_L4_3000"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L4_3000/Ev/Lev4_A?/Run/"
# runs_to_plot["13_set1_L4_1500"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L4_1500/Ev/Lev4_A?/Run/"
# runs_to_plot["13_set1_L4_1500"] = "/resnick/groups/sxs/hchaudha/spec_runs/13_set1_L4_1500/Ev/Lev4_A?/Run/"

# runs_to_plot["sbh_23_const_AMR_obs_horizon"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/23_const_AMR_obs_horizon/Lev7_265_10_10_0.001_-1/Run/"
# runs_to_plot["sbh_Lev5_9565_longer"] = "/resnick/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/Lev5_9565_longer/Run/"

# runs_to_plot["q1_0_0_8_0_0_8"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q1_0_0_8_0_0_8/Ev/Lev6_A?/Run/"
# runs_to_plot["q1_0_0_8_0_0_8_MaxDL"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q1_0_0_8_0_0_8_MaxDL/Ev/Lev6_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q4_0_0_0_0_0_0/Ev/Lev6_A?/Run/"
# runs_to_plot["q4_0_0_0_0_0_0_MaxDL"] = "/resnick/groups/sxs/hchaudha/spec_runs/50_LISA_cat_Lev6_test/q4_0_0_0_0_0_0_MaxDL/Ev/Lev6_A?/Run/"

# runs_to_plot["q1_ns_3000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_3000M_lsds/Ecc0/Ev/Lev4_A?/Run/"
# runs_to_plot["q1_ns_3000M"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_3000M/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q1_ns_2000M"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_2000M/Ecc0/Ev/Lev4_A?/Run/"
# runs_to_plot["q1_ns_2000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_2000M_lsds/Ecc0/Ev/Lev4_A?/Run/"

# runs_to_plot["q1_ns_10000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/53_large_ext_boundary_less_sds/q1_ns_10000M_lsds/Ecc0/Ev/Lev4_A[A-H]/Run/"

# runs_to_plot["q1_0_0_0_0_0_0_no_NegExp"] = "/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_0_0_0_0_no_NegExp/Ecc0/Ev/Lev4_A?/Run/"


# runs_to_plot["f128_q1_n8_8_3000M_lsds_var1"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_n8_8_3000M_lsds_var1/Ev/Lev4_A?/Run/"
# runs_to_plot["f128_q1_ns_2000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_ns_2000M_lsds/Ev/Lev4_A?/Run/"
# runs_to_plot["f128_q1_ns_3000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_ns_3000M_lsds/Ev/Lev4_A?/Run/"
# runs_to_plot["f128_q1_ns_10000M_lsds"] = "/resnick/groups/sxs/hchaudha/spec_runs/55_f128_large_ob_lsds/q1_ns_10000M_lsds/Ev/Lev4_A?/Run/"


# for lev in range(2,7):
for lev in range(2,5):
# for lev in [6]:
    runs_to_plot[f"01_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501/Ecc0/Ev/Lev{lev}_A?/Run/"
    # runs_to_plot[f"02_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102502/Ecc0/Ev/Lev{lev}_A?/Run/"
    # runs_to_plot[f"09_L{lev}"] = f"/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102509/Ecc0/Ev/Lev{lev}_A?/Run/"

ringdown_only = True
ringdown_only = False
if ringdown_only:
    for key in list(runs_to_plot.keys()):
        runs_to_plot[key] = runs_to_plot[key].replace("/Ev/","/Ev/**/")


# data_file_path = "ApparentHorizons/AhACoefs.dat"
# data_file_path = "ApparentHorizons/AhBCoefs.dat"

# data_file_path = "PowerDiagnostics.h5@ConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@HighestThirdConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfFilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfNonFilteredNonZeroModes.dat"
# data_file_path = "PowerDiagnostics.h5@NumberOfPiledUpModes.dat"
# data_file_path = "PowerDiagnostics.h5@PowerInFilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@PowerInHighestUnfilteredModes.dat"
# data_file_path = "PowerDiagnostics.h5@PredictedTruncationErrorForOneLessMode.dat"
# data_file_path = "PowerDiagnostics.h5@RawConvergenceFactor.dat"
# data_file_path = "PowerDiagnostics.h5@SpectrumIsDegenerate.dat"
# data_file_path = "PowerDiagnostics.h5@TruncationError.dat"

# all_m = ['DrJ.dat', 'H.dat', 'J.dat', 'Q.dat', 'U.dat' ]
# some_m = ['Beta.dat', 'DuR.dat',  'R.dat', 'W.dat']

# WT_var = 'DrJ.dat'
# WT_var = 'H.dat'
# WT_var = 'J.dat'
# WT_var = 'Q.dat'
# WT_var = 'U.dat'
# WT_var = 'Beta.dat'
# WT_var = 'DuR.dat'
WT_var = 'R.dat'
# WT_var = 'W.dat'


radius_list = ['0317']
# radius_list = ["0900"]
# radius_list = ['0252', '0260', '0268', '0277', '0287', '0297', '0309', '0321', '0334', '0348', '0364', '0381', '0399', '0420', '0443', '0468', '0497', '0529', '0565', '0607', '0656', '0713', '0782', '0865']
# radius_list = ['0309', '0318', '0327', '0338', '0349', '0361', '0373', '0387', '0402', '0417', '0434', '0453', '0473', '0495', '0519', '0546', '0575', '0608', '0645', '0686', '0734', '0788', '0851']
# radius_list = ['0880',"1910","2700"]
# radius_list = ["0257","0265","0274","0283","0293","0304","0315","0328","0341","0356","0372","0390","0409","0431","0454","0481","0510","0544","0582","0627","0678","0739","0811","0900"]
# radius_list = ["0257","0328","0431","0510","0627","0739","0811","0900"]
# radius_list = ['0300', '0309', '0320', '0331', '0342', '0355', '0369', '0384', '0400', '0417', '0436', '0457', '0480', '0505', '0533', '0565', '0600', '0640', '0686', '0739', '0801', '0874', '0962', '1070']
# radius_list = ['0250', '0258', '0266', '0276', '0285', '0296', '0308', '0320', '0333', '0348', '0364', '0381', '0401', '0422', '0446', '0472', '0502', '0536', '0574', '0619', '0672', '0734', '0808', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800']
# radius_list = ['1000']
# radius_list = ['0257', '0266', '0277', '0289', '0302', '0315', '0331', '0347', '0366', '0386', '0409', '0435', '0464', '0498', '0537', '0582', '0636', '0701', '0780', '0880', '1009', '1182', '1427', '1800']
# radius_list = ['0257', '0267', '0278', '0291', '0304', '0319', '0336', '0354', '0375', '0397', '0423', '0453', '0486', '0526', '0572', '0626', '0693', '0775', '0880', '1017', '1205', '1478', '1910', '2700']
# radius_list = ['0100', '0150', '0200', '0250', '0300', '0350', '0500', '0700', '0900']
# radius_list = ['0025', '0029', '0035', '0045', '0062', '0100']
# radius_list = ['0100', '0250', '0500', '0900']

# radius_list = [radius_list[0]] + radius_list[2:-1:6] + [radius_list[-1]]
# radius_list = radius_list[::-4]  # Reverse order to get large radii
# radius_list = radius_list[::4]

# data_file_path = "GW2/BondiCceR"

# data_file_path = f"GW2/PhiMinus_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/PhiPlus_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/r2Psi3_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/r3Psi2_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/r4Psi1_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/r5Psi0_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/RawStrahlkorperIntegrals.h5"
data_file_path = f"GW2/rh_FiniteRadii_CodeUnits.h5"
# data_file_path = f"GW2/rPsi4_FiniteRadii_CodeUnits.h5"

runs_data_dict = {}
psi_or_kappa = "both"  # "psi", "kappa", or "both"

for radius in radius_list:
    runs_data_dict_local = {} 
    if "BondiCceR" in data_file_path:
        data_file_path_with_radius = data_file_path+f"{radius}.h5@{WT_var}"
    else:
        data_file_path_with_radius = data_file_path+f"@{radius}"
    if psi_or_kappa == "both" and "PowerDiagnostics" in data_file_path_with_radius:
        column_names_kappa, runs_data_dict_kappa = load_data_from_levs(runs_to_plot,f"{data_file_path_with_radius}@kappa@{top_number}")
        column_names_psi, runs_data_dict_psi = load_data_from_levs(runs_to_plot,f"{data_file_path_with_radius}@psi@{top_number}")

        for key in runs_data_dict_psi:
            runs_data_dict_local[key] =pd.merge(runs_data_dict_kappa[key], runs_data_dict_psi[key], on='t(M)', how='outer')
        # column_names = runs_data_dict[key].columns.tolist()
    else:
        if "PowerDiagnostics" in data_file_path_with_radius:
            data_file_path_with_radius = f"{data_file_path_with_radius}@{psi_or_kappa}@{top_number}"
        column_names, runs_data_dict_local = load_data_from_levs(runs_to_plot,data_file_path_with_radius)

    # Each radii will have it's own runs_data_dict
    for key in runs_data_dict_local:
        runs_data_dict[key + f"_R{radius}"] = runs_data_dict_local[key]

column_names = runs_data_dict[key + f"_R{radius}"].columns.tolist()

print(column_names)
print(runs_data_dict.keys())

In [ ]:
for key in runs_data_dict:
    runs_data_dict[key] = runs_data_dict[key].sort_values(by='t(M)', ascending=True).drop_duplicates(subset='t(M)', keep='last').reset_index(drop=True)

In [ ]:
# Make it consistent with the other power stuff
if "Coefs" in data_file_path:
    for key in runs_data_dict:
        runs_data_dict[key] = runs_data_dict[key].reset_index(drop=True)
        if 'index' in runs_data_dict[key].columns:
            runs_data_dict[key] = runs_data_dict[key].drop('index', axis=1)
        runs_data_dict[key] = runs_data_dict[key].drop(columns=['Center-x', 'Center-y', 'Center-z'])
        # runs_data_dict[key] = runs_data_dict[key].rename(columns={'t(M)': 'Coef(t(M))'})
        # runs_data_dict[key] = runs_data_dict[key].rename(columns=lambda x: x[5:-1])

In [ ]:
# change_to_power_in_L_modes(runs_data_dict)
# change_to_power_in_L_modes(runs_data_dict,RMS_Power=False)
# change_to_power_in_L_modes(runs_data_dict,RMS_Power=True)

## Plots

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
constant_shift_val_time = None
plot_abs_diff = False
modification_function = None
append_to_title = ""
y_axis_list = None
x_axis = 't(M)'
take_abs = False

take_abs = True

# diff_base = '36_segs_L6_R0309'
# diff_base = f'q1_ns_3000M_R{radius_list[0]}'
# diff_base = f'q1_ns_3000M_lsds_R{radius_list[0]}'
# diff_base = f'q1_ns_10000M_lsds_R{radius_list[0]}'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

# constant_shift_val_time = 7206
# constant_shift_val_time = 2300
# constant_shift_val_time = 400
# constant_shift_val_time = 6400
if constant_shift_val_time is not None:
    take_abs = True

# y_axis = '16,0'
y_axis = '2,0'
# y_axis = '2,2'
# y_axis = '3,3'
# y_axis = '4,4'
# y_axis = '5,5'
y_axis = '6,0'
# y_axis = '7,7'
# y_axis = '7,2'
# y_axis = '6,4'
# y_axis = '8,8'
# y_axis = '12,12'
# y_axis = '0'
y_axis = '8,0'
# y_axis = '0,0'

# y_axis_list = ['2,2','4,4','8,8']

L = 8
# y_axis_list = [f"{l},{0}" for l in range(2,L+1)]
# y_axis_list = [f"{L},{l}" for l in range(L,-1,-1)]
# y_axis_list = [f"{L},{l}" for l in range(L,-L-1,-1)]
# y_axis_list = [f"{l},{l}" for l in range(2,9)] + ['2,0','3,0','2,1']
# y_axis_list = [f"{l},{l}" for l in range(2,12)]
# y_axis_list = [f"{l},{l}" for l in range(8,17)]
# y_axis_list = runs_data_dict[list(runs_data_dict.keys())[0]].columns.tolist()[1:]  # Exclude 't(M)' column
# y_axis_list = [str(i) for i in range(0,30) if str(i) in y_axis_list]

minT = -10000
# minT = 85
# minT = 470
# minT = 1200
# minT = 2200
# minT = 4000
# minT = 11000
# minT = 11000

maxT = 400000
# maxT = minT+100
# maxT = 700
# maxT = 2250
# maxT = 4500
# maxT = 8500
# maxT = 11000
# moving_avg_len = 50
# moving_avg_len = 10
# maxT = 47100
# maxT = 46600


# if "GhCe" in y_axis:
plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)

# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label) 
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x",alpha=0.4)
# save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
# save_path = "/groups/sxs/hchaudha/rough/plots/"
# save_path = "/home/hchaudha/notes/spec_accuracy/figures/"
# save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/"
# save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/L15_no_tol/"
legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None


# take_abs,modification_function = True,derivative_function
# take_abs,modification_function = True, lambda x, y, df, y_axis: derivative_function(x,np.real(y),df,y_axis,order=1)
# take_abs,modification_function = True,compute_power_in_modes
# take_abs,modification_function = False, lambda x, y, df, y_axis: (x,np.real(y),f"{y_axis} Real")
# take_abs,modification_function = False, lambda x, y, df, y_axis: (x,np.imag(y),f"{y_axis} Imag")
# take_abs,modification_function = False, lambda x, y, df, y_axis: (x,np.unwrap(np.angle(y)),f"{y_axis} Unwrapped Phase")
# take_abs,modification_function = True, lambda x, y, df, y_axis: compute_power_in_modes(x, y, df, y_axis, L_list=[2])
# take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=list(np.arange(1,7)))
# take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=[2])
# take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=[2,3,4,5,6,7,8])
# take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=[9,10,11,12,13,14,15,16])


# if 'Horizons.h5@' in data_file_path:
#   append_to_title += " HorizonBH="+data_file_path.split('@')[-1]
if 'AhA' in data_file_path:
    append_to_title += " AhA"
if 'AhB' in data_file_path:
    append_to_title += " AhB"
if 'BondiCceR' in data_file_path:
    append_to_title += f"\n{WT_var}"
if 'FiniteRadii_CodeUnits' in data_file_path:
    append_to_title += f"\n{data_file_path.split('/')[-1]}"


# with plt.style.context('default'):
with plt.style.context('ggplot'):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:20]
    # colors = plt.cm.tab10.colors
    line_styles = ['-','--',':',  '-.']
    combined_cycler = cycler(linestyle=line_styles)*cycler(color=colors[:11])
    # combined_cycler = cycler(color=colors)*cycler(linestyle=line_styles[:2])
    plt.rcParams['axes.prop_cycle'] = combined_cycler
#   plt.rcParams["figure.figsize"] = (15,10)
#   plt.rcParams["figure.figsize"] = (4,4)
    plt.rcParams["figure.figsize"] = (10,8)
    # plt.rcParams["figure.figsize"] = (6,5)
    plt.rcParams["figure.autolayout"] = True
    # plt.ylim(1e-10,5e-6)
    if y_axis_list is None:
        plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)
    else:
        plot_graph_for_runs_wrapper(runs_data_dict, x_axis, y_axis_list, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)

#   plt.title("")
#   plt.ylabel("Constraint Violations near black holes")
#   plt.tight_layout()
    # plt.legend(loc='upper left')
    # plt.ylim(1e-14, 1e-7)
    # plt.ylim(1e-20, 1e-13)
    # save_name = "change_L16_set1_SphereC22_Linf_NormalizedGhCe.png"

    # save_name = Path(f"/home/hchaudha/code_for_paper_plots/spec_accuracy/{save_name}")
    # if save_name.exists():    
    #     raise Exception("Change name")
    # plt.savefig(save_name)
    plt.yscale('log')
    # plt.yscale('symlog',linthresh=1e-7)
    # plt.yscale('symlog',linthresh=1e-12)
    # plt.gca().legend([])  # This removes the legend
    plt.tight_layout()
    plt.show()

In [ ]:
for L in range(2,9):
    for m in range(-L,L+1):
        save_path = f"/resnick/groups/sxs/hchaudha/spec_runs/del/temp_figures/"
        take_abs,modification_function = True,derivative_function
        # take_abs,modification_function = False,derivative_function
        plot_graph_for_runs(runs_data_dict, x_axis, f"{L},{m}", minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)
        plt.clf()

In [ ]:
minT = 3000
data = runs_data_dict[key]
data = data[data['t(M)']>=minT]
# Drop t(M) column
data = data.drop(columns=['t(M)'])
# Take abs and find the max value in each column
# data = data.abs().max()
data_mean = data.abs().mean()
data_std = data.abs().std()
data_mean, data_std


In [ ]:
for i in data_mean.index:
    print(i, data_mean[i], data_std[i])

In [ ]:
plt.plot(data[data>1e-6])
plt.yscale('log')
plt.xticks(rotation=90)
plt.show()

In [ ]:

def compute_interpolated_diff(data, res1, res2, l, m):
    """
    Compute difference between two resolutions after interpolating the higher
    time-resolution data onto the lower resolution timeline.
    
    Parameters:
    -----------
    data : dict
        Dictionary containing dataframes for each resolution
    res1, res2 : str
        Resolution keys
    l, m : int
        Spherical harmonic mode indices
        
    Returns:
    --------
    t_interp : np.ndarray
        Time array used for interpolation (from lower resolution)
    diff : np.ndarray
        Complex difference array
    """
    from scipy.interpolate import interp1d
    
    # Get time arrays
    t1 = data[res1]['t(M)'].values
    t2 = data[res2]['t(M)'].values
    
    # Get mode data
    mode1 = data[res1][f"{l},{m}"].values
    mode2 = data[res2][f"{l},{m}"].values
    
    # Determine which has more time steps
    if len(t1) > len(t2):
        # res1 has more time steps, interpolate onto t2
        t_interp = t2
        interp_real = interp1d(t1, mode1.real, kind='cubic', fill_value='extrapolate')
        interp_imag = interp1d(t1, mode1.imag, kind='cubic', fill_value='extrapolate')
        mode1_interp = interp_real(t2) + 1j * interp_imag(t2)
        diff = mode1_interp - mode2
        diff_normalized = diff / np.abs(mode2)
    else:
        # res2 has more time steps (or equal), interpolate onto t1
        t_interp = t1
        interp_real = interp1d(t2, mode2.real, kind='cubic', fill_value='extrapolate')
        interp_imag = interp1d(t2, mode2.imag, kind='cubic', fill_value='extrapolate')
        mode2_interp = interp_real(t1) + 1j * interp_imag(t1)
        diff = mode1 - mode2_interp
        diff_normalized = diff / np.abs(mode1)

    return t_interp, diff, diff_normalized

data = runs_data_dict
l,m = 2,2
plot_normalized_diff = True
plot_normalized_diff = False

extraction_radii = radius
variable = data_file_path
start_time = -1000
# start_time = -float(extraction_radii)
# start_time = 0
start_time = 6500

end_time = 3000
end_time = 100000

with plt.style.context('ggplot'):
    plt.figure(figsize=(6, 5))
    highest_res = '36_segs_L6'
    for low in list(data.keys())[:-1]:
        t_interp, diff, diff_normalized = compute_interpolated_diff(data, highest_res, low, l, m)
        t_plot = t_interp[(t_interp >= start_time) & (t_interp <= end_time)]
        if plot_normalized_diff:
            diff = diff_normalized[(t_interp >= start_time) & (t_interp <= end_time)]
        else:
            diff = diff[(t_interp >= start_time) & (t_interp <= end_time)]
        plt.plot(t_plot, np.abs(diff), label=f'|{highest_res} - {low}|')

    plt.ylabel(f"({l},{m}) mode for {variable}")
    if plot_normalized_diff:
        plt.title(f"{highest_res}, r={extraction_radii}M, Normalized")
    else:
        plt.title(f"{highest_res}, r={extraction_radii}M")
    plt.xlabel('t(M)')
    plt.yscale('log')
    plt.legend()
    plt.show()

In [ ]:
data = runs_data_dict['36_segs_L6']
plt.plot(data['t(M)'], data['2,2'].values.real)
plt.plot(data['t(M)'], data['2,2'].values.imag)
plt.plot(data['t(M)'], np.abs(data['2,2'].values))
plt.show()

#### Save all y-axis

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
constant_shift_val_time = None
plot_abs_diff = False
modification_function = None
append_to_title = ""
y_axis_list = None
x_axis = 't(M)'
take_abs = False

take_abs = True

# diff_base = '34_master_L16_6'
# diff_base = '6_set1_L6s6'
# diff_base = '36_segs_L6'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

# constant_shift_val_time = 7206
# constant_shift_val_time = 1200
# constant_shift_val_time = 400
# constant_shift_val_time = 6400
if constant_shift_val_time is not None:
    take_abs = True


for y_axis in runs_data_dict[list(runs_data_dict.keys())[0]].columns.tolist()[1:]:  # Exclude 't(M)' column
    if "diff" in y_axis:
        continue
# for y_axis in [str(i) for i in range(0,17)]:  # Exclude 't(M)' column
    append_to_title = ""
    # y_axis_list = runs_data_dict[list(runs_data_dict.keys())[0]].columns.tolist()[1:]  # Exclude 't(M)' column
    # y_axis_list = [str(i) for i in range(0,12) if str(i) in y_axis_list]

    minT = 0
    # minT = 85
    # minT = 470
    minT = 1100
    # minT = 2200
    # minT = 5800
    # minT = 11000

    maxT = 400000
    # maxT = minT+1750
    # maxT = 700
    # maxT = 1100
    # maxT = 2000
    # maxT = 8500
    # maxT = 11000
    # moving_avg_len = 50
    # moving_avg_len = 10
    # maxT = 47100
    # maxT = 46600


    # if "GhCe" in y_axis:
    plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
    # plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
    plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)

    # plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
    # plot_fun = lambda x,y,label : plt.loglog(x,y,label=label) 
    # plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x",alpha=0.4)
    # save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
    # save_path = "/groups/sxs/hchaudha/rough/plots/"
    # save_path = "/home/hchaudha/notes/spec_accuracy/figures/"
    # save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/"
    # save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/L15_no_tol/"
    legend_dict = {}
    for key in runs_data_dict.keys():
        legend_dict[key] = None


    # take_abs,modification_function = True,compute_power_in_modes
    # take_abs,modification_function = True, lambda x, y, df, y_axis: compute_power_in_modes(x, y, df, y_axis, L_list=[3])
    # take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=None)
    # take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=[2,3,4,5,6,7,8])
    # take_abs,modification_function = True, lambda x, y, df, y_axis: compute_diff_power_in_modes(x, y, df, y_axis,diff_base_df=runs_data_dict[diff_base], L_list=[9,10,11,12,13,14,15,16])


    # if 'Horizons.h5@' in data_file_path:
    #   append_to_title += " HorizonBH="+data_file_path.split('@')[-1]
    if 'AhA' in data_file_path:
        append_to_title += " AhA"
    if 'AhB' in data_file_path:
        append_to_title += " AhB"
    if 'BondiCceR' in data_file_path:
        append_to_title += f"\n{data_file_path.split('/')[-1]}"
    if 'FiniteRadii_CodeUnits' in data_file_path:
        append_to_title += f"\n{data_file_path.split('/')[-1]}"


    # with plt.style.context('default'):
    # with plt.style.context('seaborn-v0_8-paper'):
    # with plt.style.context('default'):
    with plt.style.context('ggplot'):
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:20]
        # colors = plt.cm.tab10.colors
        line_styles = ['-','--',':',  '-.']
        combined_cycler = cycler(linestyle=line_styles)*cycler(color=colors[:6])
        # combined_cycler = cycler(color=colors)*cycler(linestyle=line_styles[:2])
        plt.rcParams['axes.prop_cycle'] = combined_cycler
        # plt.figure(figsize=(8, 6))
        plt.figure(figsize=(6, 5))
        plt.rcParams["figure.autolayout"] = True
        plt.ylim(1e-14,5e-3)
        if y_axis_list is None:
            plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)
        else:
            plot_graph_for_runs_wrapper(runs_data_dict, x_axis, y_axis_list, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, plot_abs_diff=plot_abs_diff,constant_shift_val_time=constant_shift_val_time,append_to_title=append_to_title,modification_function=modification_function,take_abs=take_abs)

    #   plt.title("")
    #   plt.ylabel("Constraint Violations near black holes")
    #   plt.tight_layout()
        # plt.legend(loc='upper left')
        # plt.ylim(1e-14, 1e-7)
        # plt.ylim(1e-20, 1e-13)

        plt.tight_layout()
        save_name = f"""{data_file_path.split("/")[1].replace(".","_")}_{y_axis}_t={minT}:{maxT}.png"""
        save_name = Path(f"/groups/sxs/hchaudha/scripts/code_for_paper_plots/spec_accuracy/pow_diff_WT_set16/{save_name}")
        # if save_name.exists():
        #     raise Exception("Change name")
        plt.savefig(save_name, dpi=200)
        # plt.xscale('log')
        # plt.show()
        plt.clf()
    plt.close('all')

# Mismatch

In [ ]:
runs_data_dict.keys()

In [ ]:
base_key_root = "01_L4"
# base_key_root = "6_set1_L6s6"
# base_key_root = list(runs_data_dict.keys())[-1]


mistmatch_dict = {}
diff_dict={}
for radius in radius_list:
    base_key = f"{base_key_root}_R{radius}"
    data_base = runs_data_dict[base_key]
    for key in runs_data_dict:
        if f"_R{radius}" not in key:
            continue
        if key == base_key:
            continue
        data_other = runs_data_dict[key]
        print(key)
        # diff_dict[key] = waveform_diff_all_modes(data_base, data_other,t_min=3500,t_max=11500)
        diff_dict[key] = waveform_diff_all_modes(data_base, data_other,t_min=0,t_max=110500)
        mistmatch_dict[key] = get_cumilative_waveform_diff(diff_dict[key])

mistmatch_dict

{'6_set1_L6s1_R0257': 9.747174310831468e-08,
 '6_set1_L6s2_R0257': 1.6830154703518532e-07,
 '6_set1_L6s3_R0257': 5.769833352501015e-09,
 '6_set1_L6s4_R0257': 4.50971759244379e-12,
 '6_set1_L6s5_R0257': 1.8105921307634793e-11}

In [ ]:
diff_lev = diff_dict[list(diff_dict.keys())[-1]]
diff_lev = {key: diff_lev[key] for key in diff_lev if diff_lev[key][1]>1e-10}
# diff_lev = {getLM_from_key(key): diff_lev[key] for key in diff_lev if diff_lev[key][1]>1e-10}
diff_lev = {key: diff_lev[key][0]/diff_lev[key][1] for key in diff_lev}
# diff_lev = {getLM_from_key(key): diff_lev[key][0] for key in diff_lev}
# diff_lev = {getLM_from_key(key): diff_lev[key][0]/diff_lev[key][1] for key in diff_lev}
diff_lev

In [ ]:
big_vals=['2,-2', '2,-1', '2,0', '2,1', '2,2', '3,-3', '3,-2', '3,0', '3,2', '3,3', '4,-4', '4,-2', '4,0', '4,2', '4,4', '5,-4', '5,4', '6,-6', '6,-4', '6,4', '6,6', '8,-8', '8,8']

In [ ]:
mode_list =['2,2','2,0','2,1','4,4']
L = 8
normalize_each_mode = False
normalize_each_mode = True

mode_list = [f"{L},{m}" for m in range(0,L+1)]
# mode_list = [f"{L},{m}" for m in range(-L,L+1)]

mode_list = [f"{L},{L}" for L in range(2,9)]

wave_diff_dict={key:[] for key in mode_list}
wave_diff_dict['total_denom'] = []
wave_diff_dict['total_nume'] = []
for key in diff_dict:
    print(key)
    sum_denom = 0.0
    sum_nume = 0.0
    for mode_key in mode_list:
        diff_lev = diff_dict[key]
        # print(f"{key}  {mode_key}: ({diff_lev[mode_key][0]:.2e}, {diff_lev[mode_key][1]:.2e}) {diff_lev[mode_key][0]/diff_lev[mode_key][1]:.2e}")
        if normalize_each_mode:
            wave_diff_dict[mode_key].append(diff_lev[mode_key][0]/diff_lev[mode_key][1])
        else:
            wave_diff_dict[mode_key].append(diff_lev[mode_key][0])
        sum_nume += diff_lev[mode_key][0]
        sum_denom += diff_lev[mode_key][1]
    wave_diff_dict['total_nume'].append(sum_nume)
    wave_diff_dict['total_denom'].append(sum_denom)


wave_diff_dict['total'] = [wave_diff_dict['total_nume'][i]/wave_diff_dict['total_denom'][i] for i in range(len(wave_diff_dict['total_nume']))]
wave_diff_dict.pop('total_nume')
wave_diff_dict.pop('total_denom')


y_vals_dict = wave_diff_dict
wave_diff_dict

In [ ]:
diff_int_dict = {
    "6_set1_L6s1@6_set1_L6s2": 1.2858028858326789,
    "6_set1_L6s1@6_set1_L6s3": 2.952014392741012,
    "6_set1_L6s1@6_set1_L6s4": 3.8928176255488096,
    "6_set1_L6s1@6_set1_L6s5": 3.9671436667026736,
    "6_set1_L6s1@6_set1_L6s6": 3.914069956217513,
    "6_set1_L6s2@6_set1_L6s3": 4.233594523893546,
    "6_set1_L6s2@6_set1_L6s4": 5.176475788978051,
    "6_set1_L6s2@6_set1_L6s5": 5.250682223344887,
    "6_set1_L6s2@6_set1_L6s6": 5.197502238165702,
    "6_set1_L6s3@6_set1_L6s4": 0.9429371466287805,
    "6_set1_L6s3@6_set1_L6s5": 1.0171364351233168,
    "6_set1_L6s3@6_set1_L6s6": 0.9639556750278575,
    "6_set1_L6s4@6_set1_L6s5": 0.07440602543930704,
    "6_set1_L6s4@6_set1_L6s6": 0.02289045491236974,
    "6_set1_L6s5@6_set1_L6s6": 0.05318158328457974,
}

# diff_phi = {
diff_int_dict = {
    "6_set1_L6s1@6_set1_L6s2": 0.18291673620686666,
    "6_set1_L6s1@6_set1_L6s3": 0.42241567036236083,
    "6_set1_L6s1@6_set1_L6s4": 0.5564967966590947,
    "6_set1_L6s1@6_set1_L6s5": 0.5671463644311162,
    "6_set1_L6s1@6_set1_L6s6": 0.5595623478412944,
    "6_set1_L6s2@6_set1_L6s3": 0.6048938796815856,
    "6_set1_L6s2@6_set1_L6s4": 0.7392498744431973,
    "6_set1_L6s2@6_set1_L6s5": 0.7498889427836771,
    "6_set1_L6s2@6_set1_L6s6": 0.7422949585840114,
    "6_set1_L6s3@6_set1_L6s4": 0.1343604960424211,
    "6_set1_L6s3@6_set1_L6s5": 0.14499928674451112,
    "6_set1_L6s3@6_set1_L6s6": 0.13740503738952842,
    "6_set1_L6s4@6_set1_L6s5": 0.010660803068585521,
    "6_set1_L6s4@6_set1_L6s6": 0.0032864203962184035,
    "6_set1_L6s5@6_set1_L6s6": 0.007594258469580951,
}

mistmatch_dict = {
    "34_main_1_257@34_main_2_257": 2.431234098674027e-06,
    "34_main_1_257@34_main_3_257": 5.292452936890286e-06,
    "34_main_1_257@34_main_4_257": 3.3682335436664007e-06,
    "34_main_1_257@34_main_5_257": 3.6149838367932497e-06,
    "34_main_1_257@34_main_6_257": 3.2930461894781516e-06,
    "34_main_2_257@34_main_3_257": 1.2224933554647637e-05,
    "34_main_2_257@34_main_4_257": 8.953595741992174e-06,
    "34_main_2_257@34_main_5_257": 9.415137037535157e-06,
    "34_main_2_257@34_main_6_257": 8.834944222807571e-06,
    "34_main_3_257@34_main_4_257": 2.545680500823587e-07,
    "34_main_3_257@34_main_5_257": 1.8376877710760494e-07,
    "34_main_3_257@34_main_6_257": 2.7530129680480055e-07,
    "34_main_4_257@34_main_5_257": 6.231294758007266e-09,
    "34_main_4_257@34_main_6_257": 6.3965423848261e-10,
    "34_main_5_257@34_main_6_257": 9.521390013079992e-09,
}

mistmatch_dict_FR_all = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 9.691173399793742e-09,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 5.582821623776524e-08,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 9.634173231920067e-08,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 1.0012000387332597e-07,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 9.747267234242399e-08,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 1.1176474885869486e-07,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.668258294750136e-07,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 1.7177939577073267e-07,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 1.6830341576626154e-07,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 5.496792371651053e-09,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 6.426830715542033e-09,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 5.769846655140213e-09,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 3.6726266095554097e-11,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 4.509717888492286e-12,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.810591947900653e-11,
}
mistmatch_dict_FR_3 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 1.176258588817942e-08,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 6.165531185012641e-08,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 1.0651261262886684e-07,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 1.1124053411780667e-07,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 1.0928744390796267e-07,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 1.2474555910530625e-07,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.8620953058299842e-07,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 1.9221979319575064e-07,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 1.892722059434233e-07,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 6.330097059371655e-09,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 7.957686068520404e-09,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 8.201665623700853e-09,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 3.7222334803438665e-10,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 1.1900946097029719e-09,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 4.1755309956887246e-10,
}
mistmatch_dict_FR_4 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 4.0876736528390056e-08,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 2.4568427856263186e-07,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 4.228173276804809e-07,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 4.395691155964665e-07,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 4.28147262544654e-07,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 4.843832173288682e-07,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 7.237696025641046e-07,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 7.455332191591245e-07,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 7.30544109437928e-07,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 2.4035054658703722e-08,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 2.8255726158445044e-08,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 2.5508232018686196e-08,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 2.3046966213704724e-10,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 2.0107304992161404e-10,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.32760976243525e-10,
}
mistmatch_dict_FR_5 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 5.986315008341556e-06,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 8.746194906405613e-06,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 1.5468248992122915e-05,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 2.6358887100022743e-05,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 3.559502256598818e-05,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 3.4390890083687526e-06,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 1.1192658283801559e-05,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 2.0220105316254554e-05,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 2.8429052986874537e-05,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 4.496382861258997e-06,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 1.3253619549021454e-05,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 2.236338906935221e-05,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 4.4018005215639296e-06,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 1.2820626652204688e-05,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 3.7251104296524853e-06,
}

mistmatch_dict_FR_6 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 1.931007373824053e-06,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 3.1815504687829584e-06,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 3.5735224668999955e-06,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 5.14564266914458e-06,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 9.036105849235031e-06,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 2.1736240027795306e-06,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 3.5522855501308405e-06,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 4.971980059687422e-06,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 8.599596845437626e-06,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 6.870842012686385e-07,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 2.6549243914712077e-06,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 6.417413739876474e-06,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 1.1463076566244746e-06,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 4.256999589754828e-06,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 1.3612209394894923e-06,
}
mistmatch_dict_FR_7 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 0.01857643112835293,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 0.01577560284329549,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 0.018978635596919508,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 0.026428740367623316,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 0.04450042148671994,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 0.009998960263060793,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 0.017971297245776344,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 0.02358519522315965,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 0.04067588101847044,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 0.005946561824416723,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 0.015140001799637403,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 0.03348889914522287,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 0.007571306120762246,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 0.023108932718440913,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 0.007328929906606428,
}
mistmatch_dict_FR_8 = {
    "6_set1_L6s1_0257@6_set1_L6s2_0257": 0.0008436574718200077,
    "6_set1_L6s1_0257@6_set1_L6s3_0257": 0.0011948021047756044,
    "6_set1_L6s1_0257@6_set1_L6s4_0257": 0.0010416524780482038,
    "6_set1_L6s1_0257@6_set1_L6s5_0257": 0.0012101079339250042,
    "6_set1_L6s1_0257@6_set1_L6s6_0257": 0.0020797418165750814,
    "6_set1_L6s2_0257@6_set1_L6s3_0257": 0.0005219328987594512,
    "6_set1_L6s2_0257@6_set1_L6s4_0257": 0.0007417865285151334,
    "6_set1_L6s2_0257@6_set1_L6s5_0257": 0.0008851361054985489,
    "6_set1_L6s2_0257@6_set1_L6s6_0257": 0.0016800558898570797,
    "6_set1_L6s3_0257@6_set1_L6s4_0257": 0.00021021564410308817,
    "6_set1_L6s3_0257@6_set1_L6s5_0257": 0.0006245669646834853,
    "6_set1_L6s3_0257@6_set1_L6s6_0257": 0.0014236982870111218,
    "6_set1_L6s4_0257@6_set1_L6s5_0257": 0.00031973852799309984,
    "6_set1_L6s4_0257@6_set1_L6s6_0257": 0.0010042733170450983,
    "6_set1_L6s5_0257@6_set1_L6s6_0257": 0.000351805521060114,
}


def find_levs_from_key(key):
    parts = key.split("@")
    lev1 = int(parts[0].split("_")[-2][-1])
    lev2 = int(parts[1].split("_")[-2][-1])
    return lev1, lev2


def filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict):
    x_out = []
    y_out = []
    for diff_int, mistmatch in zip(diff_int_dict.keys(), mistmatch_dict.keys()):
        # print(diff_int,mistmatch)
        lev1, lev2 = find_levs_from_key(mistmatch)
        # print(lev1,lev2)
        if lev1 + 1 == lev2:
            x_out.append(diff_int_dict[diff_int])
            y_out.append(mistmatch_dict[mistmatch])
    # print(x_out,y_out)
    return x_out, y_out


def xticks_vals(diff_int_dict, mistmatch_dict):
    tick_vals = []
    tick_labels = []
    for diff_int, mistmatch in zip(diff_int_dict.keys(), mistmatch_dict.keys()):
        # print(diff_int,mistmatch)
        lev1, lev2 = find_levs_from_key(mistmatch)
        # print(lev1,lev2)
        if lev1 + 1 == lev2:
            tick_vals.append(diff_int_dict[diff_int])
            tick_labels.append(f"{lev1}@{lev2}")
    # print(x_out,y_out)
    return tick_vals, tick_labels


with plt.style.context("ggplot"):
    plt.rcParams["figure.figsize"] = (6, 4.5)
    plt.rcParams["figure.autolayout"] = True

    # Unpack data for plotting and labelling
    x_vals, y_vals = filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_all)

    plt.scatter(x_vals, y_vals, marker="o", label="2/all")

    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_3), marker="o", label="3")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_4), marker="o", label="4")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_5), marker="o", label="5")
    plt.scatter(*filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_6), marker="o", label="6")

    # plt.scatter(x, mistmatch_dict_FR_all.values(), marker="o", label="all")
    # plt.scatter(x, mistmatch_dict_FR_3.values(), marker="o", label="3")
    # plt.scatter(x, mistmatch_dict_FR_4.values(), marker="o", label="4")
    # plt.scatter(x, mistmatch_dict_FR_5.values(), marker="o", label="5")
    # plt.scatter(x, mistmatch_dict_FR_6.values(), marker="o", label="6")
    # # plt.scatter(x, mistmatch_dict_FR_7.values(), marker="o", label="7")
    # # plt.scatter(x, mistmatch_dict_FR_8.values(), marker="o", label="8")

    plt.legend(loc="lower right")
    plt.xscale("log")
    plt.yscale("log")
    plt.ylabel("Finite Radius Mismatch")

    # --- BOTTOM AXIS (Now shows Mismatch Values) ---
    # We explicitly set the ticks to be the mismatch values (y_vals)
    # plt.xticks(x_vals, [f"{val:.2e}" for val in y_vals], rotation=90, ha="right")
    plt.xlabel("Phase Drift")

    # --- TOP AXIS (Now shows "5@6" Labels) ---
    ax = plt.gca()
    ax2 = ax.twiny()

    ax2.set_xscale("log")
    ax2.set_xlim(ax.get_xlim())

    # We use your helper function to get the text labels ("5@6", etc)
    top_ticks, top_labels = xticks_vals(diff_int_dict, mistmatch_dict_FR_all)

    ax2.set_xticks(top_ticks)
    ax2.set_xticklabels(top_labels, rotation=90)

    # save_name = Path("/groups/sxs/hchaudha/scripts/code_for_paper_plots/mismatch")/"mismatch_vs_center_diff_set1.pdf"
    # plt.savefig(save_name, dpi=300)
    plt.grid(False)
    plt.show()


#### Usual mismatch different levs same radii

In [ ]:
def filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict):
    x_out = []
    y_out = []
    for diff_int, mistmatch in zip(diff_int_dict.keys(), mistmatch_dict.keys()):
        # print(diff_int,mistmatch)
        lev1, lev2 = find_levs_from_key(mistmatch)
        # print(lev1,lev2)
        if lev2 == 6:
            x_out.append(diff_int_dict[diff_int])
            y_out.append(mistmatch_dict[mistmatch])
    # print(x_out,y_out)
    return x_out, y_out

def xticks_vals(diff_int_dict, mistmatch_dict):
    tick_vals = []
    tick_labels = []
    for diff_int, mistmatch in zip(diff_int_dict.keys(), mistmatch_dict.keys()):
        # print(diff_int,mistmatch)
        lev1, lev2 = find_levs_from_key(mistmatch)
        # print(lev1,lev2)
        # if lev1 + 1 == lev2:
        if lev2 == 6:
            tick_vals.append(diff_int_dict[diff_int])
            tick_labels.append(f"{lev1}@{lev2}")
    # print(x_out,y_out)
    return tick_vals, tick_labels

with plt.style.context("ggplot"):
    plt.rcParams["figure.figsize"] = (8, 6)
    # plt.rcParams["figure.figsize"] = (6, 4.5)
    plt.rcParams["figure.autolayout"] = True

    # Unpack data for plotting and labelling
    x_vals, y_vals = filtered_and_to_lev_dict(diff_int_dict, mistmatch_dict_FR_all)
    # plt.plot(sorted(x_vals), sorted(y_vals), label='all')
    # plt.plot(x_vals, y_vals, label='all')
    # y_vals_dict = {}
    for label, y_vals in y_vals_dict.items():
        if label == 'total':
            continue
            plt.scatter(x_vals, y_vals, marker="*", label=str(L))
        else:
            plt.scatter(x_vals, y_vals, marker="o", label=label)


    plt.legend()
    # plt.legend(loc="lower right")
    plt.xscale("log")
    plt.yscale("log")
    plt.ylabel("Finite Radius Mismatch")

    # --- BOTTOM AXIS (Now shows Mismatch Values) ---
    # We explicitly set the ticks to be the mismatch values (y_vals)
    # plt.xticks(x_vals, [f"{val:.2e}" for val in y_vals], rotation=90, ha="right")
    plt.xlabel("Phase Drift")

    # --- TOP AXIS (Now shows "5@6" Labels) ---
    ax = plt.gca()
    ax2 = ax.twiny()

    ax2.set_xscale("log")
    ax2.set_xlim(ax.get_xlim())

    # We use your helper function to get the text labels ("5@6", etc)
    # top_ticks, top_labels = xticks_vals(diff_int_dict, mistmatch_dict_FR_all)

    ax2.set_xticks(top_ticks)
    ax2.set_xticklabels(top_labels, rotation=90)

    # save_name = Path("/groups/sxs/hchaudha/scripts/code_for_paper_plots/mismatch")/"mismatch_vs_center_diff_set1.pdf"
    # plt.savefig(save_name, dpi=300)
    plt.title(f"Set1 runs {normalize_each_mode=}")
    plt.grid(False)
    plt.show()

6_set1_L6s1_R0257  6,6: (8.78e-12, 9.16e-07) 9.59e-06
6_set1_L6s2_R0257  6,6: (9.31e-12, 9.16e-07) 1.02e-05
6_set1_L6s3_R0257  6,6: (5.89e-12, 9.16e-07) 6.43e-06
6_set1_L6s4_R0257  6,6: (3.84e-12, 9.16e-07) 4.19e-06
6_set1_L6s5_R0257  6,6: (1.25e-12, 9.16e-07) 1.37e-06

#### Different radii plots

In [ ]:
with plt.style.context("ggplot"):
    plt.rcParams["figure.figsize"] = (8, 6)
    # plt.rcParams["figure.figsize"] = (6, 4.5)
    plt.rcParams["figure.autolayout"] = True

    radius_list_num = [int(r) for r in radius_list]
    for label, y_vals in list(y_vals_dict.items()):
        if label == 'total':
            continue
        plt.plot(radius_list_num, y_vals, marker="o", label=label)
        # plt.scatter(radius_list, y_vals, marker="o", label=label)
        # plt.scatter(list(diff_dict.keys())[::-1], y_vals[::-1], marker="o", label=label)


    plt.legend()
    # plt.legend(loc="lower right")
    # plt.xscale("log")
    plt.yscale("log")
    plt.ylabel("Finite Radius Mismatch")
    plt.xticks(radius_list_num, list(diff_dict.keys()), rotation=90)

    # save_name = Path("/groups/sxs/hchaudha/scripts/code_for_paper_plots/mismatch")/"mismatch_vs_center_diff_set1.pdf"
    # plt.savefig(save_name, dpi=300)
    plt.title(f"{base_key_root} runs {normalize_each_mode=}")
    plt.grid(False)
    plt.show()

In [ ]:
list(y_vals_dict.items())[::-1]

In [ ]:
mode_key = '6,6'
for key in diff_dict:
    diff_lev = diff_dict[key]
    print(f"{key}  {mode_key}: ({diff_lev[mode_key][0]:.2e}, {diff_lev[mode_key][1]:.2e}) {diff_lev[mode_key][0]/diff_lev[mode_key][1]:.2e}")

# plots for h5 files

In [ ]:
runs_to_plot = {}
# runs_to_plot["high_accuracy_L0"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev0_A?/Run/"
# runs_to_plot["high_accuracy_L1"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev1_A?/Run/"
# runs_to_plot["high_accuracy_L2"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev2_A?/Run/"
# runs_to_plot["high_accuracy_L3"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L45"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L5"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L55"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev55_A?/Run/"
# runs_to_plot["high_accuracy_L6"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev6_A?/Run/"

# runs_to_plot["high_accuracy_L5_three_tier"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L5_three_tier_constra"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L5_three_tier_constra200"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_constra_200/Lev5_A?/Run/"
# runs_to_plot["L3_step_bound_gauss_error"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_A?/Run/"
# runs_to_plot["L3_step_bound_gauss_error_rd"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_Ringdown/Lev3_A?/Run/"

# runs_to_plot["Lev5_big_gaussian_ah_tol10"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol10/Lev5_A?/Run/"
# runs_to_plot["Lev5_big_gaussian_ah_tol100"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol100/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_cd_01_uamr"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_lapse"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_full/Lev5_A?/Run/"
# runs_to_plot["Lev5_bg_ah100_lapse_uamr"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/Lev5_A?/Run/"

# runs_to_plot["high_accuracy_L0_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev0_A?/Run/"
# runs_to_plot["high_accuracy_L1_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev1_A?/Run/"
# runs_to_plot["high_accuracy_L2_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev2_A?/Run/"
# runs_to_plot["high_accuracy_L3_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev3_A?/Run/"
# runs_to_plot["high_accuracy_L4_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev4_A?/Run/"
# runs_to_plot["high_accuracy_L45_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev45_A?/Run/"
# runs_to_plot["high_accuracy_L5_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev5_A?/Run/"
# runs_to_plot["high_accuracy_L55_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev55_A?/Run/"
# runs_to_plot["high_accuracy_L6_main"] =  "/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/Lev6_A?/Run/"

# runs_to_plot["ode_impro_Lev0"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev0_A?/Run/'
# runs_to_plot["ode_impro_Lev1"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev1_A?/Run/'
# runs_to_plot["ode_impro_Lev2"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev2_A?/Run/'
# runs_to_plot["ode_impro_Lev3"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev3_A?/Run/'
# runs_to_plot["ode_impro_Lev4"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev4_A?/Run/'
# runs_to_plot["ode_impro_Lev5"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev5_A?/Run/'
# runs_to_plot["main_Lev0"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev0_A?/Run/'
# runs_to_plot["main_Lev2"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev2_A?/Run/'
# runs_to_plot["main_Lev1"] = '/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev1_A?/Run/'

# runs_to_plot["high_accuracy_L5"] =  "/groups/sxs/hchaudha/spec_runs/truncation_error_diagnostics/high_accuracy_L35_Lev5/"
# runs_to_plot["high_accuracy_L5_main"] =  "/groups/sxs/hchaudha/spec_runs/truncation_error_diagnostics/high_accuracy_L35_master_Lev5/"

# runs_to_plot["6_set1_L3s0"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev0/'
# runs_to_plot["6_set1_L3s1"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev1/'
# runs_to_plot["6_set1_L3s2"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev2/'
# runs_to_plot["6_set1_L3s3"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev3/'

runs_to_plot["6_set1_L6s4"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev4/'
runs_to_plot["6_set1_L6s5"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev5/'
runs_to_plot["6_set1_L6s6"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev6/'

# runs_to_plot["6_set2_L6s4"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/h5_files_Lev4/'
# runs_to_plot["6_set2_L6s5"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/h5_files_Lev5/'
# runs_to_plot["6_set2_L6s6"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set2_L6/h5_files_Lev6/'

# runs_to_plot["6_set3_L6s4"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/h5_files_Lev4/'
# runs_to_plot["6_set3_L6s5"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/h5_files_Lev5/'
# runs_to_plot["6_set3_L6s6"] = '/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/h5_files_Lev6/'

domain = "SphereC5"

# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_ConvergenceFactor.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_HighestThirdConvergenceFactor.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_NumberOfFilteredModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_NumberOfModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_NumberOfNonFilteredNonZeroModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_NumberOfPiledUpModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_PowerInFilteredModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_PowerInHighestUnfilteredModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_PredictedTruncationErrorForOneLessMode.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_RawConvergenceFactor.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_SpectrumIsDegenerate.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf0I1_TruncationError.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_ConvergenceFactor.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_HighestThirdConvergenceFactor.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_NumberOfFilteredModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_NumberOfModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_NumberOfNonFilteredNonZeroModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_NumberOfPiledUpModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_PowerInFilteredModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_PowerInHighestUnfilteredModes.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_PredictedTruncationErrorForOneLessMode.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_RawConvergenceFactor.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_SpectrumIsDegenerate.dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Bf1S2_TruncationError.dat"

psi_or_kappa = "psi"
psi_or_kappa = "kappa"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf0I1(10 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf0I1(11 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf0I1(12 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf0I1(13 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(20 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(21 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(22 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(23 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(24 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(25 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(26 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(27 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(28 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(29 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(30 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(31 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(32 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(33 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(34 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(35 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(36 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(37 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(38 modes).dat"
# data_file_path = f"extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir/Bf1S2(41 modes).dat"

# data_file_path = "extracted-FilterDiagnostics/BoundaryFilters.dir/ExpChebCoef.dir/SliceLFF.SphereA0.dat"
# data_file_path = "extracted-ProjectedCon/Subdomains.dir/SphereA0.dir/NormOf2Con.dat"
data_file_path = "extracted-ProjectedCon/Subdomains.dir/SphereC5.dir/NormOf2Con.dat"
# data_file_path = "extracted-PowerDiagnostics/SphereA0.dir/Bf0I1_NumberOfModes.dat"
# data_file_path = "extracted-PowerDiagnostics/FilledCylinderEA0.dir/Bf0I1_TruncationError.dat"
# data_file_path = "extracted-RhsExpense/CostPerProc.dir/Proc00.dat"
# data_file_path = "extracted-RhsExpense/CostPerSubdomain.dir/SphereA0.dat"
# data_file_path = "extracted-RhsExpense/CostPerSubdomain.dir/SphereB2.dat"
data_file_path = "extracted-OrbitDiagnostics/OrbitalPhase.dat"
# data_file_path = "extracted-AdjustGridExtents/SphereC5.dir/Bf0I1.dat"
# data_file_path = "extracted-AdjustGridExtents/SphereA0.dir/Bf1S2.dat"
# data_file_path = "extracted-AdjustGridExtents/SphereA0.dir/Extents.dat"
# data_file_path = "extracted-AdjustGridExtents/SphereA0.dir/Size.dat"
# data_file_path = "extracted-AdjustGridExtents/SphereC5.dir/Size.dat"
# data_file_path = "extracted-ControlNthDeriv/ExpansionFactor.dir/a.dat"
# data_file_path = "extracted-ControlNthDeriv/Trans.dir/Tx.dat"
# data_file_path = "extracted-FilterDiagnostics/SubdomainFilters.dir/ExpChebCoef.dir/SphereB0.dat"

column_names, runs_data_dict = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names)

In [ ]:
moving_avg_len=0
save_path = None
diff_base = None
x_axis = 't(M)'

diff_base = '6_set1_L6s6'
# diff_base = '6_set1_L6s5'
# diff_base = '6_set1_L3s3'
# diff_base = 'high_accuracy_L5'
# diff_base = 'ode_impro_Lev3'
# add_max_and_min_val(runs_data_dict)
# y_axis = 'max_val'
# y_axis = 'min_val'

# y_axis = 'coef0'
# y_axis = 'coef1'
# y_axis = 'MaxTruncationError'
# y_axis = 'ShiftedTruncErrorMax'
# y_axis = 'TruncationErrorExcess'
# y_axis = 'MaxNextTruncationError'
# y_axis = 'MinNumberOfPiledUpModes'
# y_axis = 'GridDiagPowerkappa'
# y_axis = 'GridDiagPowerpsi'
# y_axis = 'CostPerPoint'
# y_axis = 'Cost'
# y_axis = 'NumberOfPoints'
# y_axis = 'Q'
# y_axis = 'lambda'
# y_axis = 'Bf0I1'
y_axis = 'phi'
# y_axis = 'Extent[0]'
# y_axis = 'Size'

minT = 0
minT = 1100
# minT = 2700
# minT = 9100

maxT = 40000
# maxT = 2710
maxT = 4000
# maxT = 9300
# maxT = 9375
# moving_avg_len = 50
# moving_avg_len = 10

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.plot(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label,marker='x')
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label) 
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label,s=10,marker="x")
# save_path = "/groups/sxs/hchaudha/rough/high_acc_plots/"
# save_path = "/groups/sxs/hchaudha/rough/plots/"
# save_path = "/home/hchaudha/notes/spec_accuracy/L5_comparisons/power_diagon/"
legend_dict = {}
for key in runs_data_dict.keys():
  legend_dict[key] = None

# legend_dict = { '3_DH_q1_ns_d18_L3': "Lev3",
#                 '3_DH_q1_ns_d18_L6': "Lev6",
#                 'all_10': "Lev3_all_tols_10",
#                 'all_100': "Lev3_all_tols_100",
#                 'near_bhs_10': "Lev3_sphere_AB_tols_10",
#                 'near_bhs_100': "Lev3_sphere_AB_tols_100"}
# legend_dict = {
#  '3_DH_q1_ns_d18_L3':"Lev3_ode_tol_1e-8",
#  '3_DH_q1_ns_d18_L3_tol9':"Lev3_ode_tol_1e-9",
#  '3_DH_q1_ns_d18_L3_tol10':"Lev3_ode_tol_1e-10",
#  '3_DH_q1_ns_d18_L3_tol11':"Lev3_ode_tol_1e-11",
#  '3_DH_q1_ns_d18_L3_all_100_tol10':"Lev3_AMR_tol_100_ode_tol_1e-11",
#  }
title = data_file_path
# for key in runs_data_dict:
#   title = title + "_" + 
with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, legend_dict=legend_dict, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun, diff_base=diff_base, title=title)
  plt.show()

# Plots for one

In [ ]:
main_path = Path("/groups/sxs/hchaudha/spec_runs/")
run_path = main_path/Path("3_DH_q1_ns_d18_L3/Ev/Lev3_A?/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_9/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_10/Lev3_AD/Run/")
# run_path = main_path/Path("3_DH_q1_ns_d18_L3_higher_acc/all_100_t2690_obs_grid_tol_11/Lev3_AD/Run/")

data_files = {}

# data_files["GrAdjustSubChunksToDampingTimes"] = {"path":run_path/"GrAdjustSubChunksToDampingTimes.dat", "prefix":None}
# data_files["MemoryInfo"] = {"path":run_path/"MemoryInfo.dat", "prefix":None}
# data_files["MinimumGridSpacing"] = {"path":run_path/"MinimumGridSpacing.dat", "prefix":None}
# data_files["DiagInclinationAngle"] = {"path":run_path/"DiagInclinationAngle.dat", "prefix":None}
# data_files["ApparentHorizons/Trajectory_AhA"] = {"path":run_path/"ApparentHorizons/Trajectory_AhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/MinCharSpeedAhA"] = {"path":run_path/"ApparentHorizons/MinCharSpeedAhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/Trajectory_AhB"] = {"path":run_path/"ApparentHorizons/Trajectory_AhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/SmoothCoordSepHorizon"] = {"path":run_path/"ApparentHorizons/SmoothCoordSepHorizon.dat", "prefix":None}
# data_files["ApparentHorizons/MinCharSpeedAhB"] = {"path":run_path/"ApparentHorizons/MinCharSpeedAhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/RescaledRadAhB"] = {"path":run_path/"ApparentHorizons/RescaledRadAhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/AhACoefs"] = {"path":run_path/"ApparentHorizons/AhACoefs.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/AhB"] = {"path":run_path/"ApparentHorizons/AhB.dat", "prefix":"AhB"}
# data_files["ApparentHorizons/HorizonSepMeasures"] = {"path":run_path/"ApparentHorizons/HorizonSepMeasures.dat", "prefix":None}
# data_files["ApparentHorizons/AhA"] = {"path":run_path/"ApparentHorizons/AhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/RescaledRadAhA"] = {"path":run_path/"ApparentHorizons/RescaledRadAhA.dat", "prefix":"AhA"}
# data_files["ApparentHorizons/AhBCoefs"] = {"path":run_path/"ApparentHorizons/AhBCoefs.dat", "prefix":"AhB"}
# data_files["TimeInfo"] = {"path":run_path/"TimeInfo.dat", "prefix":None}
# data_files["GrAdjustMaxTstepToDampingTimes"] = {"path":run_path/"GrAdjustMaxTstepToDampingTimes.dat", "prefix":None}
# data_files["FailedTStepperDiag"] = {"path":run_path/"FailedTStepperDiag.dat", "prefix":None}
# data_files["DiagAhSpeedA"] = {"path":run_path/"DiagAhSpeedA.dat", "prefix":"AhA"}
# data_files["DiagAhSpeedB"] = {"path":run_path/"DiagAhSpeedB.dat", "prefix":"AhB"}
# data_files["CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereA0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereA0.dat", "prefix":"Max_A0"}
# data_files["CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereA0.dat", "prefix":"Min_A0"}
# data_files["CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereB0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Min_SliceLFF.SphereB0.dat", "prefix":"Max_B0"}
# data_files["CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereB0"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Max_SliceLFF.SphereB0.dat", "prefix":"Min_B0"}
# data_files["CharSpeedNorms/CharSpeeds_Max_SliceUFF.SphereC29"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Max_SliceUFF.SphereC29.dat", "prefix":"Max_C29"}
# data_files["CharSpeedNorms/CharSpeeds_Min_SliceUFF.SphereC29"] = {"path":run_path/"CharSpeedNorms/CharSpeeds_Min_SliceUFF.SphereC29.dat", "prefix":"Min_C29"}
# data_files["ConstraintNorms/NormalizedGhCe_Norms"] = {"path":run_path/"ConstraintNorms/NormalizedGhCe_Norms.dat", "prefix":None}
# data_files["ConstraintNorms/GhCeExt_Norms"] = {"path":run_path/"ConstraintNorms/GhCeExt_Norms.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe_L2"] = {"path":run_path/"ConstraintNorms/GhCe_L2.dat", "prefix":None}
# data_files["ConstraintNorms/GhCeExt_L2"] = {"path":run_path/"ConstraintNorms/GhCeExt_L2.dat", "prefix":None}
# data_files["ConstraintNorms/GhCeExt"] = {"path":run_path/"ConstraintNorms/GhCeExt.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe"] = {"path":run_path/"ConstraintNorms/GhCe.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe_VolL2"] = {"path":run_path/"ConstraintNorms/GhCe_VolL2.dat", "prefix":None}
data_files["ConstraintNorms/GhCe_Norms"] = {"path":run_path/"ConstraintNorms/GhCe_Norms.dat", "prefix":None}
# data_files["ConstraintNorms/NormalizedGhCe_Linf"] = {"path":run_path/"ConstraintNorms/NormalizedGhCe_Linf.dat", "prefix":None}
# data_files["ConstraintNorms/GhCe_Linf"] = {"path":run_path/"ConstraintNorms/GhCe_Linf.dat", "prefix":None}
data_files["TStepperDiag"] = {"path":run_path/"TStepperDiag.dat", "prefix":None}
# data_files["DiagCutXCorrection"] = {"path":run_path/"DiagCutXCorrection.dat", "prefix":None}


# data = read_dat_file_across_AA(str(data_files['TimeInfo']['path']))
for key in data_files:
  data_files[key]["dataframe"] = read_dat_file_across_AA(str(data_files[key]['path']))
  cols = list(data_files[key]["dataframe"].columns)
  # Make new cols names s.t. the first cols is 't' and add prefix as required
  new_cols = []
  new_cols.append('t')
  if data_files[key]['prefix'] is None:
    [new_cols.append(name) for name in cols[1:]]
  else:
    [new_cols.append(name+"_"+data_files[key]['prefix']) for name in cols[1:]]
  data_files[key]["dataframe"].columns = new_cols

  # Set 't' to be a index and copy it into a column called 'time'
  # data_files[key]["dataframe"]["time"] = data_files[key]["dataframe"]["t"]
  # data_files[key]["dataframe"].set_index('t', inplace=True)

combined = None
for key in data_files:
  if combined is None:
    combined = data_files[key]["dataframe"]
    continue
  else:
    combined  = pd.merge(combined,data_files[key]["dataframe"],on='t',how='outer')


In [ ]:
combined = combined.sort_values(by='t')
for i in combined.columns:
  print(i)

In [ ]:
# plt.scatter(combined['t'],combined['SuggestedDampingTime'])

x_val = 't'
plot_list=[
  # ('Linf(GhCe) on SphereA0','semilogy',None,None,'x'),
  ('Linf(GhCe)','semilogy',None,None,'.'),
  ('dt','plot',1e-3,None,None),
]

for i in plot_list:
  y_val, plot_type, mul_factor, add_factor, marker = i

  label = y_val
  if mul_factor is not None:
    label = f"{label}*{mul_factor}"
  else:
    mul_factor = 1
  if add_factor is not None:
    label = f"{label}+{add_factor}"
  else:
    add_factor = 0


  match plot_type:
    case 'semilogy':
      plt.semilogy(combined[x_val],combined[y_val]*mul_factor+add_factor,marker=marker,label=label)
    case 'plot':
      plt.plot(combined[x_val],combined[y_val]*mul_factor+add_factor,marker=marker,label=label)

title = str(run_path).split('/')[-4]
save_name = str(run_path).split('/')[-4]
for i in str(run_path).split('/')[-3:-1]:
  title = title +"/" +i
  save_name = save_name+"&"+i


plt.title(title)
plt.xlabel(x_val)
plt.legend()
plt.tight_layout()
plt.savefig(f"/groups/sxs/hchaudha/rough/plots/{save_name}.png",dpi=500)
plt.show()

In [ ]:
import pandas as pd


def check_duplicate_rows(df, subset_column):
    """
    Function to find and print duplicate rows in a DataFrame based on a specified column.
    It checks if all duplicate rows are identical or if they differ in some columns.

    Parameters:
    df (pd.DataFrame): The DataFrame to check for duplicates.
    subset_column (str): The column to check for duplicate values.

    Returns:
    None
    """
    # Get rows with duplicate values in the subset column
    duplicate_mask = df.duplicated(subset=[subset_column], keep=False)
    duplicate_rows = df[duplicate_mask]
    
    # Sort the duplicate rows by the subset column to group them together
    duplicate_rows_sorted = duplicate_rows.sort_values(by=subset_column)
    
    # Iterate through groups of rows with the same value in the subset column
    for _, group in duplicate_rows_sorted.groupby(subset_column):
        if len(group) > 1:
            print(f"\nRows with '{subset_column}' value: {group[subset_column].iloc[0]}")
            
            # Check if all rows in the group are identical
            identical = all(group.iloc[0].equals(row) for _, row in group.iterrows())
            if identical:
                print("All rows are identical:")
                print(group)
            else:
                print("Rows differ in some columns:")
                print(group)
                
                # Optionally, show which columns differ
                differing_columns = group.columns[group.nunique() > 1].tolist()
                print(f"Columns with differences: {differing_columns}")



# Usage example
# combined = pd.read_csv('your_data.csv')  # Load your data
# check_duplicate_rows(combined, 't')

In [ ]:
check_duplicate_rows(combined,'t')

In [ ]:
import pandas as pd


def check_duplicate_rows(df, subset_column):
    # Get rows with duplicate values in the subset column
    duplicate_mask = df.duplicated(subset=[subset_column], keep=False)
    duplicate_rows = df[duplicate_mask]
    
    # Sort the duplicate rows by the subset column to group them together
    duplicate_rows_sorted = duplicate_rows.sort_values(by=subset_column)
    
    # Iterate through groups of rows with the same value in the subset column
    for _, group in duplicate_rows_sorted.groupby(subset_column):
        if len(group) > 1:
            print(f"\nRows with '{subset_column}' value: {group[subset_column].iloc[0]}")
            
            # Check if all rows in the group are identical
            if group.drop_duplicates().shape[0] == 1:
                print("All rows are identical:")
                print(group)
            else:
                print("Rows differ in some columns:")
                print(group)
                
                # Show which columns differ
                differing_columns = group.columns[group.nunique() > 1].tolist()
                print(f"Columns with differences: {differing_columns}")
                
                # Show the differences
                for col in differing_columns:
                    if col != subset_column:
                        print(f"\nDifferences in column '{col}':")
                        print(group[['t', col]])

# Usage example
# combined = pd.read_csv('your_data.csv')  # Load your data
check_duplicate_rows(combined, 't')

In [ ]:
combined = combined.sort_values(by='t')

In [ ]:
combined['MinCharSpeedAhA[7]_AhA']

In [ ]:
col_names = []
for i in data:
  print(i.columns.is_unique)
  col_names = col_names+list(i.columns)

In [ ]:
len(col_names), len(set(col_names))

In [ ]:
pd.concat(data,axis=1)

# Linf plots

In [ ]:
runs_to_plot = {}
runs_to_plot["t6115_tol8_linf"] =  "/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_high_tol/t6115_tol8_linf/Lev3_A?/Run/"

data_file_path="ConstraintNorms/Linf.dat"


column_names_linf, runs_data_dict_linf = load_data_from_levs(runs_to_plot,data_file_path)
print(column_names_linf)
print(runs_data_dict_linf.keys())

In [ ]:
data = runs_data_dict_linf['t6115_tol8_linf']

In [ ]:
domain_list = []
constraint_list = []
component_list = []

# domain_list.append("SphereB0")
domain_list.append("erMA0")

# constraint_list.append('1Con')
constraint_list.append('2Con')
# constraint_list.append('3Con')

component_list.append('t')

temp_list = copy.copy(column_names_linf)
col_domains = []
for col in temp_list:
  for domain in domain_list:
    if domain in col:
      col_domains.append(col)

temp_list = col_domains
col_domains = []
for col in temp_list:
  for constraint in constraint_list:
    if constraint in col:
      col_domains.append(col)

if len(component_list) > 0:
  temp_list = col_domains
  col_domains = []
  for col in temp_list:
    for component in component_list:
      if component in col:
        col_domains.append(col)

col_domains

In [ ]:
moving_avg_len = None

x = 'time'
# moving_avg_len = 50*3


for col in col_domains:
  if moving_avg_len is not None:
    plt.semilogy(np.array(data[x])[moving_avg_len-1:],moving_average_valid(data[col],moving_avg_len),label=col)
  else:
    plt.semilogy(data[x],data[col],label=col)
  # plt.plot(data[x],data[col],label=col)
plt.xlabel(x)
plt.legend()
plt.show()

In [ ]:
len(moving_average_valid(data[col],average_over))

# Plot horizons.h5

In [ ]:
base_path = Path("/groups/sxs/hchaudha/spec_runs")
runs_to_plot = {}

# runs_to_plot["high_accuracy_L0"] =  "high_accuracy_L35/Ev/Lev0_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L1"] =  "high_accuracy_L35/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L2"] =  "high_accuracy_L35/Ev/Lev2_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L3"] =  "high_accuracy_L35/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L4"] =  "high_accuracy_L35/Ev/Lev4_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L45"] =  "high_accuracy_L35/Ev/Lev45_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L5"] =  "high_accuracy_L35/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L55"] =  "high_accuracy_L35/Ev/Lev55_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L6"] =  "high_accuracy_L35/Ev/Lev6_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L3_master"] =  "high_accuracy_L35_master/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L4_master"] =  "high_accuracy_L35_master/Ev/Lev4_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L5_master"] =  "high_accuracy_L35_master/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L5_big_gauss"] =  "high_accuracy_L35_variations/Lev5_big_gaussian/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["Lev5_big_gaussian_constra"] =  "high_accuracy_L35_variations/Lev5_big_gaussian_constra/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["Lev5_big_gaussian_constra_200"] =  "high_accuracy_L35_variations/Lev5_big_gaussian_constra_200/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L5_a10"] =  "high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol10/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["high_accuracy_L5_a100"] =  "high_accuracy_L35_variations/Lev5_big_gaussian_ah_tol100/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["L3_step_bound_gauss_error"] =  "high_accuracy_L3_step_bound_gauss_error/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"

# runs_to_plot["eq_AMR_3_tier_const"] =  "high_accuracy_L3_contraints/eq_AMR_3_tier_const/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["eq_AMR_3_tier_const_gamma2"] =  "high_accuracy_L3_contraints/eq_AMR_3_tier_const_gamma2/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["three_tier_AMR_const_L3"] =  "high_accuracy_L3_contraints/three_tier_AMR_const/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["normal_constraints"]="high_accuracy_L3_contraints/eq_AMR_3_tier_const_variations/normal_constraints/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["normal_constraints_12_AB"]="high_accuracy_L3_contraints/eq_AMR_3_tier_const_variations/normal_constraints_12_AB/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["normal_constraints_const1"]="high_accuracy_L3_contraints/eq_AMR_3_tier_const_variations/normal_constraints_const1/Lev3_A?/Run/ApparentHorizons/Horizons.h5"

# runs_to_plot["ode_impro_Lev0"] = 'Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev0_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["ode_impro_Lev1"] = 'Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["ode_impro_Lev2"] = 'Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev2_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["ode_impro_Lev3"] = 'Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["ode_impro_Lev4"] = 'Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev4_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["ode_impro_Lev5"] = 'Lev01_test/new_ode_tol/high_accuracy_L35/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5'

# runs_to_plot["main_Lev0"] = 'Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev0_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["main_Lev2"] = 'Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev2_A?/Run/ApparentHorizons/Horizons.h5'
# runs_to_plot["main_Lev1"] = 'Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5'


runs_to_plot["6_set1_L3s0"] =  "6_segs/6_set1_L3/Ev/Lev0_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["6_set1_L3s1"] =  "6_segs/6_set1_L3/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["6_set1_L3s2"] =  "6_segs/6_set1_L3/Ev/Lev2_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["6_set1_L3s3"] =  "6_segs/6_set1_L3/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"

runs_to_plot["6_set1_L6s4"] =  "6_segs/6_set1_L6/Ev/Lev4_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["6_set1_L6s5"] =  "6_segs/6_set1_L6/Ev/Lev5_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["6_set1_L6s6"] =  "6_segs/6_set1_L6/Ev/Lev6_A?/Run/ApparentHorizons/Horizons.h5"

# runs_to_plot["all_100"] =  "3_DH_q1_ns_d18_L3_higher_acc/all_100/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
data_dict = load_horizon_data_from_levs(base_path, runs_to_plot)
data_dict = flatten_dict(data_dict)
data_dict[list(data_dict.keys())[0]].columns

In [ ]:
moving_avg_len = 0
save_path = None

x_axis = 't'
y_axis = 'ArealMass'
y_axis = 'ChristodoulouMass'
# y_axis = 'CoordCenterInertial_0'
# y_axis = 'CoordCenterInertial_1'
# y_axis = 'CoordCenterInertial_2'
# y_axis = 'DimensionfulInertialSpin_0'
# y_axis = 'DimensionfulInertialSpin_1'
# y_axis = 'DimensionfulInertialSpin_2'
# y_axis = 'DimensionfulInertialCoordSpin_0'
# y_axis = 'DimensionfulInertialCoordSpin_1'
# y_axis = 'DimensionfulInertialCoordSpin_2'
# y_axis = 'DimensionfulInertialSpinMag'
# y_axis = 'SpinFromShape_0'
# y_axis = 'SpinFromShape_1'
# y_axis = 'SpinFromShape_2'
# y_axis = 'SpinFromShape_3'
# y_axis = 'chiInertial_0'
# y_axis = 'chiInertial_1'
# y_axis = 'chiInertial_2'
# y_axis = 'chiMagInertial'



# moving_avg_len=25
minT = 0
minT = 1200
maxT = 25000
# maxT = 7000
maxT = 7500
maxT = 4000

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label)
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label)
# save_path = "/panfs/ds09/sxs/himanshu/scripts/report/not_tracked/temp2/"

filtered_dict = {}
allowed_horizons = ["AhA"]
for horizons in allowed_horizons:
  for runs_keys in data_dict.keys():
    if horizons in runs_keys:
      filtered_dict[runs_keys] = data_dict[runs_keys]
 
with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.figsize"] = (8,6)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(filtered_dict, x_axis, y_axis, minT, maxT, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun)

plt.show()

In [ ]:
bh = 'corrected_coord_spin2_AhB'
y_axis1 = 'chiInertial_0'
y_axis2 = 'CoordSpinChiInertial_0'

X = data_dict[bh][x_axis]
Y1 = data_dict[bh][y_axis1]
Y2 = data_dict[bh][y_axis2]
plt.plot(X,Y1,label=y_axis1)
plt.plot(X,Y2,label=y_axis2)
plt.xlabel(x_axis)
# plt.ylabel(y_axis1+" - "+y_axis2)
plt.legend()
# plt.title()
plt.show()

In [ ]:
filtered_dict.keys()

In [ ]:
x_axis = 't'
y_axis = 'ChristodoulouMass'
minT = 500
maxT = 800
run1 = filtered_dict['AccTest_q1ns_Lev5_AhA']
# run1 = filtered_dict['AccTest_q1ns_Lev6p_AhA']
run2 = filtered_dict['AccTest_q1ns_Lev6p_AhA']
interp_grid_pts = run1[x_axis].size

In [ ]:
interp_run1 = CubicSpline(run1[x_axis].to_numpy(),run1[y_axis].to_numpy())
interp_run2 = CubicSpline(run2[x_axis].to_numpy(),run2[y_axis].to_numpy())
interp_grid = np.arange(minT,maxT,(maxT-minT)/interp_grid_pts)

plt.plot(interp_grid, interp_run2(interp_grid) - interp_run1(interp_grid))
plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.legend()

In [ ]:
plt.plot(interp_grid, interp_run2(interp_grid) - interp_run1(interp_grid))

In [ ]:
def inertial_dist(run_name:str, data_dict):
    ct = data_dict[f"{run_name}_AhA"].t
    dx = data_dict[f"{run_name}_AhA"].CoordCenterInertial_0 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_0
    dy = data_dict[f"{run_name}_AhA"].CoordCenterInertial_1 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_1
    dz = data_dict[f"{run_name}_AhA"].CoordCenterInertial_2 - data_dict[f"{run_name}_AhB"].CoordCenterInertial_2

    dx = np.sqrt(dx**2 + dy**2 + dz**2)

    return ct,dx


In [ ]:
for run_name in runs_to_plot.keys():
    ct,dx = inertial_dist(run_name,data_dict)
    plt.plot(ct,dx,label=run_name)
    plt.legend()

In [ ]:
print(data_dict.keys())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data_dict['76_ngd_master_mr1_50_3000_AhA'].describe())

# Combine all paraview files into a single file

In [ ]:
base_folder = Path("/central/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_L3_higher_acc/near_bhs_100_obs/")

Lev = 3
file_pattern =f"Lev{Lev}_A[A-Z]/Run/GaugeVis.pvd"
file_patternGrid =f"Lev{Lev}_A[A-Z]/Run/GaugeVisGrid.pvd"
file_patternAll =f"Lev{Lev}_A[A-Z]/Run/GuageVisAll.pvd"

combine_pvd_files(base_folder,file_pattern)
combine_pvd_files(base_folder,file_patternGrid)
combine_pvd_files(base_folder,file_patternAll)

```Python
# Create GaugeVis
command = f"cd {base_folder} && mkdir ./GaugeVis"
status = subprocess.run(command, capture_output=True, shell=True, text=True)
if status.returncode == 0:
  print(f"Succesfully created GaugeVis in {base_folder}")
else:
  sys.exit(
      f"GaugeVis creation failed in {base_folder} with error: \n {status.stderr}")

# Create GaugeVis subfolder
vtu_folder_path = base_folder+"/GaugeVis/GaugeVis"
command = f"mkdir {vtu_folder_path}"
status = subprocess.run(command, capture_output=True, shell=True, text=True)
if status.returncode == 0:
  print(f"Succesfully created {vtu_folder_path}")
else:
  sys.exit(
      f"GaugeVis creation failed as {vtu_folder_path} with error: \n {status.stderr}")


# Copy vtu files
GaugeVisFolder=[]

for paths in path_collection:
  GaugeVisFolder.append(paths[:-4])

for paths in GaugeVisFolder:
  command = f"cp {paths}/*.vtu {vtu_folder_path}/"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully copied vtu files from {paths}")
  else:
    sys.exit(
        f"Copying vtu files from {paths} failed with error: \n {status.stderr}")

```

# Profiler results

# Make report (do not run randomly)

In [ ]:
with open("./report_new_gauge.json") as report_data:
  data = json.load(report_data)

os.mkdir(data['report_folder'])

subfolders = []
for folders in data['runs_to_track']:
  subfolders_path = data['report_folder'] + "/" + path_to_folder_name(folders) + "/"
  print(subfolders_path)
  os.mkdir(subfolders_path)
  subfolders.append(subfolders_path)

In [ ]:
runs_still_going_on = True
while runs_still_going_on:
  # time.sleep(data['report_generation_frequency'])

  for i,run_folder_path in enumerate(data['runs_to_track']):
    # if is_the_current_run_going_on(run_folder_path) or True:
    if True:
      plots_for_a_folder(data['things_to_plot'],subfolders[i],run_folder_path)
    print(run_folder_path)


  runs_still_going_on = False
  print("all done")


### Save all columns and data files paths

In [ ]:
# Write all the cols in the dat files for reference
lev_golb="/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/gauge_driver_kerr_target_50_50_0_16_16_01/Ev/Lev1_AA"
dat_files_glob=lev_golb+"/Run/**/**.dat"
path_pattern = dat_files_glob

path_collection = []
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name.split("/")[-1])


column_data_for_dat_files = {
  'columns_of_dat_files' : [
  ] 
}

for file_path in path_collection:
  file_name = file_path.split("/")[-1]
  columns_list =  list(read_dat_file(file_path).columns)
  column_data_for_dat_files['columns_of_dat_files'].append({
    'file_name': file_name,
    'file_path': file_path,
    'columns': columns_list
  })


with open('./column_data_for_dat_files.json', 'w') as outfile:
  json.dump(column_data_for_dat_files, outfile, indent=2)

In [ ]:
def JoinH5(h5_file_list, output_path, output_file_name):

  file_list_to_str = ""
  for h5file in h5_file_list:
    file_list_to_str += h5file + " "

  command = f"cd {output_path} && {spec_home}/Support/bin/JoinH5 -o {output_file_name} {file_list_to_str}"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran JoinH5 in {output_path}")
  else:
    sys.exit(
        f"JoinH5 failed in {output_path} with error: \n {status.stderr}")


def ExtractFromH5(h5_file, output_path):

  command = f"cd {output_path} && {spec_home}/Support/bin/ExtractFromH5 {h5_file}"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran ExtractFromH5 in {output_path}")
  else:
    sys.exit(
        f"ExtractFromH5 failed in {output_path} with error: \n {status.stderr}")



In [ ]:
output_base_path= "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/profiler_results"


base_folder = "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/49_ngd_weird_gauge_mr1"
file_pattern = base_folder+"/Ev/Lev1_A?/Run/Profiler.h5"

path_pattern = file_pattern
path_collection = []

# make a folder in the output directory
save_folder = output_base_path+"/"+base_folder.split("/")[-1]
os.mkdir(save_folder)


# Find all the files that match the required pattern of the file
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

JoinH5(path_collection,save_folder,"Profiler.h5")
ExtractFromH5("Profiler.h5",save_folder)

# Save path of all the summary files in extracted data

file_pattern = base_folder+"/Ev/Lev1_A?/Run/Profiler.h5"

path_pattern = file_pattern
path_collection = []

# Find all the files that match the required pattern of the file
for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

In [ ]:
# Find all the Summary files 
summary_file_pattern = save_folder+"/**/Summary.txt"
summary_file_collection = []

for file_path in glob.iglob(summary_file_pattern, recursive=True):
    if os.path.isdir(file_path) or os.path.isfile(file_path):
        summary_file_collection.append(file_path)
        print(file_path)

summary_file_collection.sort()

In [ ]:
file_path = "/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/profiler_results/49_ngd_weird_gauge_mr1/extracted-Profiler/Step10522.dir/Summary.txt"



## AmrTolerances.input

In [ ]:
Lev=3
TruncationErrorMax = 0.000216536 * 4**(-Lev)
ProjectedConstraintsMax = 0.216536 * 4**(-Lev)
TruncationErrorMaxA = TruncationErrorMax*1.e-4
TruncationErrorMaxB = TruncationErrorMax*1.e-4

AhMaxRes  = TruncationErrorMax
AhMinRes  = AhMaxRes / 10.0

AhMaxTrunc=TruncationErrorMax
AhMinTrunc=AhMaxTrunc / 100.0

print(f"AhMinRes={AhMinRes};")
print(f"AhMaxRes={AhMaxRes};")
print(f"AhMinTrunc={AhMinTrunc};")
print(f"AhMaxTrunc={AhMaxTrunc};")
print(f"TruncationErrorMax={TruncationErrorMax};")
print(f"TruncationErrorMaxA={TruncationErrorMaxA};")
print(f"TruncationErrorMaxB={TruncationErrorMaxB};")
print(f"ProjectedConstraintsMax={ProjectedConstraintsMax};")


In [ ]:
def ode_tol_val(Lev):
  TruncationErrorMax = 0.000216536 * 4**(-Lev)
  ode_tol = TruncationErrorMax/2000
  return ode_tol

for i in [0,1,2,3,4,4.5,5,5.5,6]:
  print(i,"->",ode_tol_val(i))

In [ ]:
#SBATCH --reservation=sxs_standing

# Simulating time steps

In [ ]:
class time_step_simulator():
  def __init__(self,average=0.05,variation_frac=0.1,running_avg_bound_N=50, step_growth_frac=0.07,chunk_time_diff=1.5, chunk_step_modify=3,chunk_start=1.5) -> None:
    self.average = average
    self.variation_frac = variation_frac

    self.ode_error_estimate = 1e-8
    self.ode_error_estimate_variation_fraction = 0.1 #How much can the error change from step to step
    self.ode_tolerance = 1e-8
    self.ode_order = 4

    # Time step  tracking
    self.time_till_now = 0
    self.time_steps = []
    self.time_after_step = []

    # Chunking stuff
    self.chunk_time_diff = chunk_time_diff
    self.chunk_step_modify = chunk_step_modify
    self.next_chunk_time = chunk_start
    
    # Running average stuff
    self.running_average_step_size = average
    self.step_growth_frac = step_growth_frac
    self.running_avg_bound_N = running_avg_bound_N
    self.running_average_step_size_arr = []


  def take_step(self):
    curr_step = self.average + (2*np.random.rand()-1)*self.variation_frac*self.average

    # Bound the step size growth
    if curr_step > self.running_average_step_size*(1+self.step_growth_frac):
      curr_step = self.running_average_step_size*(1+self.step_growth_frac)

    steps_left = (self.next_chunk_time-self.time_till_now)/curr_step
    if steps_left <=0:
      raise ValueError(f"{steps_left=} is an invalid value.")

    if steps_left < self.chunk_step_modify:
      if (steps_left < 1+1e-14):
        # Take exact time step to hit the chunk
        curr_step = (self.next_chunk_time-self.time_till_now)
        # set the next chunk time
        self.next_chunk_time = self.next_chunk_time + self.chunk_time_diff
      else:
        # We need to decrease the time steps to hit the chunk
        curr_step = (self.next_chunk_time-self.time_till_now)*1.05/(np.floor(steps_left)+1.0)

    avg_frac = (self.running_avg_bound_N-1)/self.running_avg_bound_N
    self.running_average_step_size = self.running_average_step_size*avg_frac + (1-avg_frac)*curr_step
    self.running_average_step_size_arr.append(self.running_average_step_size)

    self.time_till_now = self.time_till_now + curr_step
    self.time_after_step.append(self.time_till_now)
    self.time_steps.append(curr_step)

  def take_steps(self,num_steps=100):
    for i in range(num_steps):
      self.take_step()




In [ ]:
stepper = time_step_simulator(variation_frac=0.1,chunk_step_modify=3)
stepper.take_steps(100)

In [ ]:
plt.plot(stepper.time_after_step, stepper.time_steps,marker="x")
plt.plot(stepper.time_after_step, stepper.running_average_step_size_arr,marker="x")
# plt.plot(stepper.time_after_step, stepper.time_steps)
# plt.plot(stepper.time_after_step, stepper.running_average_step_size_arr)
plt.show()

# Extract h5 files

In [ ]:
Ev_path = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev")
save_run_path = Path("/groups/sxs/hchaudha/spec_runs/del.txt")

In [ ]:
with save_run_path.open('w') as f:
  path_list = list(Ev_path.glob("Lev*_A?/Run"))
  path_list.sort()
  ringdown_path_list = list(Ev_path.glob("Lev*_Ringdown/Lev*_A?/Run"))
  ringdown_path_list.sort()
  for i in (path_list+ringdown_path_list):
    if not os.path.islink(i.parent): # If Lev5_AA is symlink then do not write
      f.write(f"{i}\n")
  # for i in Ev_path.glob("Lev?_A?/Run/extrac*"):
    # f.write(f"rm -r {i}\n")

```bash
#!/bin/bash

folder=$1

cd $1
echo "Running in the folder: $folder"
/home/hchaudha/spec/Support/bin/ExtractFromH5 -o . -strippath ./*.h5
# cat del.txt | xargs -P 8 -I {} ./del.sh {}
```

In [ ]:
os.path.islink(Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev6_AA")),os.path.islink(Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev6_AA/Run"))

# Hist-GrDomain

In [ ]:
def parse_ResizeTheseSubdomains(input_str):
    # Remove the initial part before '=' and trailing semicolon
    input_str = input_str.split('=', 1)[1].strip(';')
    
    # Use regex to find all subdomain entries
    pattern = r'(\w+(?:\.\d+)*)\(Extents=\((\d+,\d+,\d+)\)\)'
    matches = re.findall(pattern, input_str)
    
    # Initialize the dictionary
    subdomains_dict = {}

    for name, extents in matches:
        # Convert the extents string into a tuple of integers
        extents = tuple(map(int, extents.split(',')))
        
        # Add to the dictionary
        subdomains_dict[name] = extents
    
    return subdomains_dict

def find_max_extents_for_domain_type(parsed_dict, domain_type):
  def max_vals_in_a_tuple(t1, t2):
    return tuple(max(a, b) for a, b in zip(t1, t2))
  max_extents = (0, 0, 0)
  for domain_name in parsed_dict:
    if domain_type in domain_name:
      max_extents = max_vals_in_a_tuple(max_extents, parsed_dict[domain_name])
  return max_extents


def AMR_input_content_spheres(sphere_name, MinExtent0, MinL, DoNotChangeBeforeTime,TruncationErrorMax=None):
  if TruncationErrorMax is None:
    amr_str = f"""{sphere_name}(MinExtent0={MinExtent0};MinL={MinL};
          DoNotChangeExtent0BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeLBeforeRadiusPlusTime = {DoNotChangeBeforeTime};
        ),
"""
  else:
    amr_str = f"""{sphere_name}(MinExtent0={MinExtent0};MinL={MinL};
          DoNotChangeExtent0BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeLBeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          TruncationErrorMax = {TruncationErrorMax};
          Center = 0,0,0;
        ),
"""
  return amr_str

def AMR_input_content_cylinder(cylinder_name, MinExtent0, MinExtent1, MinExtent2, DoNotChangeBeforeTime,TruncationErrorMax=None):
  if TruncationErrorMax is None:
    amr_str = f"""{cylinder_name}*(MinExtent0={MinExtent0};MinExtent1={MinExtent1};MinExtent2={MinExtent2};
          DoNotChangeExtent0BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeExtent1BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeExtent2BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
        ),
"""
  else:
    amr_str = f"""{cylinder_name}*(MinExtent0={MinExtent0};MinExtent1={MinExtent1};MinExtent2={MinExtent2};
          DoNotChangeExtent0BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeExtent1BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeExtent2BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          TruncationErrorMax = {TruncationErrorMax};
          Center = 0,0,0;
        ),
"""
  return amr_str

def make_odd(n):
  if n%2==0:
    return n+1
  else:
    return n

def AMR_input_content_spheres_min_max(sphere_name, MinExtent0, MinL, DoNotChangeBeforeTime):
  amr_str = f"""{sphere_name}(MinExtent0={MinExtent0};MinL={MinL};
        MaxExtent0={MinExtent0};MaxL={MinL};
        SplitAfterMaxExtent0IsReached=False;
        SplitAfterMaxLIsReached=False;
        DoNotChangeExtent0BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
        DoNotChangeLBeforeRadiusPlusTime = {DoNotChangeBeforeTime};
      ),
"""
  return amr_str

def AMR_input_content_cylinder_min_max(cylinder_name, MinExtent0, MinExtent1, MinExtent2, DoNotChangeBeforeTime):
  MinExtent0, MinExtent1, MinExtent2 = make_odd(MinExtent0), make_odd(MinExtent1), make_odd(MinExtent2)
  amr_str = f"""{cylinder_name}*(MinExtent0={MinExtent0};MinExtent1={MinExtent1};MinExtent2={MinExtent2};
          MaxExtent0={MinExtent0};MaxExtent1={MinExtent1};MaxExtent2={MinExtent2};
          SplitAfterMaxExtent0IsReached=False;
          SplitAfterMaxExtent1IsReached=False;
          SplitAfterMaxExtent2IsReached=False;
          DoNotChangeExtent0BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeExtent1BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
          DoNotChangeExtent2BeforeRadiusPlusTime = {DoNotChangeBeforeTime};
        ),
"""
  return amr_str

def adjust_parsed_dict_for_lev(parsed_dict, lev):
  adjustment = int(lev - 3)
  for domain_name in parsed_dict:
    parsed_dict[domain_name] = tuple([int(i+adjustment) for i in parsed_dict[domain_name]])
    if "Sphere" in domain_name:
      extents = parsed_dict[domain_name]
      # For spheres M is always 2*L
      parsed_dict[domain_name] = (extents[0], extents[1], 2*extents[1])
  return parsed_dict

input_str_base_set3_lev3 = "ResizeTheseSubdomains=SphereA0(Extents=(8,18,36)),SphereA1(Extents=(8,19,38)),SphereA2(Extents=(8,20,40)),SphereA3(Extents=(8,21,42)),SphereA4(Extents=(8,22,44)),SphereB0(Extents=(8,18,36)),SphereB1(Extents=(8,19,38)),SphereB2(Extents=(8,20,40)),SphereB3(Extents=(8,21,42)),SphereB4(Extents=(8,22,44)),SphereC0(Extents=(15,15,30)),SphereC1(Extents=(15,15,30)),SphereC2(Extents=(15,15,30)),SphereC3(Extents=(15,15,30)),SphereC4(Extents=(15,15,30)),SphereC5(Extents=(15,15,30)),SphereC6(Extents=(15,15,30)),SphereC7(Extents=(15,15,30)),SphereC8(Extents=(15,15,30)),SphereC9(Extents=(15,15,30)),SphereC10(Extents=(15,15,30)),SphereC11(Extents=(15,16,32)),SphereC12(Extents=(15,16,32)),SphereC13(Extents=(15,16,32)),SphereC14(Extents=(15,16,32)),SphereC15(Extents=(15,16,32)),SphereC16(Extents=(15,15,30)),SphereC17(Extents=(15,15,30)),SphereC18(Extents=(15,15,30)),SphereC19(Extents=(15,15,30)),SphereC20(Extents=(15,15,30)),SphereC21(Extents=(15,15,30)),SphereC22(Extents=(15,15,30)),SphereC23(Extents=(15,16,32)),SphereC24(Extents=(15,16,32)),SphereC25(Extents=(15,16,32)),SphereC26(Extents=(15,16,32)),SphereC27(Extents=(15,16,32)),SphereC28(Extents=(15,16,32)),SphereC29(Extents=(15,16,32)),CylinderEB0.0.0(Extents=(10,23,15)),CylinderEB1.0.0(Extents=(12,23,15)),CylinderCB0.0.0(Extents=(13,23,16)),CylinderCB1.0.0(Extents=(13,23,15)),CylinderEA0.0.0(Extents=(10,23,15)),CylinderEA1.0.0(Extents=(12,23,15)),CylinderCA0.0.0(Extents=(13,23,16)),CylinderCA1.0.0(Extents=(13,23,15)),FilledCylinderEB0(Extents=(10,9,21)),FilledCylinderEB1(Extents=(10,9,21)),FilledCylinderCB0(Extents=(11,8,21)),FilledCylinderCB1(Extents=(12,8,21)),FilledCylinderMB0(Extents=(10,9,19)),FilledCylinderMB1(Extents=(12,9,19)),CylinderSMB0.0(Extents=(10,21,11)),CylinderSMB1.0(Extents=(12,21,12)),FilledCylinderEA0(Extents=(10,9,21)),FilledCylinderEA1(Extents=(10,9,21)),FilledCylinderCA0(Extents=(11,8,21)),FilledCylinderCA1(Extents=(12,8,21)),FilledCylinderMA0(Extents=(10,9,19)),FilledCylinderMA1(Extents=(12,9,19)),CylinderSMA0.0(Extents=(10,21,11)),CylinderSMA1.0(Extents=(12,21,12));"
parsed_dict_set3_lev3 = parse_ResizeTheseSubdomains(input_str_base_set3_lev3)

input_str_base_set1_lev3 = "ResizeTheseSubdomains=SphereA0(Extents=(8,18,36)),SphereA1(Extents=(8,19,38)),SphereA2(Extents=(8,20,40)),SphereA3(Extents=(8,21,42)),SphereA4(Extents=(8,22,44)),SphereB0(Extents=(8,18,36)),SphereB1(Extents=(8,19,38)),SphereB2(Extents=(8,20,40)),SphereB3(Extents=(8,21,42)),SphereB4(Extents=(8,22,44)),SphereC0(Extents=(15,15,30)),SphereC1(Extents=(15,15,30)),SphereC2(Extents=(15,15,30)),SphereC3(Extents=(15,15,30)),SphereC4(Extents=(15,15,30)),SphereC5(Extents=(15,15,30)),SphereC6(Extents=(15,15,30)),SphereC7(Extents=(15,15,30)),SphereC8(Extents=(15,15,30)),SphereC9(Extents=(15,15,30)),SphereC10(Extents=(15,16,32)),SphereC11(Extents=(15,16,32)),SphereC12(Extents=(15,16,32)),SphereC13(Extents=(15,16,32)),SphereC14(Extents=(15,16,32)),SphereC15(Extents=(15,15,30)),SphereC16(Extents=(15,15,30)),SphereC17(Extents=(15,15,30)),SphereC18(Extents=(15,15,30)),SphereC19(Extents=(15,15,30)),SphereC20(Extents=(15,15,30)),SphereC21(Extents=(15,15,30)),SphereC22(Extents=(15,16,32)),SphereC23(Extents=(15,16,32)),SphereC24(Extents=(15,16,32)),SphereC25(Extents=(15,16,32)),SphereC26(Extents=(15,16,32)),SphereC27(Extents=(15,16,32)),SphereC28(Extents=(15,16,32)),SphereC29(Extents=(15,16,32)),CylinderEB0.0.0(Extents=(10,23,15)),CylinderEB1.0.0(Extents=(12,23,15)),CylinderCB0.0.0(Extents=(13,23,16)),CylinderCB1.0.0(Extents=(13,23,15)),CylinderEA0.0.0(Extents=(10,23,15)),CylinderEA1.0.0(Extents=(12,23,15)),CylinderCA0.0.0(Extents=(13,23,16)),CylinderCA1.0.0(Extents=(13,23,15)),FilledCylinderEB0(Extents=(10,9,21)),FilledCylinderEB1(Extents=(10,9,21)),FilledCylinderCB0(Extents=(11,8,21)),FilledCylinderCB1(Extents=(12,8,21)),FilledCylinderMB0(Extents=(10,9,19)),FilledCylinderMB1(Extents=(12,9,19)),CylinderSMB0.0(Extents=(10,21,11)),CylinderSMB1.0(Extents=(12,21,12)),FilledCylinderEA0(Extents=(10,9,21)),FilledCylinderEA1(Extents=(10,9,21)),FilledCylinderCA0(Extents=(11,8,21)),FilledCylinderCA1(Extents=(12,8,21)),FilledCylinderMA0(Extents=(10,9,19)),FilledCylinderMA1(Extents=(12,9,19)),CylinderSMA0.0(Extents=(10,21,11)),CylinderSMA1.0(Extents=(12,21,12));"
parsed_dict_set1_lev3 = parse_ResizeTheseSubdomains(input_str_base_set1_lev3)

input_str_base_lev6 = "ResizeTheseSubdomains=SphereA0(Extents=(11,23,46)),SphereA1(Extents=(11,24,48)),SphereA2(Extents=(11,25,50)),SphereA3(Extents=(11,26,52)),SphereA4(Extents=(13,27,54)),SphereB0(Extents=(11,22,44)),SphereB1(Extents=(11,23,46)),SphereB2(Extents=(11,24,48)),SphereB3(Extents=(11,25,50)),SphereB4(Extents=(13,26,52)),SphereC0(Extents=(18,18,36)),SphereC1(Extents=(18,17,34)),SphereC2(Extents=(18,18,36)),SphereC3(Extents=(18,18,36)),SphereC4(Extents=(18,18,36)),SphereC5(Extents=(18,19,38)),SphereC6(Extents=(18,19,38)),SphereC7(Extents=(18,19,38)),SphereC8(Extents=(18,19,38)),SphereC9(Extents=(18,19,38)),SphereC10(Extents=(18,19,38)),SphereC11(Extents=(18,19,38)),SphereC12(Extents=(18,19,38)),SphereC13(Extents=(18,19,38)),SphereC14(Extents=(18,19,38)),SphereC15(Extents=(18,19,38)),SphereC16(Extents=(18,20,40)),SphereC17(Extents=(18,20,40)),SphereC18(Extents=(18,20,40)),SphereC19(Extents=(18,20,40)),SphereC20(Extents=(18,20,40)),SphereC21(Extents=(18,20,40)),SphereC22(Extents=(18,21,42)),SphereC23(Extents=(18,21,42)),SphereC24(Extents=(18,21,42)),SphereC25(Extents=(18,22,44)),SphereC26(Extents=(18,22,44)),SphereC27(Extents=(18,22,44)),SphereC28(Extents=(18,22,44)),SphereC29(Extents=(18,21,42)),CylinderEB0.0.0(Extents=(14,29,19)),CylinderEB1.0.0(Extents=(16,29,21)),CylinderCB0.0.0(Extents=(17,29,20)),CylinderCB1.0.0(Extents=(17,27,19)),CylinderEA0.0.0(Extents=(14,29,19)),CylinderEA1.0.0(Extents=(16,29,21)),CylinderCA0.0.0(Extents=(17,29,20)),CylinderCA1.0.0(Extents=(17,27,19)),FilledCylinderEB0(Extents=(13,11,29)),FilledCylinderEB1(Extents=(14,11,29)),FilledCylinderCB0(Extents=(15,11,29)),FilledCylinderCB1(Extents=(16,10,27)),FilledCylinderMB0(Extents=(14,11,27)),FilledCylinderMB1(Extents=(16,11,23)),CylinderSMB0.0(Extents=(15,27,15)),CylinderSMB1.0(Extents=(16,25,15)),FilledCylinderEA0(Extents=(13,11,29)),FilledCylinderEA1(Extents=(14,11,29)),FilledCylinderCA0(Extents=(15,11,29)),FilledCylinderCA1(Extents=(16,10,27)),FilledCylinderMA0(Extents=(14,11,27)),FilledCylinderMA1(Extents=(16,11,23)),CylinderSMA0.0(Extents=(15,27,15)),CylinderSMA1.0(Extents=(16,25,15));"
parsed_dict_lev6 = parse_ResizeTheseSubdomains(input_str_base_lev6)

#### With CylinderSM

In [ ]:
input_str = "ResizeTheseSubdomains=SphereA0(Extents=(8,18,36)),SphereA1(Extents=(8,19,38)),SphereA2(Extents=(8,20,40)),SphereA3(Extents=(8,21,42)),SphereA4(Extents=(8,22,44)),SphereB0(Extents=(8,18,36)),SphereB1(Extents=(8,19,38)),SphereB2(Extents=(8,20,40)),SphereB3(Extents=(8,21,42)),SphereB4(Extents=(8,22,44)),SphereC0(Extents=(15,15,30)),SphereC1(Extents=(15,15,30)),SphereC2(Extents=(15,15,30)),SphereC3(Extents=(15,15,30)),SphereC4(Extents=(15,15,30)),SphereC5(Extents=(15,15,30)),SphereC6(Extents=(15,15,30)),SphereC7(Extents=(15,15,30)),SphereC8(Extents=(15,15,30)),SphereC9(Extents=(15,15,30)),SphereC10(Extents=(15,16,32)),SphereC11(Extents=(15,16,32)),SphereC12(Extents=(15,16,32)),SphereC13(Extents=(15,16,32)),SphereC14(Extents=(15,16,32)),SphereC15(Extents=(15,15,30)),SphereC16(Extents=(15,15,30)),SphereC17(Extents=(15,15,30)),SphereC18(Extents=(15,15,30)),SphereC19(Extents=(15,15,30)),SphereC20(Extents=(15,15,30)),SphereC21(Extents=(15,15,30)),SphereC22(Extents=(15,16,32)),SphereC23(Extents=(15,16,32)),SphereC24(Extents=(15,16,32)),SphereC25(Extents=(15,16,32)),SphereC26(Extents=(15,16,32)),SphereC27(Extents=(15,16,32)),SphereC28(Extents=(15,16,32)),SphereC29(Extents=(15,16,32)),CylinderEB0.0.0(Extents=(10,23,15)),CylinderEB1.0.0(Extents=(12,23,15)),CylinderCB0.0.0(Extents=(13,23,16)),CylinderCB1.0.0(Extents=(13,23,15)),CylinderEA0.0.0(Extents=(10,23,15)),CylinderEA1.0.0(Extents=(12,23,15)),CylinderCA0.0.0(Extents=(13,23,16)),CylinderCA1.0.0(Extents=(13,23,15)),FilledCylinderEB0(Extents=(10,9,21)),FilledCylinderEB1(Extents=(10,9,21)),FilledCylinderCB0(Extents=(11,8,21)),FilledCylinderCB1(Extents=(12,8,21)),FilledCylinderMB0(Extents=(10,9,19)),FilledCylinderMB1(Extents=(12,9,19)),CylinderSMB0.0(Extents=(10,21,11)),CylinderSMB1.0(Extents=(12,21,12)),FilledCylinderEA0(Extents=(10,9,21)),FilledCylinderEA1(Extents=(10,9,21)),FilledCylinderCA0(Extents=(11,8,21)),FilledCylinderCA1(Extents=(12,8,21)),FilledCylinderMA0(Extents=(10,9,19)),FilledCylinderMA1(Extents=(12,9,19)),CylinderSMA0.0(Extents=(10,21,11)),CylinderSMA1.0(Extents=(12,21,12));"

DoNotChangeBeforeTime = 40000
Lev = 3

parsed_dict = parse_ResizeTheseSubdomains(input_str)
adjust_parsed_dict_for_lev(parsed_dict, Lev) 

max_extent_dict = {
  # Domains that are not spheres will all have the maximum extents
  "FilledCylinderCA" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderCA"),
  "CylinderCA" : find_max_extents_for_domain_type(parsed_dict,"CylinderCA"),
  "FilledCylinderEA" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderEA"),
  "CylinderEA" : find_max_extents_for_domain_type(parsed_dict,"CylinderEA"),
  "CylinderSMA" : find_max_extents_for_domain_type(parsed_dict,"CylinderSMA"),
  "FilledCylinderMA" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderMA"),
  "FilledCylinderMB" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderMB"),
  "CylinderSMB" : find_max_extents_for_domain_type(parsed_dict,"CylinderSMB"),
  "CylinderEB" : find_max_extents_for_domain_type(parsed_dict,"CylinderEB"),
  "FilledCylinderEB" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderEB"),
  "CylinderCB" : find_max_extents_for_domain_type(parsed_dict,"CylinderCB"),
  "FilledCylinderCB" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderCB"),
}

for key,extent in parsed_dict.items():
  if 'Sphere' in key:
    max_extent_dict[key] = extent

for key in max_extent_dict:
  if "Sphere" in key:
    if "SphereC" in key:
      if "SphereC0" in key:
        extents = max_extent_dict["SphereC0"]
        print(AMR_input_content_spheres("SphereC*", extents[0], extents[1], DoNotChangeBeforeTime=DoNotChangeBeforeTime))
      continue
    print(AMR_input_content_spheres(key, max_extent_dict[key][0], max_extent_dict[key][1], DoNotChangeBeforeTime=DoNotChangeBeforeTime))
  elif "Cylinder" in key:
    print(AMR_input_content_cylinder(key, max_extent_dict[key][0], max_extent_dict[key][1], max_extent_dict[key][2], DoNotChangeBeforeTime=DoNotChangeBeforeTime))


In [ ]:
input_str = "ResizeTheseSubdomains=SphereA0(Extents=(8,18,36)),SphereA1(Extents=(8,19,38)),SphereA2(Extents=(8,20,40)),SphereA3(Extents=(8,21,42)),SphereA4(Extents=(8,22,44)),SphereB0(Extents=(8,18,36)),SphereB1(Extents=(8,19,38)),SphereB2(Extents=(8,20,40)),SphereB3(Extents=(8,21,42)),SphereB4(Extents=(8,22,44)),SphereC0(Extents=(15,15,30)),SphereC1(Extents=(15,15,30)),SphereC2(Extents=(15,15,30)),SphereC3(Extents=(15,15,30)),SphereC4(Extents=(15,15,30)),SphereC5(Extents=(15,15,30)),SphereC6(Extents=(15,15,30)),SphereC7(Extents=(15,15,30)),SphereC8(Extents=(15,15,30)),SphereC9(Extents=(15,15,30)),SphereC10(Extents=(15,16,32)),SphereC11(Extents=(15,16,32)),SphereC12(Extents=(15,16,32)),SphereC13(Extents=(15,16,32)),SphereC14(Extents=(15,16,32)),SphereC15(Extents=(15,15,30)),SphereC16(Extents=(15,15,30)),SphereC17(Extents=(15,15,30)),SphereC18(Extents=(15,15,30)),SphereC19(Extents=(15,15,30)),SphereC20(Extents=(15,15,30)),SphereC21(Extents=(15,15,30)),SphereC22(Extents=(15,16,32)),SphereC23(Extents=(15,16,32)),SphereC24(Extents=(15,16,32)),SphereC25(Extents=(15,16,32)),SphereC26(Extents=(15,16,32)),SphereC27(Extents=(15,16,32)),SphereC28(Extents=(15,16,32)),SphereC29(Extents=(15,16,32)),CylinderEB0.0.0(Extents=(10,23,15)),CylinderEB1.0.0(Extents=(12,23,15)),CylinderCB0.0.0(Extents=(13,23,16)),CylinderCB1.0.0(Extents=(13,23,15)),CylinderEA0.0.0(Extents=(10,23,15)),CylinderEA1.0.0(Extents=(12,23,15)),CylinderCA0.0.0(Extents=(13,23,16)),CylinderCA1.0.0(Extents=(13,23,15)),FilledCylinderEB0(Extents=(10,9,21)),FilledCylinderEB1(Extents=(10,9,21)),FilledCylinderCB0(Extents=(11,8,21)),FilledCylinderCB1(Extents=(12,8,21)),FilledCylinderMB0(Extents=(10,9,19)),FilledCylinderMB1(Extents=(12,9,19)),CylinderSMB0.0(Extents=(10,21,11)),CylinderSMB1.0(Extents=(12,21,12)),FilledCylinderEA0(Extents=(10,9,21)),FilledCylinderEA1(Extents=(10,9,21)),FilledCylinderCA0(Extents=(11,8,21)),FilledCylinderCA1(Extents=(12,8,21)),FilledCylinderMA0(Extents=(10,9,19)),FilledCylinderMA1(Extents=(12,9,19)),CylinderSMA0.0(Extents=(10,21,11)),CylinderSMA1.0(Extents=(12,21,12));"

DoNotChangeBeforeTime = 40000
Lev = 6

parsed_dict = parse_ResizeTheseSubdomains(input_str)
adjust_parsed_dict_for_lev(parsed_dict, Lev) 

max_extent_dict = {
  # Domains that are not spheres will all have the maximum extents
  "FilledCylinderCA" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderCA"),
  "CylinderCA" : find_max_extents_for_domain_type(parsed_dict,"CylinderCA"),
  "FilledCylinderEA" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderEA"),
  "CylinderEA" : find_max_extents_for_domain_type(parsed_dict,"CylinderEA"),
  "CylinderSMA" : find_max_extents_for_domain_type(parsed_dict,"CylinderSMA"),
  "FilledCylinderMA" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderMA"),
  "FilledCylinderMB" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderMB"),
  "CylinderSMB" : find_max_extents_for_domain_type(parsed_dict,"CylinderSMB"),
  "CylinderEB" : find_max_extents_for_domain_type(parsed_dict,"CylinderEB"),
  "FilledCylinderEB" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderEB"),
  "CylinderCB" : find_max_extents_for_domain_type(parsed_dict,"CylinderCB"),
  "FilledCylinderCB" : find_max_extents_for_domain_type(parsed_dict,"FilledCylinderCB"),
}

for key,extent in parsed_dict.items():
  if 'Sphere' in key:
    max_extent_dict[key] = extent

for key in max_extent_dict:
  if "Sphere" in key:
    if "SphereC" in key:
      if "SphereC0" in key:
        extents = max_extent_dict["SphereC0"]
        print(AMR_input_content_spheres_min_max("SphereC*", extents[0], extents[1], DoNotChangeBeforeTime=DoNotChangeBeforeTime))
      continue
    print(AMR_input_content_spheres_min_max(key, max_extent_dict[key][0], max_extent_dict[key][1], DoNotChangeBeforeTime=DoNotChangeBeforeTime))
  elif "Cylinder" in key:
    print(AMR_input_content_cylinder_min_max(key, max_extent_dict[key][0], max_extent_dict[key][1], max_extent_dict[key][2], DoNotChangeBeforeTime=DoNotChangeBeforeTime))


# Generate DomainFile from the hist

In [ ]:
data_file_path = "MinimumGridSpacing.dat"
data_file_path = "Hist-GrDomain.txt"

# data_file_path = "Hist-Domain.txt"


column_names, runs_data_dict_domain = load_data_from_levs(runs_to_plot,data_file_path)
# print(column_names)
print(runs_data_dict_domain.keys())

In [ ]:
key = "37_set1_L3_no_ang"
t_target = 4000

data = runs_data_dict_domain[key]
idx = np.argmin(np.abs(data['t(M)']-t_target))
sd_extent_dict = data.iloc[idx].to_dict()


base_extents = {
    "SphereA": {
        "RadialExtent": 0,
        "L": 0,
        "M": 0,
    },
    "SphereB": {
        "RadialExtent": 0,
        "L": 0,
        "M": 0,
    },
    "SphereC": {
        "RadialExtent": 0,
        "L": 0,
        "M": 0,
    },
    "CylinderCA": {
        "RadialExtents": 0,
        "PhiExtent": 0,
        "ThetaExtents": 0,
    },
    "CylinderCB": {
        "RadialExtents": 0,
        "PhiExtent": 0,
        "ThetaExtents": 0,
    },
    "CylinderEA": {
        "RadialExtents": 0,
        "PhiExtent": 0,
        "ThetaExtents": 0,
    },
    "CylinderEB": {
        "RadialExtents": 0,
        "PhiExtent": 0,
        "ThetaExtents": 0,
    },
    "CylinderSMA": {
        "RadialExtents": 0,
        "PhiExtent": 0,
        "ThetaExtents": 0,
    },
    "CylinderSMB": {
        "RadialExtents": 0,
        "PhiExtent": 0,
        "ThetaExtents": 0,
    },
    "FilledCylinderCA": {
        "RadialExtents": 0,
        "ThetaExtent": 0,
        "PhiExtent": 0,
    },
    "FilledCylinderCB": {
        "RadialExtents": 0,
        "ThetaExtent": 0,
        "PhiExtent": 0,
    },
    "FilledCylinderEA": {
        "RadialExtents": 0,
        "ThetaExtent": 0,
        "PhiExtent": 0,
    },
    "FilledCylinderEB": {
        "RadialExtents": 0,
        "ThetaExtent": 0,
        "PhiExtent": 0,
    },
    "FilledCylinderMA": {
        "RadialExtents": 0,
        "ThetaExtent": 0,
        "PhiExtent": 0,
    },
    "FilledCylinderMB": {
        "RadialExtents": 0,
        "ThetaExtent": 0,
        "PhiExtent": 0,
    },
}


In [ ]:

sd_extent_dict_keys = list(sd_extent_dict.keys())[4:]


def reduction_function(a,b):
    return max(a,b)
    # return min(a,b)


for sd_group in base_extents:
    keys = list(base_extents[sd_group].keys())

    for sd_extent in sd_extent_dict_keys:
        if sd_extent.startswith(sd_group):
            if "_R" in sd_extent:
                base_extents[sd_group][keys[0]] = reduction_function(base_extents[sd_group][keys[0]], sd_extent_dict[sd_extent])
            elif "_L" in sd_extent:
                base_extents[sd_group][keys[1]] = reduction_function(base_extents[sd_group][keys[1]], sd_extent_dict[sd_extent])
            elif "_M" in sd_extent:
                base_extents[sd_group][keys[2]] = reduction_function(base_extents[sd_group][keys[2]], sd_extent_dict[sd_extent])

for i in base_extents:
    for j in base_extents[i]:
        base_extents[i][j] = int(base_extents[i][j])

base_extents

In [ ]:
extents = copy.deepcopy(base_extents)
CylindersAndFilledCylinders="\n"
lev_change = -1
if lev_change != 0:
    print(f"changing level by {lev_change} for all extents")

for i in extents:
    for j in extents[i]:
        extents[i][j] += lev_change
        if "Sphere" in i:
            # Dealt with later
            continue
        if "SMA" in i or "SMB" in i:
            # SMA and SMB are not recognized somehow
            continue
        if "PhiExtent" in j:
            # The read value is PhiExtent+1, but note that PhiExtent should be odd
            if extents[i][j] % 2 == 0: # We just reduce by 1
                # CylindersAndFilledCylinders += f"     {i}::{j} = {extents[i][j]-1};\n"
                # Or we can just add 1, FilledCylinders and Cylinder will get more res
                CylindersAndFilledCylinders += f"     {i}::{j} = {extents[i][j]+1};\n"
            else:
                # Should not happen, but if it does just pass it on
                # CylindersAndFilledCylinders += f"     {i}::{j} = {extents[i][j]};\n"
                # Should not happen, but if it does just pass it on after adding 2
                CylindersAndFilledCylinders += f"     {i}::{j} = {extents[i][j]+2};\n"
            continue
        
        CylindersAndFilledCylinders += f"     {i}::{j} = {extents[i][j]};\n"


In [ ]:
GrDomain_text = f"""# TouchingJuggleBallBBH for unequal mass
ReadFromFiles = RefinementOptionHistory.input;
SubdomainStructure=
    TouchingCutSphereWedgesBBH
    (
     Level= __Lev__;
     DomainFilename=Dnew.input;

     CenterA=__xA__,__yA__,__zA__;
     CenterB=__xB__,__yB__,__zB__;
     CutSphereGridPlaneCoordX=<<DEFAULT>>;
     AdjustSmallerCutRadiusLimit=true;
     MinimumAllowedSmallerCutRadius=<<DEFAULT>>;

     UseSmallRotations = false;
     RotateSpheresOntoXAxis = false;

     SphereA=(RMin = __rExcA__;
              RMax = __rMaxA__; # 0.5*fabs(__xB__);
              RelativeDeltaR=1.1*__RelativeDeltaR__;  # prevent early shell-drops
              #L=__LA__;
              L={extents["SphereA"]["L"]-1};
              RadialExtents={extents["SphereA"]["RadialExtent"]};
              RadialMap = Exp;
             );
     SphereB=(RMin = __rExcB__;
              RMax = __rMaxB__; # 0.5*fabs(__xA__);
              RelativeDeltaR=1.1*__RelativeDeltaR__;  # prevent early shell-drops
              #L=__LB__;
              L={extents["SphereB"]["L"]-1};
              RadialExtents={extents["SphereB"]["RadialExtent"]};
              RadialMap = Exp;
             );
     SphereC=(Radii = __Radii__;
              L={extents["SphereC"]["L"]-1};
              RadialExtents={extents["SphereC"]["RadialExtent"]};
             );

     FilledCylinderDefaults  = ( ThetaExtent = 8+k;
                           PhiExtent = 13+2*k;
                           RadialExtents = 7+k;
                           RadialSplitFactor=2;
                          );
     CylinderDefaults  = ( ThetaExtents = 8+k;
                           PhiExtent = 13+2*k;
                           RadialExtents = 7+k;
                           RadialSplitFactor=2;
                          );
     FilledCylinderMA::ThetaSplitFactor = 2;
     FilledCylinderMB::ThetaSplitFactor = 2;
{CylindersAndFilledCylinders}
     );

BoundaryInfo = ( WarningLevel = 0;);
FileBaseName = GrDomain;
"""

print(GrDomain_text)

# 1/r sized sphereC

In [ ]:
r_max = 1001.04355689422
r_min = 54.0000021188612 + 25
numC = 14

a = 1
if a == 1:
    GM_sum = numC
else:
    GM_sum = (a**numC - 1)/(a-1)
r0 = (r_max - r_min)/GM_sum

# [r0*a**i for i in range(numC)]
size = [r0*a**i for i in range(numC)]
boundaries = [r_min]
for bnd in (np.cumsum([r0*a**i for i in range(numC)]) + r_min):
    boundaries.append(bnd)
boundaries[0] = r_min  # Ensure the first boundary is r_min
boundaries[-1] = r_max  # Ensure the last boundary is r_max

for_GR_domain = ""
for i in boundaries:
    for_GR_domain += f"{i},"
for_GR_domain = for_GR_domain[:-1]  # Remove the last comma

boundaries, size
for_GR_domain
# [int(i) for i in boundaries], [int(i) for i in size]

In [ ]:
boundaries, size

# Load data class

In [ ]:
Ev_path = Path(
    "/resnick/groups/sxs/hchaudha/spec_runs/39_obs_segs/L1_set1_metric_obs/Ev"
)
levs = list(
    set([int(i.name.split("_")[0][3:]) for i in Ev_path.iterdir() if i.is_dir()])
)

lev_segments={}
for lev in levs:
    lev_segments[lev] = {}
    for segment in Ev_path.iterdir():
        if segment.is_dir() and f"Lev{lev}" in segment.name:
            lev_segments[lev][segment.name[-2:]] = segment


for i in Ev_path.iterdir():
    if not i.is_file():
        print(i.name)


lev_segments


In [ ]:
class DataReader:
    def __init__(self):
        self.lev_path_regex = {}
        # self.

    def __str__(self):
        return f"{self.name}({self.size})"

# Rough

In [ ]:
runs_data_dict.keys()

In [ ]:
data = runs_data_dict['high_accuracy_L5']

# filtered_cols = [i.split(" ")[-1] for i in data.columns if 'SphereA' in i]
filtered_cols = ['t(M)']+[i for i in data.columns if 'e' in i]
filtered_cols

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Sample data creation (same as in the previous example)
np.random.seed(0)
data = {
    'time': np.arange(0, 10, 1),
    'velocity': np.random.uniform(50, 100, 10),
    'acceleration': np.random.uniform(-5, 5, 10),
    'fuel_left': np.random.uniform(100, 200, 10),
    'burning_rate': np.random.uniform(0, 10, 10)
}
df = pd.DataFrame(data)

# Normalize the data to [0, 1]
normalized_df = (df - df.min()) / (df.max() - df.min())

# Remove time for visualization
visual_data = normalized_df.drop('time', axis=1)

# Plot using imshow
plt.figure(figsize=(12, 6))
plt.imshow(visual_data, aspect='auto', cmap='viridis', origin='lower')

# Set x-ticks and labels
xtick_positions = np.arange(len(visual_data.columns))
plt.xticks(ticks=xtick_positions, labels=visual_data.columns, rotation=90)
# plt.xticks(ticks=np.arange(len(visual_data.columns)), labels=visual_data.columns, rotation=90)
plt.yticks(ticks=np.arange(len(visual_data)), labels=normalized_df['time'].astype(int))

plt.colorbar(label='Normalized Value')
# plt.xlabel('Features')
plt.ylabel('Time')
plt.title('Feature Intensity Over Time')
plt.tight_layout()
plt.show()

In [ ]:
x = np.arange(100)/100
y1 = np.sin(x)
y2 = np.cos(x)

styles =  plt.style.available

for style in styles:
    print(style)
    plt.style.use(style)
    plt.plot(x,y1,label="y1asfasd")
    plt.plot(x,y2,label="y3asfasd")
    plt.title("asdf")
    plt.legend()
    plt.savefig(f"/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/make_report_del/{style}.png")
    plt.close()

In [ ]:
w = 4
print(np.convolve(np.arange(1,10),np.ones(w),'valid')/w)
print(np.arange(1,10))

In [ ]:
def moving_average(array,avg_len):
    return np.convolve(array,np.ones(avg_len))/avg_len

In [ ]:
def gauss(x,cen,scale):
  return np.exp(-(x-cen)**2/scale**2)

In [ ]:
d0 = 18
q = 1
d1 = 1/(1+q)*d0
d2 = q/(1+q)*d0
cen1 = d1
cen2 = -d2
scale1 = d2
scale2 = d1

tol1 = 3.38337e-10
tol2 = 3.38337e-10
tol_base = 3.38337e-6

d = 100
x_min,x_max = -d/2,d/2
X = np.linspace(x_min,x_max,1000)

fac_base = np.ones_like(X)
fac1 = gauss(X,cen1,scale1)
fac2 = gauss(X,cen2,scale2)

fac_total = fac_base+fac1+fac2
val = fac_base*np.log10(tol_base) + fac1*np.log10(tol1)+fac2*np.log10(tol2)
val = 10**(val/fac_total)
print(val.min(),val.max())

In [ ]:
d0 = 18
q = 1
cen1 = 0
cen2 = 0
scale1 = 100
scale2 = 1000

tol1 = 3.38337e-10
tol2 = 3.38337e-9
tol_base = 3.38337e-6

d = 1000
x_min,x_max = -d/2,d/2
X = np.linspace(x_min,x_max,1000)

fac_base = np.ones_like(X)
fac1 = gauss(X,cen1,scale1)
fac2 = gauss(X,cen2,scale2)

fac_total = fac_base+fac1+fac2
val = fac_base*np.log10(tol_base) + fac1*np.log10(tol1)+fac2*np.log10(tol2)
val = 10**(val/fac_total)
print(val.min(),val.max())

In [ ]:
0.00000216536 * 4**(-3)/2

In [ ]:
f = plt.semilogy
f(X,val,label='tot')
# f(X,fac1*tol1,label='tol1')
# f(X,fac2*tol2,label='tol2')
# f(X,fac_base*tol_base,label='tol_base')
hor_lines = [val.min(),val[int(len(val)/2)],val.max()]
plt.hlines(y=hor_lines, xmin=-d/2, xmax=d/2, colors=['r', 'g', 'b'], linestyles='--', linewidth=2)
for y_value in hor_lines:
    plt.text(0, y_value*1.1, f'{y_value:.3e}', va='center', ha='left')


plt.ylabel('AMR tolerance')
plt.xlabel('x_axis')
plt.legend()
plt.show()

In [ ]:
plt.semilogy(X,gauss(X,0,50))
plt.show()

In [ ]:
np.exp(-4)

In [ ]:
def hist_files_to_dataframe(file_path):
  # Function to parse a single line and return a dictionary of values
  def parse_line(line):
      data = {}
      # Find all variable=value pairs
      pairs = re.findall(r'([^;=\s]+)=\s*([^;]+)', line)
      for var, val in pairs:
          # Hist-GrDomain.txt should be parsed a little differently
          if 'ResizeTheseSubdomains' in var:
              items = val.split('),')
              items[-1] = items[-1][:-1]
              for item in items:
                name,_,vals = item.split("(")
                r,l,m=vals[:-1].split(',')
                data[f"{name}_R"] = int(r)
                data[f"{name}_L"] = int(l)
                data[f"{name}_M"] = int(m)
          else:
              data[var] = float(val) if re.match(r'^[\d.e+-]+$', val) else val
      return data
  
  with open(file_path, 'r') as file:
    # Parse the lines
    data = []
    for line in file.readlines():
        data.append(parse_line(line.strip()))

    # Create a DataFrame
    df = pd.DataFrame(data)

  return df

hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AB/Run/Hist-GrDomain.txt')


In [ ]:
hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_AD/Run_old/Hist-GrDomain.txt')


In [ ]:
hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_etl_tol_10/Lev3_AD/Run_old/Hist-GrDomain.txt')
# hist_files_to_dataframe('/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AB/Run/Hist-FuncLambdaFactorB.txt')

In [ ]:
import re

import pandas as pd


# Function to parse a single line and return a dictionary of values
def parse_line(line):
    data = {}
    # Find all variable=value pairs
    pairs = re.findall(r'([^;=]+)=\s*([\d.e+-]+)', line)
    for var, val in pairs:
        data[var] = float(val) if re.match(r'[\d.e+-]+', val) else val
    return data

# Read the file
with open('/groups/sxs/hchaudha/spec_runs/2_SpKS_q1_sA_0_0_9_sB_0_0_9_d15/Ev/Lev3_AB/Run/Hist-FuncLambdaFactorB.txt', 'r') as file:
    lines = file.readlines()

# Parse the lines
data = []
for line in lines:
    data.append(parse_line(line.strip()))

# Create a DataFrame
df = pd.DataFrame(data)

print(df)


In [ ]:
import re

import pandas as pd

# The input string
data = """SphereA0(Extents=(12,22,44)),SphereA1(Extents=(10,23,46)),SphereA2(Extents=(10,24,48)),SphereA3(Extents=(10,25,50)),SphereA4(Extents=(13,26,52)),SphereB0(Extents=(12,22,44)),SphereB1(Extents=(10,23,46)),SphereB2(Extents=(10,24,48)),SphereB3(Extents=(10,25,50)),SphereB4(Extents=(13,26,52)),SphereC0(Extents=(15,15,30)),SphereC1(Extents=(15,15,30)),SphereC2(Extents=(15,14,28)),SphereC3(Extents=(15,15,30)),SphereC4(Extents=(15,15,30)),SphereC5(Extents=(15,15,30)),SphereC6(Extents=(15,15,30)),SphereC7(Extents=(15,16,32)),SphereC8(Extents=(15,15,30)),SphereC9(Extents=(15,16,32)),SphereC10(Extents=(15,16,32)),SphereC11(Extents=(15,15,30)),SphereC12(Extents=(15,15,30)),SphereC13(Extents=(15,15,30)),SphereC14(Extents=(15,15,30)),SphereC15(Extents=(15,15,30)),SphereC16(Extents=(16,15,30)),SphereC17(Extents=(16,16,32)),SphereC18(Extents=(16,16,32)),SphereC19(Extents=(16,16,32)),SphereC20(Extents=(15,16,32)),SphereC21(Extents=(15,16,32)),SphereC22(Extents=(15,16,32)),SphereC23(Extents=(15,16,32)),SphereC24(Extents=(15,15,30)),SphereC25(Extents=(15,16,32)),SphereC26(Extents=(15,16,32)),SphereC27(Extents=(15,16,32)),SphereC28(Extents=(15,16,32)),SphereC29(Extents=(15,16,32)),CylinderEB0.0.0(Extents=(13,31,19)),CylinderEB1.0.0(Extents=(17,25,18)),CylinderCB0.0.0(Extents=(17,23,17)),CylinderCB1.0.0(Extents=(14,21,15)),CylinderEA0.0.0(Extents=(13,31,19)),CylinderEA1.0.0(Extents=(14,25,18)),CylinderCA0.0.0(Extents=(17,23,18)),CylinderCA1.0.0(Extents=(14,21,15)),FilledCylinderEB0(Extents=(12,11,25)),FilledCylinderEB1(Extents=(12,10,25)),FilledCylinderCB0(Extents=(12,9,21)),FilledCylinderCB1(Extents=(12,8,19)),FilledCylinderMB0(Extents=(14,11,25)),FilledCylinderMB1(Extents=(16,10,21)),CylinderSMB0.0(Extents=(14,27,15)),CylinderSMB1.0(Extents=(18,25,15)),FilledCylinderEA0(Extents=(12,11,25)),FilledCylinderEA1(Extents=(12,10,25)),FilledCylinderCA0(Extents=(12,9,21)),FilledCylinderCA1(Extents=(12,8,19)),FilledCylinderMA0(Extents=(14,11,25)),FilledCylinderMA1(Extents=(14,10,21)),CylinderSMA0.0(Extents=(14,27,15)),CylinderSMA1.0(Extents=(15,25,15))"""

# Split the string into individual items
items = data.split('),')

# # Function to parse each item
# def parse_item(item):
#     name, values = re.match(r'(\w+)\(Extents=\((.*?)\)', item).groups()
#     r, l, m = map(int, values.split(','))
#     return {'Name': name, 'R': r, 'L': l, 'M': m}

# # Parse all items
# parsed_data = [parse_item(item) for item in items]

# # Create DataFrame
# df = pd.DataFrame(parsed_data)

# # Set 'Name' as index
# df.set_index('Name', inplace=True)

# # Create the specific variables for SphereA2
# SphereA2_R = df.loc['SphereA2', 'R']
# SphereA2_L = df.loc['SphereA2', 'L']
# SphereA2_M = df.loc['SphereA2', 'M']

# print(df)
# print(f"\nSphereA2_R = {SphereA2_R}")
# print(f"SphereA2_L = {SphereA2_L}")
# print(f"SphereA2_M = {SphereA2_M}")

In [ ]:
items = data.split('),')
name,_,vals = items[0].split("(")
r,l,m=vals[:-1].split(',')
{
  name+"_R":r,
  name+"_L":l,
  name+"_M":m
}

In [ ]:
items[0].split("(")

In [ ]:
vals[:-1].split(',')

In [ ]:
folder_path = Path("/groups/sxs/hchaudha/spec_runs")
del_path = Path("/groups/sxs/hchaudha/spec_runs/del.sh")

with del_path.open('w') as f:
  for i in folder_path.iterdir():
    if i.is_dir():
      if "ID" in str(i):
        continue
      if "del" in str(i):
        continue
      f.writelines(str(i)+"\n")

In [ ]:
def write_dir(folder_path:Path,del_path_opened):
  for i in folder_path.iterdir():
    if i.is_dir():
      print(i)
      if "ID" in str(i):
        continue
      if "del" in str(i):
        continue
      if "Lev" in str(i) and i.is_symlink():
        continue
      if "Run" == i.name:
        del_path_opened.writelines(str(i)+"\n")
        return
      write_dir(i,del_path_opened)

folder_path = Path("/groups/sxs/hchaudha/spec_runs")
# folder_path = Path("/groups/sxs/hchaudha/spec_runs/3_DH_q1_ns_d18_all_100_t2690/all_100_t2690_eteq_tol_8")
del_path = Path("/groups/sxs/hchaudha/spec_runs/del.sh")
with del_path.open('w') as f:
  write_dir(folder_path, f)

In [ ]:
folder_path.name == "asd"

In [ ]:
a = np.linspace(0.02,0.07,100)
a = np.array([0.03,0.05])
b = np.exp(-a*10)
plt.plot(a,b)
plt.plot(a,b**3)
plt.show()

In [ ]:
np.exp(-0.03*10)**3,np.exp(-0.05*10)**3

In [ ]:
import pickle

file = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/path_dict.pkl")
with file.open('rb') as f:
  data = pickle.load(f)

In [ ]:
asd = data['Lev3_R0050'].sort()

In [ ]:
data['Lev3_R0050']

In [ ]:
for i in range(8):
  amr_tol = 0.000216536 * 4**(-i)
  ode_tol = amr_tol/2000


In [ ]:
file_path = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev3_R0200/Lev3_R0200.h5")
file_path = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev4_R0200/Lev4_R0200.h5")
file_path = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_R0050/Lev5_R0050.h5")

In [ ]:
with h5py.File(file_path,'r') as f:
    names = []
    f.visit(names.append)
    f.visit(print)
    data = np.array(f['Beta.dat'])
    # print(np.array(data),np.array(data).shape)

print(names)

In [ ]:
plt.plot(data[:,0])
plt.show()

In [ ]:
a = 6.76675e-09
for i in range(7):
  print(i-2,a*4**(4-i))

In [ ]:
import numpy as np
import pandas as pd
import sympy as sp

# Define symbol
x = sp.symbols('x')

# Taylor polynomial generator
def taylor_poly(func, terms):
    return sp.series(func, x, 0, terms).removeO()

# Functions to test
funcs = {'sin': sp.sin(x), 'cos': sp.cos(x), 'tan': sp.tan(x)}

# Test parameters
a_vals = [1/4, 1/8, 1/16, 1/32]
terms_vals = [6, 10 ,16]

# Compute max errors
results = []
for name, f in funcs.items():
    for terms in terms_vals:
        # Generate Taylor polynomial and lambdify
        poly_expr = taylor_poly(f, terms)
        poly_func = sp.lambdify(x, poly_expr, 'numpy')
        for a in a_vals:
            xs = np.linspace(-a, a, 10001)
            true_vals = getattr(np, name)(xs)
            approx_vals = poly_func(xs)
            max_err = np.max(np.abs(true_vals - approx_vals))
            results.append({
                'Function': name,
                'Interval a': a,
                'Terms': terms,
                'Max Error': max_err
            })

# Create DataFrame
df = pd.DataFrame(results)

# Pivot tables for each function
pivot_tables = {}
for func in df['Function'].unique():
    sub = df[df['Function'] == func]
    pivot = sub.pivot(index='Interval a', columns='Terms', values='Max Error')
    pivot_tables[func] = pivot


In [ ]:
def trunc_err_max(lev):
    return 0.000216536 * 4**(-lev)

for i in range(16):
    print(f"Level {i}: {trunc_err_max(i):.2e}, {trunc_err_max(i)/100:.2e}")

In [ ]:
base_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/25_allcd_gaussEx_5_60/Ev/Lev3_AA/Run/GW2")
file_path = base_path / "BondiCceR0100.h5"
file_path = base_path / "BondiCceR0150.h5"
file_path = base_path / "BondiCceR0200.h5"
file_path = base_path / "BondiCceR0250.h5"
file_path = base_path / "BondiCceR0300.h5"
file_path = base_path / "BondiCceR0350.h5"
file_path = base_path / "BondiCceR0400.h5"
file_path = base_path / "BondiCceR0500.h5"
file_path = base_path / "BondiCceR0600.h5"
file_path = base_path / "BondiCceR0700.h5"
file_path = base_path / "BondiCceR0800.h5"
file_path = base_path / "BondiCceR0900.h5"
# file_path = base_path / "PhiMinus_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "PhiPlus_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r2Psi3_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r3Psi2_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r4Psi1_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r5Psi0_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "RawStrahlkorperIntegrals.h5"
# file_path = base_path / "rh_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "rPsi4_FiniteRadii_CodeUnits.h5"

# base_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/GW_data_lev5")
# file_path = base_path / "PhiMinus_FiniteRadii_CodeUnits/PhiMinus_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "PhiPlus_FiniteRadii_CodeUnits/PhiPlus_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r2Psi3_FiniteRadii_CodeUnits/r2Psi3_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r3Psi2_FiniteRadii_CodeUnits/r3Psi2_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r4Psi1_FiniteRadii_CodeUnits/r4Psi1_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "r5Psi0_FiniteRadii_CodeUnits/r5Psi0_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "RawStrahlkorperIntegrals/RawStrahlkorperIntegrals.h5"
# file_path = base_path / "rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5"
# file_path = base_path / "rPsi4_FiniteRadii_CodeUnits/rPsi4_FiniteRadii_CodeUnits.h5"

In [ ]:
# # load the ylm for a given radius
# radius = GetFiniteRadiusExtractionList(file_path)[0]

# df = read_finite_radius_quantaties(file_path, radius)

var = 'H.dat'

with h5py.File(file_path, "r") as f:
    # all_m = ['DrJ.dat', 'H.dat', 'J.dat', 'Q.dat', 'U.dat', ]
    # some_m = ['Beta.dat', 'DuR.dat',  'R.dat', 'W.dat']

    col_names = list(f[var].attrs['Legend'])
    col_names[0] = 't(M)'
    name_to_index_dict = {name: i for i, name in enumerate(col_names)}

    data = {
        't(M)': f[var][:,name_to_index_dict['t(M)']]
    }

    maxL = int(col_names[-1].split(",")[0][3:])
    for L in range(maxL + 1):
        for m in range(-L,L+1):
            real_key = f'Re({L},{m})'
            img_key = f'Im({L},{m})'
            if real_key in name_to_index_dict:
                data[f'{L},{m}'] = np.array(f[var][:,name_to_index_dict[real_key]],dtype=np.complex128)
            if img_key in name_to_index_dict:
                data[f'{L},{m}'] += 1j * np.array(f[var][:,name_to_index_dict[img_key]],dtype=np.complex128)

    df1 = pd.DataFrame(f[var][:], columns=col_names)
    df = pd.DataFrame(data)

In [ ]:
a  ="  1.2005000000000000e+03   8.9999996468564696e+00   6.1777068749999996e-07  -3.1738393000000001e-09     3.332189299305591490 0.0004700380924178489150 -2.51492084335359477e-11 0.0009303412704874686884   0.01193588066721216860 -1.31864069749434024e-10   0.01718358209424653699 3.912758179854723509e-10   -0.0143210366777721133 0.0002371382406535133306 -1.86070007877221378e-12 -9.37602468567065461e-07 -5.54594373978010017e-12 0.0002758568556644109583 8.183769456356657693e-12 -0.000376038270615937408 -0.000185664477264617221 5.042864839129655007e-12 0.0001039214335328891636 -2.49620610132571576e-12 0.0001566887989086071173 6.796649958786201087e-12 -0.000117787324455803901 -2.98183028003373833e-12 2.697213608891328628e-05 -1.17308601805129028e-05 2.769580921521032155e-13 5.076160129557759647e-06 -1.16536403204033356e-13 -1.24146259108430945e-06 -3.11830643050692920e-13 6.871968515477514664e-06 3.163110163225307778e-13 -6.64975881945115268e-06 -2.11065029220975700e-13 3.348408999396187087e-06 1.633358603269606111e-06 -7.89502384014298722e-14 -1.71007311303385265e-06 9.378354746722092436e-14 1.154123380546298339e-06 -4.21004995372296279e-14 1.649847673707672201e-06 1.078160891330615790e-13 -1.18688161364637627e-06 -4.62562057190461014e-14 6.612600068157355052e-08 -2.12011611826850929e-14 1.362872748144135510e-06 2.335798540034619338e-07 -9.38086137000385092e-15 -1.82126231473401040e-07 8.758243670489675380e-15 1.106063053892940786e-07 -4.52007439023940519e-15 -3.50645917293332081e-08 -9.63218534115459426e-15 1.407444836302205025e-07 8.975783370950197423e-15 -1.13643876887229632e-07 -5.40265219940290008e-15 3.619216893320841639e-08 -7.52272785274320154e-16 1.203164555392407218e-07 4.512668784349584736e-09 3.625306456927417817e-16 1.108589961823464428e-08 -1.26229100126000065e-15 -1.76854010800865363e-08 1.496278048502305254e-15 1.302661256392042684e-08 -6.48393653055586259e-16 1.686038600557780532e-08 1.518763740006054476e-15 -1.10866800202760704e-08 -5.23455919705677549e-16 -2.56741637221952408e-09 -6.37117421686327011e-16 1.602646918470430027e-08 1.040379301765658340e-15 -2.49732185233900007e-08 -1.50676749824547277e-09 1.255270106456095447e-16 3.050133643940880111e-09 -2.66196553865913935e-16 -3.14062704300778358e-09 2.215173464061235435e-16 2.186281032264250031e-09 -1.34675817548564856e-16 -7.60854056332600048e-10 -4.51521968163233060e-16 2.612532851480423705e-09 2.314351041983806143e-16 -1.85588977751334964e-09 -1.23174159725455873e-16 2.594763827329969594e-10 -4.03710157525573900e-17 1.991269176971297996e-09 2.012863416268783034e-16 -4.88057243028047513e-09 -3.80177088108055998e-10 3.213754870749806714e-17 1.745117842118223135e-10 -1.96625355745647185e-17 2.641575291854022113e-11 -2.73412125734979470e-18 -1.49712328233552422e-10 3.150758709897673418e-18 1.301551106582985598e-10 -5.86552550985539874e-18 1.327348941842248431e-10 1.627346025062404316e-17 -6.37187983462890740e-11 5.931472067405597644e-18 -9.15693493791682118e-11 4.035956715458028005e-18 2.142787425772689735e-10 2.531897643938656480e-17 -2.01949932599544745e-10 -1.40446262116423178e-17 2.234000695039622890e-11 -4.94785774066330768e-11 2.338802121676814960e-18 -1.02074837456496336e-11 3.400261694690530366e-18 4.340504604453555769e-11 7.861876903224407093e-18 -5.23161721674375768e-11 1.196237833415489003e-17 3.986017016228957894e-11 6.158284713828837710e-18 -1.45973007908626391e-11 7.645350407373415585e-17 4.484205543493853978e-11 -1.04862787823234540e-17 -2.83720456734390815e-11 3.497636346872528421e-18 -2.02006463495767698e-12 -2.45336748740270876e-17 3.793859396039950658e-11 -1.21638919347830537e-17 -6.58122695734259520e-11 -1.07328328857248833e-17 7.934780948181223204e-11 2.712183592378183271e-12 7.609744628276970201e-18 -4.91535013307655285e-12 1.573428203118574605e-17 4.191686269908831521e-12 9.568156678117299867e-18 -1.84417609367429994e-12 -6.39312705152172773e-18 -2.31382963427821001e-13 -3.30443548835612538e-17 7.816105725753411349e-13 -1.39759235918026738e-17 -7.38978652071830885e-14 9.252100024143377914e-18 7.744900085690106098e-13 -8.19060353008836363e-18 -2.16892312638966040e-12 -5.93178468302321432e-18 2.667514791284960528e-12 1.634685549344026702e-17 -1.03563163937363380e-12 -6.38499236979831897e-18 -2.63425763436490992e-12 -1.59300631949739269e-17 7.837357465437129118e-12"
a  ="  1.9215000000000000e+03   8.9999996468564696e+00   6.1777068749999996e-07  -3.1738393000000001e-09     3.707199981358661578  -0.00135248340999583874 7.182079924569097855e-12  -0.00140438885689326935   0.01650593596404972829 -2.74671005674575688e-10   0.02204685060673725050 7.772181214432470835e-10   -0.0180452183318158682 0.0003433942449901832093 -3.88141915128143338e-12 -7.41462552261861374e-06 -1.02337508086455627e-11 0.0003608561745306319195 1.501043249066243559e-11 -0.000474731233980739940 -0.000290309863222545244 1.226928345271066276e-11 0.0001652672878177250315 -6.08170276464566633e-12 0.0002354773823621160586 1.567106142214171343e-11 -0.000171384865911951853 -6.35284657205939511e-12 1.715953088230414052e-05 -1.92869861020438771e-05 6.819315118006494559e-13 8.555672165674628522e-06 -2.91033139207891349e-13 -2.13648878445511099e-06 -7.28425640102346117e-13 1.073660377969895389e-05 7.326024776732605775e-13 -1.01139754260058103e-05 -4.59473137664395234e-13 3.680736651936993184e-06 2.698118925507486649e-06 -2.15608694265093385e-13 -3.07706622012715829e-06 2.643379741778396779e-13 2.132656338423456484e-06 -1.19856553505344866e-13 2.907330129486780960e-06 2.910376999132823255e-13 -2.01479024705921951e-06 -1.12798580447508636e-13 -1.34560897264234052e-07 -7.88269137408758570e-14 2.851817949323754363e-06 4.171182171999205903e-07 -2.65238661073783627e-14 -3.48469783310996461e-07 2.550291656343498486e-14 2.164440431274898666e-07 -1.33608304460436540e-14 -6.92058116698392468e-08 -2.70029494450165188e-14 2.585905415584507728e-07 2.459869014605975785e-14 -2.00507339227777822e-07 -1.37039799704659398e-14 3.823032228157437389e-08 -4.81143111088610060e-15 2.858000299741590708e-07 1.730152706645742066e-08 8.357025518901757959e-16 2.003060048903548858e-08 -3.97019625377270873e-15 -3.68849333876522558e-08 4.900113732267915919e-15 2.809594198155590750e-08 -2.19754001789382875e-15 3.506276184959309205e-08 4.827459530425419907e-15 -2.20724752837785928e-08 -1.49880511729319783e-15 -8.24455068788903467e-09 -2.44082831119350807e-15 3.692405023608957224e-08 3.643192780992657040e-15 -5.33731481323632960e-08 -1.61987201189713064e-09 4.038624800988681506e-16 6.245494916773841864e-09 -7.83318947815226617e-16 -6.95789402957544191e-09 7.614445317493613542e-16 4.974417070084850377e-09 -4.31856827126567407e-16 -1.74755775607419612e-09 -1.02590720581959537e-15 5.628378431523701629e-09 6.559336197086425373e-16 -3.78874469617914825e-09 -2.76726010674344966e-16 -4.09705219838837163e-11 -2.10017650331081755e-16 5.286066885171328343e-09 6.046803053326739251e-16 -1.16065482189929809e-08 -9.82512860016088739e-10 5.884171109489596660e-17 4.983123235997021566e-10 -6.04357227780951475e-18 2.278491096489023718e-11 -4.04890076852520207e-17 -3.63296597439707237e-10 7.720191848160074075e-17 3.301481779250980331e-10 -4.27921146695173820e-17 3.338442646025282199e-10 6.252062829806215533e-17 -1.52505826080513915e-10 5.637305198861873977e-18 -2.48339334348801109e-10 -4.04049151648992345e-17 5.438448069671617007e-10 6.732495291922702373e-17 -4.63260738319972807e-10 -5.34125367670493781e-17 -7.32424642000071648e-11 -1.74814434464530908e-10 -2.72315913956789280e-18 9.324425398557333047e-13 -8.21128056884856569e-19 1.007393004158844036e-10 -2.59170636040265859e-17 -1.34255166475529511e-10 3.650115891448223428e-17 1.056630557591728076e-10 1.893037364638080428e-18 -3.91431012314762000e-11 3.342976533583412666e-16 1.133084719339728450e-10 3.108140897336375596e-17 -6.69746058792308151e-11 -3.01626330632754986e-17 -1.72468168515076930e-11 -4.14683326063310023e-17 1.129842819865252380e-10 2.398664452717314180e-17 -1.78272162719918826e-10 -4.17551500278488877e-18 1.871250353080171034e-10 4.127811134708544736e-12 2.330841150530388599e-17 -1.32191879754368059e-11 -9.37197701872028515e-18 1.231330686675457881e-11 -2.34989830174157239e-17 -5.65643733739969966e-12 1.594051867486306851e-17 -6.63654577043539424e-13 8.816745769418163103e-18 2.411533193030147637e-12 6.952643703114522898e-18 2.213672872354624946e-13 -4.42585173441242026e-18 1.938516014751858888e-12 -2.54359676696084370e-17 -6.12816850000662479e-12 -5.20475218366331510e-17 7.308884699628076635e-12 -9.55902346205967605e-18 -1.81127908525430017e-12 -1.57009298533005327e-17 -9.52413664902358269e-12 3.526333880196931378e-17 2.422604187962788843e-11 5.374033056552432883e-12 -4.59250923493614363e-17 -3.06517283124343443e-12 -3.17815765075361981e-17 6.204288311549453605e-13 -1.90818104713208554e-17 1.463030013622088357e-12 -8.29813052583631352e-18 -2.41974555632191435e-12 -1.19824107861017764e-17 2.076764004240480033e-12 2.018656285417600298e-17 -8.00885943140845208e-13 -2.95974525569377742e-16 2.100863347206131153e-12 -5.09841163124302948e-18 -1.06774956133828089e-12 -3.09557803158770190e-18 -6.55936211083282115e-13 -3.59279864659342034e-17 2.348139685788188850e-12 -5.16922667182617523e-18 -3.07794341610365521e-12 3.425583814257462691e-18 2.150480886692975502e-12 -2.38191518059008319e-17 8.019723531381991551e-13"
a_size = len(a.strip().split())
np.sqrt(a_size-4)

In [ ]:
def read_power_diagnostics_non_power_spectrum(
    file_path, dat_file_name, psi_or_kappa, top_num
):
    top_num = int(top_num)
    with h5py.File(file_path, "r") as f:
        data_dict = {}
        if psi_or_kappa == "psi":
            data_index = 1
        else:
            data_index = 2
        for sd_name in f.keys():
            top_name = get_top_name_from_number(top_num, sd_name)
            data_dict[
                f"{psi_or_kappa}_{dat_file_name.split('.')[0]}_{top_name} on {sd_name[:-4]}"
            ] = f[sd_name][f"{top_name}_{dat_file_name}"][:, data_index]

        # get time var
        any_subdomain = next(iter(f.keys()))
        top_name = get_top_name_from_number(top_num, any_subdomain)
        data_dict["t(M)"] = f[any_subdomain][f"{top_name}_{dat_file_name}"][:, 0]

        df = pd.DataFrame(data_dict)

    return df

In [ ]:
h5_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/GW_data_lev3/rPsi4_FiniteRadii_CodeUnits/rPsi4_FiniteRadii_CodeUnits.h5")
# h5_path = Path("/resnick/groups/sxs/hchaudha/spec_runs/36_segs/L15/GW_data_lev3/rh_FiniteRadii_CodeUnits/rh_FiniteRadii_CodeUnits.h5")

with h5py.File(h5_path, "r") as f:
    names = []
    f.visit(names.append)
    f.visit(print)

    file = 'R0300.dir/Y_l6_m6.dat'
    file = 'R0300.dir/Y_l2_m0.dat'
    data = f[file][:]

In [ ]:
def start_and_end_idx_for_time_range(t_arr, t_start, t_end):
    start_idx = np.searchsorted(t_arr, t_start, side='left')
    end_idx = np.searchsorted(t_arr, t_end, side='right')
    return start_idx, end_idx


start_idx, end_idx = start_and_end_idx_for_time_range(data[:,0], 1000, 10000)
# start_idx, end_idx = start_and_end_idx_for_time_range(data[:,0], 12100, 12130)

# plt.plot(data[start_idx:end_idx,0],data[start_idx:end_idx,2], label='Im')
plt.plot(data[start_idx:end_idx,0],data[start_idx:end_idx,1], label='Re', marker='.')
# plt.plot(data[start_idx:end_idx,0],np.sqrt(data[start_idx:end_idx,1]**2+data[start_idx:end_idx,2]**2), label='abs')
plt.xlabel('t(M)')
# plt.ylabel('rPsi4 (Code Units)')
plt.title(file)
plt.legend()
plt.show()

In [ ]:
df1 = runs_data_dict['q1_0_0_8_0_0_8_MaxDL_R0309']
df2 = runs_data_dict['q1_0_0_8_0_0_8_R0309']

diff_dict = waveform_diff_all_modes(df1, df2,t_min=1500)
get_cumilative_waveform_diff(diff_dict)

In [ ]:
diff_dict

In [ ]:
def generate_radii_1_over_r(Rmin, Rmax, NumRadii):
    """
    Generate extraction radii evenly spaced in 1/r.

    Args:
        Rmin: Minimum radius
        Rmax: Maximum radius
        NumRadii: Number of radii to generate

    Returns:
        List of radii (as integers)
    """
    if NumRadii == 1:
        return [Rmax]

    # Spacing in 1/r
    delta = (1.0 / Rmax - 1.0 / Rmin) / (NumRadii - 1)

    radii = []
    for i in range(NumRadii - 1):
        # Compute radius evenly spaced in 1/r, then round to integer
        r = int(1.0 / (1.0 / Rmin + i * delta))
        radii.append(r)

    # Last radius is always exactly Rmax
    radii.append(int(Rmax))

    return radii


In [ ]:
csvals = ""
for i in generate_radii_1_over_r(250,900,24):
    csvals += f"{i}, "
print(csvals)

In [ ]:
# q1 cost

inspiral = 15000
ringdown = 10000

Lev6_rate = 20
Lev5_rate = Lev6_rate
Lev4_rate = Lev6_rate * 2
Lev3_rate = Lev6_rate * 3
Lev2_rate = Lev6_rate * 4

hours6 = (inspiral + ringdown) / Lev6_rate
hours5 = (inspiral + ringdown) / Lev5_rate
hours4 = (inspiral + ringdown) / Lev4_rate
hours3 = (inspiral + ringdown) / Lev3_rate
hours2 = (inspiral + ringdown) / Lev2_rate

total_hours = hours6 + hours5 + hours4 + hours3 + hours2
# total_hours = hours6 + hours5 + hours4 
total_hours


In [ ]:
diff_int_dict = {
    "6_set1_L6s1@6_set1_L6s2": 1.2858028858326789,
    "6_set1_L6s1@6_set1_L6s3": 2.952014392741012,
    "6_set1_L6s1@6_set1_L6s4": 3.8928176255488096,
    "6_set1_L6s1@6_set1_L6s5": 3.9671436667026736,
    "6_set1_L6s1@6_set1_L6s6": 3.914069956217513,
    "6_set1_L6s2@6_set1_L6s3": 4.233594523893546,
    "6_set1_L6s2@6_set1_L6s4": 5.176475788978051,
    "6_set1_L6s2@6_set1_L6s5": 5.250682223344887,
    "6_set1_L6s2@6_set1_L6s6": 5.197502238165702,
    "6_set1_L6s3@6_set1_L6s4": 0.9429371466287805,
    "6_set1_L6s3@6_set1_L6s5": 1.0171364351233168,
    "6_set1_L6s3@6_set1_L6s6": 0.9639556750278575,
    "6_set1_L6s4@6_set1_L6s5": 0.07440602543930704,
    "6_set1_L6s4@6_set1_L6s6": 0.02289045491236974,
    "6_set1_L6s5@6_set1_L6s6": 0.05318158328457974,
}

diff_phi = {
 '6_set1_L6s1@6_set1_L6s2': 0.18291673620686666,
 '6_set1_L6s1@6_set1_L6s3': 0.42241567036236083,
 '6_set1_L6s1@6_set1_L6s4': 0.5564967966590947,
 '6_set1_L6s1@6_set1_L6s5': 0.5671463644311162,
 '6_set1_L6s1@6_set1_L6s6': 0.5595623478412944,
 '6_set1_L6s2@6_set1_L6s3': 0.6048938796815856,
 '6_set1_L6s2@6_set1_L6s4': 0.7392498744431973,
 '6_set1_L6s2@6_set1_L6s5': 0.7498889427836771,
 '6_set1_L6s2@6_set1_L6s6': 0.7422949585840114,
 '6_set1_L6s3@6_set1_L6s4': 0.1343604960424211,
 '6_set1_L6s3@6_set1_L6s5': 0.14499928674451112,
 '6_set1_L6s3@6_set1_L6s6': 0.13740503738952842,
 '6_set1_L6s4@6_set1_L6s5': 0.010660803068585521,
 '6_set1_L6s4@6_set1_L6s6': 0.0032864203962184035,
 '6_set1_L6s5@6_set1_L6s6': 0.007594258469580951}

In [ ]:
plt.plot(diff_phi.values(), diff_int_dict.values())
plt.show()

In [ ]:
import h5py
import numpy as np

filepath = "/resnick/groups/sxs/hchaudha/HighAccuracy1025/HighAccuracy102501/Ecc0/GW_data_lev4/BondiCceR0317/red_cce.h5"

with h5py.File(filepath, "r") as f:
    strain = f["/SpectreR0317.cce/Strain"]
    legend = list(strain.attrs["Legend"])
    data = strain[:]

    # Get time
    time = data[:, 0]

    # Extract (l=2, m=2) mode
    l, m = 2, 2
    real_idx = legend.index(f"Real Y_{l},{m}")
    imag_idx = legend.index(f"Imag Y_{l},{m}")
    h22 = data[:, real_idx] + 1j * data[:, imag_idx]

    # Get polarizations: h = h_+ - i*h_x
    h_plus = h22.real
    h_cross = -h22.imag
